# Chart Analysis Functions

In [1]:
## Basic stuff
%load_ext autoreload
%autoreload

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))
#IPython.Cell.options_default.cm_config.lineNumbers = true;

################################################################################
## Python Version
################################################################################
import sys


################################################################################
## General Stuff
################################################################################
from multiprocessing import Pool
from tqdm import tqdm


################################################################################
## Util Stuff
################################################################################
from timeUtils import clock, elapsed
from ioUtils import saveFile, getFile


################################################################################
## Music DB
################################################################################
from mainDB import mainDB
from musicDBMap import musicDBMap
from masterDBMatchClass import masterDBMatchClass
from matchDBArtist import matchDBArtist


################################################################################
## Music Names
################################################################################
from masterArtistNameDB import masterArtistNameDB


################################################################################
## Chart Stuff
################################################################################
from artistIgnores import getArtistIgnores
from billboardData import billboardData
from top40Data import top40Data
from spotifyData import spotifyData
from chartArtistAlbumData import chartArtistAlbumData
from chartUtils import *
from extraArtists import extraKnownArtists


################################################################################
## Pandas Stuff
################################################################################
import pandas as pd
from pandas import DataFrame
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

chartType = None

print("Python: {0}".format(sys.version))
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Python: 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2021-04-07 22:59:08.803545


# Rename Input Data

In [2]:
def updateMulti():
    updateMultiDB(1)
    updateMultiDB(2)

def updateDB():
    retval = updateManDB(1)
    if retval:
        updateManDB(2)
        print("Good!")
        return True
    else:
        print("ERROR!")
        updateManDB(1, True)
        return False

def updateRelDB():
    retval = updateManDB(3)
    if retval:
        updateManDB(4)
        print("Good!")
        return True
    else:
        print("ERROR!")
        updateManDB(3, True)
        return False

def updateMultiDB(step=1, show=False):
    retval = True    
    if step == 1:
        tmpmanDB = masterArtistNameDB("multi", init=True)
        tmpmanDB.forceReload(getFile("multi.yaml"))
        tmpmanDB.checkForRecursives()
        
    if step == 2:
        tmpmanDB = masterArtistNameDB("multi", init=True)
        tmpmanDB.forceReload(getFile("multi.yaml"))
        tmpmanDB.save()
        
        multimanDB = masterArtistNameDB("multi", init=False)
        saveFile(idata=multimanDB.getRenames(), ifile="multi.yaml")

def updateManDB(step=1, show=False):
    retval = True
    if step == 1:
        if show is False:
            try:
                tmpmanDB = masterArtistNameDB("main", init=True)
                tmpmanDB.forceReload(getFile("main.yaml"))
                tmpmanDB.checkForRecursives()
            except:
                retval = False
        else:
            tmpmanDB = masterArtistNameDB("main", init=True)
            tmpmanDB.forceReload(getFile("main.yaml"))
            tmpmanDB.checkForRecursives()
            retval = False            

    if step == 2:
        tmpmanDB = masterArtistNameDB("main", init=True)
        tmpmanDB.forceReload(getFile("main.yaml"))
        tmpmanDB.save()

        manDB = masterArtistNameDB("main", init=False)
        saveFile(idata=manDB.getRenames(), ifile="main.yaml")
        
    if step == 3:
        if show is False:
            try:
                tmpmanDB = masterArtistNameDB("main", init=False)
                tmpmanDB.addRenames(getFile("relDBRenames2.yaml"))
                tmpmanDB.checkForRecursives()
            except:
                retval = False
        else:
            tmpmanDB = masterArtistNameDB("main", init=False)
            tmpmanDB.addRenames(getFile("relDBRenames2.yaml"))
            tmpmanDB.checkForRecursives()
            retval = False
            
        
    if step == 4:
        tmpmanDB = masterArtistNameDB("main", init=False)
        tmpmanDB.addRenames(getFile("relDBRenames2.yaml"))
        tmpmanDB.save()
        
        manDB = masterArtistNameDB("main", init=False)
        saveFile(idata=manDB.getRenames(), ifile="main.yaml")
        
    
    _, _ = clock("Last Ran")
    return retval
    
    
def loadMainDB(recreate=False):
    if recreate:
        start, cmt = clock("Loading ArtistDBs Renaming Artists")
        maindb = mainDB(mdb=None, create=True, debug=True)
        maindb.setDBArtists(recreate=False)
        maindb.loadDBDataMap()
        elapsed(start, cmt)    
    else:
        start, cmt = clock("Loading ArtistDBs Without Renaming")
        maindb = mainDB(mdb=None, create=False, debug=True)
        maindb.setDBArtists(recreate=False)
        maindb.loadDBDataMap()
        elapsed(start, cmt)
        
    return maindb
    
manDB      = masterArtistNameDB("main", init=False)
multimanDB = masterArtistNameDB("multi", init=False)
maindb = loadMainDB(True)
_, _ = clock("Last Run")

========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  There are currently 13234 artist keys.
  There are currently 15823 renamed artist keys.
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
  There are currently 521 artist keys.
  There are currently 560 renamed artist keys.
Current Time is Wed Apr 07, 2021 22:59:09 for Loading ArtistDBs Renaming Artists
Setting Basic Database Objects
=========================   discogs   =========================
=========================   allmusic   =========================
=========================   musicbrainz   =========================
=========================   lastfm   =========================
=========================   rateyourmusic   =========================
=========================   deezer   ======================

Creating Pandas DataFrame for 48605 Artists
	Shape --> (48605, 1)
	Shape --> (48605, 2)
  Finding Real Artist Name
	Shape --> (48605, 4)
  Removing None Artist
	Shape --> (48605, 4)
  Finding Disc Artist Name
	Shape --> (48605, 5)
DataFrame Shape is (48605, 7)
Current Time is Wed Apr 07, 2021 23:00:13 for Done with 
=================================== Creating Artist DB ===================================
Process [Done with 
=================================== Creating Artist DB ===================================] took 1 seconds.
Saving Master Artist DB File: /Users/tgadfort/Music/Discog/db-deezer/MasterSlimArtistDB.p
  --> This file is 6.6MB.
SlimArtistDB(Deezer) DataFrame Shape is (48605, 7)
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  There are currently 13234 artist keys.
  There are currently 15823 renamed artist keys.
Adding 1 known multi delim artists.
Current T

Summary Statistics For DB: KWorbiTunes
    Using Known Artists: False
    Found 11922 ID -> Name entries
    Found 11883 Name -> ID entries
    Found 0 Albums
Current Time is Wed Apr 07, 2021 23:00:48 for Done with Loading ArtistDBs Renaming Artists
Process [Done with Loading ArtistDBs Renaming Artists] took 1.6 minutes.
Current Time is Wed Apr 07, 2021 23:00:48 for Last Run


In [ ]:
dbData = getFile("/Users/tgadfort/Music/Discog/db-rateyourmusic/MasterSlimArtistDB.p")
print(dbData.shape)
dbData[dbData["Name"] == "Life & Time"]

In [ ]:
dbData = maindb.slimArtistDB['RateYourMusic']
print(dbData.shape)
dbData[dbData["Name"] == "Life & Time"]

# Reverse Chart Lookup

In [ ]:
updateMultiDB(1)

In [ ]:
updateMultiDB(2)

In [ ]:
updateManDB(1)

In [ ]:
updateManDB(2)

In [ ]:
if False:
    from multiArtist import multiartist
    mularts  = multiartist(cutoff=0.9, discdata=None, exact=False)
    mularts.setKnownMultiDelimArtists(getFile("../multiartist/knownMultiArtists.yaml"))
    knownMultiArtists = getFile("../multiartist/knownMultiArtists.yaml")
    print(len(knownMultiArtists),"Known Artists")

    ignoresComp = ['Soundtrack', 'Various Artists', 'Original Broadway Cast Recording', 'Read-Along','Veggietales', 'Pokemon', 
                   'Barney', 'Walt Disney Read-Along', 'The Powerpuff Girls',
                   'Superstar Kidz', 'Original Cast', 'Original Cast Recording']
    ignores = ignoresComp    

    def matchArtist(artistName):
        subNames = mularts.getArtistNames(artistName)
        result = {subName: mdbmap.isKnownByName(subName) for subName in subNames}
        for subName in result.keys():
            if subName in ignores:
                result[subName] = True
        return result


    for chartName in chartDFs.keys():
        try:
            chartDFs[chartName]['DB'] = chartDFs[chartName]['RenamedArtist'].apply(matchArtist)
        except:
            print("Error with {0}".format(chartName))

In [ ]:
vals = {}
for chartName in chartDFs.keys():
    try:
        vals[chartName] = chartDFs[chartName][~chartDFs[chartName]["DB"].apply(lambda x: all(x.values()))].shape[0]
    except:
        pass
from pandas import Series

In [ ]:
Series(vals).sort_values(ascending=False)

In [ ]:
mdbmap.isKnownByName("Salt-N-Pepa")

In [ ]:
chartDF = chartDFs["vinyl-albums"]
def getMissingArtists(chartDF):
    missing = chartDF[~chartDF["DB"].apply(lambda x: all(x.values()))]
    return list(set(getFlatList(missing["DB"].apply(lambda x: [k for k,v in x.items() if v is False]))))
getMissingArtists(chartDF)

In [ ]:
def getMissingArtists(chartDF):
    ichartDF[~chartDF["DB"].apply(lambda x: all(x.values()))]

In [ ]:
chartName = "independent-albums"
chartDFs[chartName][~chartDFs[chartName]["DB"].apply(lambda x: all(x.values()))]

In [ ]:
bMap = {}
for chartTitle,chartName in initMap.items():
    newName = newMap.get(chartTitle)
    if newName is not None:
        bMap[newName] = chartName


In [ ]:
saveFile(ifile="billboardMapping.p", idata=bMap)

In [ ]:
bMap = getFile("billboardMapping.p")

# Top40 Data

In [ ]:
%load_ext autoreload
%autoreload
from top40Data import top40Data
td = top40Data(minYear=1, maxYear=2021)
td.setChartUsage(rank=[0,1,2,3,4,5,6])
td.setDBRenames(manDB)
td.setMultiDBRenames(multimanDB)
td.setFullChartData()
td.setArtistAlbumData()
td.saveArtistAlbumData()
td.saveFullChartData()

_, _ = clock("Last Run")

# Billboard Data

In [3]:
%load_ext autoreload
%autoreload
from billboardData import billboardData
bd = billboardData(minYear=1, maxYear=2021)
bd.setChartUsage(rank=[0,1,2,3,4,5,6,7,8])
bd.setDBRenames(manDB)
bd.setMultiDBRenames(multimanDB)
bd.setFullChartData()
bd.setArtistAlbumData()
bd.saveArtistAlbumData()
bd.saveFullChartData()
chartType = "Billboard"

_, _ = clock("Last Run")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Found 62 files.
  Getting Chart For top
name = top
  Using 6 Charts
  Using 6 Charts
  Getting Chart For hot
name = hot
  Using 12 Charts
  Getting Chart For adult
name = adult
  Using 4 Charts
  Using 16 Charts
  Getting Chart For alternative
name = alternative
  Using 4 Charts
  Getting Chart For countryMusic
name = countryMusic
  Using 10 Charts
  Getting Chart For rock
name = rock
  Using 14 Charts
  Using 28 Charts
  Getting Chart For christian
name = christian
  Using 21 Charts
  Getting Chart For rnb
name = rnb
  Using 18 Charts
  Using 39 Charts
  Getting Chart For canadian
name = canadian
  Using 4 Charts
  Getting Chart For comedy
name = comedy
  Using 2 Charts
  Getting Chart For general
name = general
  Using 4 Charts
  Getting Chart For twitter
name = twitter
  Using 4 Charts
  Using 14 Charts
  Getting Chart For folkblue
name = folkblue
  Using 6 Charts
  Getting Chart For classical
na

# BillboardYE Data

In [206]:
%load_ext autoreload
%autoreload
from billboardYE import billboardYE
bYE = billboardYE(minYear=1, maxYear=2021)
bYE.setChartUsage(rank=[0,1,2,3,4,5,6,7]) #,2,3,4,5,6,7]) #,6,7])
bYE.setDBRenames(manDB)
bYE.setFullChartData()
bYE.setArtistAlbumData()
bYE.saveFullChartData()
bYE.saveArtistAlbumData()
chartType = "BillboardYE"

_, _ = clock("Last Run")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
  Getting Chart For hot
  Using 5 Charts
  Using 5 Charts
  Getting Chart For adult
  Using 4 Charts
  Using 4 Charts
  Getting Chart For alternative
  Using 7 Charts
  Getting Chart For countryMusic
  Using 15 Charts
  Getting Chart For rock
  Using 17 Charts
  Getting Chart For rnb
  Using 37 Charts
  Using 53 Charts
  Getting Chart For alternative
  Using 7 Charts
  Getting Chart For countryMusic
  Using 15 Charts
  Getting Chart For rock
  Using 17 Charts
  Getting Chart For rnb
  Using 37 Charts
  Using 76 Charts
  Getting Chart For christian
  Using 32 Charts
  Getting Chart For canadian
  Using 4 Charts
  Getting Chart For comedy
  Using 2 Charts
  Getting Chart For radio
  Using 2 Charts
  Using 30 Charts
  Getting Chart For christian
  Using 32 Charts
  Getting Chart For canadian
  Using 4 Charts
  Getting Chart For comedy
  Using 2 Charts
  Getting Chart For radio
  Using 2 Charts
  Using 

Renamed 3847 single artists
Saving 3698 Full Artist Data
Saving data to currentBillboardYEFullChartArtistAlbumData.p
  --> This file is 375.6kB.
Saved data to currentBillboardYEFullChartArtistAlbumData.p
  --> This file is 375.6kB.
Saving 3698 Artist Album Data to currentBillboardYEArtistAlbumData.p
Saving data to currentBillboardYEArtistAlbumData.p
  --> This file is 179.8kB.
Saved data to currentBillboardYEArtistAlbumData.p
  --> This file is 179.8kB.
Current Time is Wed Apr 07, 2021 20:47:21 for Last Run


# Russia

In [ ]:
%load_ext autoreload
%autoreload
from russia import russiaData
rud = russiaData(minYear=0, maxYear=2021)
#lad.setDBRenames(manDB)
#lad.setMultiDBRenames(multimanDB)
rud.setChartUsage(rank=[0])
rud.setFullChartData()
rud.setArtistAlbumData()
rud.saveArtistAlbumData()
rud.saveFullChartData()
_, _ = clock("Last Run")

# Latin America

In [ ]:
%load_ext autoreload
%autoreload
from latinAmerica import latinAmericaData
lad = latinAmericaData(minYear=0, maxYear=2021)
lad.setDBRenames(manDB)
lad.setMultiDBRenames(multimanDB)
lad.setChartUsage(rank=[0])
lad.setFullChartData()
lad.setArtistAlbumData()
lad.saveArtistAlbumData()
lad.saveFullChartData()
_, _ = clock("Last Run")

# East Asia

In [ ]:
%load_ext autoreload
%autoreload
from eastAsia import eastAsiaData
ead = eastAsiaData(minYear=0, maxYear=2021)
ead.setDBRenames(manDB)
ead.setMultiDBRenames(multimanDB)
ead.setChartUsage(rank=[2])
ead.setFullChartData()
ead.setArtistAlbumData()
ead.saveArtistAlbumData()
ead.saveFullChartData()
_, _ = clock("Last Run")

# MusicVF

In [ ]:
%load_ext autoreload
%autoreload
from musicVFData import musicVFData
mvf = musicVFData(minYear=0, maxYear=2021)
mvf.setDBRenames(manDB)
mvf.setMultiDBRenames(multimanDB)
#mvf.setChartUsage(rank=[0,1,2,3,4,5])
mvf.setChartUsage(rank=[0,1,2,3,4,5])
mvf.setFullChartData()
mvf.setArtistAlbumData()
mvf.saveArtistAlbumData()
mvf.saveFullChartData()
chartType = "MusicVF"
_, _ = clock("Last Run")

# Spotify Code

## Parse

In [ ]:
%load_ext autoreload
%autoreload
from spotifyData import spotifyData
sd = spotifyData(minYear=1, maxYear=2021)
sd.parse()

## Create

In [ ]:
%load_ext autoreload
%autoreload
from spotifyData import spotifyData
sd = spotifyData(minYear=1, maxYear=2021)
sd.setDBRenames(manDB)
sd.setMultiDBRenames(multimanDB)
sd.setChartUsage(rank=[0,1,2,3,4,5,6,7,8,9,10,11,12,13]) #,2,3,4,5,6,7,8,9,10,11,12,13]) #,1,2,3,4,5,7,8,9,10,11,12])
sd.setFullChartData()
sd.setArtistAlbumData()
sd.saveArtistAlbumData()
sd.saveFullChartData()
chartType="Spotify"
_, _ = clock("Last Run")

# Spotify Viral

In [24]:
%load_ext autoreload
%autoreload
from spotifyViral import spotifyViralData
sd = spotifyViralData(minYear=1, maxYear=2021)
sd.setDBRenames(manDB)
sd.setMultiDBRenames(multimanDB)
sd.setChartUsage(rank=[0,1,2,3,4]) #,1,2,3,4,5,6,7,8,9,10,11,12,13]) #,2,3,4,5,6,7,8,9,10,11,12,13]) #,1,2,3,4,5,7,8,9,10,11,12])
sd.setFullChartData()
sd.setArtistAlbumData()
sd.saveArtistAlbumData()
sd.saveFullChartData()
chartType="SpotifyViral"
_, _ = clock("Last Run")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using Charts For Rank 0
    Categories: ['unitedstates', 'canada']
	Chart: unitedstates
  Using 1 Charts
	Chart: canada
  Using 1 Charts
Using 2 Charts For Rank 0
Using Charts For Rank 1
    Categories: ['unitedkingdom', 'australia', 'ireland', 'newzealand']
	Chart: unitedkingdom
  Using 1 Charts
	Chart: australia
  Using 1 Charts
	Chart: ireland
  Using 1 Charts
	Chart: newzealand
  Using 1 Charts
Using 4 Charts For Rank 1
Using Charts For Rank 2
    Categories: ['germany', 'austria', 'switzerland']
	Chart: germany
  Using 1 Charts
	Chart: austria
  Using 1 Charts
	Chart: switzerland
  Using 1 Charts
Using 3 Charts For Rank 2
Using Charts For Rank 3
    Categories: ['italy', 'france', 'spain', 'netherlands', 'luxembourg']
	Chart: italy
  Using 1 Charts
	Chart: france
  Using 1 Charts
	Chart: spain
  Using 1 Charts
	Chart: netherlands
  Using 1 Charts
	Chart: luxembourg
  Using 1 Charts
Using 5 Char

# UDiscoverMusic

In [23]:
%load_ext autoreload
%autoreload
from uDiscoveryMusic import uDiscoverMusicData
ud = uDiscoverMusicData(minYear=1, maxYear=2021)
ud.setDBRenames(manDB)
ud.setMultiDBRenames(multimanDB)
ud.setChartUsage(rank=[0,1,2,3,4,5,7,8,9,10,11,12])
ud.setFullChartData()
ud.setArtistAlbumData()
ud.saveArtistAlbumData()
ud.saveFullChartData()
chartType = "uDiscover"
1/0
_, _ = clock("Last Run")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using Charts For Rank 0
    Categories: ['artists']
	Chart: artists
  Using 1 Charts
Using 1 Charts For Rank 0
Using Charts For Rank 1


KeyError: 1

# Africa

In [ ]:
%load_ext autoreload
%autoreload
from Africa import africaData
africa = africaData()
africa.setDBRenames(manDB)
africa.setMultiDBRenames(multimanDB)
africa.setChartUsage(rank=[0,1,2,3,4,5,6,7])
africa.setFullChartData()
africa.setArtistAlbumData()
africa.saveArtistAlbumData()
africa.saveFullChartData()
_, _ = clock("Last Run")

# Rate Your Music Albums

In [ ]:
%load_ext autoreload
%autoreload
from rateYourMusic import rateYourMusicData
ryd = rateYourMusicData(minYear=0, maxYear=2021)
ryd.setDBRenames(manDB)
ryd.setMultiDBRenames(multimanDB)
#ryd.setChartUsage(rank=list(range(16, 30+1)))  # 0-3 Best Decade, 4-6 Diverse Decade, 7-9 Esoteric Decade, 10-13 Worst Decade, 14-15 Mixes, 16-30 Best Year
#ryd.setChartUsage(rank=list(range(10, 13+1)))  # 0-3 Best Decade, 4-6 Diverse Decade, 7-9 Esoteric Decade, 10-13 Worst Decade, 14-15 Mixes, 16-30 Best Year
#ryd.setChartUsage(rank=list(range(0, 9+1)))  # 0-3 Best Decade, 4-6 Diverse Decade, 7-9 Esoteric Decade, 10-13 Worst Decade, 14-15 Mixes, 16-30 Best Year
#ryd.setChartUsage(rank=list(range(14, 15+1)))  # 0-3 Best Decade, 4-6 Diverse Decade, 7-9 Esoteric Decade, 10-13 Worst Decade, 14-15 Mixes, 16-30 Best Year
ryd.setChartUsage(rank=list(range(0,30+1)))
ryd.setFullChartData()
ryd.setArtistAlbumData()
ryd.saveArtistAlbumData()
ryd.saveFullChartData()
_, _ = clock("Last Run")

# Rate Your Music Songs

In [ ]:
%load_ext autoreload
%autoreload
from rateYourMusicSong import rateYourMusicSongData
ryd = rateYourMusicSongData(minYear=0, maxYear=2021)
ryd.setDBRenames(manDB)
ryd.setMultiDBRenames(multimanDB)
#ryd.setChartUsage(rank=list(range(0,9+1))) # 0-4 Best Decade, 5-9 Worst Decade, 10-25 Best Yearly, 26-40 Worst Yearly
#ryd.setChartUsage(rank=list(range(10,25+1))) # 0-4 Best Decade, 5-9 Worst Decade, 10-25 Best Yearly, 26-40 Worst Yearly
#ryd.setChartUsage(rank=list(range(26,40+1))) # 0-4 Best Decade, 5-9 Worst Decade, 10-25 Best Yearly, 26-40 Worst Yearly
ryd.setChartUsage(rank=list(range(0,40+1))) # 0-4 Best Decade, 5-9 Worst Decade, 10-25 Best Yearly, 26-40 Worst Yearly
ryd.setFullChartData()
ryd.setArtistAlbumData()
ryd.saveArtistAlbumData()
ryd.saveFullChartData()
_, _ = clock("Last Run")

# Rate Your Music Lists

## 1

In [ ]:
%load_ext autoreload
%autoreload
from rateYourMusicList import rateYourMusicListData
ryd = rateYourMusicListData(minYear=0, maxYear=2021)
ryd.setDBRenames(manDB)
ryd.setMultiDBRenames(multimanDB)
ryd.setChartUsage(rank=list(range(0,16+1)))
#ryd.setChartUsage(rank=list(range(30,39+1))) # Best Albums 16-29 script, 30-44 other
ryd.setFullChartData()
ryd.setArtistAlbumData()
ryd.saveArtistAlbumData()
ryd.saveFullChartData()
chartType = "RYMList"
_, _ = clock("Last Run")

## 2

In [ ]:
%load_ext autoreload
%autoreload
from rateYourMusicList2 import rateYourMusicList2Data
ryd = rateYourMusicList2Data(minYear=0, maxYear=2021)
ryd.setDBRenames(manDB)
ryd.setMultiDBRenames(multimanDB)
#ryd.setChartUsage(rank=list(range(0,12+1)))
ryd.setChartUsage(rank=list(range(0,13+1)))
ryd.setFullChartData()
ryd.setArtistAlbumData()
ryd.saveArtistAlbumData()
ryd.saveFullChartData()
chartType = "RYMList2"
_, _ = clock("Last Run")

## Script

In [ ]:
%load_ext autoreload
%autoreload
from rateYourMusicScript import rateYourMusicScriptData
ryd = rateYourMusicScriptData(minYear=0, maxYear=2021)
ryd.setDBRenames(manDB)
ryd.setMultiDBRenames(multimanDB)
ryd.setChartUsage(rank=list(range(0,13+1))) # 0-13
ryd.setFullChartData()
ryd.setArtistAlbumData()
ryd.saveArtistAlbumData()
ryd.saveFullChartData()
chartType = "RYMScript"
_, _ = clock("Last Run")

# Analyze Data

In [25]:
## Basic stuff
%load_ext autoreload
%autoreload
from chartArtistAlbumData import chartArtistAlbumData

singleArtistAlbumData = {}
manyArtistAlbumData   = {}

mType     = "Full"
cad = chartArtistAlbumData(chartType, ignoreMultiNames=False)
cad.createIndivArtistAlbumData()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
chartArtistAlbumData(SpotifyViral)
Found 62 files.
Adding 2643 known multi delim artists.
  Assigning 2643 known multi-name artists
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  There are currently 13234 artist keys.
  There are currently 15823 renamed artist keys.
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
  There are currently 521 artist keys.
  There are currently 560 renamed artist keys.
There are 19290 unique artists in artist albums data
Corr   | 1755  | Janelle MonÃ¡e                                     ---> Janelle MonÃe
Corr   | 3063  | Lovegnarly /// Azahriah                            ---> Lovegnarly --- Azahriah
Corr   | 3604  | Take/Five                  

# Create Master Match Class & Ignores List

In [5]:
def showMatchedStatus(chartType, mdbmaps, mdbmc):
    print("Matched {0} / {1} Artists for {2} ChartType".format(mdbmaps[chartType].getSize(), len(mdbmc.matchData[chartType]), chartType))
    
def showRemainingMatches(chartType, mdbmc):
    print('='*100)
    print("="*10,"Artists That Were Not Tested (Low Albums)",'='*10)
    toMatch   = mdbmc.getDataToMatch(chartType, maxValues=10000, maxAlbums=100000, minAlbums=0, useKnown=True, ignores=ignoresList)
    print("="*10,"Previously Tested Artists Without A Match",'='*10)
    toMatch   = mdbmc.getDataToMatch(chartType, maxValues=10000, maxAlbums=100000, minAlbums=0, useKnown=False, dbMatches=0, ignores=ignoresList)
    print('='*100)

    
def showStatus(chartType, mdbmaps, mdbmc):
    showMatchedStatus(chartType, mdbmaps, mdbmc)
    showRemainingMatches(chartType, mdbmc)
    
    
def analyzeMatches(maindb, mdbmaps, chartType, maxVal=1.0, diffVal=0.1):
    dbRenames = {}
    toget   = {}
    togetID = {}
    for primaryKey,artistName in mdbmaps[chartType].getArtists().items():
        primaryArtistName = artistName
        artistData = mdbmaps[chartType].getArtistDataByKey(primaryKey)
        for db,dbID in artistData.getDict().items():
            if db in ["DatPiff", "MetalStorm", "CDandLP", "AceBootlegs", "RockCorner", "MusicStack"]:
                continue
            if dbID is not None:
                secondaryArtistName = maindb.getArtistDBNameFromID(db, dbID)
                if secondaryArtistName is None:
                    if toget.get(db) is None:
                        toget[db] = []
                        togetID[db] = {}
                    toget[db].append(primaryArtistName)
                    togetID[db][dbID] = primaryArtistName
                    continue
                s = SequenceMatcher(None, primaryArtistName, secondaryArtistName)
                ratio = s.ratio()
                maxRatio = maxVal
                if ratio >= maxRatio:
                    continue

                if ratio >= maxRatio-diffVal:
                    if dbRenames.get(secondaryArtistName) is not None:
                        if dbRenames[secondaryArtistName] == primaryArtistName:
                            continue
                        else:
                            pass
                    dbRenames[secondaryArtistName] = primaryArtistName
                    print("[{0: <30} {1: <4} {2: >30}] \t --> ({3}) {4} / {5} ({6})".format(primaryArtistName,round(ratio,2),secondaryArtistName,chartType,primaryKey,dbID,db))


    print("Found {0} artists to get.".format(len(toget)))
    print("Found {0} artists to rename.".format(len(dbRenames)))
    return toget, togetID, dbRenames    
    
    
def removeDBByNames(mdbmaps, chartType, names):
    for name in names:
        kKey = mdbmaps[chartType].getPrimaryKey(artistName=name, artistID=None)
        if kKey is not None:
            mdbmaps[chartType].removeArtistByKey(kKey)
    mdbmaps[chartType].save()
            
    
def removeDBRenames(mdbmaps, chartType, dbRenames):
    for k,v in dbRenames.items():
        kKey = mdbmaps[chartType].getPrimaryKey(artistName=k, artistID=None)
        if kKey is not None:
            mdbmaps[chartType].removeArtistByKey(kKey)

        kKey = mdbmaps[chartType].getPrimaryKey(artistName=v, artistID=None)
        if kKey is not None:
            mdbmaps[chartType].removeArtistByKey(kKey)
    mdbmaps[chartType].save()
    
    
def matchItAllLoosely(mdbmaps, thresholds, useAlbums=True, mdbmc=None, db=None, toMatch=None, nProcs=3):
    if len(toMatch) == 0:
        return

    num_processes = nProcs
    if useAlbums is True:
        func = matchDBArtistWithAlbums
    else:
        func = matchDBArtistWithoutAlbums
    pfunc = partial(func, **thresholds) # Giving some arguments for kwargs
    #argument_list = list(inputs.items()) # [random.randint(0, 100) for _ in range(num_jobs)]
    dbName = list(toMatch.keys())[0]

    argument_list = toMatch[dbName]
    if len(argument_list) == 0:
        return
    print("Running imap multiprocessing for {0} artists ...".format(len(argument_list)))
    result_list = multiProc(func=pfunc, argument_list=argument_list,
                                           num_processes=num_processes)

    return result_list


def analyzeLooseMatches(maindb, matchResults, maxVal=1.0, diffVal=0.1):
    dbRenames = {}
    
    for item in matchResults:
        primaryKey = item[0] 
        primaryArtistName = item[1]
        artistID   = item[2]
        mcs        = item[3]
        for db,matchData in mcs.items():
            matchN  = matchData.matchN
            if matchN is None:
                continue
            dbID = matchData.matchID
            dbScore = matchData.matchScore
            
            secondaryArtistName = maindb.getArtistDBNameFromID(db, dbID)
            if primaryArtistName == secondaryArtistName:
                continue
            dbRenames[primaryArtistName] = secondaryArtistName
            
    print("Found {0} artists to rename.".format(len(dbRenames)))
    return dbRenames      
    
    
def analyzeRenames(manDB, dbRenames):
    redos = {}
    dels = []
    if len(dbRenames) == 0:
        print("Nothing to process.")
        return
    for oldername,bestname in dbRenames.items():
        if oldername in manDB.artistNameDB.keys() and bestname not in manDB.artistNameDB.keys():
            print(oldername,bestname)

            redos[bestname] = oldername
            dels.append(oldername)
            print('\t',oldername,'\t',bestname)

    print("# Renames: {0}".format(len(dbRenames)))
    print("# Dels: {0}".format(len(dels)))
    for oldername in dels:
        del dbRenames[oldername]

    print("# Redos: {0}".format(len(redos)))
    dbRenames.update(redos)
    print("# Renames: {0}".format(len(dbRenames)))
    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")




    def isAscii(ele):
        return len(ele) == len(ele.encode())

    dbRenames = getFile("relDBRenames2.yaml")
    redos = {}
    dels = []
    for k,v in dbRenames.items():    
        if isAscii(v) and not isAscii(k):
            if '’' in k or "“" in k:
                continue
            redos[v] = k
            dels.append(k)
            print('\t',k,'\t',v)
    #print(len(dbRenames))
    for k in dels:
        del dbRenames[k]

    #print(len(dbRenames))
    dbRenames.update(redos)
    #print(len(dbRenames))
    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")




    dbRenames = getFile("relDBRenames2.yaml")
    redos = {}
    dels = []
    for k,v in dbRenames.items():    
        if " the " in v and " The " in k:
            if k == v.replace(" the ", " The "):
                redos[v] = k
                dels.append(k)
                print('\t',k,'\t',v)

    #print(len(dbRenames))
    for k in dels:
        del dbRenames[k]

    #print(len(dbRenames))
    dbRenames.update(redos)
    #print(len(dbRenames))
    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")




    dbRenames = getFile("relDBRenames2.yaml")
    redos = {}
    dels = []
    for k,v in dbRenames.items():    
        if '’' in v or "“" in v:
            if not '’' in k and not "“" in v:
                redos[v] = k
                dels.append(k)
                print('\t',k,'\t',v)

    #print(len(dbRenames))
    for k in dels:
        del dbRenames[k]

    #print(len(dbRenames))
    dbRenames.update(redos)
    #print(len(dbRenames))


    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")








    def isAscii(ele):
        return len(ele) == len(ele.encode())

    dbRenames = getFile("relDBRenames2.yaml")
    redos = {}
    dels = []
    for k,v in dbRenames.items():    
        if isAscii(v) and not isAscii(k):
            if '’' in k or "“" in k:
                continue
            redos[v] = k
            dels.append(k)
            print('\t',k,'\t',v)
    #print(len(dbRenames))
    for k in dels:
        del dbRenames[k]

    #print(len(dbRenames))
    dbRenames.update(redos)
    #print(len(dbRenames))


    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")








    dbRenames = getFile("relDBRenames2.yaml")
    redos = {}
    dels = []
    for k,v in dbRenames.items():    
        if " the " in v and " The " in k:
            if k == v.replace(" the ", " The "):
                redos[v] = k
                dels.append(k)
                print('\t',k,'\t',v)

    #print(len(dbRenames))
    for k in dels:
        del dbRenames[k]

    #print(len(dbRenames))
    dbRenames.update(redos)
    #print(len(dbRenames))


    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")







    dbRenames = getFile("relDBRenames2.yaml")
    redos = {}
    dels = []
    for k,v in dbRenames.items():    
        if '’' in v or "“" in v:
            if not '’' in k and not "“" in v:
                redos[v] = k
                dels.append(k)
                print('\t',k,'\t',v)

    #print(len(dbRenames))
    for k in dels:
        del dbRenames[k]

    #print(len(dbRenames))
    dbRenames.update(redos)
    #print(len(dbRenames))


    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")
    #print(len(dbRenames))
    
    
        
    
def manualAppends(toMatch, chartType, minAlbums=0, maxAlbums=None, add=True, dbs=["AllMusic", "Deezer"], showSongs=True, pattern=None):
    if len(toMatch[chartType]) == 0:
        print("Nothing to append...")
        return
    
    toMatchDF = DataFrame({item[0]: {"ArtistName": item[1]["ArtistName"], "ArtistAlbums": item[1]["ArtistAlbums"]} for item in toMatch[chartType]}).T
    toMatchDF["NumAlbums"] = toMatchDF["ArtistAlbums"].apply(len)
    toMatchDF = toMatchDF.sort_values(by="NumAlbums", ascending=False)
    
    for idx,row in toMatchDF.iterrows():
        if add:
            artistName = row["ArtistName"]
            albumNames = row["ArtistAlbums"]
            nAlbums    = row["NumAlbums"]
            
            if pattern is not None:
                if artistName.find(pattern) == -1:
                    continue
            
            if nAlbums < minAlbums:
                continue
            if maxAlbums is not None:
                if nAlbums > maxAlbums:
                    continue                
            for db in dbs:
                print("add{0}(mdbmaps, {1}, {2}, {3})   ### [{4}]".format(db, "\"{0}\"".format(chartType), "\"{0}\"".format(idx), "\"{0}\"".format(""), artistName))
            if showSongs:
                print("### {0}".format("  ,  ".join(albumNames)))            
                for albumName in albumNames:
                    print("### {0}  ---> [{1}]".format(albumName, cad.getAlbumsData(albumName)))
            print("\n")
        else:
            print("ignores.append(\"{0}\")".format(artistName))
            if " & " in artistName:
                continue
                print("ignores.append(\"{0}\")".format(artistName))

    print("mdbmaps[{0}].save()".format("\"{0}\"".format(chartType)))    
    





        
def copyMapData(mdbmap):
    return
    mdbmap.saveCopy()
    
    
def getFoundMatches(result_list):
    results = 0
    
    for item in result_list:
        mcs        = item[3]
        for db,mc in mcs.items():
            matchID    = mc.matchID
            matchScore = mc.matchScore
            if matchID is not None:
                results += 1
                
    return results


def saveMapData(mdbmap, result_list):
    for item in result_list:
        primaryKey = item[0] 
        artistName = item[1]
        artistID   = item[2]
        mcs        = item[3]
        mdbmap.addArtistByKey(primaryKey, artistName=artistName, artistID=artistID)
        for db,mc in mcs.items():
            matchID    = mc.matchID
            matchScore = mc.matchScore
            if matchID is not None:
                mdbmap.addArtistDataByKey(primaryKey, db, matchID)

    mdbmap.save()
    

def matchItAll(mdbmaps, thresholds, useAlbums=True, mdbmc=None, db=None, toMatch=None, saveIt=True, nProcs=3):
    if toMatch is None:
        toMatch   = mdbmc.getDataToMatch(db, maxValues=10000, maxAlbums=50000)

    if len(toMatch) == 0:
        return

    num_processes = nProcs
    if useAlbums is True:
        func = matchDBArtistWithAlbums
    else:
        func = matchDBArtistWithoutAlbums
    pfunc = partial(func, **thresholds) # Giving some arguments for kwargs
    #argument_list = list(inputs.items()) # [random.randint(0, 100) for _ in range(num_jobs)]
    dbName = list(toMatch.keys())[0]    
    #copyMapData(mdbmaps[dbName])

    argument_list = toMatch[dbName]
    if len(argument_list) == 0:
        return
    print("Running imap multiprocessing for {0} artists ...".format(len(argument_list)))
    result_list = multiProc(func=pfunc, argument_list=argument_list,
                                           num_processes=num_processes)


    
    nSaved = getFoundMatches(result_list)
    if nSaved > 0 or True:
        print("Found {0} matching artist/db matches")
        start, cmt = clock("Saving...")
        saveMapData(mdbmaps[dbName], result_list)
        elapsed(start, cmt)
        return True
    else:
        return False
    #print("\nSleeping for 10 seconds...\n")
    #sleep(10)
    
    #mdbmc.matchMutualMaps()

    
def matchDBArtistWithoutAlbums(item, *args, **kwargs):
    primaryKey = item[0]
    artistData = item[1]
    artistName   = artistData["ArtistName"]
    artistID     = primaryKey
    artistAlbums = None
    mdbMatcher = matchDBArtist(maindb)
    mdbMatcher.setArtistInfo(artistName, artistID, artistAlbums)
    mdbMatcher.setThresholds(matchNumArtistName=kwargs['numArtistName'], matchArtistNameCutoff=kwargs['artistNameCutoff'],
                             matchArtistAlbumCutoff=kwargs['artistAlbumCutoff'], matchNumArtistAlbums=kwargs['numArtistAlbums'],
                             matchScore=kwargs['score']),
    mcs    = mdbMatcher.findPotentialArtistNameMatchesWithoutAlbums()
    retval = [primaryKey,artistName,artistID,mcs]
    return retval


def matchDBArtistWithAlbums(item, *args, **kwargs):    
    #time.sleep(0.0025)
    primaryKey = item[0]
    artistData = item[1]
    artistName   = artistData["ArtistName"]
    artistID     = primaryKey
    artistAlbums = artistData["ArtistAlbums"]

    
    #print(item)
    #print("{0: <50}{1: <50}{2}".format(artistName,artistID,len(artistAlbums)))
    #1/0
    
    mdbMatcher = matchDBArtist(maindb)
    mdbMatcher.setArtistInfo(artistName, artistID, artistAlbums)
    mdbMatcher.setThresholds(matchNumArtistName=kwargs['numArtistName'], matchArtistNameCutoff=kwargs['artistNameCutoff'], 
                             matchArtistAlbumCutoff=kwargs['artistAlbumCutoff'], matchNumArtistAlbums=kwargs['numArtistAlbums'],
                             matchScore=kwargs['score'])
    mcs    = mdbMatcher.findPotentialArtistAlbumMatches()
    retval = [primaryKey,artistName,artistID,mcs]
    return retval


def multiProc(func, argument_list, num_processes):
    pool = Pool(processes=num_processes)
    result_list_tqdm = []
    for result in tqdm(pool.imap(func=func, iterable=argument_list), total=len(argument_list)):
        result_list_tqdm.append(result)
    return result_list_tqdm



# MDB Maps

In [26]:
mdbmaps = {}
print("ChartType = {0}".format(chartType))
print("MType     = {0}".format(mType))
mdbmaps[chartType] = musicDBMap(chartType, init=False, copy=False)
mdbmc = masterDBMatchClass(maindb, mdbmaps)
ignoresList = createIgnoreList()

manDB      = masterArtistNameDB("main", init=False)
multimanDB = masterArtistNameDB("multi", init=False)


def reMatch(mdbmaps=mdbmaps, mdbmc=mdbmc, mtype="Single"):
    if mtype == "Single":
        matchData = {mdbmaps[chartType].getPrimaryKey(artistName=artistName, artistID=None): {"ArtistName": artistName, "ArtistAlbums": artistAlbums} for artistName,artistAlbums in cad.getSingleArtistAlbumData().items()}
        mdbmc.setDBMatchData(chartType, matchData)
    elif mtype == "Many":
        matchData = {mdbmaps[chartType].getPrimaryKey(artistName=artistName, artistID=None): {"ArtistName": artistName, "ArtistAlbums": artistAlbums} for artistName,artistAlbums in cad.getManyArtistAlbumData().items()}
        mdbmc.setDBMatchData(chartType, matchData)
    elif mtype == "Full":
        matchData = {mdbmaps[chartType].getPrimaryKey(artistName=artistName, artistID=None): {"ArtistName": artistName, "ArtistAlbums": artistAlbums} for artistName,artistAlbums in cad.getFullArtistAlbumData().items()}
        mdbmc.setDBMatchData(chartType, matchData)
        
reMatch(mtype=mType)

ChartType = SpotifyViral
MType     = Full
  Valid DBs: ['Discogs', 'AllMusic', 'MusicBrainz', 'AceBootlegs', 'RateYourMusic', 'LastFM', 'DatPiff', 'RockCorner', 'CDandLP', 'MusicStack', 'MetalStorm', 'Deezer', 'AppleMusic', 'AlbumOfTheYear', 'Genius', 'IHeart', 'KWorbSpotify', 'KWorbiTunes', 'KWorbYouTube']
  Loaded 3979 previously matched entries
Loading Artist Names
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  There are currently 13234 artist keys.
  There are currently 15823 renamed artist keys.
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
  There are currently 521 artist keys.
  There are currently 560 renamed artist keys.
  Setting matchData for SpotifyViral


## Match Using Names And Albums

In [27]:
for minAlbums,minAlbumsData in getIterItems().items():
    print("="*10,minAlbums)
    maxAlbums = minAlbumsData["Max"]
    maxValues = minAlbumsData["Vals"]
    for i in range(15):
        toMatch   = mdbmc.getDataToMatch(chartType, maxValues=maxValues, maxAlbums=maxAlbums, minAlbums=minAlbums, ignores=ignoresList)
        if len(toMatch[chartType]) == 0:
            break
        retval = matchItAll(mdbmaps, getThresholds(minAlbums), toMatch=toMatch, nProcs=3)
        if retval is False:
            break
reMatch(mtype=mType)
print("Done.\n\n")

========== 50
========== 20
Total                -> 19539
After Ignores        -> 19512
After Known          -> 15533
After MaxAlbums      -> 15533
After MinAlbums      -> 30
After MaxValues      -> 30
Running imap multiprocessing for 30 artists ...


100%|██████████| 30/30 [00:09<00:00,  3.17it/s]


Found {0} matching artist/db matches
Current Time is Wed Apr 07, 2021 23:50:13 for Saving...
Saving data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyViralMap.p
  --> This file is 967.5kB.
Saved data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyViralMap.p
  --> This file is 967.5kB.
Current Time is Wed Apr 07, 2021 23:50:15 for Done with Saving...
Process [Done with Saving...] took 2 seconds.
========== 10
Total                -> 19539
After Ignores        -> 19512
After Known          -> 15503
After MaxAlbums      -> 15503
After MinAlbums      -> 217
After MaxValues      -> 200
Running imap multiprocessing for 200 artists ...


100%|██████████| 200/200 [01:12<00:00,  2.75it/s]


Found {0} matching artist/db matches
Current Time is Wed Apr 07, 2021 23:51:32 for Saving...
Saving data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyViralMap.p
  --> This file is 1.0MB.
Saved data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyViralMap.p
  --> This file is 1.0MB.
Current Time is Wed Apr 07, 2021 23:51:35 for Done with Saving...
Process [Done with Saving...] took 2 seconds.
Total                -> 19539
After Ignores        -> 19512
After Known          -> 15303
After MaxAlbums      -> 15303
After MinAlbums      -> 17
After MaxValues      -> 17
Running imap multiprocessing for 17 artists ...


...

Found {0} matching artist/db matches
Current Time is Thu Apr 08, 2021 01:45:58 for Saving...
Saving data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyViralMap.p
  --> This file is 3.8MB.
Saved data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyViralMap.p
  --> This file is 3.8MB.
Current Time is Thu Apr 08, 2021 01:46:13 for Done with Saving...
Process [Done with Saving...] took 14 seconds.
Total                -> 19539
After Ignores        -> 19512
After Known          -> 1875
After MaxAlbums      -> 1875
After MinAlbums      -> 1875
After MaxValues      -> 1000
Running imap multiprocessing for 1000 artists ...


100%|██████████| 1000/1000 [11:01<00:00,  1.51it/s]


Found {0} matching artist/db matches
Current Time is Thu Apr 08, 2021 01:57:16 for Saving...
Saving data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyViralMap.p
  --> This file is 4.0MB.
Saved data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyViralMap.p
  --> This file is 4.0MB.
Current Time is Thu Apr 08, 2021 01:57:31 for Done with Saving...
Process [Done with Saving...] took 14 seconds.
Total                -> 19539
After Ignores        -> 19512
After Known          -> 875
After MaxAlbums      -> 875
After MinAlbums      -> 875
After MaxValues      -> 875
Running imap multiprocessing for 875 artists ...


100%|██████████| 875/875 [10:59<00:00,  1.33it/s]


Found {0} matching artist/db matches
Current Time is Thu Apr 08, 2021 02:08:32 for Saving...
Saving data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyViralMap.p
  --> This file is 4.2MB.
Saved data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyViralMap.p
  --> This file is 4.2MB.
Current Time is Thu Apr 08, 2021 02:08:49 for Done with Saving...
Process [Done with Saving...] took 16 seconds.
  Setting matchData for SpotifyViral
Done.




In [28]:
#showStatus(chartType, mdbmaps, mdbmc)


## Match Only Using Names (Artists Without A Previous Match)

In [29]:
maxValues = 1000
for i in range(10):
    print("\n\n{0}> {1} <{2}\n".format("="*50,i,"="*50))    
    toMatch   = mdbmc.getDataToMatch(chartType, maxValues=maxValues, maxAlbums=100000, minAlbums=0, useKnown=False, dbMatches=0, ignores=ignoresList)
    nToMatch  = len(toMatch[chartType])
    print("\n{0}> {1: <3}: {2} <{3}\n".format("="*35,i,nToMatch, "="*35))    
    if nToMatch == 0:
        break
    matchItAll(mdbmaps, getThresholdsWithoutAlbums(1.0), toMatch=toMatch, useAlbums=False, nProcs=3)
    if nToMatch < maxValues:
        break
reMatch(mtype=mType)



==================================================> 0 <==================================================

Total                -> 19539
After Ignores        -> 19512
After DB Matches     -> 11684
After MaxAlbums      -> 11684
After MinAlbums      -> 11684
After MaxValues      -> 1000

===================================> 0  : 1000 <===================================

Running imap multiprocessing for 1000 artists ...


100%|██████████| 1000/1000 [04:44<00:00,  3.51it/s]


Found {0} matching artist/db matches
Current Time is Thu Apr 08, 2021 02:13:56 for Saving...
Saving data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyViralMap.p
  --> This file is 4.2MB.
Saved data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyViralMap.p
  --> This file is 4.2MB.
Current Time is Thu Apr 08, 2021 02:14:09 for Done with Saving...
Process [Done with Saving...] took 13 seconds.


==================================================> 1 <==================================================

Total                -> 19539
After Ignores        -> 19512
After DB Matches     -> 10813
After MaxAlbums      -> 10813
After MinAlbums      -> 10813
After MaxValues      -> 1000

===================================> 1  : 1000 <===================================

Running imap multiprocessing for 1000 artists ...


100%|██████████| 1000/1000 [05:38<00:00,  2.95it/s]


Found {0} matching artist/db matches
Current Time is Thu Apr 08, 2021 02:20:11 for Saving...
Saving data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyViralMap.p
  --> This file is 4.2MB.
Saved data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyViralMap.p
  --> This file is 4.2MB.
Current Time is Thu Apr 08, 2021 02:20:25 for Done with Saving...
Process [Done with Saving...] took 14 seconds.


==================================================> 2 <==================================================

Total                -> 19539
After Ignores        -> 19512
After DB Matches     -> 10152
After MaxAlbums      -> 10152
After MinAlbums      -> 10152
After MaxValues      -> 1000

===================================> 2  : 1000 <===================================

Running imap multiprocessing for 1000 artists ...


...

Found {0} matching artist/db matches
Current Time is Thu Apr 08, 2021 03:07:37 for Saving...
Saving data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyViralMap.p
  --> This file is 4.3MB.
Saved data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyViralMap.p
  --> This file is 4.3MB.
Current Time is Thu Apr 08, 2021 03:07:53 for Done with Saving...
Process [Done with Saving...] took 15 seconds.


==================================================> 8 <==================================================

Total                -> 19539
After Ignores        -> 19512
After DB Matches     -> 8397
After MaxAlbums      -> 8397
After MinAlbums      -> 8397
After MaxValues      -> 1000

===================================> 8  : 1000 <===================================

Running imap multiprocessing for 1000 artists ...


100%|██████████| 1000/1000 [08:06<00:00,  2.05it/s]


Found {0} matching artist/db matches
Current Time is Thu Apr 08, 2021 03:16:20 for Saving...
Saving data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyViralMap.p
  --> This file is 4.3MB.
Saved data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyViralMap.p
  --> This file is 4.3MB.
Current Time is Thu Apr 08, 2021 03:16:36 for Done with Saving...
Process [Done with Saving...] took 15 seconds.


==================================================> 9 <==================================================

Total                -> 19539
After Ignores        -> 19512
After DB Matches     -> 8291
After MaxAlbums      -> 8291
After MinAlbums      -> 8291
After MaxValues      -> 1000

===================================> 9  : 1000 <===================================

Running imap multiprocessing for 1000 artists ...


100%|██████████| 1000/1000 [08:25<00:00,  1.98it/s]


Found {0} matching artist/db matches
Current Time is Thu Apr 08, 2021 03:25:22 for Saving...
Saving data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyViralMap.p
  --> This file is 4.3MB.
Saved data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyViralMap.p
  --> This file is 4.3MB.
Current Time is Thu Apr 08, 2021 03:25:37 for Done with Saving...
Process [Done with Saving...] took 15 seconds.
  Setting matchData for SpotifyViral


In [30]:
#showStatus(chartType, mdbmaps, mdbmc)

## Try To Match Artists That Were Not Previously Tested (Low Albums)

In [31]:
for i in range(1):
    toMatch   = mdbmc.getDataToMatch(chartType, maxValues=10000, maxAlbums=100000, minAlbums=0, useKnown=True, ignores=ignoresList)
    nToMatch  = len(toMatch[chartType])
    if nToMatch == 0:
        print("Nothing else to match!")
        break
    matchItAll(mdbmaps, getThresholdsWithoutAlbums(1.0), toMatch=toMatch, useAlbums=False, nProcs=2)
reMatch(mtype=mType)

Nothing else to match!
  Setting matchData for SpotifyViral


In [32]:
#showStatus(chartType, mdbmaps, mdbmc)

## Extra Artists Info

In [33]:
#from extraArtists import extraKnownArtists
#extraKnownArtists(mdbmaps, chartType)
%autoreload
from chartUtils import addMusicVFExtras, addBillboardYEExtras
from chartUtils import addRateYourMusicSongExtras, addRateYourMusicAlbumExtras, addRateYourMusicListExtras, addRateYourMusicList2Extras
if True:
    if chartType == "MusicVF":
        addMusicVFExtras(mdbmaps)
    if chartType == "Spotity":
        addSpotifyExtras(mdbmaps)
    #if chartType == "SpotifyViral":
    #    addSpotifyViralExtras(mdbmaps)
    if chartType == "LatinAmerica":
        addLatinAmericaExtras(mdbmaps)
    if chartType == "BillboardYE":
        addBillboardYEExtras(mdbmaps)
    if chartType == "RYMSong":
        addRateYourMusicSongExtras(mdbmaps)
    if chartType == "RYMAlbum":
        addRateYourMusicAlbumExtras(mdbmaps)
    if chartType == "RYMList":
        addRateYourMusicListExtras(mdbmaps)
    if chartType == "RYMList2":
        addRateYourMusicList2Extras(mdbmaps)

In [34]:
toMatch   = mdbmc.getDataToMatch(chartType, maxValues=10000, maxAlbums=100000, minAlbums=0, useKnown=False, dbMatches=0, ignores=ignoresList)
nToMatch  = len(toMatch[chartType])
print("ToMatch --> {0}".format(nToMatch))
#showStatus(chartType, mdbmaps, mdbmc)

Total                -> 19539
After Ignores        -> 19512
After DB Matches     -> 8218
After MaxAlbums      -> 8218
After MinAlbums      -> 8218
After MaxValues      -> 8218
ToMatch --> 8218


In [35]:
#removeDBByNames(mdbmaps, chartType, [x[1]["ArtistName"] for x in toMatch[chartType]])

# Loose Matches

### Loose Match Code

In [ ]:
print([x[1]["ArtistName"] for x in toMatch[chartType]])

In [ ]:
removeDBByNames(mdbmaps, chartType, [x[1]["ArtistName"] for x in toMatch[chartType]])

### Loose Match Results

In [ ]:
#fixArtists=['Louis Armstrong and the All Stars', 'The George Shearing Quintet and Orchestra', 'Donald Byrd and Elmo Hope', 'His Piano and Group', 'Eric Dolphy and Booker Ervin']
#toMatchLoose = mdbmc.getDataToMatch(chartType, maxValues=10000, maxAlbums=100000, minAlbums=0, useKnown=False, dbMatches=0, fixArtists=fixArtists, ignores=ignoresList)

In [ ]:
printAllMissingArtists(toMatch, chartType)

In [161]:
%autoreload
from chartUtils import getLooseThresholds, getAllThresholds
useAlbums=True
thresholds={True: getLooseThresholds(), False: getLooseThresholds(albumCutoff=None)}
result_list=matchItAllLoosely(mdbmaps, thresholds[useAlbums], toMatch=toMatch, useAlbums=useAlbums)
dbRenames=analyzeLooseMatches(maindb, result_list)
analyzeRenames(manDB, dbRenames)
dbRenames

Running imap multiprocessing for 4 artists ...


 25%|██▌       | 1/4 [00:31<01:35, 31.95s/it]
Process ForkPoolWorker-132:
Process ForkPoolWorker-134:
Process ForkPoolWorker-133:
Traceback (most recent call last):
  File "/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/Users/tgadfort/opt/anaconda3/envs/py37/li

KeyboardInterrupt: 

  File "/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "<ipython-input-25-83f4d26aa8cc>", line 442, in matchDBArtistWithoutAlbums
    mcs    = mdbMatcher.findPotentialArtistNameMatchesWithoutAlbums()
  File "/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-25-83f4d26aa8cc>", line 442, in matchDBArtistWithoutAlbums
    mcs    = mdbMatcher.findPotentialArtistNameMatchesWithoutAlbums()
  File "/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/site-packages/dbmaster-0.0.1-py3.7.egg/matchDBArtist.py", line 80, in findPotentialArtistNameMatchesWithoutAlbums
    artistIDs = self.findPotentialArtistNameMatches()
  File "/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/Users/

In [ ]:
removeDBRenames(mdbmaps, chartType, dbRenames)
#mdbmaps["RYMSong"].getArtistDataByName("U2").show()

In [162]:
updateMulti()

========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
  Initializing a fresh DB for multi
  There are currently 0 artist keys.
  There are currently 0 renamed artist keys.
  Forced reload of 510 artist names
  Found 544 names
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
  Initializing a fresh DB for multi
  There are currently 0 artist keys.
  There are currently 0 renamed artist keys.
  Forced reload of 510 artist names
  Found 544 names
Saving 510 artist keys to /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
Saving data to /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
  --> This file is 20.6kB.
Saved data to /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
  --> This file is 20.6kB.
==========

In [ ]:
updateManDB(1, True)

In [115]:
updateDB()

========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  Initializing a fresh DB for main
  There are currently 0 artist keys.
  There are currently 0 renamed artist keys.
  Forced reload of 13221 artist names
  Found 15800 names
Current Time is Wed Apr 07, 2021 19:27:59 for Last Ran
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  Initializing a fresh DB for main
  There are currently 0 artist keys.
  There are currently 0 renamed artist keys.
  Forced reload of 13221 artist names
  Found 15800 names
Saving 13221 artist keys to /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
Saving data to /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  --> This file is 370.9kB.
Saved data to /Users/tgadfort/opt/anaconda3/envs/py37/musicnam

True

In [116]:
retval = updateManDB(3, True)

========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  There are currently 13221 artist keys.
  There are currently 15800 renamed artist keys.
Trying to add 2 renamed artist keys
There are currently 13221 artist keys.
There are currently 15800 renamed artist keys.
There are currently 13222 artist keys.
There are currently 15802 renamed artist keys.
Current Time is Wed Apr 07, 2021 19:28:06 for Last Ran


In [117]:
if updateRelDB():
    removeDBRenames(mdbmaps, chartType, dbRenames)

========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  There are currently 13221 artist keys.
  There are currently 15800 renamed artist keys.
Trying to add 2 renamed artist keys
There are currently 13221 artist keys.
There are currently 15800 renamed artist keys.
There are currently 13222 artist keys.
There are currently 15802 renamed artist keys.
Current Time is Wed Apr 07, 2021 19:28:09 for Last Ran
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  There are currently 13221 artist keys.
  There are currently 15800 renamed artist keys.
Trying to add 2 renamed artist keys
There are currently 13221 artist keys.
There are currently 15800 renamed artist keys.
There are currently 13222 artist keys.
There are currently 15802 renamed artist keys.
Saving 13222 artist keys to

In [118]:
from multiArtist import multiartist
knownMulti = getFile("knownMultiArtists.yaml")
print("Add the following to knownMultiArtists.yaml")
print("="*43)
mularts  = multiartist(cutoff=0.9, discdata=None, exact=False)
for renamedArtist in getFile("relDBRenames2.yaml").values():    
    N = mularts.getArtistNames(renamedArtist)
    if len(N) > 1:
        if renamedArtist not in knownMulti:
            print("- {0}".format(renamedArtist))

Add the following to knownMultiArtists.yaml


In [ ]:
#removeDBRenames(mdbmaps, chartType, dbRenames)
manDB      = masterArtistNameDB("main", init=False)
multimanDB = masterArtistNameDB("multi", init=False)
maindb     = loadMainDB(True)

# Manual Appends

In [16]:
#manualAppends(toMatch, chartType, minAlbums=10, add=True, dbs=["Discogs", "Deezer", "AllMusic"], showSongs=True)
#manualAppends(toMatch, chartType, minAlbums=0, add=True, dbs=["Discogs"], showSongs=False, pattern="Orchestra")

manualAppends(toMatch, chartType, minAlbums=0, maxAlbums=None, add=True, dbs=["Discogs"], showSongs=True)

addDiscogs(mdbmaps, "Billboard", "ab57e7cccc317dd923eca645ea91f778", "")   ### [Joe Rene And Orchestra]
### My True Story
### My True Story  ---> [['The Jive Five ::: Joe Rene And Orchestra']]


addDiscogs(mdbmaps, "Billboard", "0a77a78dfd95b2c6f7769977d755ad45", "")   ### [Tina Ray]
### Moonlight
### Moonlight  ---> [['Wilber Pan & Yuan Yawei']]


mdbmaps["Billboard"].save()


# Print Missing Artists

## Print Code Code

In [37]:
from googletrans import Translator
from langdetect import detect
import re

def printAllMissingArtists(toMatch, chartType):
    artists = [item[1]["ArtistName"] for item in toMatch[chartType]]
    print(artists)
    
def printMissingArtists(toMatch, chartType):
    t = Translator()
    
    def hasSpecial(artist):
        regex = re.compile('[@_!#$%^&*()<>?/\|}{~:]')
        return regex.search(artist) != None

    def isAscii(artist):
        return artist.isascii()

    def isDJ(artist):
        if len(artist) > 2:
            if artist[:2].upper() == "DJ":
                return True
            if artist[:3].upper() == "LIL":
                return True
        return False

    def isMulti(artist):
        if " And " in artist:
            return True
        if " and " in artist:
            return True
        if " & " in artist:
            return True
        return False
    
    def isShort(artist):
        if len(artist) <= 4:
            return True
        return False
    
    def isMid(artist):
        if len(artist) <= 7 and len(artist) > 4:
            return True
        return False
    
    def isLong(artist):
        if len(artist) > 7:
            return True
        return False
    
    def splitList(lvals, n=10):
        return [lvals[i:i + n] for i in range(0, len(lvals), n)]


    artists = [item[1]["ArtistName"] for item in toMatch[chartType]]
    print("="*25,"Non-Ascii Artists","="*25)
    langs = {}
    for x in [y for y in artists if not isAscii(y)]:
        try:
            lang = detect(x)
        except:
            lang = 'other'
        if langs.get(lang) is None:
            langs[lang] = []
        langs[lang].append(x)
    for lang,langData in langs.items():
        print("\t","-"*20,lang,"-"*25)
        for lval in splitList(langData):
            print(lval)

    print("="*25,"Ascii,Special Artists","="*25)
    for lval in splitList([x for x in artists if (isAscii(x) and hasSpecial(x))]):
        print(lval)

    print("="*25,"Ascii,Non-Special,DJ Artists","="*25)
    for lval in splitList([x for x in artists if (isAscii(x) and not hasSpecial(x) and isDJ(x))]):
        print(lval)
    
    print("="*25,"Ascii,Non-Special,Non-DJ,Non-Multi,Short Artists","="*25)
    for lval in splitList([x for x in artists if (isAscii(x) and not hasSpecial(x) and not isDJ(x) and not isMulti(x) and isShort(x))]):
        print(lval)

    print("="*25,"Ascii,Non-Special,Non-DJ,Non-Multi,Mid Artists","="*25)
    for lval in splitList([x for x in artists if (isAscii(x) and not hasSpecial(x) and not isDJ(x) and not isMulti(x) and isMid(x))]):
        print(lval)

    print("="*25,"Ascii,Non-Special,Non-DJ,Non-Multi,Long Artists","="*25)
    for lval in splitList([x for x in artists if (isAscii(x) and not hasSpecial(x) and not isDJ(x) and not isMulti(x) and isLong(x))]):
        print(lval)

## Print Code

In [38]:
printAllMissingArtists(toMatch, chartType)

['Bobby Cole', 'Obskür', 'Wellington Sea Shanty Society', 'Secession Studios', 'Cali Cave Records', 'Erick the Architect', 'Emma Langford', '遠野高志', 'DunkMemer', 'Minette Martin', 'lildeath', 'Tańcząca Krowa', 'Öwnboss', 'Extreme Music', 'Ed Ivory', 'A92', 'Ladytron', 'PattyCake', 'CMAT', 'badly', '20 Fingers', 'Robin M', 'Wacomo', 'Stegosaurus Rex', 'Ahmet Cinkaya', 'Cao White', 'Tom Felton', 'CMTEN', '-Darrell-', 'Fei Yu-ching', 'PAX', 'Smilez', 'MizOrMac', 'JAMIE WEBSTER', 'Brando', 'DJ Nazil', 'Lotte Kestner', 'KINGMOSTWANTED', 'Jack Shore', 'Caru', 'PS1', 'Nerina Pallot', 'Carlos Santos', 'TNAN', 'Aluna', 'Ivorian Doll', 'Lucy McWilliams', 'GLN', 'Lesley Roy', 'VICTORIA', 'Jealous of the Birds', 'Efendi', 'The Sei', 'Hope Sandoval and the Warm Inventions', 'Stephan F', 'The Blanks', 'Keshawn', 'Bow Anderson', 'Garth Brooks', 'Yola Recoba', 'Nothingtosay', 'Grace Davies', 'THAT KIND', 'Auntie Hammy', 'Ben Nicky', 'Full Tac', 'Ava Louise', 'DaÃ°i Freyr', 'Daniel Blume', 'Rucka Rucka 

In [39]:
printMissingArtists(toMatch, chartType)

========================= Non-Ascii Artists =========================
	 -------------------- de -------------------------
['Obskür', 'BeowÃ¼lf', 'Marcus Schössow', 'Susanne SundfÃ¸r', 'GERENDĀS', 'Kállay Saunders', 'Dér Heni', 'Matthias Schweighöfer', 'Löwenherz', 'FJØRT']
['Tinguely dä Chnächt', 'Klischée', 'dä Brüeder vom Heinz', 'BØJET', 'GATTÜSO', 'Büssi', 'Höhner', 'Te Nūtube', 'BÃ˜RNS', 'SCARFÉ†']
['SCARFɆ', 'B£NZA', 'Inüit', 'Ängie', 'Jean‐Michel Jarre', 'Erlend Øye', 'Artù', 'RØYLS', 'Karen Méndez', 'Émile Waldteufel']
['Nicolaï Dunger', 'BAMBIKÄÄRME', 'SIRÉN', 'SKÄM', 'SØNIN', 'GROÜNDLINE', 'SLIM RÄT', 'Röyh G', 'The Meänland', 'SKÁLD']
['Chlöe Howl', 'Einstök', 'Einar Ágúst', 'GÓSS', 'Sigrún Stella', 'Stefán S. Stefánsson', 'Stuðmenn', 'Guðmundur R', 'Ultra Mega Technobandið Stefán', 'Veðurguðirnir']
['Pláhnetan', 'Einar Bárðarson', 'Arnór Dan', 'Teitur Magnússon', 'Guðmundur Þórarinsson', 'Grossstadtgeflüster', 'Die Draufgänger', 'Der traurige Gärtner', 'Gloriettenstürmer', 

In [231]:
updateMulti()
updateDB()

========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
  Initializing a fresh DB for multi
  There are currently 0 artist keys.
  There are currently 0 renamed artist keys.
  Forced reload of 515 artist names
  Found 551 names
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
  Initializing a fresh DB for multi
  There are currently 0 artist keys.
  There are currently 0 renamed artist keys.
  Forced reload of 515 artist names
  Found 551 names
Saving 515 artist keys to /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
Saving data to /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
  --> This file is 20.8kB.
Saved data to /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
  --> This file is 20.8kB.
==========

True

In [ ]:
toMatchDF = DataFrame({item[0]: {"ArtistName": item[1]["ArtistName"], "ArtistAlbums": item[1]["ArtistAlbums"]} for item in toMatch[chartType]}).T
toMatchDF["NumAlbums"] = toMatchDF["ArtistAlbums"].apply(len)
toMatchDF = toMatchDF.sort_values(by="NumAlbums", ascending=False)

for idx,row in toMatchDF.iterrows():
    artistName = row["ArtistName"]
    albumNames = row["ArtistAlbums"]
    nAlbums    = row["NumAlbums"]
    if nAlbums >= 1:
        print(artistName,'\t',nAlbums)
    else:
        break
#https://kworb.net/itunes/extended.html



# Renames

In [ ]:
#printAllMissingArtists(toMatch, chartType)
chartType

In [256]:
toget, togetID, dbRenames = analyzeMatches(maindb, mdbmaps, chartType, maxVal=1.0, diffVal=1.0)
analyzeRenames(manDB, dbRenames)

[Juan Luis Guerra 440           0.93        Juan Luis Guerra y 4.40] 	 --> (Billboard) 9368a8f4c96684d45fbcf3e4afef275f / 15386 (RateYourMusic)
[Little Eva                     0.9                      Little Eve] 	 --> (Billboard) b61e7d6fa97e4f018d0dd57496e8f763 / 40588375576 (LastFM)
[George Michael-Elton John      0.92    George Michael & Elton John] 	 --> (Billboard) 029b4631146fa8f4482d5c8a14434dd2 / 58767118100 (LastFM)
[Clivillés & Cole               0.94               Clivillés + Cole] 	 --> (Billboard) 595c5a45a39a5c811908a01b78fbfce7 / 33341 (RateYourMusic)
[Izzy Stradlin And The Ju Ju Hounds 0.91 Izzy Stradlin & the Ju Ju Hounds] 	 --> (Billboard) b9d0361546e4d63ba32d358f8269fadc / 0001267850 (AllMusic)
[Izzy Stradlin And The Ju Ju Hounds 0.94 Izzy Stradlin and the Ju Ju Hounds] 	 --> (Billboard) b9d0361546e4d63ba32d358f8269fadc / 212250053545971339439873725862280725822 (MusicBrainz)
[Coverdale Page                 0.93                 Coverdale-Page] 	 --> (Billboard) 06203

In [280]:
updateMulti()

========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
  Initializing a fresh DB for multi
  There are currently 0 artist keys.
  There are currently 0 renamed artist keys.
  Forced reload of 517 artist names
  Found 555 names
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
  Initializing a fresh DB for multi
  There are currently 0 artist keys.
  There are currently 0 renamed artist keys.
  Forced reload of 517 artist names
  Found 555 names
Saving 517 artist keys to /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
Saving data to /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
  --> This file is 20.9kB.
Saved data to /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
  --> This file is 20.9kB.
==========

In [281]:
updateDB()

========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  Initializing a fresh DB for main
  There are currently 0 artist keys.
  There are currently 0 renamed artist keys.
  Forced reload of 13238 artist names
  Found 15828 names
Current Time is Wed Apr 07, 2021 22:20:26 for Last Ran
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  Initializing a fresh DB for main
  There are currently 0 artist keys.
  There are currently 0 renamed artist keys.
  Forced reload of 13238 artist names
  Found 15828 names
Saving 13238 artist keys to /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
Saving data to /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  --> This file is 371.5kB.
Saved data to /Users/tgadfort/opt/anaconda3/envs/py37/musicnam

True

In [264]:
updateManDB(3,True)

========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  There are currently 13222 artist keys.
  There are currently 15802 renamed artist keys.
Trying to add 26 renamed artist keys
There are currently 13222 artist keys.
There are currently 15802 renamed artist keys.
There are currently 13238 artist keys.
There are currently 15828 renamed artist keys.
Current Time is Wed Apr 07, 2021 21:39:35 for Last Ran


False

In [265]:
if updateRelDB():
    removeDBRenames(mdbmaps, chartType, dbRenames)
#updateManDB(2)
#updateMultiDB(2)
#mdbmaps["MusicVF"].addArtistDataByID("a4b9fb7275d480a00e15bf30c533c3cd", "Deezer", "69904")
#mdbmaps["MusicVF"].save()

========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  There are currently 13222 artist keys.
  There are currently 15802 renamed artist keys.
Trying to add 26 renamed artist keys
There are currently 13222 artist keys.
There are currently 15802 renamed artist keys.
There are currently 13238 artist keys.
There are currently 15828 renamed artist keys.
Current Time is Wed Apr 07, 2021 21:39:50 for Last Ran
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  There are currently 13222 artist keys.
  There are currently 15802 renamed artist keys.
Trying to add 26 renamed artist keys
There are currently 13222 artist keys.
There are currently 15802 renamed artist keys.
There are currently 13238 artist keys.
There are currently 15828 renamed artist keys.
Saving 13238 artist keys 

In [ ]:
updateMultiDB(2)

In [282]:
from multiArtist import multiartist
knownMulti = getFile("knownMultiArtists.yaml")
print("Add the following to knownMultiArtists.yaml")
print("="*43)
mularts  = multiartist(cutoff=0.9, discdata=None, exact=False)
adds = {}
for renamedArtist in getFile("main.yaml").values():
    try:
        N = mularts.getArtistNames(renamedArtist)
    except:
        print("Error with [{0}]".format(renamedArtist))
        continue
    if len(N) > 1:
        if renamedArtist not in knownMulti:
            adds[renamedArtist] = True
            
for renamedArtist in adds.keys():
    print("- {0}".format(renamedArtist))
print("="*43)

Add the following to knownMultiArtists.yaml
- André Rieu & Johann Strauss Orchestra
- Jimmy Page & Robert Plant
- Juan Gabriel & Banda El Recodo
- Judy Garland & Gene Kelly
- Boney James & Rick Braun
- George Michael & Elton John
- Johnny Mathis & Deniece Williams
- Ricky Kej & Wouter Kellerman


In [283]:
updateDB()
updateMulti()

========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  Initializing a fresh DB for main
  There are currently 0 artist keys.
  There are currently 0 renamed artist keys.
  Forced reload of 13234 artist names
  Found 15823 names
Current Time is Wed Apr 07, 2021 22:23:57 for Last Ran
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  Initializing a fresh DB for main
  There are currently 0 artist keys.
  There are currently 0 renamed artist keys.
  Forced reload of 13234 artist names
  Found 15823 names
Saving 13234 artist keys to /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
Saving data to /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  --> This file is 371.3kB.
Saved data to /Users/tgadfort/opt/anaconda3/envs/py37/musicnam

In [284]:
#removeDBRenames(mdbmaps, chartType, dbRenames)
manDB      = masterArtistNameDB("main", init=False)
multimanDB = masterArtistNameDB("multi", init=False)
maindb     = loadMainDB(True)

========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  There are currently 13234 artist keys.
  There are currently 15823 renamed artist keys.
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
  There are currently 521 artist keys.
  There are currently 560 renamed artist keys.
Current Time is Wed Apr 07, 2021 22:24:10 for Loading ArtistDBs Renaming Artists
Setting Basic Database Objects
=========================   discogs   =========================
=========================   allmusic   =========================
=========================   musicbrainz   =========================
=========================   lastfm   =========================
=========================   rateyourmusic   =========================
=========================   deezer   ======================

Creating Pandas DataFrame for 48605 Artists
	Shape --> (48605, 1)
	Shape --> (48605, 2)
  Finding Real Artist Name
	Shape --> (48605, 4)
  Removing None Artist
	Shape --> (48605, 4)
  Finding Disc Artist Name
	Shape --> (48605, 5)
DataFrame Shape is (48605, 7)
Current Time is Wed Apr 07, 2021 22:25:23 for Done with 
=================================== Creating Artist DB ===================================
Process [Done with 
=================================== Creating Artist DB ===================================] took 1 seconds.
Saving Master Artist DB File: /Users/tgadfort/Music/Discog/db-deezer/MasterSlimArtistDB.p
  --> This file is 6.6MB.
SlimArtistDB(Deezer) DataFrame Shape is (48605, 7)
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  There are currently 13234 artist keys.
  There are currently 15823 renamed artist keys.
Adding 1 known multi delim artists.
Current T

Summary Statistics For DB: KWorbiTunes
    Using Known Artists: False
    Found 11922 ID -> Name entries
    Found 11883 Name -> ID entries
    Found 0 Albums
Current Time is Wed Apr 07, 2021 22:26:14 for Done with Loading ArtistDBs Renaming Artists
Process [Done with Loading ArtistDBs Renaming Artists] took 2.1 minutes.


# Extra Artists

## Latin America

In [ ]:
def addLatinAmericaExtras(mdbmaps):
    addDeezer(mdbmaps, "LatinAmerica", "11851fcabe9d3c13a05675874669de73", "288166")   ### [Justin Bieber[a]]
    addAllMusic(mdbmaps, "LatinAmerica", "11851fcabe9d3c13a05675874669de73", "mn0002165952")   ### [Justin Bieber[a]]
    addDeezer(mdbmaps, "LatinAmerica", "b9d30acfe05c6d1e6be03785671ad78d", "8945638")   ### [Nío García]
    addAllMusic(mdbmaps, "LatinAmerica", "b9d30acfe05c6d1e6be03785671ad78d", "mn0003736054")   ### [Nío García]
    addDeezer(mdbmaps, "LatinAmerica", "a35e5dd04d8d05df287bf566147c2546", "8945")   ### [Raúl di Blasio]
    addAllMusic(mdbmaps, "LatinAmerica", "a35e5dd04d8d05df287bf566147c2546", "mn0000399559")   ### [Raúl di Blasio]
    addAllMusic(mdbmaps, "LatinAmerica", "1fd19f1829afdf8e5bc2f346ff09f0c0", "mn0000695328")   ### [Zezé di Camargo]

    mdbmaps["LatinAmerica"].save()
    
addLatinAmerica(mdbmaps)

## East Asia

In [ ]:
def addEastAsiaExtras(mdbmaps): 
    addDiscogs(mdbmaps, "EastAsia", "6463a43f6360a078b9a7b49b1d17c4a2", "112800")   ### [Exile]
    addDeezer(mdbmaps, "EastAsia", "feb3b72ee68ab3aac4ae1415da3066c1", "1823771")   ### [Greeeen]
    addDeezer(mdbmaps, "EastAsia", "a0479e9fb983e4db5fee1412a62f17ed", "1116794")   ### [Shin Seung-hun]
    addDeezer(mdbmaps, "EastAsia", "33810f2e107fc2e0aa0778371bc4ecac", "2562901")   ### [Busker Busker]
    addDeezer(mdbmaps, "EastAsia", "9560ca87e6638a37f7c05109cb8d6b07", "5414528")   ### [Jo Kwan-woo]
    addDeezer(mdbmaps, "EastAsia", "5378aab4a9c8f5849b659f4d1b26d5f5", "122718062")   ### [Seo Taiji and Boys]
    addDeezer(mdbmaps, "EastAsia", "dee730d9df12fb835277f59df0041533", "3582581")   ### [015B]
    addDeezer(mdbmaps, "EastAsia", "2b6c187e872fb9f94d4bde11b2397ff4", "6340658")   ### [Jo Sung-mo]
    addDeezer(mdbmaps, "EastAsia", "f044ea992b2a7b3fbe9589b0a9381c03", "14494599")   ### [Byun Jin-sub]
    addDeezer(mdbmaps, "EastAsia", "506e1de63c3c1863ccd927c02f76d0af", "9836070")   ### [Leessang]
    mdbmaps["EastAsia"].save()

addEastAsiaExtras(mdbmaps)

## MusicVF Extras

In [ ]:
def addMusicVFExtras(mdbmaps):    
    addDiscogs(mdbmaps, "MusicVF", "5973c632cd2c12611b2935a2ab748642", "280863")   ### [The Cascades]
    addDiscogs(mdbmaps, "MusicVF", "fcbd34ca9378360c610401125531b34e", "356575")   ### [Frank Skinner]
    addDiscogs(mdbmaps, "MusicVF", "5f4b9d57e3c7739ebcd7d2e662495eb9", "3265278")   ### [Déjà [soul duo]]
    addDiscogs(mdbmaps, "MusicVF", "34578b29b07a064a2488b2b9aebdc119", "445147")   ### [Kiara]
    addDeezer(mdbmaps, "MusicVF", "4a97b8cd5080a4a51b0660462085b8da", "56728")   ### [Solo [2°]]
    addDiscogs(mdbmaps, "MusicVF", "14056437503039b9427022275173259d", "3587388")   ### [Grady Martin and His Slew Foot Five]
    addDiscogs(mdbmaps, "MusicVF", "c2c5b209a710a0991f8bc3180ea9a68b", "881501")   ### [Little Lola Wiggins]

    addDiscogs(mdbmaps, "MusicVF", "17ffc53d0baa1b1a4aae81565c80b9d5", "5379389")   ### [Lennie Lewis and His Orchestra]
    addDeezer(mdbmaps, "MusicVF", "094af66ba7c5ae6c4bbe8e9d8a4b3161", "2279151")   ### [DJ Pied Piper & the Master of Ceremonies]
    addDeezer(mdbmaps, "MusicVF", "b320c6e988db1ce7c61ca9b1e13bc6c7", "6157")   ### [Fedde le Grand]
    addDeezer(mdbmaps, "MusicVF", "dc238f0a8b049fab371e39c6a3f8b11a", "304064")   ### [LMC vs U2]
    addDeezer(mdbmaps, "MusicVF", "c4605d3fd3f580ba93eae9983de09d82", "68540642")   ### [Stardust [electronic music trio]]
    addDeezer(mdbmaps, "MusicVF", "51ee90b8b70178242dfaf8d5c078cd78", "66635")   ### [The Bluebells]
    addDeezer(mdbmaps, "MusicVF", "a4b9fb7275d480a00e15bf30c533c3cd", "69904")   ### [Nicole [German singer]]
    addDeezer(mdbmaps, "MusicVF", "63b28376a3b62e50435d28bf9a5baeef", "93101")   ### [Edelweiss]
    addDeezer(mdbmaps, "MusicVF", "c2e7357ed0144ab31623a8ed366be77a", "167801")   ### [Karmina]
    addDeezer(mdbmaps, "MusicVF", "e76ed180cf5a7f5dfd1ccfa2ebc8374d", "192520")   ### [His Solid Senders]
    addDeezer(mdbmaps, "MusicVF", "ee5a98eeb6a19322027298fe56066a8c", "265467")   ### [Jump 'N the Saddle]
    addDeezer(mdbmaps, "MusicVF", "c3bba8cdb2cf4ae9365d0141ac23bb8f", "566111")   ### [Whatnauts]
    addDiscogs(mdbmaps, "MusicVF", "66fe5552fc95386fd601d784bc7d3c53", "2406884")   ### [the Doolittle Band]
    addDeezer(mdbmaps, "MusicVF", "2aa85c3687153b518cba59cf60ceca4f", "5702452")   ### [the Citizen's Band]
    addDeezer(mdbmaps, "MusicVF", "167d27edb31e4739511f735006e11563", "7899968")   ### [the GB Experience]
    addDeezer(mdbmaps, "MusicVF", "0461a66d048d94906b4d7fb375cd1429", "9056410")   ### [the Soul Partners]
    addDeezer(mdbmaps, "MusicVF", "06243e8d97e5946e72105936264e4694", "373755")   ### [the Blue Notes]
    addDiscogs(mdbmaps, "MusicVF", "8f873f75d62e0300beca8330e2340e0f", "384078")   ### [the Family Cookin']
    addDiscogs(mdbmaps, "MusicVF", "f402f5751df4d0ff3dc056f87c651c80", "1210897")   ### [His Blues Jumpers]
    addDeezer(mdbmaps, "MusicVF", "f675d3e6570cd2fd49c1b52d7abe873d", "122265")   ### [the Emblems]
    addDiscogs(mdbmaps, "MusicVF", "424dd1539f00cf3d73a4b07c67db904b", "1441198")   ### [The Peter Rabbit Trio]
    addDiscogs(mdbmaps, "MusicVF", "e7615f8e1a2330c7e768e0fdef346578", "1441199")   ### [Jimmie Nelson]
    addDiscogs(mdbmaps, "MusicVF", "838d1958aeb06a6c82336554dba7f0d7", "1792949")   ### [Sandy Stewart [singer]]
    addDiscogs(mdbmaps, "MusicVF", "46def99045ce5dc20a899b18445a4608", "3997961")   ### [Robert Maxwell and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "872c2f41f0738213bee984fda7cbab00", "287849")   ### [Sonny Boy Williamson [1°]]
    addDiscogs(mdbmaps, "MusicVF", "77cfb477c4beec19eb6cb910242a161d", "2031417")   ### [The Larks [North Carolina band]]
    addDiscogs(mdbmaps, "MusicVF", "c3320468eff8af69fc5aa123f70090c0", "3857723")   ### [Little Caesar [Harry Caesar]]
    addDiscogs(mdbmaps, "MusicVF", "109dffbb6d1c886cde6d75199a0c90ef", "689758")   ### [Marvin and Johnny]
    addDiscogs(mdbmaps, "MusicVF", "44fce31f19a26f10dc035b59f857e3d7", "274061")   ### [Earl King [Earl Johnson]]
    addDiscogs(mdbmaps, "MusicVF", "8a25d81a61e6556a4b59a881ba48a696", "901095")   ### [Earl King [Earl Connelly]]
    addDiscogs(mdbmaps, "MusicVF", "86e51aea8bc05364e9dcb848aab30a65", "383325")   ### [The Regents [American doo-wop group]]
    addDiscogs(mdbmaps, "MusicVF", "b099b9102dfad935eff4e6df6f5412a7", "1220747")   ### [Rochell and the Candles]
    addDeezer(mdbmaps, "MusicVF", "7f6c8f3b3e74a1dc07f11fa6bacbd748", "66014")   ### [Mongo Santamaría]
    addDiscogs(mdbmaps, "MusicVF", "f7587cb7da7c1062ba2807da903ce436", "419733")   ### [The Reflections [Detroit rock and roll band]]
    addDiscogs(mdbmaps, "MusicVF", "803e0a4da43dfb4db0404926c7d01701", "425364")   ### [The Shades of Blue]
    addDiscogs(mdbmaps, "MusicVF", "6dcd2659e26b9d40ab869e6163202773", "7844868")   ### [Bobby Moore [saxophonist]]
    addDiscogs(mdbmaps, "MusicVF", "499904c4d7e9132976d54aebac6573be", "282993")   ### [Ko Ko Taylor]
    addDiscogs(mdbmaps, "MusicVF", "5f0242a44aa86d07bfa00793135e4ae4", "60631")   ### [Dyke and the Blazers]
    addDiscogs(mdbmaps, "MusicVF", "6528839ca77756f8422bd688a64f0b3c", "29852")   ### [B.M.U. (Black Men United)]
    addDiscogs(mdbmaps, "MusicVF", "5a4c9a2699f80579ba01f8c93ac93d98", "125017")   ### [B-Rock & the Bizz]
    addDiscogs(mdbmaps, "MusicVF", "e4c2176a80ad1d252fccb3eba74bedcb", "2778849")   ### [iAMSU!]
    addDeezer(mdbmaps, "MusicVF", "70a68e769fe0f78f69c255af4b116594", "4665980")   ### [TK n Cash]
    addDiscogs(mdbmaps, "MusicVF", "b3a5b1023d0fa4b649efc7762d3097f4", "428493")   ### [Arthur Guitar Boogie Smith]    

    if False:
        addDiscogs(mdbmaps, "MusicVF", "17ffc53d0baa1b1a4aae81565c80b9d5", "")   ### [Lennie Lewis and His Orchestra]
        addDiscogs(mdbmaps, "MusicVF", "f402f5751df4d0ff3dc056f87c651c80", "")   ### [His Blues Jumpers]
        addDiscogs(mdbmaps, "MusicVF", "f35e132f30a7648b992b826748c5e936", "")   ### [Willows]
        addDiscogs(mdbmaps, "MusicVF", "a17aee903392db6b3a995f5b55e833d7", "")   ### [The Falcons]
        addDiscogs(mdbmaps, "MusicVF", "b3a5b1023d0fa4b649efc7762d3097f4", "")   ### [Arthur Guitar Boogie Smith]
        addDiscogs(mdbmaps, "MusicVF", "95410013638a2fd9009ed15beacb2057", "")   ### [Burl Ives and Grady Martin and His Slew Foot Five]
        addDiscogs(mdbmaps, "MusicVF", "772ce8ec88a626147f3dad18edbf795a", "")   ### [Gene Woods]
        addDiscogs(mdbmaps, "MusicVF", "e76ed180cf5a7f5dfd1ccfa2ebc8374d", "")   ### [His Solid Senders]
        addDiscogs(mdbmaps, "MusicVF", "872c2f41f0738213bee984fda7cbab00", "")   ### [Sonny Boy Williamson [1°]]
        addDiscogs(mdbmaps, "MusicVF", "ad2409a681fc6b5f315390c3e5d144a1", "")   ### [Eunice]
        addDiscogs(mdbmaps, "MusicVF", "2c7dab3f67344e99f98ad9d1dd63dc84", "")   ### [Johnnie]
        addDiscogs(mdbmaps, "MusicVF", "6d2b188443bfecfa0bdf8189d3b56e2b", "")   ### [Marilyn]
        addDiscogs(mdbmaps, "MusicVF", "e3b261c8784e1a07aa28c2050279cda9", "")   ### [The Heartbeats]
        addDiscogs(mdbmaps, "MusicVF", "8a25d81a61e6556a4b59a881ba48a696", "")   ### [Earl King [Earl Connelly]]
        addDiscogs(mdbmaps, "MusicVF", "44fce31f19a26f10dc035b59f857e3d7", "")   ### [Earl King [Earl Johnson]]
        addDiscogs(mdbmaps, "MusicVF", "38a01d2c5d6f9d62a3d326a19f720c3e", "")   ### [Jack Dupree and Mr. Bear]
        addDiscogs(mdbmaps, "MusicVF", "6f3c65a5375f4d1f870d819e0ddc51a9", "")   ### [The Four Jacks]
        addDiscogs(mdbmaps, "MusicVF", "0184d4e1718e6ce082b96d6820eac9bc", "")   ### [Original Five Blind Boys]
        addDiscogs(mdbmaps, "MusicVF", "77cfb477c4beec19eb6cb910242a161d", "")   ### [The Larks [North Carolina band]]
        addDiscogs(mdbmaps, "MusicVF", "26f42521224025ad024b098cfa48fd08", "")   ### [Little Willie Littlefield and Little Lola Wiggins]
        addDiscogs(mdbmaps, "MusicVF", "c3320468eff8af69fc5aa123f70090c0", "")   ### [Little Caesar [Harry Caesar]]
        addDiscogs(mdbmaps, "MusicVF", "4cf00a46ac591b4bd05d379c7ada71f7", "")   ### [Little Esther and Mel Walker]
        addDiscogs(mdbmaps, "MusicVF", "109dffbb6d1c886cde6d75199a0c90ef", "")   ### [Marvin and Johnny]
        addDiscogs(mdbmaps, "MusicVF", "cb249cefeb1507f39d9ebb3c6f56b9b6", "")   ### [Jimmy Wilson and His All-Stars]
        addDiscogs(mdbmaps, "MusicVF", "015755c973d95256988b47f2456410ea", "")   ### [The Spiders]
        addDiscogs(mdbmaps, "MusicVF", "d1c1dbfda9db2dbf8f47159810c053f9", "")   ### [Sonny Boy Williamson [2°]]
    
    
    addDiscogs(mdbmaps, "MusicVF", "6b623b28dbfc1558c22c20ae1fce91b0", "911933")   ### [Eddie Vinson]
    addDiscogs(mdbmaps, "MusicVF", "9ddc6e10e85b2d5b637438cc045f05a9", "1562432")   ### [Paul Gayten and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "2ee7e0138ad434c15ba4b67e090fa3eb", "2209834")   ### [the Sparkletones]
    addDiscogs(mdbmaps, "MusicVF", "d8e04d4872a8679829c4b134105a2162", "1504473")   ### [Captain Stubby and the Buccaneers]
    addDiscogs(mdbmaps, "MusicVF", "fb7bb7ef5d36763bd30f6ead78cbdf5c", "1377553")   ### [Ernest Tubb and the Texas Troubadors]
    addDiscogs(mdbmaps, "MusicVF", "9d4cdf09aaa470c4e8315d0d65679d33", "1400007")   ### [the Wilburn Brothers]
    addDiscogs(mdbmaps, "MusicVF", "ff2665100d749d771237f87780ad6b68", "5880996")   ### [Willie Dixon and the Allstars]
    addDiscogs(mdbmaps, "MusicVF", "401656dc414aa3ef8ec244b58bbd46c0", "2024647")   ### [Shirley Gunter and the Queens]
    addDiscogs(mdbmaps, "MusicVF", "300e8fb4f7af15e15da7112dac8e9a3c", "5893418")   ### [Wini Brown and Her Boyfriends]
    addDiscogs(mdbmaps, "MusicVF", "504f895bbb59b80b82d09b2a5f03fef0", "881738")   ### [Willie Mabon and His Combo]
    addDiscogs(mdbmaps, "MusicVF", "1cae9107084baa0927a5f3fc44e48ad9", "446642")   ### [Earl Bostic and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "fb57e6254c02bf7ccae8686ce903bcdc", "2765919")   ### [John Greer and the Rhythm Rockers]
    addDiscogs(mdbmaps, "MusicVF", "467a1e2bde3283f3157daed751dba418", "368181")   ### [Illinois Jacquet and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "75acbe4304cf5fbd5c48b4a09e9911da", "6647129")   ### [Danny Overbea and His Combo]
    addDiscogs(mdbmaps, "MusicVF", "1021117cdfb815d5078d81534edc514c", "502590")   ### [Swinging Sax Kari and Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "f46044e69b5c3c5a825060973e0c4e5e", "869117")   ### [Louis Brooks and His Hi-Toppers]

    addDiscogs(mdbmaps, "MusicVF", "77cf2d0e3c03a69200813cf38fcb07d2", "843761")   ### [Roy Milton and His Solid Senders]
    addDiscogs(mdbmaps, "MusicVF", "0128431770823378f248b4fae5186996", "4078040")   ### [Boyd Heath]
    addDiscogs(mdbmaps, "MusicVF", "acbcdf89f8eedfa0b832bec9adc93c40", "3162587")   ### [Calvin Boze and His All-Stars]
    addDiscogs(mdbmaps, "MusicVF", "5c4b6d840de1b278ed57a4f41be572f0", "331401")   ### [The Robins]
    addDiscogs(mdbmaps, "MusicVF", "41e6bdeb2106f290a35afacc54b8480c", "3709446")   ### [Johnny Sparrow and His Bows and Arrows]
    addDiscogs(mdbmaps, "MusicVF", "ddb049d759639ca09404616d131e0c0e", "1367456")   ### [Foy Willing and His Riders of the Purple Sage]
    addDiscogs(mdbmaps, "MusicVF", "303e5796c123f31b9da14452340e7de3", "1366313")   ### [Bill Boyd and His Cowboy Ramblers]
    addDiscogs(mdbmaps, "MusicVF", "a70d47e7fef67d9d26620a5ce84c00f8", "2099671")   ### [Wiley and Gene]
    addDiscogs(mdbmaps, "MusicVF", "fe1d9b16ffef3ed107c93f7f09c91571", "373807")   ### [Luis Russell and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "671397454cf28fcb5db4c6ce7f2a8e4a", "1353345")   ### [The Carlisles Brothers]
    addDiscogs(mdbmaps, "MusicVF", "7210b848a0b5339c603cc24d4ed726b9", "1444077")   ### [Jerry Irby and His Texas Ranchers]
    addDiscogs(mdbmaps, "MusicVF", "4b3beae9c37bdbb95e9a7b5c13ce04f9", "170356")   ### [The Shadows [vocal group]]
    addDiscogs(mdbmaps, "MusicVF", "2a3968a2454cdd2d2ea507689079262f", "1210897")   ### [Roy Byrd and His Blues Jumpers]
    addDiscogs(mdbmaps, "MusicVF", "3d571f2d665708173315e3dbbdbd5acd", "6381534")   ### [Ed Wiley and His After Hour Rhythm]
    addDiscogs(mdbmaps, "MusicVF", "0aa06fee44e4fe7fc7171c208a599711", "2152387")   ### [Chubby Hip Shakin' Newsom]
    addDiscogs(mdbmaps, "MusicVF", "7e83b9ca3e2e849d050e0720236600a3", "3239072")   ### [The Cats 'N Jammer Three]
    addDiscogs(mdbmaps, "MusicVF", "049675dc13a3e048490bb90849bbf185", "320593")   ### [Eddie Vinson and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "fc4ef2798ad5aacb911c50bd1ab531af", "2082434")   ### [Gene Ammons and His Sextet]
    addDiscogs(mdbmaps, "MusicVF", "a47e9e688632c969ccfa3705c9b7d529", "4679197")   ### [Floyd Hunt Quartette]
    addDiscogs(mdbmaps, "MusicVF", "b7e23303db0ae92a2344741af20f552f", "4141135")   ### [Do Ray and Me]
    addDiscogs(mdbmaps, "MusicVF", "2491d4d03f5ede698a11ec05e2238bd1", "597586")   ### [Paul Williams [saxophonist]]
    addDiscogs(mdbmaps, "MusicVF", "e85602adc1500a774c8a2c66a490f780", "1759768")   ### [Stick McGhee and His Buddies]
    addDiscogs(mdbmaps, "MusicVF", "d441c773639869a7c8d587cbf0a5d21b", "3774491")   ### [Jimmy Preston and His Prestonians]
    addDiscogs(mdbmaps, "MusicVF", "915d78a220bc1924b5e1a728dbd153eb", "648347")   ### [Freddie Mitchell and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "13c00f874bf139b0e84362767b12c5c7", "603388")   ### [Joe Thomas [American saxophonist]]
    addDiscogs(mdbmaps, "MusicVF", "db8f4ae34dbf2f6a22863156649095c6", "4383478")   ### [Dave Landers]    
    
    addDiscogs(mdbmaps, "MusicVF", "72e367a9278e1e84c8c5901de4f22da5", "235314")   ### [Disco Tex And The Sex-O-Lettes]
    addDiscogs(mdbmaps, "MusicVF", "60577606a1ec511148859d450ae7d1c8", "892896")   ### [Senator Everett McKinley Dirksen]
    addAllMusic(mdbmaps, "MusicVF", "60577606a1ec511148859d450ae7d1c8", "mn0001199524")   ### [Senator Everett McKinley Dirksen]
    addDiscogs(mdbmaps, "MusicVF", "31763b616bbe089982b49001415bc111", "3169685")   ### [Schuyler, Knobloch, & Overstreet]
    addDiscogs(mdbmaps, "MusicVF", "04e36152e78cf5c20c3414638107a095", "39727")   ### [B.B. & Q. (Brooklyn, Bronx & Queens)]
    addAllMusic(mdbmaps, "MusicVF", "04e36152e78cf5c20c3414638107a095", "mn0000764166")   ### [B.B. & Q. (Brooklyn, Bronx & Queens)]
    addDiscogs(mdbmaps, "MusicVF", "8024ef0e8f888ec62354d22d8c3bdbd9", "29956")   ### [Groove Holmes]
    addAllMusic(mdbmaps, "MusicVF", "8024ef0e8f888ec62354d22d8c3bdbd9", "mn0000848952")   ### [Groove Holmes]
    addDeezer(mdbmaps, "MusicVF", "c6e228da38d1cd7730275876e347fe91", "177614")   ### [Jackie Lee [American singer]]
    addDeezer(mdbmaps, "MusicVF", "7b9bdbf9faa241c9dde7e60ecbc9970d", "188144")   ### [The Baja Marimba Band]
    addDiscogs(mdbmaps, "MusicVF", "7aa8ff3b8d2bb8e7881500c19c174aee", "382426")   ### [Carl Butler and Pearl]
    addAllMusic(mdbmaps, "MusicVF", "7aa8ff3b8d2bb8e7881500c19c174aee", "mn0000795407")   ### [Carl Butler and Pearl]
    
    addDeezer(mdbmaps, "MusicVF", "d96ab4c9bae2db0fcda7fad5ecb9c707", "1343481")   ### [The Flaming Ember]
    addAllMusic(mdbmaps, "MusicVF", "d96ab4c9bae2db0fcda7fad5ecb9c707", "mn0000797353")   ### [The Flaming Ember]
    addDeezer(mdbmaps, "MusicVF", "cf013207f0f852cc61fea44160f10ce0", "78937")   ### [The Gatlin Brothers]
    addAllMusic(mdbmaps, "MusicVF", "cf013207f0f852cc61fea44160f10ce0", "mn0000073906")   ### [The Gatlin Brothers]
    addAllMusic(mdbmaps, "MusicVF", "ec872a148b01a90d965041d7cebcb16d", "mn0000075801")   ### [The Highwaymen [country supergroup]]
    addDeezer(mdbmaps, "MusicVF", "a5ea547b06d6ed9edfedbb502ad940b5", "135410")   ### [S.O.U.L. S.Y.S.T.E.M.]
    addAllMusic(mdbmaps, "MusicVF", "a5ea547b06d6ed9edfedbb502ad940b5", "mn0000418079")   ### [S.O.U.L. S.Y.S.T.E.M.]
    addDeezer(mdbmaps, "MusicVF", "70e591b0d0dc135319a9674bf69a6b4c", "7531")   ### [The Outlaws [US band]]
    addDeezer(mdbmaps, "MusicVF", "d51f2dd6f21efce5cf4c3d26fd869491", "54541862")   ### [Diesel [1°]]
    addAllMusic(mdbmaps, "MusicVF", "d51f2dd6f21efce5cf4c3d26fd869491", "mn0001346219")   ### [Diesel [1°]]
    addDeezer(mdbmaps, "MusicVF", "2e3484ca1d6a8daff3d7c258920feb72", "7059")   ### [The Firm [English rock supergroup]]
    addDeezer(mdbmaps, "MusicVF", "c1db298eab1887692e00e50f7466c84c", "15490")   ### [Camouflage [German dance group]]
    addDeezer(mdbmaps, "MusicVF", "c39e8321ed7989d62d1ca99f9dc1e609", "1414458")   ### [Disco-Tex and the Sex-O-Lettes]
    addAllMusic(mdbmaps, "MusicVF", "c39e8321ed7989d62d1ca99f9dc1e609", "mn0001202301")   ### [Disco-Tex and the Sex-O-Lettes]
    addDeezer(mdbmaps, "MusicVF", "ee73a1b7003868b1835f69385a41627c", "80454")   ### [Dana [Dana Rosemary Scallon]]
    addAllMusic(mdbmaps, "MusicVF", "ee73a1b7003868b1835f69385a41627c", "mn0000671478")   ### [Dana [Dana Rosemary Scallon]]
    addAllMusic(mdbmaps, "MusicVF", "c7306bb4c95f15d67b041fdb180814d7", "mn0001190753")   ### [Trio [American country supergroup]]
    addDeezer(mdbmaps, "MusicVF", "e7214f06fe78c6ccc2dfc6a211ae27ce", "1447749")   ### [LaCosta Tucker]
    addAllMusic(mdbmaps, "MusicVF", "e7214f06fe78c6ccc2dfc6a211ae27ce", "mn0001634091")   ### [LaCosta Tucker]
    addDeezer(mdbmaps, "MusicVF", "521c91ff56ddba0264d4251403dc83d0", "981")   ### [Michael Zager Band]
    addAllMusic(mdbmaps, "MusicVF", "521c91ff56ddba0264d4251403dc83d0", "mn0000468962")   ### [Michael Zager Band]
    addDeezer(mdbmaps, "MusicVF", "f2ac4b5467eacb1d1a869675b777dc22", "16538")   ### [Karen Young [US singer]]
    addAllMusic(mdbmaps, "MusicVF", "f2ac4b5467eacb1d1a869675b777dc22", "mn0000300963")   ### [Karen Young [US singer]]
    addDeezer(mdbmaps, "MusicVF", "d27f9f5125beb918b683c52573bc6d22", "10394")   ### [Switch [funk band]]
    addAllMusic(mdbmaps, "MusicVF", "d27f9f5125beb918b683c52573bc6d22", "mn0000048757")   ### [Switch [funk band]]
    addAllMusic(mdbmaps, "MusicVF", "601a98dfca6212de8e032fd8f6e5eb5f", "mn0001388438")   ### [Fantasy [US soul band]]
    addDeezer(mdbmaps, "MusicVF", "ade864bca82d1536c58404052bbf8f0e", "449389")   ### [Chéri]
    addAllMusic(mdbmaps, "MusicVF", "ade864bca82d1536c58404052bbf8f0e", "mn0000107814")   ### [Chéri]
    addAllMusic(mdbmaps, "MusicVF", "dc1e7da97c6a3ea8923c6956a8f722b6", "mn0001285388")   ### [Howard Johnson [singer]]
    addAllMusic(mdbmaps, "MusicVF", "bb65324c757ee8ece9b14022cce3a409", "mn0000454773")   ### [Regina [Regina Richards
    addDeezer(mdbmaps, "MusicVF", "fbe6a9681b143afef10f8ab8dae46a9e", "1591592")   ### [Bill Anderson and Jan Howard]
    addDeezer(mdbmaps, "MusicVF", "4a760cca3c179622f447493731d91cf8", "5453369")   ### [Lisa Hartman Black]
    addAllMusic(mdbmaps, "MusicVF", "4a760cca3c179622f447493731d91cf8", "mn0000835160")   ### [Lisa Hartman Black]
    
    addDeezer(mdbmaps, "MusicVF", "4e39aa0574e41e66299ffbd6344fbe22", "78133")   ### [Sweethearts of the Rodeo]
    addAllMusic(mdbmaps, "MusicVF", "4e39aa0574e41e66299ffbd6344fbe22", "mn0000048168")   ### [Sweethearts of the Rodeo]
    addDeezer(mdbmaps, "MusicVF", "5eca107611a447bfed38d5faac961f45", "4584473")   ### [Baillie & the Boys]
    addAllMusic(mdbmaps, "MusicVF", "5eca107611a447bfed38d5faac961f45", "mn0000077815")   ### [Baillie & the Boys]
    addDeezer(mdbmaps, "MusicVF", "f237a8b44d779a52755eff8cfc587b3a", "884151")   ### [Terry McBride & the Ride]
    addAllMusic(mdbmaps, "MusicVF", "f237a8b44d779a52755eff8cfc587b3a", "mn0000340446")   ### [Terry McBride & the Ride]
    addDeezer(mdbmaps, "MusicVF", "420090923fe77ba30d505abc9208fcaf", "1324")   ### [Rainbow [British rock band]]
    addAllMusic(mdbmaps, "MusicVF", "420090923fe77ba30d505abc9208fcaf", "mn0000391933")   ### [Rainbow [British rock band]]
    addDeezer(mdbmaps, "MusicVF", "5b616b5b25c58a92b322b4049d1150e0", "183230")   ### [The Boys [US R&B quartet]]
    addAllMusic(mdbmaps, "MusicVF", "5b616b5b25c58a92b322b4049d1150e0", "mn0002444397")   ### [The Boys [US R&B quartet]]
    addDeezer(mdbmaps, "MusicVF", "0031de31b36ae76c87b2f9fb2532a8cc", "76152")   ### [Sylvia [Sylvia Kirby aka Sylvia Hutton]]
    addDiscogs(mdbmaps, "MusicVF", "0031de31b36ae76c87b2f9fb2532a8cc", "648667")   ### [Sylvia [Sylvia Kirby aka Sylvia Hutton]]    
    addAllMusic(mdbmaps, "MusicVF", "0bfea6608222050d4baff34cfff034a3", "mn0001752569")   ### [Page and Plant]
    addDeezer(mdbmaps, "MusicVF", "93399776eefe74a02bf94e1d87816c79", "78111")   ### [The Chimes [British dance music act]]
    addDeezer(mdbmaps, "MusicVF", "585f2a42ade6493d2d1687076f94cca6", "2828")   ### [The Geto Boys]
    addDeezer(mdbmaps, "MusicVF", "cba2178fd54b0213a3f8a9366ea44f55", "6810")   ### [The Mission [English band]]
    addDeezer(mdbmaps, "MusicVF", "e6150ea414ffd1f20826c91debcbe291", "5199")   ### [The Flys]
    addDeezer(mdbmaps, "MusicVF", "3670acca6154d80a0478b9a48e1f45b3", "9286974")   ### [Mono [British pop duo]]
    addDeezer(mdbmaps, "MusicVF", "26f21eb99326a926c7d2ef91c20ae577", "5251658")   ### [J Roddy Walston & The Business]
    addAllMusic(mdbmaps, "MusicVF", "f7fc6c3c60f4bd1db1805b930fd07e1e", "mn0000417717")   ### [The Raybon Brothers]
    addDeezer(mdbmaps, "MusicVF", "7876aed952a9bd4c34484ed3fab4a017", "479116")   ### [Skip]
    addDeezer(mdbmaps, "MusicVF", "2b60fc9e40cb42185b2fb1212f046f0f", "395317")   ### [Zac Brown]
    addDeezer(mdbmaps, "MusicVF", "56733603e5633b144b05458b809926e4", "143005")   ### [Ray Kennedy]
    addDeezer(mdbmaps, "MusicVF", "deddba800e308ec4b5274f5381b1d69a", "9208")   ### [Ideal [R&B quartet]]
    addDeezer(mdbmaps, "MusicVF", "c5b392a2242fab6b28ebdca261ad4536", "252311")   ### [Rampage]

    addDeezer(mdbmaps, "MusicVF", "866830b0d16b7e8803ce68faa87197ee", "384310")   ### [Troy Olsen]
    addDeezer(mdbmaps, "MusicVF", "eb22252aca5ac0377b931e1ef0ea5bb0", "76067")   ### [Angels and Airwaves]
    addAllMusic(mdbmaps, "MusicVF", "633f2eb095606c09c1ed3816e5ee13ef", "mn0000816418")   ### [The Purple Ribbon All-Stars]
    addDeezer(mdbmaps, "MusicVF", "27dad384cfd99402e3da71d74a975f62", "6762519")   ### [Børns]
    addAllMusic(mdbmaps, "MusicVF", "918b54ebd7eca1a1aa0dd2f22687b3da", "mn0002061816")   ### [Jade [Canadian singer]]
    addDeezer(mdbmaps, "MusicVF", "345274824dd3ea2b369667fb9f458f2e", "353")   ### [TRUSTcompany]
    addDeezer(mdbmaps, "MusicVF", "1244558bdd61482c0d02386af8a0532e", "281")   ### [Pendulum [drum and bass band]]
    addDeezer(mdbmaps, "MusicVF", "f816a192a41210e2dacde4e1cd0c66b8", "145298")   ### [Device [metal band]]
    addDeezer(mdbmaps, "MusicVF", "7ce5e1378aa18016b435d27f59661e32", "1060989")   ### [Deborah Gibson]
    addDeezer(mdbmaps, "MusicVF", "0da9c6260e3266ea5db169966594378f", "78470")   ### [Randy Jackson]
    addDeezer(mdbmaps, "MusicVF", "44ca3342c1d0396c83e56a316327c328", "289749")   ### [Sasha [Sasha Alexander]]
    addDeezer(mdbmaps, "MusicVF", "2db710ebeef331b5cd0945caa8e07fd2", "1204744")   ### [Bradley Gaskin]
    addAllMusic(mdbmaps, "MusicVF", "d859194c918107250ba6240fa70b561e", "mn0002065192")   ### [James Wesley Prosser]
    addDeezer(mdbmaps, "MusicVF", "73cd965c3c97b3127f47ef90915d6606", "178009")   ### [Slim [Marvin Scandrick]]
    addDeezer(mdbmaps, "MusicVF", "61a67ef8c7a5b81ed721f78c8ca62e25", "345092")   ### [Fast Life Yungstaz (F.L.Y.)]
    addDeezer(mdbmaps, "MusicVF", "ccbe2043b22bc7dd3c8d723d122dfee5", "456953")   ### [Heartland [2°]]
    addDeezer(mdbmaps, "MusicVF", "9a41660ec016647dbe40480ac2aeed08", "535924")   ### [The JaneDear Girls]
    addDeezer(mdbmaps, "MusicVF", "43474f5f561c795b358a07edfc2b4739", "6043158")   ### [Goldlink]
    addDeezer(mdbmaps, "MusicVF", "3083b053076722885e09648e30048f40", "1461356")   ### [The Farm Inc.]
    addDeezer(mdbmaps, "MusicVF", "077d47811ab93e443989db077e2caef0", "13915")   ### [Baby D [dance group]]
    addDeezer(mdbmaps, "MusicVF", "8de695a5c77eb65f9f49e1329a2f68b5", "8825")   ### [Rui da Silva]
    addDeezer(mdbmaps, "MusicVF", "63a10f4666a92c1070800215eefc5a55", "5147523")   ### [Mike Will Made-It]    
    
    addDeezer(mdbmaps, "MusicVF", "7c5cd618a022d2cf9b28049b6eb6ea44", "4248")   ### [The Blues Magoos]
    addDeezer(mdbmaps, "MusicVF", "1f56a1b675ef76d0c767875dd1297b3d", "1511")   ### [Nicole Kidman]
    
    addDeezer(mdbmaps, "MusicVF", "aed80849f9d8fd1acbc76fa7cd7f6b34", "103103")   ### [Mabel [Mabel McVey]]
    addDeezer(mdbmaps, "MusicVF", "26b76ac354e7683d10b82243208cf0f4", "11256100")   ### [Dave [rapper]]
    addDeezer(mdbmaps, "MusicVF", "6e90609be6f5d8b76726e5b4e7280087", "1194083")   ### [Tyler the Creator]
    addDeezer(mdbmaps, "MusicVF", "1b6e632809b254d580d1e428c74a142c", "50903332")   ### [VickeeLo]
    addDeezer(mdbmaps, "MusicVF", "355210e4b53e40e826d0ad16f42f3c9f", "51204222")   ### [Kid Laroi]    
    addDeezer(mdbmaps, "MusicVF", "9388aad491c98c85f6f695be4f0f65dc", "5171069")   ### [The Vamps [2°]]
    addDeezer(mdbmaps, "MusicVF", "3b202907e68bf07780092ca6f22607da", "398073")   ### [Psy [Korean singer]]
    addDeezer(mdbmaps, "MusicVF", "e1ffa7b9b42c48b4c4aee6c3f92c8af4", "802470")   ### [Hunter Hayes [US country music singer]]
    addDeezer(mdbmaps, "MusicVF", "870518aaf3165832108c217f2e98857b", "10189506")   ### [NHS Choir]
    addDeezer(mdbmaps, "MusicVF", "fff8fb4baecfbf6af7af65d7a45060ad", "334170")   ### [Sam Bailey]
    addDeezer(mdbmaps, "MusicVF", "d2519ea2e5f6c8c2572971f8b275fde5", "165930")   ### [Future [rapper]]
    addDeezer(mdbmaps, "MusicVF", "07511b3a5dc0de3b6abd1fe3b967fe88", "6989379")   ### [Gareth Malone's All-Star Choir]
    addDeezer(mdbmaps, "MusicVF", "b15ed83b286dcf8f0ca4124d60275589", "76030882")   ### [Grace [Australian singer]]    
    
    addDeezer(mdbmaps, "MusicVF", "6711279de31a902e87fb83b53c50b2bb", "785")   ### [The Spice Girls]
    addDeezer(mdbmaps, "MusicVF", "8af29c4df992d65be6801d8dad0568c0", "489")   ### [5ive]
    addDeezer(mdbmaps, "MusicVF", "f1605645e0f8ad7efbc7409b57bc38cd", "144")   ### [Blue [2000s band]]
    addDeezer(mdbmaps, "MusicVF", "ee4c6ac37f65da62e9a59af1fc02430c", "72519")   ### [Duffy [Aimee Anne Duffy]]
    addDeezer(mdbmaps, "MusicVF", "296b6e403facfd7e66defa0ab952dc34", "183716")   ### [Florence and the Machine]
    addDeezer(mdbmaps, "MusicVF", "7ddc26e21490e0d42f1548dae38c646c", "293378")   ### [The Wanted [boy band]]
    addDeezer(mdbmaps, "MusicVF", "5dbd6fc92953f38ba81cc7ec6129f9f1", "70667")   ### [Agnes [Agnes Carlsson]]
    addDeezer(mdbmaps, "MusicVF", "7fa1e50e2b59fd1673060d750a3dc02c", "397487")   ### [Divine [R&B girl group]]
    addDeezer(mdbmaps, "MusicVF", "3ac428bd029b54309ea44679259de33c", "12117")   ### [Cherish]
    addDeezer(mdbmaps, "MusicVF", "d1c43ff6993a5054d7e673b8b8142b0f", "248263")   ### [LSG (Levert, Sweat & Gill)]
    addDeezer(mdbmaps, "MusicVF", "224fe79382efa84c7a1b7a39ceae9102", "258829")   ### [Dizzee Rascal and Armand Van Helden]
    addDeezer(mdbmaps, "MusicVF", "b50fca73487050c0cb2d82e3df935ddd", "5324557")   ### [Shout for England]
    addDeezer(mdbmaps, "MusicVF", "0b651d315ae7a964520eb71e27b000a0", "542824")   ### [D Cup]    
    addDeezer(mdbmaps, "MusicVF", "e321e9cdc9edfb09f50c56e4926056ed", "167384")   ### [911 [british boy band]]
    addDeezer(mdbmaps, "MusicVF", "dd6e108356051470001cf887e2896c20", "5095")   ### [a1]
    addDeezer(mdbmaps, "MusicVF", "1553808126e025ed9aabb69c408fdde6", "417645")   ### [D-12]
    addDeezer(mdbmaps, "MusicVF", "8a8a6ba0ad1e4146c9e3a60679ae66c8", "597")   ### [Jet [Australian band]]
    addDeezer(mdbmaps, "MusicVF", "4578b25315f7437501e5f0aa79a96a40", "15207279")   ### [Lyte Funky Ones (LFO)]
    addDeezer(mdbmaps, "MusicVF", "31e908c9350d92890d910da5a810eb6f", "4430633")   ### [JXL]
    addDeezer(mdbmaps, "MusicVF", "2ad2141aad4a62d927fdf7d8ddfcb621", "7300")   ### [Len]
    addDeezer(mdbmaps, "MusicVF", "38e3830417a0d646566b2631a218f473", "16724")   ### [Voices of Theory]
    addDeezer(mdbmaps, "MusicVF", "ac3bff96e0d2babe6035b80ebc873e55", "2371")   ### [Somethin' for the People]
    addDeezer(mdbmaps, "MusicVF", "3b9d59bfbbcbbb007f9fec196331a766", "7766")   ### [Space [english pop band]]    
    
    addDiscogs(mdbmaps, "MusicVF", "67fce06c2a1437c18d6f84117db291eb", "419131")   ### [Robson Green & Jerome Flynn]
    addDeezer(mdbmaps, "MusicVF", "a25688aed5817d6f072d9153bea4756d", "1685662")   ### [Marky Mark & the Funky Bunch]    
    
    addDeezer(mdbmaps, "MusicVF", "642b5ad5ba31c2c034188b2ae18b1bcc", "5733")   ### [Poison [hard rock band]]
    addDeezer(mdbmaps, "MusicVF", "25fc449f0de6e4dbead30d2f006036f0", "7448")   ### [Vanessa Williams [Vanessa Lynn Williams]]
    addDeezer(mdbmaps, "MusicVF", "97a3e9fd21ff48f082ce2a5779a489b1", "415")   ### [Nirvana [American band]]
    addDeezer(mdbmaps, "MusicVF", "da6a972bc80cc500f459cb2a577f006a", "3526")   ### [The Jets [pop band]]
    addDeezer(mdbmaps, "MusicVF", "2285045d649fd03859d0a0006a97cbd0", "57267")   ### [Jade [American band]]
    addDeezer(mdbmaps, "MusicVF", "89c08013c817a7d57d629c92612b49e1", "2402")   ### [Enigma [german electronic music project]]
    addAllMusic(mdbmaps, "MusicVF", "407c6f5950271ed58c15fdb29c2c651e", "mn0000771838")   ### [IMx]
    addDeezer(mdbmaps, "MusicVF", "08fde96818c1dde60ab642e32b4276c2", "2667")   ### [Mr. Big [US band]]
    addDeezer(mdbmaps, "MusicVF", "9c830dfc527627333c6ebca6cfe31c76", "1808031")   ### [4 P.M.]
    addDeezer(mdbmaps, "MusicVF", "880abe0df6030ac2d8c20328de6ccb2f", "7549")   ### [Domino [American rapper]]
    addDeezer(mdbmaps, "MusicVF", "8e1a97fbdbbabc33fa3bbb6efd0c383c", "13604")   ### [Altern-8]
    addDeezer(mdbmaps, "MusicVF", "4906a255a6c377ca0702d42ecd4649b4", "111817")   ### [Candyman]
    addDeezer(mdbmaps, "MusicVF", "6cd63ada8bd0c670a3e0ad1232a6e5c8", "7365")   ### [Mel and Kim]
    addDeezer(mdbmaps, "MusicVF", "7cd09b4f40631db866403fa556bdaf2b", "367290")   ### [Partners in Kryme]
    addDeezer(mdbmaps, "MusicVF", "0930095900cc7de51f1a7d0b3a25e09b", "169264")   ### [Sweet Sensation [American band]]
    addDeezer(mdbmaps, "MusicVF", "88cecf53871bf08f5ad9687f63842c7c", "4016289")   ### [The Adventures of Stevie V]
    addDeezer(mdbmaps, "MusicVF", "b0d7a98d5833146a7acc0c013a56a526", "16321")   ### [Sonia [Sonia Evans]]
    addDeezer(mdbmaps, "MusicVF", "b76e723753b2ac17b303eebfbe64b88e", "289897")   ### [D:Mob]
    addDeezer(mdbmaps, "MusicVF", "61e900e6362defe3136e19d6753b96be", "2363")   ### [Johnny Kemp]
    addDeezer(mdbmaps, "MusicVF", "db2597d4c66bab23e2ed811a5383648b", "1502417")   ### [Doctor and the Medics]
    addDeezer(mdbmaps, "MusicVF", "e14a693cfb27201cbd404751156696dd", "3792")   ### [Don Johnson [actor]]
    addDeezer(mdbmaps, "MusicVF", "f050c4d82861ee7870fc4ebcacb095d3", "9916")   ### [Dave Stewart [David Allan Stewart]]
    addDiscogs(mdbmaps, "MusicVF", "8ed3b973a02c78a2a17862a5179b2af6", "1224507")   ### [Synch]
    
    addDiscogs(mdbmaps, "MusicVF", "6047ae45fdcbb1a61452be74c2e09458", "102404")   ### [Stars on 45/Starsound]
    addDeezer(mdbmaps, "MusicVF", "6047ae45fdcbb1a61452be74c2e09458", "451347")   ### [Stars on 45/Starsound]
    addAllMusic(mdbmaps, "MusicVF", "6047ae45fdcbb1a61452be74c2e09458", "mn0000019073")   ### [Stars on 45/Starsound]    
    
    addDiscogs(mdbmaps, "MusicVF", "92def4f0947b2b3a2b6deb83b698e0dd", "177974")   ### [Toyah]
    addDeezer(mdbmaps, "MusicVF", "92def4f0947b2b3a2b6deb83b698e0dd", "125542")   ### [Toyah]
    addAllMusic(mdbmaps, "MusicVF", "92def4f0947b2b3a2b6deb83b698e0dd", "mn0000012059")   ### [Toyah]
    addDiscogs(mdbmaps, "MusicVF", "928153b2815c2f2ee8f03e3b7d30f31e", "88740")   ### [Junior]
    addDeezer(mdbmaps, "MusicVF", "928153b2815c2f2ee8f03e3b7d30f31e", "64303492")   ### [Junior]
    addAllMusic(mdbmaps, "MusicVF", "928153b2815c2f2ee8f03e3b7d30f31e", "mn0000249552")   ### [Junior]
    addDiscogs(mdbmaps, "MusicVF", "38a08457915c65ef3cb1ff0b8a2db435", "31726")   ### [Ollie and Jerry]
    addDeezer(mdbmaps, "MusicVF", "38a08457915c65ef3cb1ff0b8a2db435", "5517")   ### [Ollie and Jerry]
    addAllMusic(mdbmaps, "MusicVF", "38a08457915c65ef3cb1ff0b8a2db435", "mn0000889962")   ### [Ollie and Jerry]
    addDiscogs(mdbmaps, "MusicVF", "06f8252aff2ada19a262b651bc610b42", "65957")   ### [Arcadia]
    addDeezer(mdbmaps, "MusicVF", "06f8252aff2ada19a262b651bc610b42", "211291")   ### [Arcadia]
    addAllMusic(mdbmaps, "MusicVF", "06f8252aff2ada19a262b651bc610b42", "mn0000501561")   ### [Arcadia]
    addDiscogs(mdbmaps, "MusicVF", "7ae3fb2747978b2d93a22910fb1bb293", "150891")   ### [Dave Stewart [David Lloyd Stewart]]
    addAllMusic(mdbmaps, "MusicVF", "7ae3fb2747978b2d93a22910fb1bb293", "mn0001737671")   ### [Dave Stewart [David Lloyd Stewart]]
    addDiscogs(mdbmaps, "MusicVF", "97845c5ac2a1eda6bc7a40a52550f6cf", "397333")   ### [Scandal [rock band]]
    addDeezer(mdbmaps, "MusicVF", "97845c5ac2a1eda6bc7a40a52550f6cf", "7943564")   ### [Scandal [rock band]]
    addAllMusic(mdbmaps, "MusicVF", "97845c5ac2a1eda6bc7a40a52550f6cf", "mn0000315142")   ### [Scandal [rock band]]

    addDiscogs(mdbmaps, "MusicVF", "e4b3177fc3046cf9d6bef9f8de838668", "152707")   ### [Mel Tormé and His Mel-Tones]
    addDeezer(mdbmaps, "MusicVF", "e4b3177fc3046cf9d6bef9f8de838668", "5036455")   ### [Mel Tormé and His Mel-Tones]    
    addDiscogs(mdbmaps, "MusicVF", "51bdddcad9ec3f5a5e37350c4fd05bf7", "319856")   ### [Keith]
    addDeezer(mdbmaps, "MusicVF", "51bdddcad9ec3f5a5e37350c4fd05bf7", "177616")   ### [Keith]    
    addDiscogs(mdbmaps, "MusicVF", "e1d965d3065c3be9e7bd57c63a6e20e5", "31208")   ### [Focus [1°]]
    addDiscogs(mdbmaps, "MusicVF", "c35939027e0e7ec06e8320dd46e790ef", "273394")   ### [John Williams [composer]]
    addDiscogs(mdbmaps, "MusicVF", "5329dea1b8ecda6841c5ab6bdab61a43", "384584")   ### [Rick Dees & His Cast of Idiots]
    addDeezer(mdbmaps, "MusicVF", "5329dea1b8ecda6841c5ab6bdab61a43", "183782")   ### [Rick Dees & His Cast of Idiots]
    addDiscogs(mdbmaps, "MusicVF", "f42685ea4f66c85a4635a8d3f95366a6", "283288")   ### [Delaney, Bonnie & Friends]
    addDiscogs(mdbmaps, "MusicVF", "dece6e0c0c1c1cad29e0e0f625d4018c", "319103")   ### [Gallery]
    addDiscogs(mdbmaps, "MusicVF", "47f792069a03acd651a6ecbde65895a0", "511467")   ### [Climax [US band]]
    addDiscogs(mdbmaps, "MusicVF", "22f9b012c07c6890e1bed160182226b3", "115131")   ### [The Dramatics [1°]]
    addDiscogs(mdbmaps, "MusicVF", "2c947f7a889c2d431cd00e5e85913df6", "3452")   ### [M.F.S.B.]
    addDiscogs(mdbmaps, "MusicVF", "b26d4ae19a5b1cf2ddb07b32371e69d9", "139260")   ### [Eruption]
    addDiscogs(mdbmaps, "MusicVF", "e6972ff5d00f1d66b88746d2fc81260a", "86248")   ### [The Emotions [female R&B trio]]
    addDiscogs(mdbmaps, "MusicVF", "085f10fce6a33c786f967fc1729f2042", "303048")   ### [Brian and Michael]
    addDiscogs(mdbmaps, "MusicVF", "b1014004f336ff817d2c905413665d71", "223235")   ### [Jigsaw [English group]]
    addDiscogs(mdbmaps, "MusicVF", "c79c6b1627a8e4bc4027a3fcd8617b72", "166278")   ### [Dave and Ansil Collins]
    addDiscogs(mdbmaps, "MusicVF", "f14ae50f772d6187b5e821f5a1547a4a", "325408")   ### [Sweet Sensation [english band]]
    addDiscogs(mdbmaps, "MusicVF", "5ff0fb159078d09e3d20d23c36e36a70", "234332")   ### [The Free Movement]    
    
    addDiscogs(mdbmaps, "MusicVF", "7cc6279cab5f001ecbf8b848727bb272", "324860")   ### [The Scaffold]
    addDiscogs(mdbmaps, "MusicVF", "a863ec7ba236ab6614b59e37a8bd69e6", "458405")   ### [The Outsiders [1°]]
    addDiscogs(mdbmaps, "MusicVF", "672a7e5a27ae7c8ea520df12753fb5d7", "630848")   ### [Staff Sgt. Barry Sadler]
    addDiscogs(mdbmaps, "MusicVF", "d6952fda07639466bf3efbc89b46e63b", "372980")   ### [Unit 4 + 2]
    addDiscogs(mdbmaps, "MusicVF", "0b68011a45365670d3c914265302fa59", "62500")   ### [The Larks [Los Angeles band]]
    addDiscogs(mdbmaps, "MusicVF", "000e2d4b0fff826c1aef2bfe7634590f", "277351")   ### [The Four Pennies [English pop group]]
    addDiscogs(mdbmaps, "MusicVF", "7ed47eccfad0f0538664bf35989eab92", "368275")   ### [The Angels [US girl group]]
    addDiscogs(mdbmaps, "MusicVF", "29c31ab9aa41caa9389c02d6fec4461d", "318675")   ### [B. Bumble and the Stingers]
    addDiscogs(mdbmaps, "MusicVF", "3f59c400561a7b838e9ac29a06e49c04", "886357")   ### [Joe Henderson [American R&B singer]]
    addDiscogs(mdbmaps, "MusicVF", "a8c211fe0d027c386fe16e2ddb15753f", "337529")   ### [Ned Miller]
    addDiscogs(mdbmaps, "MusicVF", "210876931ba27271afa8197663ccc915", "1888215")   ### [Little Caesar & the Romans]
    addDiscogs(mdbmaps, "MusicVF", "2da77bc7d525ec9d446c3055bd4a22c2", "1214698")   ### [Kokomo [Jimmy Wisner]]
    addDiscogs(mdbmaps, "MusicVF", "6685fc6e664ad472deb9461525248eb8", "593039")   ### [The Sims Twins]
    addDiscogs(mdbmaps, "MusicVF", "81fb128ce380557b0464045d4bbd5713", "294706")   ### [Bobby Lewis]
    addDeezer(mdbmaps, "MusicVF", "81fb128ce380557b0464045d4bbd5713", "104995")   ### [Bobby Lewis]
    addAllMusic(mdbmaps, "MusicVF", "81fb128ce380557b0464045d4bbd5713", "mn0000071409")   ### [Bobby Lewis]
    addDiscogs(mdbmaps, "MusicVF", "dbe12b0fd1f39d4baa1b03990374f660", "275791")   ### [Them]
    addDeezer(mdbmaps, "MusicVF", "dbe12b0fd1f39d4baa1b03990374f660", "2463")   ### [Them]
    addAllMusic(mdbmaps, "MusicVF", "dbe12b0fd1f39d4baa1b03990374f660", "mn0000925181")   ### [Them]
    addDiscogs(mdbmaps, "MusicVF", "fd64625ef0ce3f6a1d6d91b8b932cd94", "2082135")   ### [The Cuff Links]
    addDiscogs(mdbmaps, "MusicVF", "f427959753a9b354845e51fdb7ab8dda", "4278444")   ### [Every Mother's Son]
    addDiscogs(mdbmaps, "MusicVF", "1b9252d1f463dd59a80a09a9a6049afa", "319856")   ### [Keith [American pop singer]]
    addDiscogs(mdbmaps, "MusicVF", "afceff565d12dc99275867186100b959", "336642")   ### [Ruby & the Romantics]
    addDiscogs(mdbmaps, "MusicVF", "2f4f399b889d3e70513490344c3fc647", "731174")   ### [Dick and Dee Dee]

    addDiscogs(mdbmaps, "MusicVF", "a7efe1bba5bb56d1f0ade2a48c82c260", "677033")   ### [The Highwaymen [folk group]]
    addDeezer(mdbmaps, "MusicVF", "a7efe1bba5bb56d1f0ade2a48c82c260", "112328")   ### [The Highwaymen [folk group]]
    addAllMusic(mdbmaps, "MusicVF", "a7efe1bba5bb56d1f0ade2a48c82c260", "mn0000075801")   ### [The Highwaymen [folk group]]

    addDiscogs(mdbmaps, "MusicVF", "585e0b8fbd3e1e6109ccdd0b924a15ed", "337625")   ### [Freddie and the Dreamers]
    addDeezer(mdbmaps, "MusicVF", "585e0b8fbd3e1e6109ccdd0b924a15ed", "61723")   ### [Freddie and the Dreamers]
    addAllMusic(mdbmaps, "MusicVF", "585e0b8fbd3e1e6109ccdd0b924a15ed", "mn0000177558")   ### [Freddie and the Dreamers]    
    
    addDeezer(mdbmaps, "MusicVF", "7f3f30dd052dda98c4d387a3522b107e", "14998")   ### [Sonny and Cher]
    addAllMusic(mdbmaps, "MusicVF", "7f3f30dd052dda98c4d387a3522b107e", "mn0000042993")   ### [Sonny and Cher]    
    
    addDiscogs(mdbmaps, "MusicVF", "688def85e032254c234d8441f307033f", "292376")   ### [Martha and the Vandellas]
    addDeezer(mdbmaps, "MusicVF", "688def85e032254c234d8441f307033f", "272982")   ### [Martha and the Vandellas]
    addAllMusic(mdbmaps, "MusicVF", "688def85e032254c234d8441f307033f", "mn0001199135")   ### [Martha and the Vandellas]    
    
    addDiscogs(mdbmaps, "MusicVF", "7a97c74c2b75a68121523083e4170d4e", "3761848")   ### [Harold Stern & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "1f7b5453ba1804f413bf127e230278ad", "4892300")   ### [The Castillian Troubadors]
    addAllMusic(mdbmaps, "MusicVF", "9c31b906e799c6be454297e9368d9aa0", "mn0003550643")   ### [The Castillians]

    addDiscogs(mdbmaps, "MusicVF", "0f50f13de71c3fe2c345ddf05b969181", "3547291")   ### [Tex Grande and His Range Riders]
    addDiscogs(mdbmaps, "MusicVF", "c8ced3e0213ee278030a5010d9c99cf3", "317903")   ### [Bennie Moten's Kansas City Orch.]
    addDiscogs(mdbmaps, "MusicVF", "959f839dd111bbb6bd2a099d8b8438d8", "1836420")   ### [Fred Hillebrand]
    addDiscogs(mdbmaps, "MusicVF", "0f74724d746e8e4bb2e38c48d99d18d0", "5562397")   ### [Dick Barrie]
    addDiscogs(mdbmaps, "MusicVF", "8b75992b1dd9207bbc405a144f70fbcf", "7369644")   ### [Hod Williams]
    addDiscogs(mdbmaps, "MusicVF", "a9bd1784af29d4a358ac6994aeae8294", "3338780")   ### [Red Jessup]
    addDiscogs(mdbmaps, "MusicVF", "36e533d418ac3f05e29bfa4a1390c4f4", "258518")   ### [the Mills Brothers]
    addDiscogs(mdbmaps, "MusicVF", "fc0b440ba7ef6104661b11a433a85acc", "1514960")   ### [Joe Sanders]
    addDiscogs(mdbmaps, "MusicVF", "f256e9c891c17308d9dfc8ca8844bf61", "1417290")   ### [King Garcia]
    addDiscogs(mdbmaps, "MusicVF", "8e141aaf19712fdc32189ddfeb80430e", "781771")   ### [Bob Howard]
    addDiscogs(mdbmaps, "MusicVF", "50d3706b2605b07b8b8a9b9961bea6ba", "1335225")   ### [Raymond Paige]
    addDiscogs(mdbmaps, "MusicVF", "ef9bf813076f90b5f8eaf2da0d244357", "899557")   ### [Rhythm Wreckers]
    addDiscogs(mdbmaps, "MusicVF", "0b9550156cafd474b09fe8778504d2ad", "905976")   ### [Rose Marie]
    addDiscogs(mdbmaps, "MusicVF", "bc1d9ac4203fedd720c413be4fa0a8f4", "383328")   ### [The Ravens]
    addDiscogs(mdbmaps, "MusicVF", "fd6d53a7d15d0844f39f16da39a5b360", "847610")   ### [Slim Gaillard Trio]
    addDiscogs(mdbmaps, "MusicVF", "7294bddc35185cd867b705196438ea18", "332327")   ### [Roosevelt Sykes and His Piano]
    addDiscogs(mdbmaps, "MusicVF", "9ed340e7c406c35d62e51ecb4c47ffb0", "320605")   ### [Pvt. Cecil Gant]
    addDiscogs(mdbmaps, "MusicVF", "e17c9d6b12e51038367a15bc40d84107", "337637")   ### [Don Reid]

    addDiscogs(mdbmaps, "MusicVF", "4f7f88f7e39b56a37e1f6cfcab989684", "4109908")   ### [The Havana Novelty Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "8517343b6aafcd5373bd348b2940f107", "864317")   ### [The Royal Teens]
    addDiscogs(mdbmaps, "MusicVF", "6c7dda5c7af0d386d430c3d096eacc75", "229181")   ### [The Big Bopper]
    addDiscogs(mdbmaps, "MusicVF", "3bf6a6337a24644243ccede70fd1d4e0", "635174")   ### [Charles McDevitt Skiffle Group]
    addDiscogs(mdbmaps, "MusicVF", "4e9789b4dbc9c9ff3c21edafeea91691", "1434057")   ### [Johnny Brandon]
    addDiscogs(mdbmaps, "MusicVF", "ae9f6d8935c41a1f5d78bf5650dec443", "1405959")   ### [Ray Burns]
    addDiscogs(mdbmaps, "MusicVF", "e04a5e161d882c31c39a80f6dfbe3efb", "656752")   ### [Somethin' Smith & the Redheads]
    addDiscogs(mdbmaps, "MusicVF", "d6f83baaad3622cf3bbcfea3dcf7db72", "661725")   ### [David Carroll]
    addDiscogs(mdbmaps, "MusicVF", "a7fb814f78370c1709f8d71d89cdbbb7", "907304")   ### [Norman Petty Trio]
    addDiscogs(mdbmaps, "MusicVF", "d1fcd1acf91f461760f5db89d6371243", "1554497")   ### [José Ferrer]
    addDiscogs(mdbmaps, "MusicVF", "7fe410b9139c911fce973d3214c59b7e", "3727380")   ### [Frank Weir & His Saxophone, Chorus & Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "c596cadb17d03d7e0a1a100fd6de5f3b", "1262946")   ### [The DeCastro Sisters]
    addDiscogs(mdbmaps, "MusicVF", "4796166bde6ef44bc2ce9eff6552fd8f", "1388910")   ### [Mariners]
    addDiscogs(mdbmaps, "MusicVF", "8cf7b312ec780e576720661724408a48", "417192")   ### [Les Compagnons de la chanson]
    addDiscogs(mdbmaps, "MusicVF", "7ebc2c92dbda26a5f35ec6609d9fec6e", "3598682")   ### [The Harry Grove Trio]
    addDiscogs(mdbmaps, "MusicVF", "2aa2c3d7a02d9717285ce61414a40520", "2504095")   ### [Eddie Wilcox Orch.]
    addDiscogs(mdbmaps, "MusicVF", "724e838678ee9e6c035146a1061a5d78", "8567655")   ### [The Wesson Brothers]
    addDiscogs(mdbmaps, "MusicVF", "b9639af619090eaddce0680eb25f93ea", "317205")   ### [Stardusters]
    addDiscogs(mdbmaps, "MusicVF", "7437e24d08744ee791f60c08f71913ab", "938654")   ### [Dan Russo & His Oriole Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "f019a54512ffd89d0308b1aef5166ea0", "4917055")   ### [Vaughn Horton and His Polka Debs]
    addDiscogs(mdbmaps, "MusicVF", "f962c6513c6bd518c2ff896bf46d2b79", "3888608")   ### [Jack Lathrop and the Drugstore Cowboys]
    addDiscogs(mdbmaps, "MusicVF", "ae29b7c1d80d4e045d5390a50bb10d70", "1667729")   ### [Brother Bones and His Shadows]
    addDiscogs(mdbmaps, "MusicVF", "d9451d357d4099416723ea60d0ed6c8f", "1558243")   ### [Anne Vincent]
    addDiscogs(mdbmaps, "MusicVF", "96ab13605e07ad3b97b398b307da8eb7", "1469965")   ### [The Poni-Tails]
    addDiscogs(mdbmaps, "MusicVF", "8095bcff1d8303d5575f1b4f937d0622", "383343")   ### [The Monotones]
    addDiscogs(mdbmaps, "MusicVF", "5c5707abdb7762182cbab8090143a83b", "5102087")   ### [Dick Two Ton Baker and His Music Makers]
    addDiscogs(mdbmaps, "MusicVF", "c90554db46d2a16fa43a47fecef71127", "395324")   ### [Hugo Winterhalter And His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "e1641cc8a8376ec4d76d74693b3390f4", "1968385")   ### [Billie Anthony]
    addDiscogs(mdbmaps, "MusicVF", "9c8b8d608c91561c6b88cee25361bf34", "367959")   ### [Mantovani And His Orchestra And Chorus]
    addDiscogs(mdbmaps, "MusicVF", "c9b54441216933c30501b0c5be4ddc72", "3587388")   ### [Grady Martin And His Slew Foot Five]
    addDiscogs(mdbmaps, "MusicVF", "3fb860cf933027fb20c35ae3fcfd6e67", "2547731")   ### [Laurie Anders]
    addDiscogs(mdbmaps, "MusicVF", "8328d465bb4eb143b2f8fba5d81eea1b", "1476286")   ### [Gloria DeHaven]
    addDiscogs(mdbmaps, "MusicVF", "3488e88f070db7ceb2400a560c01fc5f", "1514912")   ### [Victor Young And His Orchestra And Chorus]
    addDiscogs(mdbmaps, "MusicVF", "76eac162552ff5d6ac53e0a287fa453f", "1455378")   ### [The Keynotes]
    addDiscogs(mdbmaps, "MusicVF", "3f95e818916a3728097d2d5746d35d13", "1474769")   ### [Ernie Fields]
    addDiscogs(mdbmaps, "MusicVF", "980c84a5a3ac32aec4734996632fd25e", "272302")   ### [Mel Blanc and His Original Woody Woodpecker Voice]
    addDiscogs(mdbmaps, "MusicVF", "098bd0895351657a096917d224d630ef", "350223")   ### [Bob Crosby And His Bob Cats]
    addDiscogs(mdbmaps, "MusicVF", "c348f499cd1aae2a1d4da55954a5a514", "987396")   ### [Harry Cool]
    addDiscogs(mdbmaps, "MusicVF", "9c815acfd163b9034f3e09bc433acd0b", "2049936")   ### [Sally Foster]
    addDiscogs(mdbmaps, "MusicVF", "1593fbca70245e0ef0cb9d6ed375e4f8", "3155878")   ### [Muriel Lane]
    addDiscogs(mdbmaps, "MusicVF", "4ba957ba66bf53e0bb07c4da6147d24b", "573278")   ### [Gracie Allen]
    addDiscogs(mdbmaps, "MusicVF", "d4f29b4108645f92a1409b30a4ace170", "161916")   ### [Lolita]
    addDiscogs(mdbmaps, "MusicVF", "5650a29f417ad387c6921cc87b082300", "379734")   ### [Rosie]
    addDiscogs(mdbmaps, "MusicVF", "60de3b382798b1cf4bb63ce5fa675723", "901244")   ### [Miss Toni Fisher]
    addDiscogs(mdbmaps, "MusicVF", "c7c9f3935857a0bbc1ed2341040be9f4", "3111328")   ### [Billy Williams and the Pecos River Rogues]
    addDiscogs(mdbmaps, "MusicVF", "853ab232e08af792c2d65bbaa6340989", "2168178")   ### [The Uptown String Band]
    addDiscogs(mdbmaps, "MusicVF", "e3a376e87ef622aa85d8df23c574b64a", "317867")   ### [Nellie Lutcher and Her Rhythm]
    addDiscogs(mdbmaps, "MusicVF", "dd73a219cf98b7cfc16fe040b7738c6e", "3447620")   ### [Boone's Jumpin' Jacks]
    addDiscogs(mdbmaps, "MusicVF", "2a51cf04bd16e24cd0868639441d68ba", "4981989")   ### [Jimmy Lytell and His All Star Seven]
    addDiscogs(mdbmaps, "MusicVF", "ab56a6296bb617d0fe456852c9bdcae0", "3583401")   ### [The Royal Harmony Quartet]
    addDiscogs(mdbmaps, "MusicVF", "c0f175962521dbfe76a78c87e17fe44f", "1771795")   ### [Joan Merrill]
    addDiscogs(mdbmaps, "MusicVF", "b09f4a642a189c908803c168f8b32f2f", "557958")   ### [The Cats and the Fiddle]
    addDiscogs(mdbmaps, "MusicVF", "ae1f3d85e98fd09f235bcad1826ab703", "6990602")   ### [Al Kavelin and His Cascading Chords]
    addDiscogs(mdbmaps, "MusicVF", "6465abd321796236f977274ff6d6cb1f", "1678804")   ### [The Three Peppers]
    addDiscogs(mdbmaps, "MusicVF", "c024192437df576ff5cae3c7b409ec94", "374160")   ### [Jack McVea and His All Stars]
    addDiscogs(mdbmaps, "MusicVF", "02318adbba0b059a10e90b8b5a3bbc84", "1514914")   ### [Patricia Norman]
    addDiscogs(mdbmaps, "MusicVF", "a5006bb89a57d13ed942a00e1732e814", "3680685")   ### [The Milt Herth Trio]
    addDiscogs(mdbmaps, "MusicVF", "a59011ebcf56caedfe2145b6f5dad40b", "3838547")   ### [The Top Hatters]
    addDiscogs(mdbmaps, "MusicVF", "29c108cd41f644fd4253d32065929887", "1249695")   ### [The Sweet Violet Boys]
    addDiscogs(mdbmaps, "MusicVF", "b9b0d5327aa46f043b5cb674e8987d9d", "301370")   ### [Adrian Rollini & the Gang]
    addDiscogs(mdbmaps, "MusicVF", "a24ae2f1563085e1bbca2e213dd315f7", "1388909")   ### [Frank Parker]
    addDiscogs(mdbmaps, "MusicVF", "0975b95c438e2749aababfa0b9ed60b2", "1087275")   ### [Walter O'Keefe]
    addDiscogs(mdbmaps, "MusicVF", "8ea487d42f8bc4ac90c61eb7b1d72b87", "3778825")   ### [Conrad Thibault]
    addDiscogs(mdbmaps, "MusicVF", "82052cbfc3245267698272e4a3b4cfe4", "3444018")   ### [The Rondoliers & Piano Pals]
    addDiscogs(mdbmaps, "MusicVF", "1a0b3b68bbc0b6cccacfe7a463c27167", "651412")   ### [Six Hits and a Miss]
    addDiscogs(mdbmaps, "MusicVF", "3484f84ebdfee149b9b7d5638a863c54", "552073")   ### [Edward Byrnes]
    addDiscogs(mdbmaps, "MusicVF", "eb01b709e14365bf842b348bf75f5ee9", "7205117")   ### [Beverly White and Her Blues Chasers]
    addDiscogs(mdbmaps, "MusicVF", "39cf6983b7a23fb216da3bbd965539c8", "3235012")   ### [Phil Moore Four]
    addDiscogs(mdbmaps, "MusicVF", "27b50128a8ed15dfa9e696c0572f4abd", "2847852")   ### [5 Red Caps]
    addDiscogs(mdbmaps, "MusicVF", "a8ecbc08426611afc0348e1af183a49e", "1950077")   ### [Helen Carroll]
    addDiscogs(mdbmaps, "MusicVF", "55953e4539e4e5fd1b8ae761bd191f97", "895462")   ### [Dick Thomas]
    addDiscogs(mdbmaps, "MusicVF", "aef328dd8453ebcd3554224d8f28b7e5", "2081299")   ### [The Three Flames]
    addDiscogs(mdbmaps, "MusicVF", "bc78373a0bc71ca596adc70aa5dffe25", "2547592")   ### [Jerry Jerome and His Cats and Jammers]
    addDiscogs(mdbmaps, "MusicVF", "e589a798adf5ef25c8b7422f40684307", "4204858")   ### [Fred Waring and His Concert Vochestra]
    addDiscogs(mdbmaps, "MusicVF", "594076385af78f70120acafd064ce702", "4178021")   ### [Phil Hanna]

    addDiscogs(mdbmaps, "MusicVF", "5dc881691a067fa0a6cd6fcad2bba762", "759128")   ### [John Zacherle the Cool Ghoul]
    addDiscogs(mdbmaps, "MusicVF", "370d38be0e589813ad46b01a602902d2", "368604")   ### [King Brothers]
    addDiscogs(mdbmaps, "MusicVF", "4fb6ab67d1cbad34c3fc92490d43b309", "190250")   ### [Buchanan and Goodman]
    addDiscogs(mdbmaps, "MusicVF", "253643bcac6f68d5bc9f5bbb71ce0bb3", "1988086")   ### [Johnny Duncan [bluegrass musician]]
    addDiscogs(mdbmaps, "MusicVF", "eb347699e9e10d06721cfd0b48f94491", "297492")   ### [The Chords [1950s american doo wop group]]
    addDiscogs(mdbmaps, "MusicVF", "a5fd7caf3c0e9a1f04ace99d10b64836", "253855")   ### [Tommy Dorsey Ork.]
    addDiscogs(mdbmaps, "MusicVF", "5ba5fc8aad13de7cd1b6dd1d551d2b87", "1775011")   ### [the Lancers]

    addDiscogs(mdbmaps, "MusicVF", "fd724efa6b00d22d253709bb192e7941", "3387486")   ### [Frank Froeba & His Swing Band]
    addDiscogs(mdbmaps, "MusicVF", "03bdc7069772b8b97585ea378a19d88e", "312090")   ### [Otis Williams & The Charms]
    addDiscogs(mdbmaps, "MusicVF", "3463f172d0c20c18c7be1751781e2a12", "2350031")   ### [Tiny Hill]
    addDiscogs(mdbmaps, "MusicVF", "15f1ad3a8340840bfdd05ddcba0ffdf9", "1239010")   ### [Benny Strong And His Orchestraa]
    addDiscogs(mdbmaps, "MusicVF", "0a79439ac5d35709853504af6ecaa7f0", "5914209")   ### [Tom Coakley & His Palace Hotel Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "168ce0f17cb46c73cb685e5348a61583", "893226")   ### [Billy Williams]
    addDiscogs(mdbmaps, "MusicVF", "144244e3229bbe989c6db9efe6ee77f5", "418972")   ### [Delta Rhythm Boys]
    addDiscogs(mdbmaps, "MusicVF", "e7a09af3f02cd73a9731f5bdcc3a9fa1", "798934")   ### [Joe Liggins and His Honeydrippers]
    addDiscogs(mdbmaps, "MusicVF", "68c11950a8674db0ae8eb317a8bf7580", "758541")   ### [Sportsmen]
    addDiscogs(mdbmaps, "MusicVF", "f3b33db42d0205998dfcace184c74035", "312090")   ### [The Charms]
    addDiscogs(mdbmaps, "MusicVF", "950195f930ffac627fd60389448103b8", "2822201")   ### [Art and Dotty Todd]
    addDiscogs(mdbmaps, "MusicVF", "f1fdde3f9b04a07031409a4a5b919660", "281075")   ### [Jerry Wayne]
    addDiscogs(mdbmaps, "MusicVF", "d2e7f7df29489827a769427189944546", "307305")   ### [Al Morgan]
    addDiscogs(mdbmaps, "MusicVF", "8f0ddc32fccfc5ba30204410455c2c5c", "302684")   ### [Joe Fingers Carr]
    addAllMusic(mdbmaps, "MusicVF", "c5a32420d3a2fb6dfa25b74b009fe1ef", "mn0001768318")   ### [Pinetoppers]
    addDiscogs(mdbmaps, "MusicVF", "71c6249285dd6b7dad774ec4ae0e1c8b", "984446")   ### [Song Spinners]
    addDiscogs(mdbmaps, "MusicVF", "4186632a3d8f4b415cfac0b89d8e46da", "1777288")   ### [Bobby Wayne]
    addDiscogs(mdbmaps, "MusicVF", "69ceb43a1076ede530403cddd2fbe5c7", "638171")   ### [Pinky Tomlin]
    addDiscogs(mdbmaps, "MusicVF", "909d4adb2d2c30ea0f78fab4b4e0098c", "1604825")   ### [Louise Massey and the Westerners]
    addDiscogs(mdbmaps, "MusicVF", "22c802e6c5da0ed138e510b634a0110f", "585772")   ### [Dick Powell]
    addDiscogs(mdbmaps, "MusicVF", "acbceebafa88f32801748a144efc5e29", "1169164")   ### [Barry Wood]
    addDiscogs(mdbmaps, "MusicVF", "327e660219fe8294f8622cadd72892d5", "3766687")   ### [The Hotel Commodore Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "e8c09762848a72497df7757fafbd9e2e", "2755752")   ### [Jerry Blaine & His Streamline Rhythm]

    addDiscogs(mdbmaps, "MusicVF", "c5cd9627f130cf4cae0c8c7a0f5ad51d", "447537")   ### [The Jesters]
    addDiscogs(mdbmaps, "MusicVF", "cf20fd9520d3f64be55bfefde5740907", "1239803")   ### [The Stargazers]
    addDiscogs(mdbmaps, "MusicVF", "b2a52870d6d0959c214eafda098d765a", "1089961")   ### [Gray Gordon]
    addDiscogs(mdbmaps, "MusicVF", "86cc9cc2430255441266f086719425ec", "506604")   ### [Julia Lee and Her Boy Friends]
    addDiscogs(mdbmaps, "MusicVF", "48f18a34d00fc9d64dd304b1c3ace7d7", "889650")   ### [Donald Novis]
    addDiscogs(mdbmaps, "MusicVF", "25db75ac34ad19ff373c25ec27d7c0cf", "1320571")   ### [Gertrude Niesen]
    addDiscogs(mdbmaps, "MusicVF", "54ad04919641707c535b9d0e0cb78fbc", "1188913")   ### [Johnny Maddox]
    addDiscogs(mdbmaps, "MusicVF", "99a4c19c1d091eadde2361fee561dcd3", "1912920")   ### [Mitchell Ayres and His Fashions in Music]
    addDiscogs(mdbmaps, "MusicVF", "b8e5b236ab73bf0ee6f0c223eafac251", "2891073")   ### [Tempo King & His Kings of Tempo]
    addDiscogs(mdbmaps, "MusicVF", "74bcd2926d57a2bac2f56b3d2c2b62a2", "709833")   ### [Jimmie Rodgers [pop singer]]
    addDiscogs(mdbmaps, "MusicVF", "c294c15e746828bd4f3ef842856d3d9a", "1239803")   ### [The Stargazers [British vocal group]]

    addDiscogs(mdbmaps, "MusicVF", "1311428767d81e19393dca9e26fd5392", "548019")   ### [Jack Smith]
    addDiscogs(mdbmaps, "MusicVF", "78b263751efbd75720f901ab0d78c1d9", "3164458")   ### [Tommy Tucker Time]
    addDiscogs(mdbmaps, "MusicVF", "1acc9f3642a9a2ba179d9945019b8aea", "4532644")   ### [Johnnie Johnston]
    addDiscogs(mdbmaps, "MusicVF", "3ff2d787edff99136e77f7864cc8c75a", "320797")   ### [The King Cole Trio]
    addDiscogs(mdbmaps, "MusicVF", "96c6305f37ffe8e7a86f4b49f3c23851", "915906")   ### [Swing And Sway With Sammy Kaye]

    addDiscogs(mdbmaps, "MusicVF", "303c0a0d96229c9581426b1fd33bd9f9", "307369")   ### [Connee Boswell]
    addDiscogs(mdbmaps, "MusicVF", "9dd3afd3c1e8a0f00392f3a1adee1d50", "2725656")   ### [Art Kassel and His Kassels-in-the-Air]
    addDiscogs(mdbmaps, "MusicVF", "b45d6d5f8c69607588df29d0748a89df", "2891172")   ### [Dolly Dawn & Her Dawn Patrol]
    addDiscogs(mdbmaps, "MusicVF", "6d891605cca4053b7956de9ddda8bd73", "335595")   ### [Andy Kirk and His Clouds of Joy]
    addDiscogs(mdbmaps, "MusicVF", "df2027d9e5b2df8691f950d59cc5a322", "349509")   ### [Eddy Duchin and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "b52dbc06b774edfd2d32ef664958397b", "349494")   ### [Kay Kyser and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "7c833833f5a7019308daa45c0497ddff", "1348205")   ### [Ted Wallace & His Campus Boys]
    addDiscogs(mdbmaps, "MusicVF", "b04bf1b01cc83657ce2033def0b9fb1f", "1700393")   ### [Moran & Mack]
    addDiscogs(mdbmaps, "MusicVF", "c3a8e594c568d01b41ec85f6e572c1fc", "1117996")   ### [Don Azpiazu & His Havana Casino Orchestra]

    addDiscogs(mdbmaps, "MusicVF", "0cbf330d8edc6626ab13340a54c281a7", "4515599")   ### [Kitty O'Connor]
    addDiscogs(mdbmaps, "MusicVF", "3a4d9b2db9828197f6014e586219d6ab", "1637594")   ### [The Charleston Chasers]
    addDiscogs(mdbmaps, "MusicVF", "e8b8acba3d3e770d65131602dd149ede", "969363")   ### [The Cotton Pickers]
    addDiscogs(mdbmaps, "MusicVF", "dff50fd358f481f7700224a34fe7d237", "486224")   ### [The Duncan Sisters [1°]]
    addDiscogs(mdbmaps, "MusicVF", "282a55383b3b2531e2af708dcc6de186", "516930")   ### [Henry Whittier]
    addDiscogs(mdbmaps, "MusicVF", "2a3c936e4d859c36f18fe32013f52989", "3115628")   ### [Warner's Seven Aces]
    addDiscogs(mdbmaps, "MusicVF", "183b68d77047d8fc95dce60b1adfa03b", "3617667")   ### [Dennis King]
    addDiscogs(mdbmaps, "MusicVF", "849e543e93f5ecebd77ed894f71f9894", "1528216")   ### [Willie Howard]

    addDiscogs(mdbmaps, "MusicVF", "be1f16c73817b80c1dcd8a40809bb7e9", "3199669")   ### [High Hatters]
    addDiscogs(mdbmaps, "MusicVF", "536cdeb9f15951002e0b2bd4e784d1e3", "2524169")   ### [Irving Aaronson & His Commanders]
    addDiscogs(mdbmaps, "MusicVF", "1b95f8d64ddaf4ef32b4ee53c6d80c73", "1753513")   ### [Art Gillham]
    addDiscogs(mdbmaps, "MusicVF", "039a4ed21c34d2b6691ee89a55aab12d", "3219276")   ### [Jane Green]
    addDiscogs(mdbmaps, "MusicVF", "fd65e5da58193dfbef86bbc2d6309e14", "2979161")   ### [Marcia Freer]
    addDiscogs(mdbmaps, "MusicVF", "e83a5984d733718d4b9f23323aa8022a", "1157658")   ### [Scrappy Lambert]
    addDiscogs(mdbmaps, "MusicVF", "ccd4999de700cf53032bb55bdb7e6e68", "2370466")   ### [The Clicquot Club Eskimos]
    addDiscogs(mdbmaps, "MusicVF", "a9fcddd5a046d89fca079eadafd0538c", "1534535")   ### [The Regent Club Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "59a8f074247f3df46dd7429e4e12cd3a", "4442232")   ### [The Beverly Hill Billies]
    addDiscogs(mdbmaps, "MusicVF", "7d3f3000f025191d1746f1053b8c878b", "3370337")   ### [Arthur Gibbs & His Gang]
    addDiscogs(mdbmaps, "MusicVF", "cf2d49073c56b8d5edc714d54cc69618", "4643811")   ### [The Associated Glee Clubs of America]

    addDiscogs(mdbmaps, "MusicVF", "637a650477c0567a4b6bca7ad49333cc", "826064")   ### [Vaughn De Leath]
    addDiscogs(mdbmaps, "MusicVF", "0cde569a9f694c8814c68dbb02cf71d1", "708256")   ### [The California Ramblers]
    addDiscogs(mdbmaps, "MusicVF", "1ae7eba65460ecfed11cb1af7c069eb3", "1313374")   ### [Ford & Glenn]
    addDiscogs(mdbmaps, "MusicVF", "3b1c44282812439bd5e48c39fa0f214b", "2903429")   ### [Edwin Dale]
    addDiscogs(mdbmaps, "MusicVF", "bfa0c0324e7acc417df47e2bda8bf364", "1533274")   ### [The Brox Sisters]
    addDiscogs(mdbmaps, "MusicVF", "0f7952d54f822eb899ba72ccd8419e18", "1261670")   ### [Billy Jones]
    addDiscogs(mdbmaps, "MusicVF", "8141e4ed4b735c19d9b566ea802d9fd9", "1048195")   ### [George Olsen and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "74a0461718b14d24860a657cb45b837f", "1538599")   ### [Weber & Fields]

    addDiscogs(mdbmaps, "MusicVF", "7a0ff86e52a88b8c87fd86505d36417d", "866093")   ### [Harry Fox]
    addDiscogs(mdbmaps, "MusicVF", "06fb4b0901135a954a5f40abeecf0efa", "1700332")   ### [Julian Rose]
    addDiscogs(mdbmaps, "MusicVF", "c2658529dbd5c96ca5d9bca9ae5606ba", "2829163")   ### [Belle Baker]
    addDiscogs(mdbmaps, "MusicVF", "9a20e6e56cd6c9e4a2c21ec0ef761677", "3374285")   ### [The Farber Sisters]
    addDiscogs(mdbmaps, "MusicVF", "e38faa8c21fd102aca30e25eec8fbeff", "1835752")   ### [Percy Hemus]
    addDiscogs(mdbmaps, "MusicVF", "0c4556e72b5f9bb1f9eb4a89d33945ee", "2898914")   ### [Hipolito Lazaro]
    addDiscogs(mdbmaps, "MusicVF", "776940af81a4e2e2ecb5bce9f6aa2d9d", "1768625")   ### [The Knickerbocker Quartet]
    addDiscogs(mdbmaps, "MusicVF", "5325038a2866515f8e86e141d6af5422", "5416455")   ### [Reginald McAll]
    addDiscogs(mdbmaps, "MusicVF", "4ada244fd430d4585f6b365ad44dd3dc", "1272386")   ### [Elsie Janis]
    addDiscogs(mdbmaps, "MusicVF", "1947751d3ff5603dd35d78a85a5e0bf3", "2708821")   ### [Kathleen Parlow]
    addDiscogs(mdbmaps, "MusicVF", "d20f2f573dc7c6c9111eb9b572b42d1c", "740344")   ### [Ernest Ball]
    addDiscogs(mdbmaps, "MusicVF", "a77d02cd9ee77fcb218744773d1e0b62", "5018453")   ### [Frank Tinney]
    addDiscogs(mdbmaps, "MusicVF", "5d2d580dd960072f06ee44bb08f1b283", "2813306")   ### [The Venetian Trio]
    addDiscogs(mdbmaps, "MusicVF", "351858939ae70d17e2bcc028aa365f8a", "2827022")   ### [Charles Kellogg]
    addDiscogs(mdbmaps, "MusicVF", "67daff7cb8f6771546b907c9cdf17960", "2667409")   ### [The Victor Mixed Chorus]
    addDiscogs(mdbmaps, "MusicVF", "44bdc5fa077e6dba2d29cadd396f19a6", "3349119")   ### [Owen McCormack]
    addDiscogs(mdbmaps, "MusicVF", "ce2af862047fe0b5117f96511b84b27b", "3529099")   ### [Virginia Asher]
    addDiscogs(mdbmaps, "MusicVF", "2eab96f5dbf0362ce569d26e04dc8fa6", "2314053")   ### [the Orpheus Quartet]
    addDiscogs(mdbmaps, "MusicVF", "3354a2245997515a40e67c1787273375", "2093757")   ### [Happy Six]
    addDiscogs(mdbmaps, "MusicVF", "b8888888649cfa8a577b012afc1fb1f6", "3160570")   ### [George Meader]
    addDiscogs(mdbmaps, "MusicVF", "bb4060e0a54ced1e5ca012a109ee716b", "1340791")   ### [the Victor Light Opera Co.]
    addDiscogs(mdbmaps, "MusicVF", "437f31cb2645204f6feed226d01b5fc0", "2567602")   ### [William Robyn]
    addDiscogs(mdbmaps, "MusicVF", "c7b4791e72f894810860eaa98d155263", "3805757")   ### [Mary Carson]
    addDiscogs(mdbmaps, "MusicVF", "ea8df109f91a975dbccbffe463f58695", "1534545")   ### [the Sterling Trio]
    addDiscogs(mdbmaps, "MusicVF", "34513adc4b08819e420b0e439e6e1ea3", "1700445")   ### [Helen Louise]
    addDiscogs(mdbmaps, "MusicVF", "15e33f0cb8ed4f76362f517d0b0b74ce", "2637024")   ### [James Harrod]
    addDiscogs(mdbmaps, "MusicVF", "7f56ae62f696a63c2f98aa6ce3c13ed5", "3681942")   ### [Frank Coombs]
    addDiscogs(mdbmaps, "MusicVF", "f1610c7f9a40df7ed45a84483b92df86", "1775471")   ### [Maurice Burkhardt]
    addDiscogs(mdbmaps, "MusicVF", "16abf00a321c0f93e4ed13e754735196", "1835757")   ### [The Neapolitan Trio]
    addDiscogs(mdbmaps, "MusicVF", "9bab68feb111f335f38311810b1baf46", "3836929")   ### [Gene Rodemich & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "0b26d0b7c70fdd67a011721cab0aff52", "2954135")   ### [The Kentucky Serenaders]
    addDiscogs(mdbmaps, "MusicVF", "5b757ced53ae65b7c226f8b84d7b8926", "2093852")   ### [Guido Deiro]
    addDiscogs(mdbmaps, "MusicVF", "e7fd8511eeda2ec20f82cfa67908c984", "925226")   ### [The Victor Concert Orchestra]

    addDiscogs(mdbmaps, "MusicVF", "f06c3716e8ae24f46ef55d2e6a961619", "2838233")   ### [Anna Wheaton]
    addDiscogs(mdbmaps, "MusicVF", "c97096206968151f344102098844e1b0", "4720328")   ### [Horace Wright]
    addDiscogs(mdbmaps, "MusicVF", "23eab09f00750b8f07697cd62ebdca11", "1233624")   ### [Lucy Gates]
    addDiscogs(mdbmaps, "MusicVF", "07ba21344cee2e24b5ecd8f3037615de", "5414420")   ### [Monroe Jockers]
    addDiscogs(mdbmaps, "MusicVF", "081ff0ea0dcea3f5fb20361ff59f42d6", "2581194")   ### [The Columbia Saxophone Sextette]
    addDiscogs(mdbmaps, "MusicVF", "e0846a5616b5118f48f91c9690bc1dd7", "1582538")   ### [Lillian Rosedale]
    addDiscogs(mdbmaps, "MusicVF", "27bc0cd46b7644dc741f18fc3b921665", "1700076")   ### [the Shannon Four]
    addDiscogs(mdbmaps, "MusicVF", "abf3f05d50e0177fa2e44c3f4fa0942e", "3785106")   ### [Gene Greene]
    addDiscogs(mdbmaps, "MusicVF", "e6112bbf0b4e22e17f368c8e9b397860", "2383029")   ### [William J. Halley]
    addDiscogs(mdbmaps, "MusicVF", "3e97fa98c40deb08a0bda3ca1dfcb8be", "6843673")   ### [Joseph Cawthorn]
    addDiscogs(mdbmaps, "MusicVF", "e61f1c29df82b036f88e56e0b0838584", "6262062")   ### [Joe Hayman]
    addDiscogs(mdbmaps, "MusicVF", "cd5cb502bc05b3c5099979df5d5ef918", "3333738")   ### [The Paul Biese Trio]
    addDiscogs(mdbmaps, "MusicVF", "dcc7cf3c7be6954fe83dd4f8519b4d67", "2918251")   ### [George Wilton Ballard]
    addDiscogs(mdbmaps, "MusicVF", "e921f75d1a98bdea5f4c01b1a44cc4b8", "2581047")   ### [George O'Connor]
    addDiscogs(mdbmaps, "MusicVF", "6d608dda152f2234a10874a7222fa061", "3621101")   ### [Elliot Shaw]
    addDiscogs(mdbmaps, "MusicVF", "2711769219ff792132a716dbb70654e8", "2309206")   ### [Dolly Connolly]
    addDiscogs(mdbmaps, "MusicVF", "94603bd68729be64bb5fbbc32d2e94fc", "4052246")   ### [Anna Chandler]
    addDiscogs(mdbmaps, "MusicVF", "4501e2cb614920d78a3402b9f885acc5", "1638060")   ### [George MacFarlane]
    addDiscogs(mdbmaps, "MusicVF", "412c8688cbfe290bd5c291d360d5ab41", "655644")   ### [Chauncey Olcott]
    addDiscogs(mdbmaps, "MusicVF", "71f6eb4ef827b0185c4d6f6bb739c47a", "3388558")   ### [Marguerite Farrell]
    addDiscogs(mdbmaps, "MusicVF", "91c249d08b5f4504141ed26e91e93ff2", "6028413")   ### [Will Robbins]
    addDiscogs(mdbmaps, "MusicVF", "3325f5e5535bbc4a6fc5f61a4aff429d", "3992295")   ### [Andrea Sarto]
    addDiscogs(mdbmaps, "MusicVF", "2879f8d7080effe00067a2d4fb2b0008", "2637033")   ### [Taylor Trio]
    addDiscogs(mdbmaps, "MusicVF", "6804ed1c99d9a717000db6c7ad970bbd", "1534556")   ### [The Avon Comedy Four]
    addDiscogs(mdbmaps, "MusicVF", "dea0214ca7949a4aa1eb6f66c47923dc", "2174340")   ### [Charles Hart]
    addDiscogs(mdbmaps, "MusicVF", "6c7c6aabcb657383b24b5929b7a2819d", "1534545")   ### [The Sterling Trio]
    addDiscogs(mdbmaps, "MusicVF", "b15bf8ba743e67310ecdb628f024d1b5", "1534360")   ### [John Steel]
    addDiscogs(mdbmaps, "MusicVF", "ac24cb5f7ea5ca9ebca46d55863e1d40", "1775463")   ### [The Heidelberg Quintet]
    addDiscogs(mdbmaps, "MusicVF", "4b2640d5a6fee565f16dad701a6d0d0d", "1700076")   ### [The Shannon Four]
    addDiscogs(mdbmaps, "MusicVF", "1aef553674f63531f0676521eb0affc0", "2577868")   ### [The Columbia Stellar Quartet]
    addDiscogs(mdbmaps, "MusicVF", "87b2ed88b0d4233172cfb30042a6b602", "6069200")   ### [Ada Jones & Walter Van Brunt]
    addDiscogs(mdbmaps, "MusicVF", "156cef596c313b8bf47c7e2be7597e86", "3247552")   ### [Marguerite Dunlap]
    addDiscogs(mdbmaps, "MusicVF", "55c203544f7f87cb2ce4e5cd8891f7bb", "1313719")   ### [the Peerless Quartet]
    addDiscogs(mdbmaps, "MusicVF", "8bbc5188169d6feabd282c2801b70d9a", "2314053")   ### [The Orpheus Quartet]
    addDiscogs(mdbmaps, "MusicVF", "124194725e14cb8c2ee774e14c5b7a39", "5524645")   ### [Elizabeth Brice]
    addDiscogs(mdbmaps, "MusicVF", "4fc68f4e1c4b66b614e5879f76dfc3e1", "413940")   ### [Charles King]
    addDiscogs(mdbmaps, "MusicVF", "2d1d7c9d09fa004dff58e99be8383626", "1334935")   ### [The All-Star Trio]
    addDiscogs(mdbmaps, "MusicVF", "687f2f78766b4afb3db125e47991f64f", "2570607")   ### [John Barnes Wells]
    addDiscogs(mdbmaps, "MusicVF", "0b6cb5335a14477392a57d8332866d62", "6104768")   ### [Florodora Girls]
    addDiscogs(mdbmaps, "MusicVF", "1d98f1125ac4b7fc3d9cb6e8eddb5abf", "307346")   ### [The Fisk University Jubilee Quartet]
    addDiscogs(mdbmaps, "MusicVF", "86744afb227c06b3ebe5c211f2d051e2", "3068289")   ### [Kitty Cheatham]
    addDiscogs(mdbmaps, "MusicVF", "800742427bca8a3bc53b1e90813a8d7f", "4196180")   ### [Paul Southe]
    addDiscogs(mdbmaps, "MusicVF", "46cb836b572875c6f1f897e2af03d75e", "2705492")   ### [Gina Viafora]
    addDiscogs(mdbmaps, "MusicVF", "8f5104cd2c3c40e2f3e360b5bbdf8727", "4080215")   ### [Harold Jarvis]
    addDiscogs(mdbmaps, "MusicVF", "7f23916f0099bc98609cd31175c85048", "3068292")   ### [May Irwin]
    addDiscogs(mdbmaps, "MusicVF", "bf7387ba2045dda4d45bfe6a09cc44ab", "6701051")   ### [Helene Noldi]
    addDiscogs(mdbmaps, "MusicVF", "6b893f72ffbcd4e203ec16862a9e93ac", "562820")   ### [James McCool]
    addDiscogs(mdbmaps, "MusicVF", "0ba7e39c46e0c125312e1acb245c4fb6", "1517508")   ### [DeWolf Hopper]
    addDiscogs(mdbmaps, "MusicVF", "6b94b70722ce20ac987d0039deac7c36", "495702")   ### [The Columbia Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "16aa31b2a6d648944f0d8bdb1511f4f9", "3445561")   ### [William H. Thompson]
    addDiscogs(mdbmaps, "MusicVF", "983d919e863d0234fe414427f0051cdf", "3776429")   ### [William Redmond]
    addDiscogs(mdbmaps, "MusicVF", "89ab8ecd02d67b4f5033616146a3016e", "1337794")   ### [the American Quartet]

    addDiscogs(mdbmaps, "MusicVF", "044f611b92098846fb92fbc73fada281", "847619")   ### [George J. Gaskin]
    addDiscogs(mdbmaps, "MusicVF", "14f2e63b3a16d0d8b3befafee0abc84a", "1534417")   ### [Alan Turner]
    addDiscogs(mdbmaps, "MusicVF", "79fcadfb9c285562c09025ba9db0e58c", "3253082")   ### [Arthur Clough]
    addAllMusic(mdbmaps, "MusicVF", "3249ec1b23042ab6b80f9a8822101189", "mn0000318081")   ### [Jere Mahoney]
    addDiscogs(mdbmaps, "MusicVF", "d9ff09bf53dbb7e72dc5ea0b6b9d4c70", "4887325")   ### [Silas Leachman]
    addDiscogs(mdbmaps, "MusicVF", "2d777270139f2209e47b6f9b14cebcd7", "1202354")   ### [Jules Levy]
    addDiscogs(mdbmaps, "MusicVF", "37c6c9812cf67fcbb68f932897435408", "4812494")   ### [Grace Spencer]
    addDiscogs(mdbmaps, "MusicVF", "d78a0aa58ab85928515788aa76459b1e", "1487761")   ### [Nat Wills]
    addDiscogs(mdbmaps, "MusicVF", "1e0178d363cd693f6b22baf3b403fed5", "3570381")   ### [Will Denny

    addDiscogs(mdbmaps, "MusicVF", "f2ba906e2c895afa7cc274f84a8a64f5", "3854419")   ### [S. H. Dudley]
    addDiscogs(mdbmaps, "MusicVF", "2659712a0675f0121db7f4dd1dea1495", "3113189")   ### [John Bieling]
    addDiscogs(mdbmaps, "MusicVF", "e85d85cf298584efbbd5b83f0db7ee82", "4557912")   ### [Richard Jose]
    addDiscogs(mdbmaps, "MusicVF", "8dee48bab325476580ec7f96e94f286f", "8005580")   ### [Mina Hickman]
    addDiscogs(mdbmaps, "MusicVF", "77d495aa4446778d87d0606a28ecd595", "847633")   ### [Dan Quinn]
    addDiscogs(mdbmaps, "MusicVF", "b71835d352d8570c70b3e5faa0a0b4df", "2905804")   ### [Harry Tally]

    addDiscogs(mdbmaps, "MusicVF", "40da9295fedc06007599b5aed7ed23c4", "915906")   ### [Swing and Sway with Sammy Kaye]
    addDiscogs(mdbmaps, "MusicVF", "424a198fbe9cdb4b9824e0dab2d02b8d", "2577546")   ### [Frank Stanley]

    addDiscogs(mdbmaps, "MusicVF", "4a0e76a3f7d60126def2ad9eb4fc797d", "833291")   ### [The Green Brothers Novelty Band]
    addDiscogs(mdbmaps, "MusicVF", "7ae4466bc08021ac9bb000a08b4ec534", "1725437")   ### [the Watts 103rd Street Rhythm Band]
    addDiscogs(mdbmaps, "MusicVF", "10401ed282fb35b566c27c2ff3ac5325", "1435727")   ### [The Eli Young Band]
    addDiscogs(mdbmaps, "MusicVF", "78897201543b498aacf1c88ea26530eb", "518900")   ### [The Debroy Somers Band]
    addDiscogs(mdbmaps, "MusicVF", "62c5ee996268ddf0959c4900f7a3c565", "3678154")   ### [Metronome All Star Band]
    addDiscogs(mdbmaps, "MusicVF", "8a8d9f24b674503156f61945d2bfeb17", "1319719")   ### [The New York Military Band]
    addDiscogs(mdbmaps, "MusicVF", "b6f449a458feffa45d1733690050b813", "6620157")   ### [U.S. Naval Academy Band]
    addDiscogs(mdbmaps, "MusicVF", "89ee96133f18678875fb588a1804534d", "4846155")   ### [The Vernon Country Club Band]
    addDiscogs(mdbmaps, "MusicVF", "b214de55c5016792e857b4e1cea9d547", "631489")   ### [Ferko String Band]
    addDiscogs(mdbmaps, "MusicVF", "59a1f622916a78cb86f486057fe398a1", "341400")   ### [Jimmy Dorsey and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "5e91727a080c9c2d39e24fda912f1e97", "750213")   ### [Isham Jones & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "1162e06ab796e03f2569674315c517cb", "323993")   ### [Freddy Martin and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "9d16f4b2bc7dc28c9ca1a391910d269b", "342802")   ### [Leo Reisman and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "90eaa040177c2811146f9bd9596f0e9a", "348903")   ### [Hal Kemp and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "364a3b80de20af42c4d91b29ea9fb24f", "1277416")   ### [Glen Gray and the Casa Loma Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "663d60521dc70e807c1597f6b4699085", "3453989")   ### [Nat Shilkret & the Victor Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "afce6faff59a28d344434f9a1b0695ca", "349510")   ### [Ray Noble and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "df05c6f55851e6578477ca388660dc56", "1253756")   ### [Jan Garber and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "02ab9faaa95a329ed8dd672e5904c0a5", "1424900")   ### [Ted Weems and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "9d7bcbfee811d2f50c06f801c79dcf98", "341410")   ### [Larry Clinton and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "dc4a1e1f2b07b534353aa2d67fc18a17", "1089954")   ### [Ozzie Nelson and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "1b945945f43c0420f9d20ccd59c83d88", "1369041")   ### [Vincent Lopez & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "68876fa095048150249bafa168d67f69", "342361")   ### [Teddy Wilson & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "7143b00323c00bd261ad17f645d7ab64", "2426367")   ### [The Benson Orchestra of Chicago]
    addDiscogs(mdbmaps, "MusicVF", "2241f0ef784c8ac08ea7a47a48255f35", "339917")   ### [Roger Wolfe Kahn & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "a8e52909def4852558fe2437e06f43b1", "2610502")   ### [Arden-Ohman Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "3aa0308fe7e68b05fd8934e39e3cd390", "349522")   ### [Orrin Tucker and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "7777c383c810e40a3eadfce4d1b91a6c", "708255")   ### [Gus Arnheim & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "9048dc7c26905c398e086c8c67d4b508", "3197262")   ### [Paul Specht & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "21c1f7f1d5e459accd5b35621978b191", "1916128")   ### [Xavier Cugat and His Waldorf-Astoria Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "93976a04e70e7d73ac764770c54072b3", "2509255")   ### [Ted Fio Rito & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "85453843117a5eff96a9f102eff93666", "708247")   ### [Ray Miller & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "13ba349d06b706dd6f11f1f26bc1fac8", "2053208")   ### [Benny Krueger and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "1aa9688887d53658cc41a1db9518301c", "1514912")   ### [Victor Young & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "5d848b88a6e1b36b110abf3b61014b41", "341414")   ### [Tony Pastor and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "7a27db8bf0450bb84b58eb8ec5de9735", "338450")   ### [Jean Goldkette & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "07e3044e4b2eebfd6a89411bddf15019", "376962")   ### [Paul Weston and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "03c254c24017c8cb8aaf73ea2cf5be8b", "1909656")   ### [Dick Robertson & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "9fc8fbd76fd9d408157fa351c801f62e", "393014")   ### [Red Norvo & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "71fc57fe60a582a0ca3973f88711ddf1", "349507")   ### [Frankie Carle and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "86841c06a640a2717210bf54841c7a00", "3778766")   ### [Richard Himber & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "d3eb01fde0fb4a6de0df0228cc9184be", "1089948")   ### [Alvino Rey and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "2fd2396c6f988059397b0d6c36096fb5", "1270963")   ### [Sam Donahue and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "91f0224dc363f481debfdb3190f7ec21", "340734")   ### [Erskine Hawkins and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "c2d52ce3f15eb769eaab0be4a12e9486", "2705133")   ### [Henry King and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "21ab9c4037c261145865b119786c877e", "778418")   ### [Johnny Green & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "8739692acbb74242899157014478c777", "3349535")   ### [The Coon-Sanders Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "06126254a90f73d2a3ad4b8adc4c332a", "317889")   ### [Frankie Trumbauer & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "7c4f29dd44fb3859a09fe764e44aa2c9", "2765469")   ### [Al Donahue and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "4b7739a3b430a7ebafdcff9a908fc537", "540268")   ### [Jacques Renard and His Orchestra]
    addAllMusic(mdbmaps, "MusicVF", "f00e5a681822186c70c9c022c85cc2cb", "mn0001664426")   ### [Victor Herbert & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "9fa16a83d97e89319c28125903ce9a55", "1922766")   ### [Johnny Long and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "079674fb465a2b67123724e55b3bb32c", "357493")   ### [Ben Pollack & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "1b3dec386442752641ec283ff3f684cd", "2584575")   ### [Bert Lown and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "647c7ba1a93ac1ea8bc71c5225085e99", "843797")   ### [Freddie Slack and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "5fde257835593d3b4a6e17c07e946db2", "349512")   ### [Henry Busse & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "e805bb06ad2e064554515ad6f44b3a9b", "2990501")   ### [Johnny Hamp & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "1a49b43d30bb4dbfe0440149cd07e8d8", "3463207")   ### [Smith Ballew & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "efa149b82b5e9d0b8f0e16588436acf6", "349525")   ### [Will Bradley and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "1a1a29a71d2a9768f13fee5ec9397f13", "8051598")   ### [Earl Fuller's Novelty Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "d22e5ffe8e7ba9346455332509216ba5", "3194253")   ### [Jack Denny & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "e7e918c211119950c538271fe7c140c2", "708258")   ### [Fred Rich & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "ead3f3d7aa3c0975da0db2cb703be35c", "429644")   ### [Jimmie Grier & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "b4d9d721d37807d83145744914e57e9b", "7902851")   ### [Tom Gerun & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "53202b03bb71bd095ddab26432935d16", "349492")   ### [Clyde McCoy & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "49d5ea8f8e052f6b95fd68036b924e97", "2499512")   ### [Harry Reser's Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "ba2374b01c2c8a238f3da50946e6e04c", "446654")   ### [Don Redman & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "e1fe7d8022f0c55e7199d3ba780efa12", "1909635")   ### [Ruby Newman & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "46dea3fde8edeae6b2675d7e4736b630", "1527229")   ### [Phil Spitalny & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "18d6881a09e9ebea3713ce08534df60c", "1386796")   ### [Jan Savitt and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "8d7ee1984988bf4cc92ffdc3372b7106", "349501")   ### [Frankie Masters and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "399579d73e82eea2403c0e968aac3e74", "4317302")   ### [Buddy Rogers & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "c7dc152e3675b6c89a854edaf86e8001", "3330524")   ### [Ted Black & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "9fba994839e0e20edf5f74a478dea5ea", "3113509")   ### [Art Landry & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "09662762e6b1222572e121db07d3d397", "5189304")   ### [Lud Gluskin & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "fd6f7dbd728e896265fa836c2fb9c206", "432780")   ### [Henri René and His Musette Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "d67f1d65c8d4b80acfb2be380db375a3", "830957")   ### [Joe Venuti & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "4f26c243c4fb5dd9974009f128fabf8f", "2469413")   ### [Yerkes Jazarimba Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "d6a525fc93bafc6654c3b633e5ba917a", "917501")   ### [The Waldorf-Astoria Dance Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "7034518e9f13f25730f0220d0990fd1e", "1089955")   ### [Bob Chester and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "71753da0e75b7620e5a5d253d77e0e3b", "4699749")   ### [Paul Pendarvis & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "991d13db85dc8c149a3e262395f987f6", "2884360")   ### [Joe Green Novelty Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "c0dbd0a98ee1f2dbc5f821e4c7069ad0", "4370332")   ### [Al Trace and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "9cebc11d4538d5b8e5841a30001ab0ea", "341397")   ### [Raymond Scott and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "cbc2144c31d44c14b213c41095bd5730", "419475")   ### [Jack Jackson & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "c81486bf147b732e66ae95374e6c43cf", "1807615")   ### [The Hudson-DeLange Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "44dc319f4a15889d4a227e2697f6d4ce", "7342653")   ### [Reginald Foresythe & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "5464a3bac47d402ec094fdf37cb01326", "1612846")   ### [Art Jarrett and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "9053fe6b892fde478309af2e1316247b", "2773668")   ### [Mal Hallett & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "504b4f75a95d06335a1e3cb5dac95315", "3476694")   ### [Nat Brandwynne & His Stork Club Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "18cebbf9b4cfb8bda1ca957e50a2c992", "349502")   ### [Hal McIntyre and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "16a6048c3ef87052e05da99ab5dad8f2", "3087019")   ### [Anson Weeks & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "0f3c6b721541e1c741ff87fdc85d62a3", "1201611")   ### [Henry Allen & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "16f75b62f43e6501109d499b164706b3", "5675216")   ### [Peter van Steeden & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "7bd2b23381955fe428d95261ab6e9e1e", "1201977")   ### [The Victor Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "0fa3a0a5620ad16a47643b256005a830", "1541489")   ### [The International Novelty Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "f695a12ad76d87fc9f224ced8bf2cf0f", "3978868")   ### [Seger Ellis & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "15d524fcf25c0fe5e277ea476d4283d5", "3247982")   ### [Meyer Davis & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "1b5dca776f0ef2fd1205107cd9556ef7", "368180")   ### [Billy Butterfield and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "493e9274729503ebd82138eb04ee09a5", "1239010")   ### [Benny Strong and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "43cd3c56c657b29dc1d2adbc2d8e0f91", "1089157")   ### [Pee Wee Hunt and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "07649958fd7cd4f1035de4ebfd157771", "368184")   ### [Benny Carter and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "6386ec97b809d7afa589c33227c0dd95", "869942")   ### [Francis Craig and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "1e6336eeedc3f9436cb166b74b2db142", "349504")   ### [Elliot Lawrence and His Orchestra
    addDiscogs(mdbmaps, "MusicVF", "9ca5dd24ed8df06840865fc9021a0933", "341971")   ### [Ray Martin Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "872966b95880bc6fb65eb39f4828b111", "3325840")   ### [Carmen Cavallaro and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "975e3ed288a38b89797172a17ac201a2", "200966")   ### [Ted Heath Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "eb59c6b4e73764bca0d99e00acafb757", "271871")   ### [Frank Chacksfield and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "66ebcf967b25b1a72cfb576fefca80a1", "1403626")   ### [Will Osborne & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "011453b5586099bc69971661d18d8b0f", "3211715")   ### [Willie Kelly and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "5d450e2ef51fcdcd50e47aae756e0012", "2571353")   ### [Eddie Elkins & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "aaf6f171b47aa462dd6fdb8fe6531a4f", "3101048")   ### [W.C. Handy's Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "db7fac12df5fe8f9afdabf8bd48a0ef2", "1733554")   ### [Hilo Hawaiian Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "33208f2c0f69046675d175be287066f4", "925226")   ### [The Victor Symphony Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "df089b055fbe08f588b219e1e2e3032b", "5503818")   ### [Roy Ingraham & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "4d9fd245aedc6712a02f9f8bec978fd0", "886413")   ### [Johnny Johnson & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "a2d457fb1677103212bdc008fd531cfe", "6175352")   ### [Frank Black & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "763ea2bd44882528710c405e3a70c80b", "7125528")   ### [Jack Shilkret & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "7855f2793904d80de446edcc71252207", "3174998")   ### [Ernie Golden & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "10f0ac04981d8f7b1ebfe46dfbd18ad2", "3836928")   ### [Harry Archer & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "44b045bcf9e9d424f36716838e514a51", "3444587")   ### [Ross Gorman & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "59f5fc538500a11839f9b6f539bae568", "2884387")   ### [The Jaudas' Society Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "9a92789023d11881fd968cd8a73f090b", "1667817")   ### [McKee's Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "f772619fd4153349fb47f9ccda97a310", "3444582")   ### [Don Voorhees & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "778429d8a9821e88445969e3dc18b89a", "4299574")   ### [Phil Ohman & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "b44f3520e071957b8503ecced88767a8", "2891082")   ### [Paul Tremaine & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "3cdafda9a575a60eef2885004d65bc06", "312154")   ### [Stanley Black and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "740d289bb972ffc20b965a5317cef883", "1534133")   ### [Freddie Fisher & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "5a056efffa4ce2f92cf8b0550ec393fb", "1981924")   ### [Johnny Messner & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "0ef1e7b623c27d1f250229532484bb9d", "2064042")   ### [Bobby Byrne and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "fc34aae20cab9df2ae938b00a8d0efbf", "341409")   ### [Teddy Powell and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "a237529600f01e769b1d9cea9cc654f7", "341406")   ### [Claude Thornhill and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "e0f7b00f6c4950bde5da78645f0b1569", "367945")   ### [Jack Hylton & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "973a6531397e89bb78d2de6bcb702bc7", "3973861")   ### [Hal Goodman and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "76db57e97f4f99db7db7d2625e634123", "340731")   ### [Jack Teagarden & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "7523b97dfeb2ac3209e47d93180622dd", "5688823")   ### [Joe Moss & His Society Dance Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "8c4a58fa2eeffdeee25d3773bbb264a6", "2873355")   ### [Chick Bullock & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "18b04a7c320c287bfd63d7d1c48c8d9e", "5321173")   ### [Primo Scala's Banjo and Accordion Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "7cb82c5a2e57a82389be69200a4f44f0", "395324")   ### [Hugo Winterhalter's Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "ab86beec7d0969d43d784a6cedc8405d", "1968386")   ### [Eric Jupp and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "bdc6a1f249f43e4555ef2a4150f37611", "6197004")   ### [Jerry Johnson & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "eb56f846b0ab36eb3d63221181e29308", "3108182")   ### [Jolly Coburn & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "6d9be0f6509804108684fdaf84017ef4", "5100070")   ### [Ferde Grofé & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "310edf853dbd21626e09ecb9832a7b8c", "1658630")   ### [Arthur Lally & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "9859472117697df7d6862943e443bc2e", "1514952")   ### [Slatz Randall & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "6eca07976f58c020d2e54bafa2961e58", "1668971")   ### [The Riley-Farley Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "ab12a4348377acb624df0dbbf4906cc8", "1020569")   ### [The Roy Fox Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "4ae5cd21c231b532ab007a0091e6bc50", "2236664")   ### [Art Kahn & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "34302d19c358666521a535411af530bd", "1206210")   ### [The RKO Studio Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "c83cb0386f904d73f9755c0df8d9dc86", "1598356")   ### [Lew Stone & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "45d72ae61b67005d696a79a88fce60e1", "5283673")   ### [Charles Wolcott and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "5a216494307b7ec3f2ace9e110a93d9b", "4693278")   ### [Sonny Boy Williams and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "bb84eb39cf49cd24a54065551d655304", "2130269")   ### [Ceelle Burke and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "465897d2140b35204079a686166ba7aa", "3108653")   ### [Allen Miller and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "d4c22b725181e370189f32dd098bc8ff", "950513")   ### [Clyde Lucas and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "f225c614925408fc8e100d356a5f54de", "2685342")   ### [Terry Shand and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "10aaf6361bd752ce70c5407eec78b3d4", "3252185")   ### [Andy Iona & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "2e21c0926346a9be0990e62880deb9ea", "1089946")   ### [Skinnay Ennis & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "964b316d1ce04bb3ab186d6349ec1423", "1113839")   ### [Emery Deutsch & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "9d8eda72f4e03acbd0db2805843bf73c", "2534757")   ### [Louis Levy & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "cfc2b7abcc61165ce9af15dcdca80001", "5115585")   ### [Orville Knapp & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "b555c01db44d2054f8b5291707d66f9a", "2076665")   ### [Willie Bryant & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "60107fd180a7085a8a0b662fc8d93ab6", "8599059")   ### [Frank Westphal & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "a7f8728d0464ba2cdb280d60cf73e757", "3424996")   ### [Nathan Glantz & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "9d068924c00e547bbca5bfab17298b21", "1699772")   ### [Zez Confrey & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "39a1a3ec713ed1a4a4b30a3cca2617bf", "388185")   ### [The Philharmonic Orchestra of New York]
    addDiscogs(mdbmaps, "MusicVF", "c87626c18a8ef0fc98373b827f1a39c2", "834227")   ### [The Cincinnati Symphony Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "7a5b2a00960a6fcc7906e44241a40c6f", "383340")   ### [Leroy Holmes and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "6814c6d3e5509475cc7d82f96106c966", "1717539")   ### [Morris Stoloff and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "da127666e61c0670709952d3b5ba6f95", "1911338")   ### [Jerry Wald and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "60fe62c5faa8e16170d8349a687db147", "2048947")   ### [Herbie Fields and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "86e56ff8c3825915d78bcc4dfbd36cd5", "700576")   ### [Ray Bloch and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "3bffe9403c94e4303d4c50b720187264", "951382")   ### [Al Sack and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "6a5ef6dd4e7ccb755866d76edc3e9d11", "3476192")   ### [Gray Rains and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "da2331602a1eeb3b191638bce3ebd15c", "843805")   ### [Johnny Hodges and His Orchestra]

    addDiscogs(mdbmaps, "MusicVF", "b70f953700e5d0088335c84a223cce94", "6269664")   ### [Billy Jones & Ernest Hare]
    addDiscogs(mdbmaps, "MusicVF", "5de22f3396df806061d6fb1178d55495", "1541488")   ### [Carl Fenton and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "68876fa095048150249bafa168d67f69", "342361")   ### [Teddy Wilson & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "1b945945f43c0420f9d20ccd59c83d88", "1369041")   ### [Vincent Lopez & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "70020d3d9c374709e82a2a49e304f074", "1089971")   ### [Abe Lyman and His Californians]
    addDiscogs(mdbmaps, "MusicVF", "73337b116f577d25dceb545c9819c436", "1340791")   ### [The Victor Light Opera Co.]
    addDiscogs(mdbmaps, "MusicVF", "e02b9b1454c7e363bad165d512bc20df", "170357")   ### [The Carpenters]
    addDiscogs(mdbmaps, "MusicVF", "46abd922d7c4fdcfe654a39f6920e088", "1534354")   ### [Elsie Baker]
    addDiscogs(mdbmaps, "MusicVF", "b5cffbaa3ef63048522c90adb10af06a", "621943")   ### [Bob Roberts]
    addDiscogs(mdbmaps, "MusicVF", "fe178b6d86ceada3c50c26af07d55185", "1534185")   ### [Van & Schenck]
    addDiscogs(mdbmaps, "MusicVF", "d70e60bc1664eef96d37b0d8ee9369f3", "3704749")   ### [Earl Burtnett & His Los Angeles Biltmore Hotel Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "6a3aa27e6c3914c63ff83ac4b9231982", "317885")   ### [Red Nichols & His Five Pennies]
    addDiscogs(mdbmaps, "MusicVF", "7143b00323c00bd261ad17f645d7ab64", "2426367")   ### [The Benson Orchestra of Chicago]
    addDiscogs(mdbmaps, "MusicVF", "31e1781613731885e10f9204b566dfc2", "732768")   ### [Vess Ossman]
    addDiscogs(mdbmaps, "MusicVF", "cc654ba68ea4a88eefbe969cbc9438b6", "424111")   ### [the Haydn Quartet]
    addDiscogs(mdbmaps, "MusicVF", "58c56f36fb19ccbb45f390f0ab01a01a", "307205")   ### [Boswell Sisters]
    addDiscogs(mdbmaps, "MusicVF", "2241f0ef784c8ac08ea7a47a48255f35", "339917")   ### [Roger Wolfe Kahn & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "336d8488cbe2d09af93b92728a1bcd2d", "3251382")   ### [Corrine Morgan]
    addDiscogs(mdbmaps, "MusicVF", "a8e52909def4852558fe2437e06f43b1", "2610502")   ### [Arden-Ohman Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "3557232a1330590e9e500f6af7ce566c", "1551026")   ### [Don Bestor & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "3aa0308fe7e68b05fd8934e39e3cd390", "349522")   ### [Orrin Tucker and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "a98b66aaebcf635f7a8eb6c4c07bfd30", "1667953")   ### [The Glee Cast]
    addDiscogs(mdbmaps, "MusicVF", "330315f48c9ba8187fe75e7c1a8e2f6b", "82735")   ### [The Spinners [US band]]
    addDiscogs(mdbmaps, "MusicVF", "db72c76196f0469fbf1cbd901b8b6646", "1089953")   ### [Enric Madriguera and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "618d1ac69d767a02600e17fd21ad75d0", "1639763")   ### [Art Hickman & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "ec7168be46b5f1b985ee45c04e79cfe2", "341398")   ### [Charlie Spivak and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "b3ec2cbefce16cdcfb717302b4949560", "1186529")   ### [Dick Jurgens and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "7777c383c810e40a3eadfce4d1b91a6c", "708255")   ### [Gus Arnheim & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "364a3b80de20af42c4d91b29ea9fb24f", "1277416")   ### [Glen Gray and the Casa Loma Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "663d60521dc70e807c1597f6b4699085", "3453989")   ### [Nat Shilkret & the Victor Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "56a7191be8bbfc305156ca789bf5f31d", "1369825")   ### [Ben Bernie & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "afce6faff59a28d344434f9a1b0695ca", "349510")   ### [Ray Noble and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "599389f3b040e49d82e6a8385e17bb1a", "424111")   ### [The Haydn Quartet]
    addDiscogs(mdbmaps, "MusicVF", "df05c6f55851e6578477ca388660dc56", "1253756")   ### [Jan Garber and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "ada1f66f40ecb5b1f068b2ee780d1f3f", "1534207")   ### [Charles Harrison]
    addDiscogs(mdbmaps, "MusicVF", "02ab9faaa95a329ed8dd672e5904c0a5", "1424900")   ### [Ted Weems and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "dc4a1e1f2b07b534353aa2d67fc18a17", "1089954")   ### [Ozzie Nelson and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "9d7bcbfee811d2f50c06f801c79dcf98", "341410")   ### [Larry Clinton and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "78954f5f199505d1100853141aa92c92", "893947")   ### [Shep Fields and His Rippling Rhythm]
    addDiscogs(mdbmaps, "MusicVF", "9d16f4b2bc7dc28c9ca1a391910d269b", "342802")   ### [Leo Reisman and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "90eaa040177c2811146f9bd9596f0e9a", "348903")   ### [Hal Kemp and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "1206cc734327d41b89b7b21b614b24c1", "388298")   ### [Fred Waring's Pennsylvanians]
    addDiscogs(mdbmaps, "MusicVF", "a0da070b4d4b8481e517575e25da0d0f", "1337794")   ### [The American Quartet]
    addDiscogs(mdbmaps, "MusicVF", "8141e4ed4b735c19d9b566ea802d9fd9", "1048195")   ### [George Olsen and His Orchestra]
    addAllMusic(mdbmaps, "MusicVF", "572af4b5e5c28b35d0c01bc32f35969c", "mn0000032614")   ### [Paul Whiteman & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "572af4b5e5c28b35d0c01bc32f35969c", "82656")    ### [Paul Whiteman & His Orchestra]
    addAllMusic(mdbmaps, "MusicVF", "73318e203d3d01d134b78d76fdea6ede", "mn0000028948")   ### [Ted Lewis & His Band]
    addDiscogs(mdbmaps, "MusicVF", "73318e203d3d01d134b78d76fdea6ede", "344672")    ### [Ted Lewis & His Band]
    addAllMusic(mdbmaps, "MusicVF", "e73b5eb55c8d9cbd6c4e94ef34e857b4", "mn0000252794")   ### [The Peerless Quartet]
    addDiscogs(mdbmaps, "MusicVF", "e73b5eb55c8d9cbd6c4e94ef34e857b4", "459440")    ### [The Peerless Quartet]
    addAllMusic(mdbmaps, "MusicVF", "cfd29fb73a1df61969cc4e2c0d528889", "mn0000108905")   ### [Arthur Collins & Byron Harlan]
    addDiscogs(mdbmaps, "MusicVF", "cfd29fb73a1df61969cc4e2c0d528889", "1102536")    ### [Arthur Collins & Byron Harlan]
    addAllMusic(mdbmaps, "MusicVF", "7e48771184a11d7a02b2809bbeae73c6", "mn0000855695")   ### [Rudy Vallée & His Connecticut Yankees]
    addDiscogs(mdbmaps, "MusicVF", "7e48771184a11d7a02b2809bbeae73c6", "1309989")    ### [Rudy Vallée & His Connecticut Yankees]
    addAllMusic(mdbmaps, "MusicVF", "59a1f622916a78cb86f486057fe398a1", "mn0000358248")   ### [Jimmy Dorsey and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "59a1f622916a78cb86f486057fe398a1", "147557")    ### [Jimmy Dorsey and His Orchestra]
    addAllMusic(mdbmaps, "MusicVF", "5e91727a080c9c2d39e24fda912f1e97", "mn0000773745")   ### [Isham Jones & His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "5e91727a080c9c2d39e24fda912f1e97", "296197")    ### [Isham Jones & His Orchestra]
    addAllMusic(mdbmaps, "MusicVF", "1162e06ab796e03f2569674315c517cb", "mn0000186373")   ### [Freddy Martin and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "1162e06ab796e03f2569674315c517cb", "333548")    ### [Freddy Martin and His Orchestra]
    addAllMusic(mdbmaps, "MusicVF", "b52dbc06b774edfd2d32ef664958397b", "mn0001712608")   ### [Kay Kyser and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "b52dbc06b774edfd2d32ef664958397b", "159370")    ### [Kay Kyser and His Orchestra]
    addAllMusic(mdbmaps, "MusicVF", "df2027d9e5b2df8691f950d59cc5a322", "mn0000145904")   ### [Eddy Duchin and His Orchestra]
    addDiscogs(mdbmaps, "MusicVF", "df2027d9e5b2df8691f950d59cc5a322", "428029")    ### [Eddy Duchin and His Orchestra]
    addAllMusic(mdbmaps, "MusicVF", "40da9295fedc06007599b5aed7ed23c4", "mn0000045220")   ### [Swing and Sway with Sammy Kaye]
    addDiscogs(mdbmaps, "MusicVF", "40da9295fedc06007599b5aed7ed23c4", "273876")    ### [Swing and Sway with Sammy Kaye]

    mdbmaps["MusicVF"].save()
    
addMusicVFExtras(mdbmaps)

In [ ]:
print([item[1]["ArtistName"] for item in toMatch[chartType]])

## Spotify

## Top40

In [ ]:
#https://www.allmusic.com/artist/boa-mn0000322754
addAllMusic(mdbmaps, "Top40", "6dfb791d61d3dd8a44fa5b5c1c8dbde0", "mn0000322754")   ### [Boa]
addDeezer(mdbmaps, "Top40", "6dfb791d61d3dd8a44fa5b5c1c8dbde0", "792")    ### [Boa]
addDeezer(mdbmaps, "Top40", "688f28c172f877020462ae0dcd158f7d", "5337130")    ### [Antytila]
addDeezer(mdbmaps, "Top40", "c4bfdaef6a75c5b92d4314bed5324ac9", "4251519")    ### [V. Meladze]
#https://www.discogs.com/artist/1027379-The-Gospellers
addDiscogs(mdbmaps, "Top40", "2874db4bf2285180da2878da30719d98", "1027379")    ### [Gospellers]
addDeezer(mdbmaps, "Top40", "2874db4bf2285180da2878da30719d98", "7722644")    ### [Gospellers]




addDeezer(mdbmaps, "Top40", "996dc2db2e60fc3de7a14828df795171", "10470285")    ### [Z. E.]
addDeezer(mdbmaps, "Top40", "73ea300db5aa1c2154adfe42186a887e", "5418")    ### [Ark]
addDeezer(mdbmaps, "Top40", "c29dd858b42c5e8d6caecd0302c47dfe", "9060182")    ### [Hov 1]
addDeezer(mdbmaps, "Top40", "5127ec6e19c0a6008623adfad8b778c8", "72144882")    ### [Moe]
addDeezer(mdbmaps, "Top40", "2bc0fe9eb315a876af7ff1b91988daf3", "4523116")    ### [E.m.m.a.]
addDeezer(mdbmaps, "Top40", "919603bf283bab261d6607bea9234ce3", "75100")    ### [Crashdiet]
addDeezer(mdbmaps, "Top40", "25c0271edadaa16ad11b4016508ad57c", "10540205")    ### [Sebastian]
addDiscogs(mdbmaps, "Top40", "89946a817a31189393f24f92b32be466", "60136")   ### [Catch]
addDeezer(mdbmaps, "Top40", "0b8ff2909d235838583a92b539b640ef", "1164412")    ### [Floodgate]
addDeezer(mdbmaps, "Top40", "680c0282e232b1f64a3ef9d56da97275", "251483")    ### [Wallstones]
addDiscogs(mdbmaps, "Top40", "cb4133f2381241be2ae73d3365e746e8", "3143651")    ### [Honey]
addDeezer(mdbmaps, "Top40", "b59c32c628ceae2c2c6b59aacce5c3fe", "66502")    ### [Nic]
addDeezer(mdbmaps, "Top40", "6269dc570069e12a96dddc0a674e4ff2", "75178")    ### [Charlotte]
addDeezer(mdbmaps, "Top40", "b0e8915fa1946cafcbb743ec3f4a6ea9", "122866")    ### [Charizma]
addDeezer(mdbmaps, "Top40", "afadfe17376050c150bb0f9cb78e4181", "64411")    ### [Fame]
addDeezer(mdbmaps, "Top40", "5984fbaf75a5530e9a3319136ee014cf", "76721")    ### [Dj Roine]
addDiscogs(mdbmaps, "Top40", "214906164a747a8ce969529a640b9eec", "346187")   ### [Jeanette]
addDeezer(mdbmaps, "Top40", "214906164a747a8ce969529a640b9eec", "450752")    ### [Jeanette]
addDeezer(mdbmaps, "Top40", "c652ef0598d18f7655b92b18d87b8bcd", "56449262")    ### [Cuz]
addAllMusic(mdbmaps, "Top40", "89caf52a1eb268d2335ef2f09c39deb1", "mn0002185241")   ### [H.e.a.t.]
addDiscogs(mdbmaps, "Top40", "89caf52a1eb268d2335ef2f09c39deb1", "1890182")    ### [H.e.a.t.]
addDeezer(mdbmaps, "Top40", "336f4261abc48ea6247320a9b25885b0", "13720517")    ### [Chris Klafford]
addDeezer(mdbmaps, "Top40", "5ae548889336487db92e7746dea26566", "513525")    ### [Jay Smith]
addDeezer(mdbmaps, "Top40", "f412cf34b7f042c6e70017898a977805", "428632")    ### [Jazzattacks]
addDeezer(mdbmaps, "Top40", "71bbc65fd1a58cfe3b7eab94bc666fe4", "4588048")    ### [Ria Hall]
addDiscogs(mdbmaps, "Top40", "b1b3b97ec0478725400c7c8729607be9", "3037345")    ### [Carol Anthony]
addAllMusic(mdbmaps, "Top40", "29c175308260a766645f88a48fda8cb5", "mn0002141576")   ### [Sasha]
addDeezer(mdbmaps, "Top40", "ace5274eac9ac83ecc8be584f2f89678", "389681")    ### [Jezabels]
addAllMusic(mdbmaps, "Top40", "9eb2641c9d801ac6a24cd6d40a99cfcc", "mn0001488122")   ### [J.williams]
addDeezer(mdbmaps, "Top40", "9eb2641c9d801ac6a24cd6d40a99cfcc", "1226315")    ### [J.williams]
addDeezer(mdbmaps, "Top40", "3b51465a51f53de3f9406c8c4084b9eb", "4771547")    ### [Hannah]
addDeezer(mdbmaps, "Top40", "4660f8fef24f1de99d82c58cb6d558ae", "11373")    ### [Solitaire]
addDeezer(mdbmaps, "Top40", "bad689c3aa55f468721a0f0e31e00490", "862972")    ### [Knocks]
addAllMusic(mdbmaps, "Top40", "df7a414766c07b227b947fe19d735cca", "mn0000527296")   ### [P.I.]
addDeezer(mdbmaps, "Top40", "ee1825b598532d857e97ba0c03a6c10b", "9049402")    ### [They]
addDeezer(mdbmaps, "Top40", "bdf6a38ef1a824d2fbc71eb0866ec945", "5817516")    ### [Kucka]
addDeezer(mdbmaps, "Top40", "4e61bdac79b1a4b86d16d9a0121b1a77", "1655320")    ### [Tobtok]
addDeezer(mdbmaps, "Top40", "66eaf372536540197d6edb36afe6a9ce", "595")    ### [Destiny`s Child]
addDeezer(mdbmaps, "Top40", "8b3f077c9506a80b91565e354f5939ec", "6756461")    ### [Tony Ansell]
addDeezer(mdbmaps, "Top40", "4dbc5b8f4dabea79e3ae50712cb6a994", "2830461")    ### [Yebba]
addDeezer(mdbmaps, "Top40", "6e9d556d993ea0981f932c68de4c1b77", "9852222")    ### [Kelly Kiara]
addDeezer(mdbmaps, "Top40", "b80ed1fc4530c8fcc8e780deccf26b26", "5358213")    ### [Tom Thum]
addDeezer(mdbmaps, "Top40", "96db126f7d9fb560896964cc9019cfc7", "5922055")    ### [Montaigne]
addDeezer(mdbmaps, "Top40", "ddb7f9b0e8260c58c6f6829d149920a2", "5680005")    ### [James Chatburn]
addDeezer(mdbmaps, "Top40", "7b58a1b910d5311157faf0d7ba943ffa", "6800529")    ### [Benjamin Joseph]
addDeezer(mdbmaps, "Top40", "110294f1b077f24129fc995ebe23fbe8", "6514885")    ### [Beau Ryan]
addDeezer(mdbmaps, "Top40", "620e410c22308e859ec128e1d31cd98f", "323884")    ### [Pez]
addDeezer(mdbmaps, "Top40", "208619acbc541f424ed93ef7efd3f3cd", "6756463")    ### [Peter Wall]
addDeezer(mdbmaps, "Top40", "6b56ba9a8034ebbfd087f70a7f702a1a", "4079487")    ### [Ceekay Jones]
addDeezer(mdbmaps, "Top40", "04d84fa03fde087f3259c8bdc33752bf", "80523")    ### [Tv Rock]
addDeezer(mdbmaps, "Top40", "1be93f066d18ad80f8bfd2e70f363c0b", "1697944")    ### [Mt Raskil Preservation]
addDeezer(mdbmaps, "Top40", "ce801880919906527590fa4ddaa32c45", "1503955")    ### [Musicians Of The Sydney Symphony]
addDiscogs(mdbmaps, "Top40", "f90e3b3a282edb1c217d6e0fcf997d6c", "2931966")   ### [Prophet]
addDeezer(mdbmaps, "Top40", "f31c59cc71f57ef742aa0db40beb9b12", "280294")    ### [B.o.b]
addDeezer(mdbmaps, "Top40", "5f5e50d6614f04aaefd4995dca22f6b0", "443623")    ### [Jes Hudak]
addDeezer(mdbmaps, "Top40", "2176f53302606c207715827ebaa28e2a", "5523244")    ### [George Krikes]
addAllMusic(mdbmaps, "Top40", "1ea871f975022d52aae6eab3e9f86e5e", "mn0000999409")   ### [Geoffrey Gurrumul Yunupingu]
addDeezer(mdbmaps, "Top40", "5b4514a1943a8b0ecfc49c1feb0b95d7", "173576")    ### [R Kelly]
addDeezer(mdbmaps, "Top40", "95f4e2183d8f5105c6f825eae0181a52", "11645017")    ### [Mzwetwo]
addDeezer(mdbmaps, "Top40", "caf2a08a94f016a97e44a10130f3e3cf", "7374704")    ### [G-Easy]
addDeezer(mdbmaps, "Top40", "897216c66805bf28461c18040b04b67b", "10173710")    ### [Esther Elliott]
addAllMusic(mdbmaps, "Top40", "2f31f75ae3cb6d34dd619a1f3287b9ab", "mn0002135283")   ### [Lz7]
addDeezer(mdbmaps, "Top40", "2f31f75ae3cb6d34dd619a1f3287b9ab", "532751")    ### [Lz7]
addDeezer(mdbmaps, "Top40", "9993a84fddfdfd82804bbfed434d9885", "1825991")    ### [Ruby Frost]
addDeezer(mdbmaps, "Top40", "b242e187ccc3073a56e38ae2ee55e650", "243446")    ### [Drew]
addDeezer(mdbmaps, "Top40", "8e3afdb94e098cd079c307fb95c9218e", "4379877")    ### [Spawnbreezie]
addDeezer(mdbmaps, "Top40", "c8f6f45297ea4a12ea03624a875a212e", "406155")    ### [The Auckland Philharmonia Orchestra]
addDiscogs(mdbmaps, "Top40", "520556e00e6d980c7a428f88a6658c68", "5925200")   ### [Jade Louise]
addDeezer(mdbmaps, "Top40", "2f953cbb100b34dcc4f2d90a8000eacc", "4877604")    ### [Jessie G]
addDeezer(mdbmaps, "Top40", "7b1225826f059f0b7da7a22e128f37fb", "106014482")    ### [Mike Puru]
addDeezer(mdbmaps, "Top40", "79024f1f81e52227802f50f57ad5523d", "1405492")    ### [Seth Haapu]
addDeezer(mdbmaps, "Top40", "ec743dea831270125f51df1df5da8e42", "1182264")    ### [Frisko]
addDiscogs(mdbmaps, "Top40", "d5df77c4abe71b7e73ced4bbc963a71f", "6231462")   ### [Timesthree]
addDeezer(mdbmaps, "Top40", "d5df77c4abe71b7e73ced4bbc963a71f", "1658696")    ### [Timesthree]
addDeezer(mdbmaps, "Top40", "eac8f9698979b290371bd2a392c661fc", "1226318")    ### [Tiki]
addDeezer(mdbmaps, "Top40", "49230df907f220c6fc887f8b142492be", "85563822")    ### [Starboy]
addDeezer(mdbmaps, "Top40", "fca25c35f21674eabb0671a206c03930", "208194")    ### [Kin]
addDeezer(mdbmaps, "Top40", "4b4fd591656c8499d7d511fff78579b7", "11732531")    ### [Pipe Down Project]
addDeezer(mdbmaps, "Top40", "ed0eefd23b87ef67f3d0dd9eadf6226b", "62745")    ### [Arise]
addDeezer(mdbmaps, "Top40", "0e92ee4147f408112ff68c135d5202df", "8915798")    ### [#kiwiscurebatten]
addDeezer(mdbmaps, "Top40", "7f25cab06931e5c754eb74229144ca3a", "1031587")    ### [L. A. B.]
addDiscogs(mdbmaps, "Top40", "26c097ad9f7f55270b85ae02a22bb359", "266816")   ### [Jack L]
addDeezer(mdbmaps, "Top40", "26c097ad9f7f55270b85ae02a22bb359", "1438219")    ### [Jack L]
addDeezer(mdbmaps, "Top40", "dabd78a618b6ed0505657303ac766f81", "6788749")    ### [Broken Hill]
addDeezer(mdbmaps, "Top40", "5a73b17da495da0dc6ac4fff5054f0b8", "292468")    ### [Smash Proof]
addAllMusic(mdbmaps, "Top40", "00d5d59798eeffef591f272260b0bbcc", "mn0000124572")   ### [Chuck-N-Blood]
addAllMusic(mdbmaps, "Top40", "21da687c5162d957e0e8aaa39eff1e80", "mn0003750925")   ### [The Carters]
addAllMusic(mdbmaps, "Top40", "c00c9a5d2e3053716de9b79898cb5691", "mn0001225951")   ### [Mustard]
addAllMusic(mdbmaps, "Top40", "78f1feb9204bce3173d42798bc9bcb91", "mn0001808325")   ### [Kobe Bryant]
addAllMusic(mdbmaps, "Top40", "5edcd28a4505dc29e4edebd67052e614", "mn0000795894")   ### [Dolla]
addAllMusic(mdbmaps, "Top40", "5a888aa5d5b0835533b2a73dd895b461", "mn0002528230")   ### [DCUP]
addAllMusic(mdbmaps, "Top40", "7bcb752a835787affb80072d61387b22", "mn0001411030")   ### [Hollis]
addAllMusic(mdbmaps, "Top40", "c0051ae12f6048a807213c44b434aa1d", "mn0003348925")   ### [Jennifer Lawrence]
addAllMusic(mdbmaps, "Top40", "1f9b34897f6440e2484f1ac5e00953f7", "mn0003462822")   ### [Gnash]
addAllMusic(mdbmaps, "Top40", "9d337e01e83fc2b9d9fcbd6930bfb107", "mn0003715465")   ### [Casper Magico]
addAllMusic(mdbmaps, "Top40", "7670c11b0a85e757c3cd25e1cb4f0d2b", "mn0003496227")   ### [Dakota]
addAllMusic(mdbmaps, "Top40", "29c175308260a766645f88a48fda8cb5", "mn0002141576")   ### [Sasha]
addAllMusic(mdbmaps, "Top40", "20cf770061e851a2768a67afad49a089", "mn0001884165")   ### [Chris Malinchak]
addAllMusic(mdbmaps, "Top40", "80891387fa9995ead4c3a41b8eb58cac", "mn0003165846")   ### [Islove]
addAllMusic(mdbmaps, "Top40", "927d313b814225911b0cc45f6d101384", "mn0003525593")   ### [Neales]
addDiscogs(mdbmaps, "Top40", "000a8119d7146d68b89c0a1190ec7e11", "7474592")    ### [Young T & Bugsey]
addDiscogs(mdbmaps, "Top40", "8869a02f414bf8aa44ffa2491e057c18", "5690424")    ### [Sile Seoige]
addAllMusic(mdbmaps, "Top40", "23766e9564639a78dd2aa4469c656e33", "mn0000186917")   ### [Gabriella]
addAllMusic(mdbmaps, "Top40", "ee06f338805166cc652a5ffa29bfcaa4", "mn0000792831")   ### [Shane Macgowan]
addDiscogs(mdbmaps, "Top40", "338160050dd8760f6e21fd314da0c262", "282163")    ### [Glen Keating]
addDiscogs(mdbmaps, "Top40", "8f7bba2ad693b7bd7a54e9f5561df5fd", "670431")    ### [Redone]
addAllMusic(mdbmaps, "Top40", "61e0cad6ba67b77cf03b21bdcc855000", "mn0001072817")   ### [Ndubz]
addAllMusic(mdbmaps, "Top40", "8c59920da3b9afafbe8064702b8ae417", "mn0000866772")   ### [Levada Louca]
addAllMusic(mdbmaps, "Top40", "f9902337667b007c4cb0bd56853d6f77", "mn0001499382")   ### [Don Mescall]
addDiscogs(mdbmaps, "Top40", "3a78a66700126f076bc9880499d46098", "6754053")    ### [Theresa Rex]
addAllMusic(mdbmaps, "Top40", "45ec0da62cf46f95d1d19d6477e15223", "mn0003492788")   ### [Lewisham & Greenwich NHS Choir]
addAllMusic(mdbmaps, "Top40", "b3a0765a3d6a1f9b5f0218cc1a78167d", "mn0001585547")   ### [DJ Champion]
addAllMusic(mdbmaps, "Top40", "4101fb8a2e7d70d355cb78bb670cafb2", "mn0000615641")   ### [N Trance]
addAllMusic(mdbmaps, "Top40", "d4afc32b214e23fb75edc998fa24f102", "mn0000116049")   ### [D'side]
addAllMusic(mdbmaps, "Top40", "0edfaa56ae9a7023ef89e6dc038b27e7", "mn0000556741")   ### [Marshals]
addDiscogs(mdbmaps, "Top40", "ef5682c651f7c5295bc121ad58f18381", "1627595")    ### [St. Julien]
addAllMusic(mdbmaps, "Top40", "50eab1669ad5105c900ba050d23f7b37", "mn0002085581")   ### [Ruth-anne]
addAllMusic(mdbmaps, "Top40", "123b17da9b4f0bc068b6211be4ba6eeb", "mn0002334760")   ### [Erick]
addDiscogs(mdbmaps, "Top40", "0f4330d2f262dae9ff367c6c07b07ebd", "1551192")    ### [Temper Trap]
addAllMusic(mdbmaps, "Top40", "08de644904d4cf3503bdb90e1095eb0a", "mn0003330733")   ### [High Hopes Choir]
addAllMusic(mdbmaps, "Top40", "1b7dbb15e26894ee42a4088cd348af5c", "mn0002840885")   ### [Frankie Sandford]
addAllMusic(mdbmaps, "Top40", "7eac99ffb3a009020ae826491a7328a2", "mn0002300128")   ### [Calvin Goldspink]
addAllMusic(mdbmaps, "Top40", "a937fec3aae407f22faf4d34080e3c53", "mn0001093965")   ### [Frank Skinner]
addAllMusic(mdbmaps, "Top40", "3772bea79ec33ccc235bd5eb74ec4188", "mn0000662225")   ### [H2O]
addAllMusic(mdbmaps, "Top40", "2bb54463a4b2619f4cddb21626fd43f3", "mn0000443133")   ### [Donaeo]
addDiscogs(mdbmaps, "Top40", "5a56911227b3087c65fa0c64cfc22858", "7175745")    ### [Rani]
addAllMusic(mdbmaps, "Top40", "b44c3d5c561812173cb1fadc6bbadad0", "mn0000663567")   ### [Node]
addAllMusic(mdbmaps, "Top40", "94a91dcb424b476f725b107e3d561dd2", "mn0003085199")   ### [A Typisk]
addDiscogs(mdbmaps, "Top40", "244b104a4ecf5d29056a30135da49164", "5558809")    ### [Amanda]
addDiscogs(mdbmaps, "Top40", "9547f484677db99f07f0d2ce639fbcdb", "2644206")    ### [KESI]
addDiscogs(mdbmaps, "Top40", "cf2ec7a5779606dc225eb1ff52a125b5", "610327")    ### [Six]
addDiscogs(mdbmaps, "Top40", "682ff892fde0720742da7539be5cbfd4", "3769696")    ### [Mellemfingamuzik]
addDiscogs(mdbmaps, "Top40", "964b8471bb5b3317ff80c658789fb5c6", "2178850")    ### [Nadja Ga]
addDiscogs(mdbmaps, "Top40", "5702f798cedf95408e7998cae7160f75", "5094759")    ### [Adhd]
addDiscogs(mdbmaps, "Top40", "5f6a0d2ee2e29501ee8668e87c668653", "8290907")    ### [David O'connor]
addDiscogs(mdbmaps, "Top40", "be31dd84b00112c792101e2ed6bd66b3", "1086751")    ### [Donna & Joe]
addDiscogs(mdbmaps, "Top40", "cec35a4d1e39f4804f0329f8a891b635", "606524")    ### [Johnson]
addDiscogs(mdbmaps, "Top40", "00d39e303b291175cb7dc306b9316607", "1003686")    ### [Kyla]
addAllMusic(mdbmaps, "Top40", "c2a35b01b931ff0ea34c41bb944ed7ad", "mn0000302025")   ### [Rune Rk]
addDiscogs(mdbmaps, "Top40", "906b28e54bbb4b4cf8c91f488d978719", "5024518")    ### [Emile Kruse]
addDiscogs(mdbmaps, "Top40", "b4ac9bf7c517d486d0dfbb31267e4d3f", "6257928")    ### [Pattersutter]
addAllMusic(mdbmaps, "Top40", "ff33e0cca2bc86ca1c8aca2edf6ad878", "mn0000175401")   ### [M.V.P.]
addDiscogs(mdbmaps, "Top40", "c3eb573746b90008c43fb9ed843e01c4", "396225")    ### [U$o]
addDiscogs(mdbmaps, "Top40", "a5e6e308d31db00836b85dec9a28fac6", "120009")    ### [lum]
addDiscogs(mdbmaps, "Top40", "9cc336f08daf6b60fdc7df51c4951cd7", "771237")    ### [Soren Huss]
addDiscogs(mdbmaps, "Top40", "2f8ef80aa737b08ccbf4a4f597272dd4", "585244")    ### [Jimmy Jorgensen]
addDiscogs(mdbmaps, "Top40", "9361a0dd987d763afb7ebda59733dd4b", "164681")    ### [Michael Andr]
addDiscogs(mdbmaps, "Top40", "8c4759f7357435c07209f893bd031627", "500375")    ### [Goleo Vi]
addDiscogs(mdbmaps, "Top40", "df5925386d5e772c54fab9c8ee4afc1a", "2449670")    ### [Bjornskov]
addDiscogs(mdbmaps, "Top40", "ea6d8ca839c5321096bf52075b5913a3", "1091530")    ### [Blaes Bukki]
addAllMusic(mdbmaps, "Top40", "dff7682d6b7f9e652467f1d387940420", "mn0002116244")   ### [Michael Caro]
addDiscogs(mdbmaps, "Top40", "8293b2b928ccdcfda3a5b282235ed4ba", "489717")    ### [L.o.c.]
addAllMusic(mdbmaps, "Top40", "f101734989e2fb6b8dd4f0192a44424e", "mn0001546498")   ### [Sha]
addDiscogs(mdbmaps, "Top40", "c88659abef342c45aaeee57a7208f24b", "3871612")    ### [Jimillian]
addDiscogs(mdbmaps, "Top40", "cde97ecc04c4fb0ab06efcceb6ba1766", "393367")    ### [Monday]
addDiscogs(mdbmaps, "Top40", "f7a9e3bb639bf960c442d3854cc8e7f7", "709641")    ### [Herrelandsholdet]
addDiscogs(mdbmaps, "Top40", "9c0415d6fbfcac5c210c5cb9cd921ffd", "4181123")    ### [Fruhstuck Bei Stefanie]
addAllMusic(mdbmaps, "Top40", "56fb2c6c60bd0165fcec8d4622f92b80", "mn0002139381")   ### [Buergermeister]
addDiscogs(mdbmaps, "Top40", "bb4ac57918a08f80cb48be770d9e9979", "5134105")    ### [Cinco De Mayo]
addDiscogs(mdbmaps, "Top40", "853397c4ed3d2cd920a67cc9c17067f5", "1883733")    ### [M0]
addAllMusic(mdbmaps, "Top40", "31c4d92d280c3a99ad3e1a44e9adaf5a", "mn0003793253")   ### [Mathias Og Henriette]
addAllMusic(mdbmaps, "Top40", "0d437db44c23416f96cf5975535f221f", "mn0001963267")   ### [B.a.n.g.e.r.s]
addAllMusic(mdbmaps, "Top40", "2c045db8ead27ec90b26be684dee48a8", "mn0002329134")   ### [Blaojne]
addDiscogs(mdbmaps, "Top40", "2c045db8ead27ec90b26be684dee48a8", "100681")    ### [Blaojne]
addDiscogs(mdbmaps, "Top40", "ce330beec86bfeb671dc2a2d08562025", "1943966")    ### [Fresh]
addDiscogs(mdbmaps, "Top40", "396a9170c68a28370c6a415e5cb0acdd", "6119064")    ### [LOTTE]
addDiscogs(mdbmaps, "Top40", "112cbf6ca3a878116b385274aa05438d", "1482382")    ### [RAF]
addDiscogs(mdbmaps, "Top40", "a07c6ac1b552021b88cfd2e02e78ade3", "5072342")    ### [Laura Van Den Elzen]
addDiscogs(mdbmaps, "Top40", "dce6aefe16db0f22d8509a37392fbda0", "34669")    ### [Mia.]
addDiscogs(mdbmaps, "Top40", "a5d240665753e6ec5149a4cfec868fe1", "518132")    ### [Black Fooss]
addDiscogs(mdbmaps, "Top40", "5016655838213c82095f3cd9941ab8a8", "86482")    ### [J-luv]
addDiscogs(mdbmaps, "Top40", "014056a5b370a16da25725557f53497b", "1399728")    ### [Zipfelbuben]
addDiscogs(mdbmaps, "Top40", "2cfa11ab68fa15392a28ba3473bfafd5", "1383823")    ### [Dschungel-allstars 2009]
addDiscogs(mdbmaps, "Top40", "a9d6b7af886dba4eaa94ee0e6a0400f9", "264574")    ### [The Frank and Walters]
addAllMusic(mdbmaps, "Top40", "c74211123c6662f96a81ef1dd607613b", "mn0003687505")   ### [The Greatest Showman Ensemble]
addDiscogs(mdbmaps, "Top40", "977d879a9ef599db0179842a90eb0829", "2986568")    ### [Nik & Ras]
addAllMusic(mdbmaps, "Top40", "27b4e2613c000e26afc0dcbf6ab8137a", "mn0001560738")   ### [Milk & Honey]
addDiscogs(mdbmaps, "Top40", "27b4e2613c000e26afc0dcbf6ab8137a", "506822")    ### [Milk & Honey]
addDiscogs(mdbmaps, "Top40", "4fb5ad8251f32ebc71a4caa19a79671c", "1404551")    ### [Rald Schmitz]
addDiscogs(mdbmaps, "Top40", "35a5e92b985eb2530625c792eb8794fd", "2261153")    ### [Ell & Nikki]
addDiscogs(mdbmaps, "Top40", "108e43f1076de906cdd8ba014d4b16ff", "2068922")    ### [Bonez Mc]
addDiscogs(mdbmaps, "Top40", "225ac6dbdf0fffd4a9e113da51ca214d", "6081758")    ### [Brudi30]
addDiscogs(mdbmaps, "Top40", "446d29ccb5a2a3de8ac79412defdc3b0", "4255795")    ### [YouNotUs]
addDiscogs(mdbmaps, "Top40", "c3516450970181d37943c2cd92289bfd", "4367398")    ### [Anthony]
addDiscogs(mdbmaps, "Top40", "df31d4c938e84e9804579b48368955ba", "4453381")    ### [Eicka Jane]
addDiscogs(mdbmaps, "Top40", "5921f9df84764cd6c69a4f92a205f390", "7671579")    ### [Lolo]
addAllMusic(mdbmaps, "Top40", "9be0a055a02f9facc0f0fb7fc693c793", "mn0000753347")   ### [Snap!]
addAllMusic(mdbmaps, "Top40", "a0c07f2c1e5e8b60724c92fb83410552", "mn0003380067")   ### [Coleman Hell]
addDeezer(mdbmaps, "Top40", "768af9dd3025f4e218099084284a501b", "54647592")    ### [Mero]
addDeezer(mdbmaps, "Top40", "52752c575988eb5edea512020cec4858", "2211")    ### [Nena]
addDeezer(mdbmaps, "Top40", "e309c2e3fc905eae304b71e10e82eb99", "75530")    ### [Dream]
addDeezer(mdbmaps, "Top40", "8df03772e7cf5b3390e10574a0a90db2", "1407")    ### [Frames]
addDeezer(mdbmaps, "Top40", "40841ab93e4bc05dbd354f23e76fa1ae", "537158")    ### [Sean Og]
addDeezer(mdbmaps, "Top40", "1f50464766f3a9ce94f1754b0463ec8f", "182603")    ### [Sarah]
addDeezer(mdbmaps, "Top40", "c3e16affa11432c13e8be7cc6c8d736b", "261212")    ### [XX]
addDeezer(mdbmaps, "Top40", "fbf1e430c96f43c82045103a7c1407fd", "253597")    ### [George Murphy]
addDeezer(mdbmaps, "Top40", "03ece6e9a07c0a5915cfe676a05567a2", "543560")    ### [Kaka]
addAllMusic(mdbmaps, "Top40", "cdcec0b0558677aa394a85b105a3699c", "mn0000084053")   ### [The Jam]
addDeezer(mdbmaps, "Top40", "cdcec0b0558677aa394a85b105a3699c", "3944")    ### [The Jam]
addDeezer(mdbmaps, "Top40", "410637b6027dc82a2ce4a93308e58dca", "90401")    ### [Aura]
addAllMusic(mdbmaps, "Top40", "91f266ac5ec2839f80f07b9c5385c524", "mn0000057556")   ### [Bill & Gloria Gaither]
addDeezer(mdbmaps, "Top40", "856bd94164974e377d780028096bc128", "70715")    ### [Martin]
addDeezer(mdbmaps, "Top40", "f9793851d6cb8247b6b003a729ce9dc6", "264387")    ### [Nash]
addDeezer(mdbmaps, "Top40", "6f21b97f1010287aa78ee44d6046702f", "77424")    ### [Karen]
addDeezer(mdbmaps, "Top40", "50201d87ecab39af8f2f0aabc1cffc24", "166116")    ### [Jasmin]
addDiscogs(mdbmaps, "Top40", "f7c1455d4147cbf4dc094aa9c2a39356", "3594438")    ### [Zoo]
addAllMusic(mdbmaps, "Top40", "35d8d65ac040e8756273c2f76884e5f5", "mn0003842449")   ### [Beast Coast]
addDeezer(mdbmaps, "Top40", "c4d67d31024c80e72927ac50751161f7", "16891")    ### [Nicole]
addDiscogs(mdbmaps, "Top40", "b1b3b97ec0478725400c7c8729607be9", "3037345")    ### [Carol Anthony]
addDeezer(mdbmaps, "Top40", "154e9916e9dc4d52c6dcc9ea8c5934f7", "211032")    ### [Departure]
addDeezer(mdbmaps, "Top40", "a52581e5c20a50979faeecde53d01381", "247134")    ### [Yasmin]
addDeezer(mdbmaps, "Top40", "86058aed9a9a3acf13a9a98a36b7e7b1", "16065")    ### [Louise]
addDeezer(mdbmaps, "Top40", "f4223a2e29ad572bdd62408309749e55", "4068228")    ### [M.A.D.]
addDeezer(mdbmaps, "Top40", "dd0c42047d7063ec53285cda72c0610d", "3244121")    ### [Tough Love]
addDeezer(mdbmaps, "Top40", "e29a4723271638f34847fc292ffa48c3", "271117")    ### [James Fox]
addDeezer(mdbmaps, "Top40", "95b9824e26389cf67674f50cb242ef70", "205804")    ### [Unk]
addAllMusic(mdbmaps, "Top40", "914aff29bbb33d9bd8742b9b8d7bbb7c", "mn0001453952")   ### [Chima]
addDeezer(mdbmaps, "Top40", "914aff29bbb33d9bd8742b9b8d7bbb7c", "283635")    ### [Chima]
addDiscogs(mdbmaps, "Top40", "9545ffea8117bf350955621bae797917", "39162")   ### [Clever]
addDeezer(mdbmaps, "Top40", "9545ffea8117bf350955621bae797917", "1261918")    ### [Clever]
addDeezer(mdbmaps, "Top40", "a0eaef106d85a20c4f40697e6be08ade", "1219224")    ### [Rory & The Island]
addDeezer(mdbmaps, "Top40", "7b9aa21b6614e8381a912da660081515", "4791900")    ### [Jordan Egan]
addDeezer(mdbmaps, "Top40", "6ce57bb13155b6a976267ae7fea24a81", "5061058")    ### [The Suspects and Guests]
addDiscogs(mdbmaps, "Top40", "dc9afe97092e87a01ec4127299268b14", "97648")    ### [Tone]
addDeezer(mdbmaps, "Top40", "331b04fbaf10fec210a976adb4b1d562", "62971")    ### [Heavy]
addDeezer(mdbmaps, "Top40", "4534407ec1d2e60bd79803fcac70ddab", "5613402")    ### [Jimmy Magee]
addDeezer(mdbmaps, "Top40", "93e2f319eb80a4d47ed7830e34472de8", "164956192")    ### [Steffany Gretzinger]
addDeezer(mdbmaps, "Top40", "d027967af998dd89ee301fbfadc2a9c6", "7274")    ### [Down]
addDeezer(mdbmaps, "Top40", "aaa8882482fb1c672c7048f6ad980017", "72652862")    ### [Poppy Fields]
addDeezer(mdbmaps, "Top40", "84ea1e6425431af3e3c9132c3ef0dff2", "5359329")    ### [Presentation Secondary School Clonmel]
addDeezer(mdbmaps, "Top40", "23a4949a6ef2a59b0a1992d55a5e7119", "100307")    ### [Versatile]
addDeezer(mdbmaps, "Top40", "387f8b95bf312f4e1f06c3fa8a2e702d", "7204178")    ### [The Zog Chorus]
addDeezer(mdbmaps, "Top40", "d571e0c4adaf9473e43a55048b524222", "248033")    ### [Chuckie]
addDeezer(mdbmaps, "Top40", "83e36a7fbb4e47d90df2c7a5f41ceca5", "5283366")    ### [Avener]
addDiscogs(mdbmaps, "Top40", "d8af7b562fb5b6896e9caf208a7631fe", "2500803")    ### [Daisy Dares You]
addDeezer(mdbmaps, "Top40", "3771c6784980f14a97a79960df44683f", "7416924")    ### [Emzee A]
addDeezer(mdbmaps, "Top40", "ea418f0376767c59aea193fb88f2fbed", "11807729")    ### [Audrey Trainor]
addDeezer(mdbmaps, "Top40", "f39917252c23494a2bffc34608080e34", "4566327")    ### [Edmee]
addDeezer(mdbmaps, "Top40", "13568fea713fb70f4a72987863820910", "67704502")    ### [Paradise Hotel 2019]
addDeezer(mdbmaps, "Top40", "a552723762e26b8bf2ff3cf00abb742e", "76729")    ### [Vera]
addDeezer(mdbmaps, "Top40", "e20047c96a03c801a7a761d639303baa", "105146")    ### [Christina]
addDeezer(mdbmaps, "Top40", "747e53fad10278c0be9f12e5064e1a8a", "9435020")    ### [Ung Cezar]
addDeezer(mdbmaps, "Top40", "59755826fe21e0d913917f38e519bd97", "4037981")    ### [Trinidad James]
addDiscogs(mdbmaps, "Top40", "b091868e0d9e66c4f3cfc3d57c0e15c1", "6754464")    ### [Pauline]
addDiscogs(mdbmaps, "Top40", "fdef672eb79aa1e42d68559231f9e91a", "2891027")    ### [Contiez]
addDiscogs(mdbmaps, "Top40", "376424d25e7c667172ea571a9bd5ecf7", "34061")   ### [DT8]
addDeezer(mdbmaps, "Top40", "376424d25e7c667172ea571a9bd5ecf7", "16653")    ### [DT8]
addDeezer(mdbmaps, "Top40", "298dc80023436fd7d1327755dc0f3aea", "5406531")    ### [Virginia To Vegas]
addAllMusic(mdbmaps, "Top40", "df7a414766c07b227b947fe19d735cca", "mn0000527296")   ### [P.I.]
addDiscogs(mdbmaps, "Top40", "9ffc319f3524de0eb5857b3a041e89d8", "4115550")   ### [Rin]
addDiscogs(mdbmaps, "Top40", "6c757d349b981f59ce15f8050351c33b", "286345")   ### [Androids]
addDiscogs(mdbmaps, "Top40", "2580311e731b9bca913825f23a494d96", "4511617")    ### [Alina]
addDeezer(mdbmaps, "Top40", "f8c6ee3fdec7c8e8d68666db7bd07a5b", "1217914")    ### [Zion & Lennox]
addDeezer(mdbmaps, "Top40", "04b0ea321844b974fa623332ec9e81a8", "1234739")    ### [Nadine]
addDiscogs(mdbmaps, "Top40", "5b813121e727842a70d17d40279c8060", "3915565")    ### [Tieks]
addDeezer(mdbmaps, "Top40", "e6852c19581c5c2842941d6585011de8", "507529")    ### [New Sunset Hotel]
addDeezer(mdbmaps, "Top40", "1ee6a7f79efe291ad8bc9ba50f14ee39", "110592")    ### [Tarja]
addDeezer(mdbmaps, "Top40", "f70f7318448d90d9e2648fa729d0ccd9", "7569860")    ### [Mendoza]
addDeezer(mdbmaps, "Top40", "87019c83d1d9002251d5e0123d73de2f", "4158299")    ### [Ladywell Primary School]
addDeezer(mdbmaps, "Top40", "70fd12ba96ac516bed74bb1918828f96", "7215388")    ### [Wealdstone Raider]
addDeezer(mdbmaps, "Top40", "18a4ef47c6e65a8cd0686f17dcb5a2d6", "458443")    ### [Mike Sheridan]
addDeezer(mdbmaps, "Top40", "b6f0eeb7241d52315a563b41a83f03ab", "12544804")    ### [LIV'n'G]
addDeezer(mdbmaps, "Top40", "3203329df817eab4fd383d2515def0c7", "61557012")    ### [RSPB]
addDeezer(mdbmaps, "Top40", "3de5a67ac001eda9da009b4269f64d72", "84950802")    ### [Tobi & Manny]
addDeezer(mdbmaps, "Top40", "8476f77796a006e9d4cbafa3404aebac", "55142")    ### [Virgo]
addDeezer(mdbmaps, "Top40", "7befb4ecd12fe5e883dc1785be298dab", "134048")    ### [Fabric Bent]
addDiscogs(mdbmaps, "Top40", "9d224a59b3df753402c55e6974c203b5", "1335584")   ### [Eddy Bar]
addDeezer(mdbmaps, "Top40", "9d224a59b3df753402c55e6974c203b5", "152363")    ### [Eddy Bar]
addDeezer(mdbmaps, "Top40", "c039dcafb38ef291cd3576f6f2d96498", "14492701")    ### [Ransom]
addDeezer(mdbmaps, "Top40", "538d641f7c607731ce3c97de84888ddc", "10904")    ### [Trooper]
addAllMusic(mdbmaps, "Top40", "576086654e3dac2a5660989c51366659", "mn0000043629")   ### [Bushido Produziert Sonny Black]
addDeezer(mdbmaps, "Top40", "576086654e3dac2a5660989c51366659", "143975")    ### [Bushido Produziert Sonny Black]
addDeezer(mdbmaps, "Top40", "d4e420120a039e03455ee8536740a0a0", "2128")    ### [Goldfinger]
addDeezer(mdbmaps, "Top40", "d9de59336d2d408662aa03ad71e870b1", "4036827")    ### [Andreas Kummert]
addDeezer(mdbmaps, "Top40", "5edc174fb5fefd278c05b53dbd1bd2e0", "14740207")    ### [Prince Karma]
addDeezer(mdbmaps, "Top40", "603304c35dea9d2995f5ea307fc3e437", "5361924")    ### [Judith Van Hel]
addDeezer(mdbmaps, "Top40", "18e08ec319bb5434ce1be10ede35b88d", "15224255")    ### [KMN Gang]
addDeezer(mdbmaps, "Top40", "043c614068c79f134a62512dd92dd205", "4747977")    ### [Meltem Acikgoz]
addDeezer(mdbmaps, "Top40", "f97c6800d97bc9b645de82758b1b616e", "4844456")    ### [Minds Of 99]
addDeezer(mdbmaps, "Top40", "1f84610afa18b37896d9cb8c3ffb929a", "9356090")    ### [POL1Z1STENS0HN]
addAllMusic(mdbmaps, "Top40", "7dbc379090f7843f537ea0df634b1aad", "mn0000748795")   ### [Shifty]
addDeezer(mdbmaps, "Top40", "7dbc379090f7843f537ea0df634b1aad", "6677")    ### [Shifty]
addDeezer(mdbmaps, "Top40", "a17bf82b0c99a5d6a9fdb61e1cba475b", "567150")    ### [Leanne Moore]
addDeezer(mdbmaps, "Top40", "3a9adb2ba05512379660af30c4499d9e", "255246")    ### [Seneca]
addDeezer(mdbmaps, "Top40", "18091df6740eb23606268e1fe70a832e", "262682")    ### [Dustin The Turkey]
addDeezer(mdbmaps, "Top40", "ab0121020ea6f52f5dcc2b0ad8417684", "1569593")    ### [Kanyu Tree]
addDeezer(mdbmaps, "Top40", "1feb0b498fd6b76e794cd4994499bd66", "471918")    ### [Scuba Dice]
addDeezer(mdbmaps, "Top40", "3531e20a4eaab8cd5b085fc9605f6fba", "261093")    ### [Martin Staunton Band]
addDeezer(mdbmaps, "Top40", "8d229bc5952f811627abcecf1a1d9f6d", "404641")    ### [Soul Control]
addAllMusic(mdbmaps, "Top40", "c87cad8f7c42f2e678a0c18782869407", "mn0001951902")   ### [Conway Sisters]
addDeezer(mdbmaps, "Top40", "c87cad8f7c42f2e678a0c18782869407", "5441329")    ### [Conway Sisters]
addDeezer(mdbmaps, "Top40", "bf39144a5e6eb48401d5732ec4edddf4", "15811")    ### [Delirious]
addDeezer(mdbmaps, "Top40", "d1029792b9078894873b1965b343540c", "262762")    ### [Walls]
addDiscogs(mdbmaps, "Top40", "009bc268f4161c893c3469d0f475fa03", "393367")    ### [The Revs]
addDeezer(mdbmaps, "Top40", "0166cabfe7594a261af1250ad91c5ebe", "3039")    ### [Di-rect]
addDeezer(mdbmaps, "Top40", "79947bd82f85385cf579852761e3bcca", "5432668")    ### [Grafa]
addAllMusic(mdbmaps, "Top40", "996dc2db2e60fc3de7a14828df795171", "mn0003729938")   ### [Z. E.]
addDeezer(mdbmaps, "Top40", "73ea300db5aa1c2154adfe42186a887e", "5418")    ### [Ark]
addDeezer(mdbmaps, "Top40", "c0e4ba1ede3532ccfde5b29e8aacc50b", "789194")    ### [Skryabin]
addDeezer(mdbmaps, "Top40", "12a8a35953b0633264cd439f7dc723ad", "13244969")    ### [Elias]
addDiscogs(mdbmaps, "Top40", "60044c2592b77d0e4a7d4df0d5fdcbba", "908078")    ### [Julia Savicheva]
addDiscogs(mdbmaps, "Top40", "c8c1ade4a56b82b143856c2c590e3fc1", "369368")    ### [Gola]
addDiscogs(mdbmaps, "Top40", "8422de1228cdf9aa9ee5888a858a98b1", "8707189")   ### [Mihaela Fileva]
addDeezer(mdbmaps, "Top40", "8422de1228cdf9aa9ee5888a858a98b1", "5471912")    ### [Mihaela Fileva]
addDeezer(mdbmaps, "Top40", "8e0a54291a8ae6add5c89ae6bd744591", "2077221")    ### [Pavell]
addDeezer(mdbmaps, "Top40", "5793bffe6cf2b42fbcf5ecc4cdf66911", "4796386")    ### [Venci Venc']
addDeezer(mdbmaps, "Top40", "e4a1ed07636cc650fcd92f2ee99a44fe", "1288678")    ### [Lazza]
addDeezer(mdbmaps, "Top40", "c29dd858b42c5e8d6caecd0302c47dfe", "9060182")    ### [Hov 1]
addDeezer(mdbmaps, "Top40", "981871e41d004689023b3867cca2003c", "4239461")    ### [Maan]
addDeezer(mdbmaps, "Top40", "6eb8df7aacfb80b8a5aefbb8155dc13a", "247284")    ### [George Sampanis]
addDeezer(mdbmaps, "Top40", "daf03202751046b2d8cc380e7535cc39", "529")    ### [Raphael]
addDeezer(mdbmaps, "Top40", "8a65b04059e971e76dc736fcf1ab4dd2", "13440379")    ### [Nino]
addDeezer(mdbmaps, "Top40", "1533082f4491090f90aa6f086be1b2b6", "4018571")    ### [Arif]
addDeezer(mdbmaps, "Top40", "3e5169154b281b19469ce1fb806a3919", "795146")    ### [Machete]
addDeezer(mdbmaps, "Top40", "cd5004d3b03507b93a5154c8f703ffb1", "1358419")    ### [KataStrofe]
addDeezer(mdbmaps, "Top40", "f4c09b9cc9b7184e363a22d1a7565ef9", "176639")    ### [Nitro]
addDeezer(mdbmaps, "Top40", "a2f3f9edf03e4bf85df289edf3084780", "58916452")    ### [Dnoteondabeat]
addDeezer(mdbmaps, "Top40", "8bbd146262d01da925cec72df725f3cf", "11937599")    ### [Blockparty]
addDeezer(mdbmaps, "Top40", "0d866c18af54941ec8f33ce3df4be902", "3848")    ### [Elize]
addDeezer(mdbmaps, "Top40", "0c6423fdddb18f976061565d288baded", "1005065")    ### [Como]
addDiscogs(mdbmaps, "Top40", "0f17d781876da4732563e43039aaaa8b", "448654")   ### [L5 (Popstars)]
addDeezer(mdbmaps, "Top40", "ce3fbaee597ec5a524a308b812541f56", "214934")    ### [Rebecca]
addDeezer(mdbmaps, "Top40", "ff69c3ee307619b22c2ee2969aea521d", "11328")    ### [Dj Jose]
addDiscogs(mdbmaps, "Top40", "8f7b584aa940b5b8d46129956fee0c52", "2250857")   ### [Vremia I Steklo]
addDeezer(mdbmaps, "Top40", "8f7b584aa940b5b8d46129956fee0c52", "4878214")    ### [Vremia I Steklo]
addDeezer(mdbmaps, "Top40", "1af733e9abe9956b5d2ef9fdbb1ad764", "537741")    ### [Grigoriy Leps]
addDeezer(mdbmaps, "Top40", "a2ee02d0ff400d2ec0d77d2aae27d098", "164423")    ### [Stan]
addDeezer(mdbmaps, "Top40", "cb1a4cdd6178852f36aba64d12602d1a", "281011")    ### [India Martinez]
addDeezer(mdbmaps, "Top40", "2fbb9be1f137d28d5d01a88b2a61c3a6", "1366660")    ### [Taisiya Povaliy]
addDeezer(mdbmaps, "Top40", "43a4a6af3a411e6c9a24170f30c2815b", "13612387")    ### [Maneskin]
addDeezer(mdbmaps, "Top40", "9318518ff509296fc43d312a9d65ff56", "101471")    ### [Pegasus]
addDeezer(mdbmaps, "Top40", "0abaf01f580932db3646af80a6023868", "1710654")    ### [Potap & Nastia]
addAllMusic(mdbmaps, "Top40", "e3b7acbc2078d13eafa2a765ce38115a", "mn0002080697")   ### [Luttenberger & Klug]
addDeezer(mdbmaps, "Top40", "e3b7acbc2078d13eafa2a765ce38115a", "1495367")    ### [Luttenberger & Klug]
addAllMusic(mdbmaps, "Top40", "760ffac9c884a2e8fabc13d4eb617e75", "mn0002744178")   ### [Kevin]
addDeezer(mdbmaps, "Top40", "760ffac9c884a2e8fabc13d4eb617e75", "70615702")    ### [Kevin]
addAllMusic(mdbmaps, "Top40", "c4bfdaef6a75c5b92d4314bed5324ac9", "mn0003060329")   ### [V. Meladze]
addDeezer(mdbmaps, "Top40", "c4bfdaef6a75c5b92d4314bed5324ac9", "4251519")    ### [V. Meladze]
addDiscogs(mdbmaps, "Top40", "c4bfdaef6a75c5b92d4314bed5324ac9", "753179")
addDeezer(mdbmaps, "Top40", "299683e0e48d3508d34c59fd60d1c969", "6138354")    ### [Lora Karadzhova]
addAllMusic(mdbmaps, "Top40", "9277fef69257e9c368980e44addab30d", "mn0002598754")   ### [Tove Ostman Styrke]
addDiscogs(mdbmaps, "Top40", "9277fef69257e9c368980e44addab30d", "2023759")    ### [Tove Ostman Styrke]
addAllMusic(mdbmaps, "Top40", "a2b07050b3a3158923a626ecd79a9a99", "mn0002764822")
addDiscogs(mdbmaps, "Top40", "a2b07050b3a3158923a626ecd79a9a99", "2241934")   ### [Maksim Barskih]
addDeezer(mdbmaps, "Top40", "a2b07050b3a3158923a626ecd79a9a99", "1365309")    ### [Maksim Barskih]
addDeezer(mdbmaps, "Top40", "f5cfbe3d9410cd26c9f5db2f5d00e2af", "64976")    ### [Yasin]
addAllMusic(mdbmaps, "Top40", "2da7b3559029e16c501f19f2ca9e1f3d", "mn0001509405")   ### [Valeria]
addDiscogs(mdbmaps, "Top40", "2da7b3559029e16c501f19f2ca9e1f3d", "907332")    ### [Valeria]
addDeezer(mdbmaps, "Top40", "71495276459b2c8d9319a4f23736b1fe", "4188612")    ### [NeAngely]
addDiscogs(mdbmaps, "Top40", "df1a1cf249844e537d5f8e301999f050", "508430")   ### [Romeo]
addDeezer(mdbmaps, "Top40", "df1a1cf249844e537d5f8e301999f050", "66510")    ### [Romeo]
addDiscogs(mdbmaps, "Top40", "ddcae8e624e066c57c1a76f7a50c3be5", "5163802")   ### [Pianoboy]
addDeezer(mdbmaps, "Top40", "ddcae8e624e066c57c1a76f7a50c3be5", "4444748")    ### [Pianoboy]
addDeezer(mdbmaps, "Top40", "57dd82cd58ebc1ee03495e7111f68837", "69824902")    ### [Adel]
addDeezer(mdbmaps, "Top40", "9790bfbf128c234ff3dfe9734a29112d", "3097771")   ### [S. Tarabarova]
addDiscogs(mdbmaps, "Top40", "9790bfbf128c234ff3dfe9734a29112d", "3180922")    ### [S. Tarabarova]
addAllMusic(mdbmaps, "Top40", "e87ca03c7907697d6b985a36efec96b3", "mn0000907539")   ### [Dolcenera]
addDeezer(mdbmaps, "Top40", "e87ca03c7907697d6b985a36efec96b3", "77036")    ### [Dolcenera]
addDiscogs(mdbmaps, "Top40", "1a1c4cd2bf4d0da1c3ac37eed6c5e6dd", "349539")   ### [Seer]
addDiscogs(mdbmaps, "Top40", "479632c7dad0bc6491b1dc428c2f384b", "323925")   ### [Carola]
addDeezer(mdbmaps, "Top40", "479632c7dad0bc6491b1dc428c2f384b", "9621")    ### [Carola]
addDeezer(mdbmaps, "Top40", "1018fb9997d689d2cfbeb250cba1e0fc", "8731")    ### [Erika]
addAllMusic(mdbmaps, "Top40", "74d19220f1e9035dc9be17ab0fa7d496", "mn0003059139")   ### [Gradysi]
addDeezer(mdbmaps, "Top40", "5eb7994fa8fc5d454eb9975e8d0982b3", "8399")    ### [Wigwam]
addDeezer(mdbmaps, "Top40", "ec7bcd247bd91b161fd3e236faf60113", "8359")    ### [Laura]
addDeezer(mdbmaps, "Top40", "8cc666d82558e9a3801428ce9377e0cc", "243610")    ### [Eri Esittajia]
addDeezer(mdbmaps, "Top40", "14035f38e39640c7b8a126848f15a17f", "6329")    ### [Belinda]
addDeezer(mdbmaps, "Top40", "ace5274eac9ac83ecc8be584f2f89678", "389681")    ### [Jezabels]
addDeezer(mdbmaps, "Top40", "f808aec316082282ad04c543ea7293fc", "1532710")    ### [Gossling]
addDeezer(mdbmaps, "Top40", "8248a0d3d76dd9b6e989ed30758e1fdb", "7727636")    ### [Young Tapz]
addDeezer(mdbmaps, "Top40", "9abfa0a95115150fa9854d05888b30ad", "323")    ### [T-pain]
addDeezer(mdbmaps, "Top40", "9eb2641c9d801ac6a24cd6d40a99cfcc", "1226315")    ### [J.williams]
addDeezer(mdbmaps, "Top40", "d8efa30039a62c84a5cd7aae07baaed4", "9388984")    ### [Jess & Matt]
addDeezer(mdbmaps, "Top40", "3b51465a51f53de3f9406c8c4084b9eb", "4771547")    ### [Hannah]
addDeezer(mdbmaps, "Top40", "ff33e0cca2bc86ca1c8aca2edf6ad878", "146348")    ### [M.V.P.]
addDeezer(mdbmaps, "Top40", "7670c11b0a85e757c3cd25e1cb4f0d2b", "121457")    ### [Dakota]
addDeezer(mdbmaps, "Top40", "aca525c6189a7e1746b237ac3c8e75f1", "241381")    ### [OpShop]
addDeezer(mdbmaps, "Top40", "75addee6118eeea8b15dfa7d27a7bc56", "132842")    ### [Gin]
addDeezer(mdbmaps, "Top40", "40841ab93e4bc05dbd354f23e76fa1ae", "537158")    ### [Sean Og]
addDeezer(mdbmaps, "Top40", "8a8901e987f198d467979ba02675bf2a", "480089")    ### [360]
addDeezer(mdbmaps, "Top40", "8b8ee2fa941154779a27093f715c82c1", "2476")    ### [Whitlams]
addDeezer(mdbmaps, "Top40", "f1e6489254cab6f1fcbe17478f66812d", "9164")    ### [Presets]
addDeezer(mdbmaps, "Top40", "4cbe8a7a881e6bbb1b2ad97de6caa28c", "76000")    ### [Stellar]
addDeezer(mdbmaps, "Top40", "33db69043854dc72c7ae28f79e66b295", "134078")    ### [Bleeders]
addDeezer(mdbmaps, "Top40", "5fbe502afaeea5cd3894fbbacc76d0f0", "72186")    ### [Frontline]
addAllMusic(mdbmaps, "Top40", "8776b0955cc02e70c84df62f5d08783c", "mn0000288296")   ### [k-os]
addDeezer(mdbmaps, "Top40", "8776b0955cc02e70c84df62f5d08783c", "1666")    ### [k-os]
addDeezer(mdbmaps, "Top40", "34110bca84cc003cb7f9663d441b97f3", "404975")    ### [Swiss]
addDeezer(mdbmaps, "Top40", "cf724ae1b389235411e9848ab2492b86", "1263309")    ### [Jason Kerrison]
addDeezer(mdbmaps, "Top40", "92b769e55de140d27e26b4f52382dc3c", "4344203")    ### [JGeek]
addDeezer(mdbmaps, "Top40", "fe6ff35dd5203e256dddc02fa796c5f6", "5611287")    ### [Arise Church]
addDeezer(mdbmaps, "Top40", "b877108ea31ee744be61d4cf33ddf382", "4423398")    ### [Lavina Williams]
addDeezer(mdbmaps, "Top40", "fd41667c368acecbf8d672b7afbed008", "476858")    ### [Aftermath]
addDeezer(mdbmaps, "Top40", "3682bbc007fa90e18375fa5edb0c37b3", "76360")    ### [Toya]
addDeezer(mdbmaps, "Top40", "4157b21887eedb61b04d15ae42183eae", "4757049")    ### [Dolla & Skeet]
addDeezer(mdbmaps, "Top40", "a663ace511580671d69a6b3f916fe919", "1116771")    ### [Catfish & The Bottlemen]
addDiscogs(mdbmaps, "Top40", "dc9afe97092e87a01ec4127299268b14", "97648")    ### [Tone]
addAllMusic(mdbmaps, "Top40", "c0051ae12f6048a807213c44b434aa1d", "mn0003348925")   ### [Jennifer Lawrence]
addDeezer(mdbmaps, "Top40", "c0051ae12f6048a807213c44b434aa1d", "7196728")    ### [Jennifer Lawrence]

mdbmaps["Top40"].save()

## BillboardYE

In [ ]:
addDeezer(mdbmaps, "BillboardYE", "52a0965785c941aa958b64dc50f3d972", "176485")    ### [Fuego]
addDeezer(mdbmaps, "BillboardYE", "ccc783c2cd738ace20a5315e7d43e547", "3069")    ### [Rakim]
addDeezer(mdbmaps, "BillboardYE", "edac53ed142896746b3bd4c681c12f64", "347454")    ### [Tim / Berg]
addDeezer(mdbmaps, "BillboardYE", "a7fa9eedf990ab73ba0d7e60db65c95d", "9081")    ### [Eric Prydz Vs. Floyd]
addDeezer(mdbmaps, "BillboardYE", "fbe56cc641e9d04e1b4fcdd924a9c8cc", "318079")    ### [Carlos y Alejandra]

addDeezer(mdbmaps, "BillboardYE", "37f8c0772c31a769f56239563ad7b458", "6500875")    ### [Anuel]
addDeezer(mdbmaps, "BillboardYE", "1c014b1998bb4fab0322f211eec0c611", "445119")    ### [La Adictiva]
addDeezer(mdbmaps, "BillboardYE", "17d5bdd02ad4dee110445a03f919b8ac", "57843")    ### [Marlon]
addDeezer(mdbmaps, "BillboardYE", "33ab8be62d65b7d75df716432b86336f", "240427")    ### [Sky]
addDeezer(mdbmaps, "BillboardYE", "c133daae0111978ba09001d2e7de72f1", "5611")    ### [Willie Colon]
addDeezer(mdbmaps, "BillboardYE", "6510784503b99747dfee77f5a079476d", "50683152")    ### [Banda MS de Sergio Lizarraga
addDeezer(mdbmaps, "BillboardYE", "3f5ee815ed36b113db281ce7ed332da4", "71922")    ### [Dino]
addDeezer(mdbmaps, "BillboardYE", "8046ee19ecf0576f13f6520114447c52", "325722")    ### [James Lloyd]
addDeezer(mdbmaps, "BillboardYE", "73a58c444f3a947331cc0f62fff0cfd5", "628982")    ### [Bastien Laval]
addDeezer(mdbmaps, "BillboardYE", "e28b088f9d859934e4e4cbb39c431199", "4948050")    ### [Il Pomo d'Oro]
addDeezer(mdbmaps, "BillboardYE", "ef95f8a3355a4bbe43fceb8f72be7f9f", "145155")    ### [Divino]
addDeezer(mdbmaps, "BillboardYE", "adc972062945092b4343017780b6c242", "4065944")    ### [Hnos. Vega Jr.]
addDeezer(mdbmaps, "BillboardYE", "8e9f75d573db0b3fd1c5e8b48e3301cc", "4330924")    ### [Codigo FN]

addAllMusic(mdbmaps, "BillboardYE", "0daf21abe59aac57d10e0fe76f304077", "mn0002686289")   ### [Ben Williams]
addDeezer(mdbmaps, "BillboardYE", "0daf21abe59aac57d10e0fe76f304077", "485014")    ### [Ben Williams]
addDeezer(mdbmaps, "BillboardYE", "093b82ff928d96dddbc505cd2f1ddb1f", "469114")    ### [AB Quintanilla]
addDeezer(mdbmaps, "BillboardYE", "d98278206c2f7c2e6232599ee9cb6bc0", "145593")    ### [Adam Holzman]
addDeezer(mdbmaps, "BillboardYE", "dd88591508a27fecc4b05c47f7d9698f", "108626102")    ### [James F. Twyman]

mdbmaps["BillboardYE"].save()

## Billboard

In [ ]:
def addSpotifyViralExtras(mdbmaps):
    addDiscogs(mdbmaps, "SpotifyViral", "19b7726bba940477044f2b3120d7aead", "5383065")   ### [AminÃ©]
    addAllMusic(mdbmaps, "SpotifyViral", "19b7726bba940477044f2b3120d7aead", "mw0003059654")   ### [AminÃ©]
    addDiscogs(mdbmaps, "SpotifyViral", "c254d2ae2ba44015bdcd825b1f0890ac", "1887511")   ### [CyHi The Prynce]
    addAllMusic(mdbmaps, "SpotifyViral", "c254d2ae2ba44015bdcd825b1f0890ac", "mn0002414749")   ### [CyHi The Prynce]
    addDiscogs(mdbmaps, "SpotifyViral", "173b571160d7fb7a36ef9797cd1103d2", "5734491")   ### [D-METAL STARS]
    addAllMusic(mdbmaps, "SpotifyViral", "173b571160d7fb7a36ef9797cd1103d2", "mn0003549702")   ### [D-METAL STARS]    
    
    
def addSpotifyExtras(mdbmaps):    
    addAllMusic(mdbmaps, "Spotify", "2c7704262fe420489299bef485e21f10", "mn0003130756")   ### [VIC MENSA]
    addDeezer(mdbmaps, "Spotify", "2c7704262fe420489299bef485e21f10", "1044777")    ### [VIC MENSA]
    addDeezer(mdbmaps, "Spotify", "920c63b4fcad01b381d646d6b8b365bc", "557185")    ### [NEW CITY]
    addDeezer(mdbmaps, "Spotify", "26154d0d5295a4f04f64f0b7f1c0e067", "1521325")    ### [DiRTY RADiO]
    addDeezer(mdbmaps, "Spotify", "845f615d3f2c32f4335fe39bef730d05", "566999")    ### [Rv]
    addAllMusic(mdbmaps, "Spotify", "ae62113cf8a3f76960b374a1aefffc3d", "mn0000591459")   ### [TiÃ«sto]
    addDeezer(mdbmaps, "Spotify", "ae62113cf8a3f76960b374a1aefffc3d", "434")    ### [TiÃ«sto]
    addDeezer(mdbmaps, "Spotify", "a28cc660d82570830e3d7a652b4dda50", "5355839")    ### [DJ JEEZY]
    addDeezer(mdbmaps, "Spotify", "802f1e688970ffdae3036900c02a48d4", "46853482")    ### [ToTheMoon]
    addDeezer(mdbmaps, "Spotify", "bf1a1a76b86020560d80d25939467283", "1345313")    ### [LIBA]
    addDeezer(mdbmaps, "Spotify", "172b31efbafca7f93222e89535823c71", "6713359")    ### [ValsBezig]
    addDeezer(mdbmaps, "Spotify", "5c7f68b5c9d98dc02f75b7a052c655d6", "14251")    ### [BangBros]
    addDeezer(mdbmaps, "Spotify", "efa40ffa68a7c9b3d9b44b335959f5be", "10011140")    ### [Wikw]
    addDeezer(mdbmaps, "Spotify", "3cc604c59cfb71a740983a330b874cae", "10807780")    ### [DYSTINCT]
    addDeezer(mdbmaps, "Spotify", "84a5e716bdce0b761cdd9b3995565cfe", "7639780")    ### [RUGGED]
    addDeezer(mdbmaps, "Spotify", "8dda0d0fa1201255d30cddd370b4527a", "59466262")    ### [LUNA KI]
    addDeezer(mdbmaps, "Spotify", "6deb82dda4cd26bec6b94e8e7e970b7f", "14364799")    ### [FlowZeta]
    addDeezer(mdbmaps, "Spotify", "c4596c33a8f464e29b07bbb0f1c7c0ff", "10342534")    ### [Poorchoice]
    addDeezer(mdbmaps, "Spotify", "841b83ac5259a4b2bd769695adf387f8", "325115")    ### [Nosi]
    addDeezer(mdbmaps, "Spotify", "cc6f52fa7b2368269985744be5214b1b", "1517421")    ### [DENZO]
    addDeezer(mdbmaps, "Spotify", "28d0487969b481c095a44464af37a9a9", "67450512")    ### [GS]
    addDeezer(mdbmaps, "Spotify", "fc2b87972f8a8ec07b311e59825be184", "122381472")    ### [Dioscures]
    addDeezer(mdbmaps, "Spotify", "b253d9e0f63cfe594ad59c1573c14b64", "5794446")    ### [AXEC]
    addDeezer(mdbmaps, "Spotify", "8d3d126619a7a4757f4f2c248ade9aa6", "61471392")    ### [SKiNNY BARBER]
    addDeezer(mdbmaps, "Spotify", "9e153fd9c556ac76075c23a4e2401272", "15644")    ### [Dan Barta]
    addDeezer(mdbmaps, "Spotify", "3347bd679e845d9cb60181daf815d879", "1442177")    ### [LYSA]
    addDeezer(mdbmaps, "Spotify", "75074c0c9d0202a88c21f866c9debe9d", "4646560")    ### [Apu]
    addDeezer(mdbmaps, "Spotify", "8efaa0f6bcfb4d9d2bf43d29e47ca19d", "7637470")    ### [VALMAR]
    addDeezer(mdbmaps, "Spotify", "4645b07014e7de28b3eb4fdc2b09ece4", "117840852")    ### [SHE]
    addDeezer(mdbmaps, "Spotify", "e68cb9c121fd6da74857936c67f0f06f", "1354350")    ### [Kid1]
    addDeezer(mdbmaps, "Spotify", "68496911494cdd8e63039cccbfac733d", "4560405")    ### [JORI SJÖROOS]
    addDeezer(mdbmaps, "Spotify", "7027fdc8d392b9a06c57a4566b144445", "96643")    ### [ODE]
    addDeezer(mdbmaps, "Spotify", "d3555bd07f974448684672f85aabef02", "63712282")    ### [ZÖE]
    addDeezer(mdbmaps, "Spotify", "6d4ca17986eee42444e86dbfc7e0b137", "134065")    ### [ZENA]
    addDeezer(mdbmaps, "Spotify", "c17577a0af8e0ba087f802001b600b9b", "77554172")    ### [fuse.]
    addDeezer(mdbmaps, "Spotify", "9fa8323b0c2feb08be976c7e6b97d37c", "85045172")    ### [VICTORIA]
    addDeezer(mdbmaps, "Spotify", "78da7b843a50d1bed3ddfe556eb661a7", "98175162")    ### [Bruno x spacc]
    addAllMusic(mdbmaps, "Spotify", "3498367b46c2f47cb70bdb9fb75a2cd2", "mn0003597150")   ### [Zävodi]
    addDeezer(mdbmaps, "Spotify", "5a54b188dfb176fe6d47912e30c39e84", "12790253")    ### [Felix & Dean]
    addDeezer(mdbmaps, "Spotify", "103972014ba658abe4b40a4fa9b74495", "507314")    ### [4Korners]
    addDeezer(mdbmaps, "Spotify", "6d61811c24db9279f28e1e1921619aff", "208250")    ### [HAYES]
    addDeezer(mdbmaps, "Spotify", "0a9d2e9a751ac074778eefd4fe0449ca", "113731272")    ### [EMIR]
    addDeezer(mdbmaps, "Spotify", "2f351c66d2fa77578e1ce385cccf6578", "13609323")    ### [LayLay]
    addDeezer(mdbmaps, "Spotify", "966bac2b2a33212f907f0ead939b7f9d", "3713431")    ### [SunCity]
    addDeezer(mdbmaps, "Spotify", "d80c0934559a8ccfb558f1cbd8f7f361", "72022")    ### [EMMY]
    addDeezer(mdbmaps, "Spotify", "890fdf97a97559703a6b0751e2d15048", "114020712")    ### [IBO]
    addDeezer(mdbmaps, "Spotify", "90e4f38de02fe89733974a1e1538f4a0", "1112997")    ### [NALO]
    addDeezer(mdbmaps, "Spotify", "195e988c3cd6f12d1906aaeb5111d36c", "12089130")    ### [CASPR]
    addDeezer(mdbmaps, "Spotify", "ef407ddceb97d8e618394a0e7b2947ae", "12188928")    ### [Vkation]
    addDeezer(mdbmaps, "Spotify", "a728fbfe6dc4de2bd87407d3d3e23e76", "5653407")    ### [Meso]
    addDeezer(mdbmaps, "Spotify", "6a1eff593643f6b231caf35224485374", "11478356")    ### [Mad.S]
    addDeezer(mdbmaps, "Spotify", "ea00bcea92149aec2ba35641d5e25789", "9719800")    ### [Solguden & Mannen]
    addDeezer(mdbmaps, "Spotify", "4d68a565a29117574836e3a7f1760d22", "327360")    ### [Madd]
    addDeezer(mdbmaps, "Spotify", "33d7726b3779bb6007b25e5608bd8bcc", "70773932")    ### [Afroto]
    addDeezer(mdbmaps, "Spotify", "422213e84cc37e39b83ce21c3e94137d", "342125")    ### [ISSAM]
    addDeezer(mdbmaps, "Spotify", "b46e5c7b15b7427c94a1903b3a9f8c83", "11419694")    ### [Tflow]
    addDeezer(mdbmaps, "Spotify", "9b3766afd03eca31d6b33959b5e6b54b", "117480")    ### [Majid Almohandis]
    addDeezer(mdbmaps, "Spotify", "9e62594b9185e8fe3e2f4ee7c2380b6c", "13272315")    ### [L7or]
    addDeezer(mdbmaps, "Spotify", "4fd56f6c7826769bf01a0687ca01ca46", "4341385")    ### [Mohamed AlSalim]
    addDeezer(mdbmaps, "Spotify", "62aafefe147d236674ea7b6c31e064d9", "2469601")    ### [Fnaire]
    addDeezer(mdbmaps, "Spotify", "dd6022f7bc11f42cfe10f37db3ab3805", "14943131")    ### [ARNOLD]
    addDeezer(mdbmaps, "Spotify", "c5de08852f51c6d4b64443c6ff28b064", "14999849")    ### [micem]
    addDeezer(mdbmaps, "Spotify", "44ca8fe2e5b74e945d51c4be314345f0", "7107933")    ### [Lil Eazy]
    addDeezer(mdbmaps, "Spotify", "25ebd6fe7a146bbeba83a9430a18d22e", "83499152")    ### [DJ LYAN]
    addDeezer(mdbmaps, "Spotify", "402f040af4a7667a87491c3f81fc51dd", "13900015")    ### [FEYI]
    addDeezer(mdbmaps, "Spotify", "64fb5161ab6bf6bd9685ec461218f27d", "5414687")    ### [TYTE]
    addDeezer(mdbmaps, "Spotify", "0d6f38579b8d7a6d3b39abe2cc673918", "6051522")    ### [Ebx Music]
    addDeezer(mdbmaps, "Spotify", "b0de7a947f0ec3f64ea69422d7737a09", "114617632")    ### [Barry]
    addDeezer(mdbmaps, "Spotify", "e74c665fab1655c7214d088b1cfcc631", "59338402")    ### [Gher]
    addDeezer(mdbmaps, "Spotify", "4472f33a7bcc305660f2459c40ee56b0", "2478401")    ### [ansis]
    addDeezer(mdbmaps, "Spotify", "e0c4033e2f7913a914c7920d564c9e2b", "125477")    ### [Livi]
    addDeezer(mdbmaps, "Spotify", "09b13dabe4b1cc54728627b6958a35d2", "157690")    ### [säm]
    addDeezer(mdbmaps, "Spotify", "71ad6ba0221a38b61929597170f931b8", "14003")    ### [Ania Dabrowska]
    addDeezer(mdbmaps, "Spotify", "a3765e586e042254b4a369b16cdd6dc4", "10956326")    ### [rolands če]
    addDeezer(mdbmaps, "Spotify", "d8ee0c11cb37dbb97dbe4589a4545282", "89313432")    ### [LUISA]
    addDeezer(mdbmaps, "Spotify", "fdf70d482adf5f6fb9fe3691b66fd666", "10864428")    ### [BAZOOKA]
    addDeezer(mdbmaps, "Spotify", "c63d668bad2daac009d38d69b06709a9", "181827")    ### [STEFAN]
    addDeezer(mdbmaps, "Spotify", "3e51c2a492a3208d44ed4b68c5c0de11", "4683785")    ### [INGER]
    addDeezer(mdbmaps, "Spotify", "d0ea789adb64bde45d31ed2b957f04af", "68813")    ### [LIDA]
    addDeezer(mdbmaps, "Spotify", "efdcc76555c18b9a47da12a095d5f4da", "4660822")    ### [KHANS]
    addDeezer(mdbmaps, "Spotify", "f15e7437521872b6aa54287b4c1d3ddc", "1136614")    ### [kaw]
    addDeezer(mdbmaps, "Spotify", "1e3271d5849a140d3f6e5698d9804764", "4686396")    ### [Exiled]
    addDeezer(mdbmaps, "Spotify", "fbea3070535d01a829acf5c0580fdf32", "106388592")    ### [payton]
    addDeezer(mdbmaps, "Spotify", "87c7f6875d65a79bc2f8a97dfc3bae7a", "8954502")    ### [Kiol]
    addDeezer(mdbmaps, "Spotify", "0e74d4e6e94af3ef261e8f83483483c4", "804945")    ### [SAINTS]
    addDeezer(mdbmaps, "Spotify", "a2c82bc5baadb0c156927c3aaf2e577c", "165664")    ### [Mariño]
    addDeezer(mdbmaps, "Spotify", "e1c94b24ad823c50979510eb5df5eb29", "1085396")    ### [Stix]
    addDeezer(mdbmaps, "Spotify", "1bd82622eeaad5c700241013ffd4a39a", "4220098")    ### [VIŅA]
    addDeezer(mdbmaps, "Spotify", "783518b05ced6ee19c334d3954f71fe2", "12741165")    ### [Young DrA]
    addDeezer(mdbmaps, "Spotify", "f6e14b60135b2e9023db60d4e5aef7fe", "53173512")    ### [SEEMEE]
    addDeezer(mdbmaps, "Spotify", "a0f7810c25589860ac8973e63238e56c", "5331617")    ### [KOIT TOOME]
    addDeezer(mdbmaps, "Spotify", "dd2042c741f536dd19998a749c498899", "15166901")    ### [372kaspar]
    addDeezer(mdbmaps, "Spotify", "2f327dedb546e1d6ee6d96e41a41f27e", "54812362")    ### [IsaacMusic]
    addDeezer(mdbmaps, "Spotify", "9052f79c5866ca8509bdba6b13d7d4ff", "482837")    ### [KNUX]
    addDeezer(mdbmaps, "Spotify", "ff777d857ffd620f66c9677034ba5137", "1362521")    ### [MacShawn100]
    addDeezer(mdbmaps, "Spotify", "16395062a4e4a343c2e4fd6326837d9f", "12438312")    ### [Leol]
    addDeezer(mdbmaps, "Spotify", "b311824763f4757dd5557233781b041f", "317194")    ### [andrei]
    addDeezer(mdbmaps, "Spotify", "3d65586d18ec29c99342c6f85a7ab9ee", "3767391")    ### [VUNK]
    addDeezer(mdbmaps, "Spotify", "a722dccf85df771ce9c061e45b292990", "1664003")    ### [RAVA]
    addDeezer(mdbmaps, "Spotify", "212aee750484f97810fe3446092bd8c7", "14649413")    ### [Ali Dolla$]
    addDeezer(mdbmaps, "Spotify", "8a7a8cb8e7cad7f49869dabb19aae3a9", "143637")    ### [TRISHES]
    addDeezer(mdbmaps, "Spotify", "0500b238f4271c84348b5a4643c8895d", "7143773")    ### [Beau'Joli]
    addDeezer(mdbmaps, "Spotify", "9c8fd5547029ceca9134d5f77690d595", "7742848")    ### [Monika Linkyte]
    addDeezer(mdbmaps, "Spotify", "845974b91908722810a650988afc5416", "9476028")    ### [MEANDI]
    addDeezer(mdbmaps, "Spotify", "75115e81be8211dac1ea58283c8542bf", "53002782")    ### [Radvilas RA]
    addDeezer(mdbmaps, "Spotify", "27ae5d527bb68f4d2636a0d4e619f526", "14861157")    ### [be vis]
    addDeezer(mdbmaps, "Spotify", "1aaf83a6c22205806a04690f58f982c9", "14014275")    ### [KiROT]
    addDeezer(mdbmaps, "Spotify", "6c928d014408cbe348a78f08041e5c85", "8644134")    ### [Plamen & Ivo]
    addDeezer(mdbmaps, "Spotify", "95ab8f5a6277c25332cf0f2ae43f86a0", "6137")    ### [Heroes Del Silencio]
    addDeezer(mdbmaps, "Spotify", "283e9ad2edc7926f1225d00caa3703dc", "244980")    ### [La Mosca Tse-Tse]
    addDeezer(mdbmaps, "Spotify", "15c4a8facf210c3bfc7f0cac28b31a4c", "305771")    ### [Bobo]
    addDeezer(mdbmaps, "Spotify", "f817ce364f7768dd727d90324bc8e91e", "1217586")    ### [MOI MUSIC]
    addDeezer(mdbmaps, "Spotify", "70a347e39165362ec895f0582ec35e54", "2946291")    ### [BARBEL]
    addDeezer(mdbmaps, "Spotify", "4bbaa1fb2632cb64077188609e441aef", "4607825")    ### [Talley]
    addDeezer(mdbmaps, "Spotify", "b2f2bf4d5d78ea7e4fce5adec65f860d", "49192912")    ### [JOSEPH CHASE]
    addDeezer(mdbmaps, "Spotify", "ae14a4a2e9de5d9e2707d4ed482c1063", "14609075")    ### [His Kings]
    addDeezer(mdbmaps, "Spotify", "082d82c2cfa5bd0188d7c978b53bc58c", "9927892")    ### [BONNIE X CLYDE]
    addDeezer(mdbmaps, "Spotify", "f17f23e5351e33f30b14584ffa51e80f", "1415466")    ### [DJ Yayo]
    addDeezer(mdbmaps, "Spotify", "ae7abd8b47d277b7be0109e91015631a", "14901793")    ### [Claveles de la Cumbia]
    addDeezer(mdbmaps, "Spotify", "2ea00dcb2e06802bd4d2a594e26d6159", "12664741")    ### [VICE MENTA]
    addDeezer(mdbmaps, "Spotify", "adac75d8af78c47417607eed58ac2b58", "7816600")    ### [Urband]
    addDeezer(mdbmaps, "Spotify", "3fdec593956e5d930420d02d2666b24a", "12251454")    ### [Los inquietos del vallenato]
    addDeezer(mdbmaps, "Spotify", "5a6eb36d1241dd8991ed77c5c76f7d63", "366177")    ### [MC LEO]
    addDeezer(mdbmaps, "Spotify", "283e9ad2edc7926f1225d00caa3703dc", "244980")    ### [La Mosca Tse-Tse]
    addDeezer(mdbmaps, "Spotify", "34c0a13db2387965a5ebfaa14296c6eb", "12251454")    ### [Los Inquietos Del Vallenato]
    addDeezer(mdbmaps, "Spotify", "c4c92ea1f897b796c91ade25828b2ec6", "9231042")    ### [CHIAKI SATO]
    addDeezer(mdbmaps, "Spotify", "2f0133066d975b78c013934554e7eedb", "87908682")    ### [The New Boyz]
    addDeezer(mdbmaps, "Spotify", "8de6581b8c7f02485d8c8d4008772a07", "135962")    ### [DOLLA]
    addDeezer(mdbmaps, "Spotify", "3326a1459e0eb70a7f0fce751e60316c", "9220944")    ### [XPOSE]
    addDeezer(mdbmaps, "Spotify", "2422543147956008f31f94d6574735dd", "53217322")    ### [ABANGSAPAU]
    addDeezer(mdbmaps, "Spotify", "da2135c1ad33ea3cc20899ef9db4a0a5", "7349002")    ### [JOSIMAR Y SU YAMBU]
    addDeezer(mdbmaps, "Spotify", "fac24962ac25281899e9fb086744e8dc", "1190692")    ### [PETITE]
    addDeezer(mdbmaps, "Spotify", "012d1b92264beb5beb5b5d6b9ad107ba", "11107496")    ### [TOSSAKAN]
    addDeezer(mdbmaps, "Spotify", "10c8c10c4d508e61d5c0ce9920f4e616", "96924502")    ### [KUN]
    addDeezer(mdbmaps, "Spotify", "d21abd9a924824b3591881083f0cdd86", "49098811")    ### [PiA]
    addDeezer(mdbmaps, "Spotify", "e2a03f421834df8a3b354c0475ca45d3", "4457870")    ### [JINU]
    addDeezer(mdbmaps, "Spotify", "3fb999bcd426e08f451ae381a3e68b38", "8410882")    ### [Mr.Siro]
    addDeezer(mdbmaps, "Spotify", "cdda9cbbdb1ef951778b3cfc81c8fd5b", "88017162")    ### [COSMO.]
    addDeezer(mdbmaps, "Spotify", "7dc5ad3d308ecaefc3107719bee41b39", "13747253")    ### [3NIGMA]
    addDeezer(mdbmaps, "Spotify", "cd22a685e541843314ebbd7f5c60efd8", "12354536")    ### [Kiian]
    addDeezer(mdbmaps, "Spotify", "1b8b9edcaf6afb10abab14ba84d06fe5", "12086134")    ### [FAO>dino]
    addDeezer(mdbmaps, "Spotify", "12b828a4aec973bd2973546a08bb03b8", "11473350")    ### [Jt el Utility]
    addDeezer(mdbmaps, "Spotify", "21da20975b043559ab8b2ceaaf7b6e0a", "372689")    ### [my little airport]
    addDeezer(mdbmaps, "Spotify", "79febaab2b17a129e67afa418da8ea35", "14753211")    ### [AYA A.K.A PANDA]
    addDeezer(mdbmaps, "Spotify", "673db1aee7862515f4b04d7a8bfdea4d", "5367324")    ### [ANATII]
    addDeezer(mdbmaps, "Spotify", "8d7cca977c9983f6784d5942626bdf0a", "15316233")    ### [Dj Obza]
    addDeezer(mdbmaps, "Spotify", "f902ea414120d93bf8ca66a75a47f1a0", "13134247")    ### [WUANT]
    addDeezer(mdbmaps, "Spotify", "ac0997d0e6975ec8b40a5e9bccb99aa7", "10399494")    ### [OIK]
    addDeezer(mdbmaps, "Spotify", "82e0e6e18a81788dacfafccd0a45ffbe", "14478167")    ### [MOBBERS]
    addDeezer(mdbmaps, "Spotify", "3c7f465d7af84f658b57dde0203275b6", "8599352")    ### [xtinto]

    mdbmaps["Spotify"].save()
    

def addDeezerExtras(mdbmaps):
    addDeezer(mdbmaps, "Billboard", "4f617f7ae1994485f0c6fb1e739faa0f", "11064346")    ### [Anibal de Gracia y Sus Invitados]

    addDeezer(mdbmaps, "Billboard", "246d14ae520c0bd8adc50cb10538805a", "93555")    ### [Lalo Rodriguez]
    addDeezer(mdbmaps, "Billboard", "1f591df9ecc72cdbecc5ca37c953b722", "58203022")    ### [Joe Sambo]
    addAllMusic(mdbmaps, "Billboard", "0ac87a2703773df3ee90f35051976d6e", "mn0003205412")   ### [Yuan Yawei]
    addDeezer(mdbmaps, "Billboard", "5614aee754da6707437a67aba21ae97b", "9846490")    ### [Gabo Parisi]
    addDeezer(mdbmaps, "Billboard", "c751a299bc9b7926f25dd12c5c16c40b", "6459251")    ### [Los de La Nazza]
    addDeezer(mdbmaps, "Billboard", "f0ea2c0f6ed64d3d6bc31d1f381092b0", "5138573")    ### [Cantor Yitzchak Meir Helfgot]
    addDeezer(mdbmaps, "Billboard", "3c2a14f3f9126d17f7f2fdbfd3e04295", "5162473")    ### [Joe Vasconcelos]
    addDeezer(mdbmaps, "Billboard", "fa0c3060dc1ed49905865587788668b7", "4277041")    ### [Hector]
    addDeezer(mdbmaps, "Billboard", "d03d6af3ef8c0c4d1e104fe65d94cfb0", "111522992")    ### [Matsuri Nine.]
    addDiscogs(mdbmaps, "Billboard", "75d13abe72774129842a4beff75739ea", "7964912")    ### [The Elovaters]
    addDeezer(mdbmaps, "Billboard", "75d13abe72774129842a4beff75739ea", "11723379")    ### [The Elovaters]
    addDeezer(mdbmaps, "Billboard", "ae1cfe75bb6f91da7d223452168a3d59", "65634")    ### [Vicious]
    addAllMusic(mdbmaps, "Billboard", "1f6381c06850170b54c95e6a596ffb28", "mn0003782313")   ### [Ken Tackey]
    addDeezer(mdbmaps, "Billboard", "6b3628493b920913e8d550fd3eb75703", "10908080")    ### [Hoo Leeger]
    addDeezer(mdbmaps, "Billboard", "bee91a30c653a9d5146bdce553318cf4", "11975779")    ### [Hwang ChiYeul]
    addAllMusic(mdbmaps, "Billboard", "7270cf68e89cab0f5febe2edaed0b7d6", "mn0003487021")   ### [Subaru Shibuya]
    addAllMusic(mdbmaps, "Billboard", "42bb5146be764807cc62ea7cd44bff02", "mn0002503541")   ### [CNBLUE]
    addDeezer(mdbmaps, "Billboard", "42bb5146be764807cc62ea7cd44bff02", "1492698")    ### [CNBLUE]
    addDeezer(mdbmaps, "Billboard", "24d5cd50291e2b5c4c2eb9f3cddce277", "65336")    ### [Fiji]
    addDeezer(mdbmaps, "Billboard", "725c8f573f9f00fd36ea148172338bbc", "133611")    ### [L'arc En Ciel]
    addAllMusic(mdbmaps, "Billboard", "1d5e1f09e81138a1f8efc590f757fc8e", "mn0003255680")   ### [Kakigoori]
    addDiscogs(mdbmaps, "Billboard", "a626013386ca6bc138efacdf05aa7a96", "380314")    ### [Mickey]

    addDeezer(mdbmaps, "Billboard", "8cf125b4398ec24a60757a1ae0dbde6a", "100731")    ### [Yuri]
    addDeezer(mdbmaps, "Billboard", "1a437e23ad6fb8d3077e86589b52a1a3", "9029")    ### [Lucero]
    addDeezer(mdbmaps, "Billboard", "a1afb34f24048e019a601fbb42ae6180", "16319")    ### [Banda El Recodo]
    addDeezer(mdbmaps, "Billboard", "4b8310b8cd692ffb4a081e8bb4f69ed9", "16599")    ### [Pandora]
    addDeezer(mdbmaps, "Billboard", "e75095d73c04898447d4658e618263f3", "1641733")    ### [Jessie Morales: El Original De La Sierra]
    addDeezer(mdbmaps, "Billboard", "bff236756e7bf0dd73d59584e3e1bfe7", "4962433")    ### [Rogelio Martinez "EL RM"]
    addDeezer(mdbmaps, "Billboard", "c9e0e91d959d73ff5edfb706e6ffa684", "149")    ### [iiO]
    addDeezer(mdbmaps, "Billboard", "bb927e3aee770897812deb1e8ab31eae", "264210")    ### [Anaís]
    addAllMusic(mdbmaps, "Billboard", "ddc76b91c35812592603fed613d14f13", "mn0000392577")   ### [R. Ayala]
    addDiscogs(mdbmaps, "Billboard", "ddc76b91c35812592603fed613d14f13", "5845005")    ### [R. Ayala]
    addDeezer(mdbmaps, "Billboard", "ddc76b91c35812592603fed613d14f13", "115142")    ### [R. Ayala]

    addDeezer(mdbmaps, "Billboard", "878d720ee17c0c0a4ef8f071950f88f6", "17053")    ### [Voyage]
    addDeezer(mdbmaps, "Billboard", "d2122c257dd5b8a3c4c0ac06b2abd0e2", "78111")    ### [The Chimes]
    addDeezer(mdbmaps, "Billboard", "9ab14919e655dc9669042291d6771569", "7298")    ### [Andy Bell]
    addDeezer(mdbmaps, "Billboard", "6c9e35fa812bd11f2496be3aa1076cd2", "1436738")    ### [Kimberly Davis]
    addDeezer(mdbmaps, "Billboard", "a4952315ff2618cc4b878de65a4dfe6a", "494171")    ### [Fever]
    addDeezer(mdbmaps, "Billboard", "d79329a688889810aa7ee43aef41c59b", "975")    ### [Lime]
    addDeezer(mdbmaps, "Billboard", "e8b586570af23bcd228f54f4af667358", "10716495")    ### [Slingshot]
    addDeezer(mdbmaps, "Billboard", "37f4a4ef2c5a3db1176b0d514fc5ebf1", "264546")    ### [Bingo Boys]
    addDeezer(mdbmaps, "Billboard", "cafd8f2f1480ab5c07e5a677f94539a2", "149")    ### [iio]
    addDeezer(mdbmaps, "Billboard", "c858d2c428f54d66c563a498860fbf56", "184317")    ### [Dany]
    addDeezer(mdbmaps, "Billboard", "261a823373b1c2d3538e401b824a0705", "120034")    ### [Gia]
    addDeezer(mdbmaps, "Billboard", "52aac3cde8dbb07e987225096f7d3df7", "9710684")    ### [GOODBOYS]

    addDeezer(mdbmaps, "Billboard", "dd9ca9f1eec3ec7324764098928fa90a", "62490")    ### [Nils]
    addDeezer(mdbmaps, "Billboard", "8126b866ffdc150d799057c736ff708e", "577628")    ### [Steve Oliver]
    addDeezer(mdbmaps, "Billboard", "4eb35c41efdc8495ebca5e64cb723217", "4518873")    ### [Jeff Ryan]
    addDeezer(mdbmaps, "Billboard", "96aa7eddeddf7079189c5f9c2feb0378", "1594391")    ### [Kim Scott]

    addDeezer(mdbmaps, "Billboard", "a0ec3ffe08120ccf0c9b5345714a5c51", "2743")    ### [LTD]
    addDiscogs(mdbmaps, "Billboard", "fa00044058206494d4335094632fe4eb", "83751")    ### [Boss]
    addDeezer(mdbmaps, "Billboard", "9a0fdd63c4aa97fc9716413a6111c187", "148184")    ### [Enchantment]
    addDeezer(mdbmaps, "Billboard", "ef9d8e82a6937e87668db5702cd43b2f", "1816361")    ### [UMC's]
    addDeezer(mdbmaps, "Billboard", "71e904485de1ca65917eb99f886f9c4d", "69608322")    ### [Marty]
    addDeezer(mdbmaps, "Billboard", "7a85a9d8804e19413bab514f5f51dfb9", "8322164")    ### [Donnie Trumpet & The Social Experiment]
    addDeezer(mdbmaps, "Billboard", "4fdc8908f2ee7ad25e35f2ad9531c681", "15080705")    ### [Dennis Reed & Gap]
    addDeezer(mdbmaps, "Billboard", "6c999baddd68939e501aa0561d3e0eda", "4052173")    ### [King Bach]
    addDeezer(mdbmaps, "Billboard", "ba6cb719ca53bf4f57ecdcd6bd342bf5", "59927")    ### [Magoo]
    addDeezer(mdbmaps, "Billboard", "55be138b7b3faad5764a62dc13c9c402", "1236346")    ### [John Reilly]
    addDeezer(mdbmaps, "Billboard", "4ba36e510dd7bbd2009fb8ff90aa48c7", "1575744")    ### [Francis & The Lights]
    addDeezer(mdbmaps, "Billboard", "07c0deba90ed751e5c2ec92d0d60d7dd", "59218332")    ### [Brooke Staten]

    addDeezer(mdbmaps, "Billboard", "66dca5283eac29a7b8c9714ed919bb78", "442981")    ### [Red Sun Rising]
    addDeezer(mdbmaps, "Billboard", "7087547bca580c898106d8f7eb761f58", "251366")    ### [Michael Peterson]
    addDeezer(mdbmaps, "Billboard", "b470feb6f3d72bfae87a83e16dd83886", "76640")    ### [Sons of the Desert]

    addDeezer(mdbmaps, "Billboard", "a6fbf98a79a048834d50ce68d67ad442", "546")    ### [Mya]
    addDeezer(mdbmaps, "Billboard", "3bb7b26b6dee914d5d35651092ebbd38", "123941")    ### [Questian Mark & The Mysterians]
    addDeezer(mdbmaps, "Billboard", "c5281bc610aaf5fcb1be3ea9b5aef4a7", "1078918")    ### [Bo Donaldson And The Heywoods]
    addDeezer(mdbmaps, "Billboard", "ee476cb24bebb0cbe4d7061b0d33a707", "1533511")    ### [Rock Star Supernova]
    addDeezer(mdbmaps, "Billboard", "31aa5ad81b2a6342572b6bd54425deae", "390649")    ### [Iration]
    addDeezer(mdbmaps, "Billboard", "808c8b3a9909c7d7a3e5753792d1b995", "1225601")    ### [LV]

    mdbmaps["Billboard"].save()

# Extra

In [ ]:
from dbArtistsAllMusic import dbArtistsAllMusic
from dbArtistsDiscogs import dbArtistsDiscogs
dbAP = dbArtistsAllMusic()




artistURLs=["https://www.allmusic.com/artist/heat-mn0002185241"]
for artistURL in artistURLs:
    artistID=dbAP.dutils.getArtistID(artistURL)
    savename=dbAP.dutils.getArtistSavename(artistID)
    dbAP.dutils.downloadArtistURL(artistURL, savename)
    
dbAP = dbArtistsDiscogs()
artistURLs=['https://www.discogs.com/artist/60136-Catch', 'https://www.discogs.com/artist/3143651-Honey-31', 'https://www.discogs.com/artist/346187-Jeanette-Biedermann']
for artistURL in artistURLs:
    artistID=dbAP.dutils.getArtistID(artistURL)
    savename=dbAP.dutils.getArtistSavename(artistID)
    dbAP.dutils.downloadArtistURL(artistURL, savename)

#Artist ID [0003255680] is not a member of artistIDToName.
#Artist ID [0002503541] is not a member of artistIDToName.
#Artist ID [0003487021] is not a member of artistIDToName.
#Artist ID [7964912] is not a member of artistIDToName.
#Artist ID [0003205412] is not a member of artistIDToName.

In [ ]:
from dbArtistsDeezer import dbArtistsDeezer
from time import sleep
dbAP = dbArtistsDeezer()
print(togetID.keys())
for db in togetID.keys():
    if db != "Deezer":
        continue
    for dbID,artistName in togetID["Deezer"].items():
        artistURL = dbAP.getArtistURL(dbID)
        artistSavename = dbAP.dutils.getArtistSavename(dbID)
        try:
            dbAP.dutils.downloadArtistURL(artistURL, artistSavename, force=False, sleeptime=2)
        except:
            sleep(2)

In [ ]:
togetID['AllMusic'].values()

In [ ]:
updateManDB(4)

In [ ]:
removeDBRenames(mdbmaps, chartType, dbRenames)

In [ ]:
toget=['Vanessa Jenkins and Bryn West', 'Garcia', 'Dana [Dana Rosemary Scallon]', 'the Bizz', 'the Master of Ceremonies', 'Baby D', 'H Two O', 'Rui da Silva', 'Peters', 'Fedde le Grand', 'LMC vs U2', 'Barry De Vorzon and Perry Botkin Jr.', 'Renato', 'Kelly', 'Disco-Tex and the Sex-O-Lettes', 'Buckner', 'Michael Zager Band', 'Nicole Kidman', 'Stardust', 'Nicole', 'The Blues Magoos', 'Mike Will Made-It', 'Wing and a Prayer Fife and Drum Corps', 'The Manchester United Football Squad']
toget=['Greg Medoro', 'The Rua', 'Matt Hartke', 'Jim Brickman and Adam Crossley', 'Carnie and Wendy Wilson', 'Wilton Place Street Band', 'Daniel DeBourg', 'Grooveatech Orchestra', 'Corbette Jackson', 'Chris Emerson', "Jump 'N the Saddle", 'Dionne Warwick and Johnny Mathis', 'Groove Holmes', 'The Jones Gang', 'The Romeos', 'William Prince', 'The Castells', 'MacNeal', 'Leslie Cours Mather', 'Cnote', 'Dave Barnes', 'Charleene Closshey', 'Neil', 'Band of Gold', 'Allison Park', 'The Villains', 'Scarlett', 'Kaz Bielinski', 'Lareau', 'James Taylor and J.D. Souther', 'Carr', 'Regie Hamm', 'LeBlanc', 'Sasha', 'Sasha Alexander', 'Ryan Chernin', 'Jeff Timmons', 'Il Divo and Céline Dion', 'Nancy Wilson', 'Sunny', 'Margo Rey', 'Robert Maxwell and His Orchestra', 'Mouth', 'Deborah Gibson', 'The Gunter Kallmann Chorus', 'Greg London', 'Lebo M.', 'Sam Mizell', 'Viennie V', 'Paul Stookey', 'Danny Griego', 'TrueHeart', 'the Sunliners', 'Chris Walker', 'The Alternate Routes', 'Giant Steps', 'Jade', 'Burke Roney', 'Desol', 'Whitney Wolanin', 'Sandy Stewart', 'Jane Lynch', 'Johnny Mann Singers', 'the Allis', 'Guys', 'Sylvia', 'Sylvia Kirby', 'Sylvia Hutton', 'Kanisha K', 'Dolls', 'The Sanford', 'John Splithoff', 'Josh Krajcik', 'Randy Jackson', 'Jon', 'Glenn Mohr', 'Father Ted Berndt', 'Townsend Band', 'The Baja Marimba Band', 'John Schneider', 'Senator Everett McKinley Dirksen', 'Richard Kincaid', 'Kathryn Dean', 'King Calaway']
toget=['Janus', 'Siouxsie', 'Hoosier Hot Shots and Two Ton Baker', 'Squirrely', 'Boyd Heath', 'Walter Shrum', 'Zach Seabaugh', 'Johnny Wright', 'Baillie', 'Dave Landers', 'Jerry Irby and His Texas Ranchers', 'Heartland [2°]', 'Marilyn', 'Melissa Lawson', 'Cashman', 'The Used and My Chemical Romance', 'Billy Thunderkloud', 'Johnny Hardy', 'Leon Seago', 'Mel Tillis and Sherry Bryce', 'Carl Butler and Pearl', 'the RubberBand', 'Song Trust', 'Lisa Hartman Black', 'Marion Worth and George Morgan', 'Ray Godfrey', 'Mark Willis', 'the Ride', 'New Medicine', 'Bradley Gaskin', 'Rehab', 'Jimmy Smart', 'Coverdale', 'Page and Plant', 'K. T. Oslin', 'the On-U Sound System', 'Bill Boyd and His Cowboy Ramblers', 'TRUSTcompany', 'Greg Bates', 'Camouflage', 'Angels and Airwaves', 'Kevin Sharp', 'The Buchanan Brothers', 'Pete Scobell Band', 'Holy White Hounds', 'Jake Worthington', 'Mono', 'Gene Woods', 'Bill Anderson and Mary Lou Turner', 'Course of Nature', 'The Raybon Brothers', 'Jeffrey Steele', 'Frankie Miller', 'Jimmie Dean', 'Mike Reid', 'Norma Jean and Liz Anderson', 'COIN', 'Zac Brown', 'Young Guns', 'the Melody Makers', 'Dandy', 'Kirk Jay', 'Phil Stacey', 'Rory', 'the Doolittle Band', 'Earshot', 'Texas Jim Lewis', 'Bill Leatherwood', 'The JaneDear Girls', "Coleman O'Neal", 'The Highwaymen', 'the Chieftones', 'Susan Anton', 'Burl Ives and Captain Stubby and the Buccaneers', 'The Glaser Brothers', 'Arthur Guitar Boogie Smith', 'Karen Brooks', 'Schuyler', "the Citizen's Band", 'The Firm', 'Connie Britton', 'Ruby Wright', 'the Banshees', 'Laverne Downs', 'Andy Gibson', 'Reef', 'Trio', 'Cindy Walker', 'Sam Palladio', 'Jim Carroll Band', 'Burl Ives and Grady Martin and His Slew Foot Five', 'Gary Buck', 'Eddie Dean', 'Børns', 'Waylon Jennings and Anita Carter', 'Tompall', 'Silvertide', 'Charlie Worsham', 'Buck Owens and Buddy Alan', 'Troy Olsen', 'Jimmy Newman', 'the Wilburn Brothers', 'Terry McBride', 'LaCosta Tucker', 'Bill Anderson and Jan Howard', 'The Carlisles Brothers', 'Jimmy Edwards', 'Rainbow', 'the Boys', 'Foy Willing and His Riders of the Purple Sage', 'Elton Britt and Rosalie Allen', 'Edens Edge', 'Adam Gregory', 'The Mission', 'Rockie Lynne', 'Victoria Shaw and Bryan White', 'The Henningsens', 'Operator', 'Vernon Stewart', 'Bobby Bare and Skeeter Davis', 'Bill Nettles', 'The Jenkins', 'Ryan Shupe', 'The Farm Inc.', 'Corey Kent White', 'Jonie Mosby', 'The Andrews Sisters and Ernest Tubb and the Texas Troubadors', 'Sound', 'Charles Mitchell', 'Vince Gill and Patty Loveless', 'Luke Gordon', 'Overstreet', 'Ted Self', 'Josh Gallagher', 'Billy Gray', 'MISSIO', 'Shorty Long', 'Spensha Baker', 'Harlow Wilcox and the Oakies', 'Keith Urban and Miranda Lambert', 'Bobby Lewis', 'Chris Kroeze', 'Reno', 'the Lonesome', 'The Outlaws', 'Knobloch', 'Wiley and Gene', 'Diesel [1°]', 'Alan Riddle', 'Copilot Music', 'Doug McKenzie', 'Device', 'Doug', 'J Roddy Walston', 'Pendulum', 'Soho', 'James Wesley Prosser', 'Lajon', 'Cole Deggs', 'Bobby Edwards', 'Cody Jameson', 'The Primitives', 'Dust for Life', 'John', 'Jack Rivers', 'Esmereldy', 'Rod Hart', 'Shelby Brown']
toget=toget=['Johnny Sparrow and His Bows and Arrows', 'Whatnauts', 'Damage', 'Mercy Dee', 'Nile Rodgers and Tony Moran present Kimberly Davis', 'Little Son Jackson', 'Ideal [R', 'Mark Curry', 'Brothers in Rhythm', 'DJs from Mars', 'Eddie Williams and His Brown Buddies', 'John Greer and the Rhythm Rockers', 'DirtyFreqs', 'Nomad', 'The Shades of Love', 'The Tabulations', 'the Ramblers', 'Rocko', 'Dirty Werk', 'Company', 'Joe Bennett', 'Annie Laurie and Paul Gayten and His Orchestra', 'Regina [Regina Richards]', 'Tim Myers', 'Richard Vission and Static Revenger', 'Ja-Kki', 'James and Smith', 'Mo Thugs and Bone Thugs-N-Harmony', 'Nokio', 'the Seen', 'Dynamix', "Mad'House", 'iAMSU!', 'The Street People [2°]', "the Family Cookin'", 'John Davis and the Monster Orchestra', 'The Boogie Man Orchestra', 'The Don [Nathaniel Pierre Jones]', 'Fantasy', 'Danny Overbea and His Combo', 'The X-Rays', 'Hadda Brooks Trio', 'the FBI', 'Pulse [2°]', 'Louie Louie', 'Do Ray and Me', 'Hannah', 'Chris Cox and DJ Frankie', 'Brighter Side of Darkness', 'Howard Johnson', 'Pleasure [1°]', 'Roy C', 'Like Mike and David Guetta', 'Bill', 'B.M.U. (Black Men United)', 'Tim Letteer', 'The Moment of Truth', 'Nathaniel Mayer and the Fabulous Twilights', 'Loveland vs Darlene Lewis', 'JT', 'Whirlwind', 'Dirty Pop', 'Eddie Vinson and His Orchestra', 'Robert Knight', 'Shirley Gunter and the Queens', 'Emitt Slay Trio', 'Calvin Boze and His All-Stars', 'Grant Jones', 'the Bluebelles', 'Tami LaTrell', 'India [Linda Viera Caballero]', 'Hybrid Heights and Crystal Waters', 'TK n Cash', 'Loveland', 'Willie Mae Big Mama Thornton', 'Tim', 'Louis Brooks and His Hi-Toppers', 'The Four Blazes', 'Macho', 'Profyle', 'Andrea Brown', 'The Counts', 'the Rebel MC', 'Young Thug and Travis Scott', 'Mike', 'Bazuka', 'Bobby Moore', 'Gene Ammons and His Sextet', 'the Muffins', "The McCrary's", 'ORS', 'The Chimes', 'The Reflections', 'Metro', 'Robin S. and DJ Escape', 'James Waynes', 'The Quick', 'The Four Jacks', 'Love and Kisses', 'Red Miller Trio', 'Grace [1°]', 'The Heart and Soul Orchestra', 'Marvin and Johnny', 'Willie Dixon and the Allstars', 'Little Esther and Mel Walker', 'Roy Byrd and His Blues Jumpers', 'JR Castro', 'B.G. the Prince of Rap', 'B.B.', 'Joe Thomas', 'Rob Base and DJ E-Z Rock', 'the Soul Rockers', 'Paul Humphrey', 'D.O.N.S. and Terri B!', 'Jimmy James', 'Jack Dupree and Mr. Bear', 'Sonny Boy Williamson [2°]', 'Charo', 'Buddy Lucas and His Band of Tomorrow', 'Jorio', 'Sister Rosetta Tharpe and Marie Knight', 'David Morales and Janice Robinson', 'Chong', '2Virgins', 'the Soul Partners', 'Little Caesar [Harry Caesar]', 'Holland-Dozier', 'Joe Hinton', 'Zach Adam', 'Praxis', 'Fawn', 'Rochell and the Candles', 'Farley Project', 'Dianne Wesley', 'the Rhythm Bangers', 'Dan Grissom', 'Impact [1°]', 'Sagat', 'Electric', 'Mike Hitman Wilson', 'Rosko', 'Archibald and His Orchestra', 'Fancy Inc.', 'The Lisa Marie Experience', 'Crazibiza and Dave Audé', 'Roscoe Gordon', 'Angel City', 'Cliq', 'Like Mike and Era Istrefi', 'Cause', 'the Sparkletones', 'Dana Ordway', 'The Blues Woman', 'Joe Fritz', 'Jimmy Bo Horne', 'The Sapphires', 'The Swallows', 'Queens', 'Wini Brown and Her Boyfriends', "Chubby Hip Shakin' Newsom", 'Switch', 'Taffy', 'the Blue Notes', 'Sun [Sun Ho]', 'Little Willie Littlefield and Little Lola Wiggins', 'The Regents', 'Martha', 'Floyd Hunt Quartette', 'Sharon Paige and Harold Melvin', 'David Joseph', 'Limmie', 'DJ Suede the Remix God', 'Ta Mara', 'The Boys [US R', 'Earl Bostic and His Orchestra', 'Déjà Vu', 'Sphinx', 'RetroVision', 'Pee Wee Crayton and His Guitar', 'DJ Bill Bennett', 'Silvetti', 'Jennifer Saunders', 'Lil Louis', 'Earl King [Earl Johnson]', 'Hal Singer Sextette', 'Bouvier', 'C. J. Bolland', "Marc O'Tool Tall Paul vs INXS", 'K.I.D.', 'Effect', 'Doc Sausage and His Mad Lads', 'The Lovers [2°]', 'the Big Family', 'Memphis Slim and His House Rockers', 'Rudy Render', 'Charlotte [Charlotte Kelly]', 'Cheech', 'Ramsey Lewis and Earth, Wind & Fire', 'Les Cooper', 'Angello', 'Blaze', 'B quartet]', 'Jimmie Nelson and the Peter Rabbit Trio', 'Jackie Lee', 'Joe Morris and His Orchestra', 'Chéri', 'Tony Moran', 'Mel', 'Luvndreams', 'the GB Experience', 'Da Mighty Dub Katz', 'Nellie Lutcher and Nat King Cole', 'Ko Ko Taylor', 'Taylor Girlz', 'Jess Glynne and DJ Cassidy', 'Karel Ullner', 'Q. Brooklyn', 'JX Riders and Skylar Stecker', 'The Larks', 'Jennifer Green', 'Space', 'Paul Williams', 'E.V.E. (Ebony Vibe Everlasting)', 'Fireball', 'Slim [Marvin Scandrick]', 'Staxx', 'The Reflections', 'Jeff Morgan', 'Brenda', 'Hodges', 'Extasia', 'Goldlink', 'Absolutely Fabulous Pet Shop Boys', 'Raw Silk', 'the Emblems', 'George Lamond', 'Stick McGhee and His Buddies', 'The Rainbows', 'Laguna', 'Pet Shop Boys = The Collaboration', 'The Flaming Ember', 'Kevin Ross', 'Artie the 1 Man Party', 'Flip', 'Freddie Mitchell and His Orchestra', 'Troop 41', 'Joi', 'the Police', 'Twisted Dee', 'Pianoheadz', 'The Shades of Blue', 'Lipstique', 'the Soul Searchers', 'The Philarmonics', 'Cashis', 'Eastside Connection', 'Bell', 'The Coronets', 'Fast Life Yungstaz (F.L.Y.)', 'Bumble Bee Unlimited', 'Sheila', 'Fenix', 'Patty', 'Betty Boo and the Beatmasters', 'U.N.V.', "Suave'", 'The Fog', 'Original Five Blind Boys', 'Lennie Lewis and His Orchestra', 'Earl King [Earl Connelly]', 'His Cool Aid Chemists', 'Camille Howard and Her Trio', 'Linx', 'DJ Duke', 'Dave Matthias', 'Herb Lance and the Classics', 'The Goodfellas', "Brown's Blues Blowers", 'Dyke and the Blazers', 'Skip', 'Eniac', 'S.O.U.L. S.Y.S.T.E.M. introducing Michelle Visage', 'Jimmy Grissom', 'Trio', 'C.J. & Co.', 'The Positive Force', 'Jon of the Pleased Wimmin', 'Angel Clivillés', 'Arpeggio', 'Brainstorm', 'Destiny', 'the Egg', 'Industry [2°]', 'Ed Wiley and His After Hour Rhythm', 'Black Machine', 'Herb Fisher and His Trio', 'Barona', 'Jessie Saunders', 'the Vagabonds', 'Dillon Francis and Kygo', 'The Geto Boys', '50 Cent and Olivia', 'Swinging Sax Kari and Orchestra', 'KMC', 'Déjà', 'DJ Vicious', 'Alton McClain', 'Meechie', 'Mel Walker', 'Lydia Murdoch', 'Fatback', 'The Ones [2°]', 'Solo [2°]', 'Who Da Funk?', '3rd Party', 'Jimmy Preston and His Prestonians', 'Chip Chocolate', 'Ecstasy', 'The Shadows', 'The Purple Ribbon All-Stars', "The Cats 'N Jammer Three", "Stretch 'N' Vern Present Maddogg", 'Marvin', 'Eunice', 'Richard Dimples Fields', 'Robbie Rivera and Axwell', 'Ron Butler', 'Karen Young', 'Round2Crew', 'The Cardinals', 'Refugee Camp All-Stars', 'Mount Rushmore', 'Sonny Boy Williamson [1°]', 'Altar', 'Pacific Gas', 'Jimmy Wilson and His All-Stars', 'Willie Mabon and His Combo', 'The Corsairs', 'Mike Theodore Orchestra', 'Sunshine']

toget=['Vanessa Jenkins and Bryn West', 'Garcia', 'Dana [Dana Rosemary Scallon]', 'the Bizz', 'the Master of Ceremonies', 'Baby D [dance group]', 'H Two O', 'Rui da Silva', 'Peters', 'Fedde le Grand', 'LMC vs U2', 'Barry De Vorzon and Perry Botkin Jr.', 'Renato', 'Kelly', 'Disco-Tex and the Sex-O-Lettes', 'Buckner', 'Michael Zager Band', 'Nicole Kidman', 'Stardust [electronic music trio]', 'Nicole [German singer]', 'The Blues Magoos', 'Mike Will Made-It', 'Wing and a Prayer Fife and Drum Corps', 'The Manchester United Football Squad']
toget+=['Greg Medoro', 'The Rua', 'Matt Hartke', 'Jim Brickman and Adam Crossley', 'Carnie and Wendy Wilson', 'Wilton Place Street Band', 'Daniel DeBourg', 'Grooveatech Orchestra', 'Corbette Jackson', 'Chris Emerson', "Jump 'N the Saddle", 'Dionne Warwick and Johnny Mathis', 'Groove Holmes', 'The Jones Gang', 'The Romeos', 'William Prince', 'The Castells', 'MacNeal', 'Leslie Cours Mather', 'Cnote', 'Dave Barnes', 'Charleene Closshey', 'Neil', 'Band of Gold', 'Allison Park', 'The Villains', 'Scarlett', 'Kaz Bielinski', 'Lareau', 'James Taylor and J.D. Souther', 'Carr', 'Regie Hamm', 'LeBlanc', 'Sasha [Sasha Alexander]', 'Ryan Chernin', 'Jeff Timmons', 'Il Divo and Céline Dion', 'Nancy Wilson', 'Sunny', 'Margo Rey', 'Robert Maxwell and His Orchestra', 'Mouth', 'Deborah Gibson', 'The Gunter Kallmann Chorus', 'Greg London', 'Lebo M.', 'Sam Mizell', 'Viennie V', 'Paul Stookey', 'Danny Griego', 'TrueHeart', 'the Sunliners', 'Chris Walker', 'The Alternate Routes', 'Giant Steps', 'Jade [Canadian singer]', 'Burke Roney', 'Desol', 'Whitney Wolanin', 'Sandy Stewart [singer]', 'Jane Lynch', 'Johnny Mann Singers', 'the Allis', 'Guys', 'Sylvia [Sylvia Kirby aka Sylvia Hutton]', 'Kanisha K', 'Dolls', 'The Sanford', 'John Splithoff', 'Josh Krajcik', 'Randy Jackson', 'Jon', 'Glenn Mohr', 'Father Ted Berndt', 'Townsend Band', 'The Baja Marimba Band', 'John Schneider', 'Senator Everett McKinley Dirksen', 'Richard Kincaid', 'Kathryn Dean', 'King Calaway']
toget+=['Janus', 'Siouxsie', 'Hoosier Hot Shots and Two Ton Baker', 'Squirrely', 'Boyd Heath', 'Walter Shrum', 'Zach Seabaugh', 'Johnny Wright', 'Baillie', 'Dave Landers', 'Jerry Irby and His Texas Ranchers', 'Heartland [2°]', 'Marilyn', 'Melissa Lawson', 'Cashman', 'The Used and My Chemical Romance', 'Billy Thunderkloud', 'Johnny Hardy', 'Leon Seago', 'Mel Tillis and Sherry Bryce', 'Carl Butler and Pearl', 'the RubberBand', 'Song Trust', 'Lisa Hartman Black', 'Marion Worth and George Morgan', 'Ray Godfrey', 'Mark Willis', 'the Ride', 'New Medicine', 'Bradley Gaskin', 'Rehab', 'Jimmy Smart', 'Coverdale', 'Page and Plant', 'K. T. Oslin', 'the On-U Sound System', 'Bill Boyd and His Cowboy Ramblers', 'TRUSTcompany', 'Greg Bates', 'Camouflage [German dance group]', 'Angels and Airwaves', 'Kevin Sharp', 'The Buchanan Brothers', 'Pete Scobell Band', 'Holy White Hounds', 'Jake Worthington', 'Mono [British pop duo]', 'Gene Woods', 'Bill Anderson and Mary Lou Turner', 'Course of Nature', 'The Raybon Brothers', 'Jeffrey Steele', 'Frankie Miller [country singer]', 'Jimmie Dean', 'Mike Reid', 'Norma Jean and Liz Anderson', 'COIN', 'Zac Brown', 'Young Guns', 'the Melody Makers', 'Dandy', 'Kirk Jay', 'Phil Stacey', 'Rory', 'the Doolittle Band', 'Earshot', 'Texas Jim Lewis', 'Bill Leatherwood', 'The JaneDear Girls', "Coleman O'Neal", 'The Highwaymen [country supergroup]', 'the Chieftones', 'Susan Anton', 'Burl Ives and Captain Stubby and the Buccaneers', 'The Glaser Brothers', 'Arthur Guitar Boogie Smith', 'Karen Brooks', 'Schuyler', "the Citizen's Band", 'The Firm [English rock supergroup]',
       'Connie Britton', 'Ruby Wright [country singer]', 'the Banshees', 'Laverne Downs', 'Andy Gibson [singer]', 'Reef', 'Trio [American country supergroup]', 'Cindy Walker', 'Sam Palladio', 'Jim Carroll Band', 'Burl Ives and Grady Martin and His Slew Foot Five', 'Gary Buck', 'Eddie Dean', 'Børns', 'Waylon Jennings and Anita Carter', 'Tompall', 'Silvertide', 'Charlie Worsham', 'Buck Owens and Buddy Alan', 'Troy Olsen', 'Jimmy Newman', 'the Wilburn Brothers', 'Terry McBride', 'LaCosta Tucker', 'Bill Anderson and Jan Howard', 'The Carlisles Brothers', 'Jimmy Edwards', 'Rainbow [British rock band]', 'the Boys', 'Foy Willing and His Riders of the Purple Sage', 'Elton Britt and Rosalie Allen', 'Edens Edge', 'Adam Gregory', 'The Mission [English band]', 'Rockie Lynne', 'Victoria Shaw and Bryan White', 'The Henningsens', 'Operator', 'Vernon Stewart', 'Bobby Bare and Skeeter Davis', 'Bill Nettles', 'The Jenkins', 'Ryan Shupe', 'The Farm Inc.', 'Corey Kent White', 'Jonie Mosby', 'The Andrews Sisters and Ernest Tubb and the Texas Troubadors', 'Sound', 'Charles Mitchell', 'Vince Gill and Patty Loveless', 'Luke Gordon', 'Overstreet', 'Ted Self', 'Josh Gallagher', 'Billy Gray', 'MISSIO', 'Shorty Long [country singer]', 'Spensha Baker', 'Harlow Wilcox and the Oakies', 'Keith Urban and Miranda Lambert', 'Bobby Lewis [country singer]', 'Chris Kroeze', 'Reno', 'the Lonesome', 'The Outlaws [US band]', 'Knobloch', 'Wiley and Gene', 'Diesel [1°]', 'Alan Riddle', 'Copilot Music', 'Doug McKenzie', 'Device [metal band]', 'Doug', 'J Roddy Walston', 'Pendulum [drum and bass band]', 'Soho', 'James Wesley Prosser', 'Lajon', 'Cole Deggs', 'Bobby Edwards', 'Cody Jameson', 'The Primitives', 'Dust for Life', 'John', 'Jack Rivers', 'Esmereldy', 'Rod Hart', 'Shelby Brown']
toget+=['Johnny Sparrow and His Bows and Arrows', 'Whatnauts', 'Damage', 'Mercy Dee', 'Nile Rodgers and Tony Moran present Kimberly Davis', 'Little Son Jackson', 'Ideal [R', 'Mark Curry', 'Brothers in Rhythm', 'DJs from Mars', 'Eddie Williams and His Brown Buddies', 'John Greer and the Rhythm Rockers', 'DirtyFreqs', 'Nomad [house music act]', 'The Shades of Love', 'The Tabulations', 'the Ramblers', 'Rocko', 'Dirty Werk', 'Company', 'Joe Bennett', 'Annie Laurie and Paul Gayten and His Orchestra', 'Regina [Regina Richards]', 'Tim Myers', 'Richard Vission and Static Revenger', 'Ja-Kki', 'James and Smith', 'Mo Thugs and Bone Thugs-N-Harmony', 'Nokio', 'the Seen', 'Dynamix', "Mad'House", 'iAMSU!', 'The Street People [2°]', "the Family Cookin'", 'John Davis and the Monster Orchestra', 'The Boogie Man Orchestra', 'The Don [Nathaniel Pierre Jones]', 'Fantasy [US soul band]', 'Danny Overbea and His Combo', 'The X-Rays', 'Hadda Brooks Trio', 'the FBI', 'Pulse [2°]', 'Louie Louie', 'Do Ray and Me', 'Hannah [British singer]', 'Chris Cox and DJ Frankie', 'Brighter Side of Darkness', 'Howard Johnson [singer]', 'Pleasure [1°]', 'Roy C', 'Like Mike and David Guetta', 'Bill', 'B.M.U. (Black Men United)', 'Tim Letteer', 'The Moment of Truth', 'Nathaniel Mayer and the Fabulous Twilights', 'Loveland vs Darlene Lewis', 'JT', 'Whirlwind', 'Dirty Pop', 'Eddie Vinson and His Orchestra', 'Robert Knight', 'Shirley Gunter and the Queens', 'Emitt Slay Trio', 'Calvin Boze and His All-Stars', 'Grant Jones', 'the Bluebelles', 'Tami LaTrell', 'India [Linda Viera Caballero]', 'Hybrid Heights and Crystal Waters', 'TK n Cash', 'Loveland', 'Willie Mae Big Mama Thornton', 'Tim', 'Louis Brooks and His Hi-Toppers', 'The Four Blazes', 'Macho', 'Profyle', 'Andrea Brown', 'The Counts',
       'the Rebel MC', 'Young Thug and Travis Scott', 'Mike', 'Bazuka', 'Bobby Moore [saxophonist]', 'Gene Ammons and His Sextet', 'the Muffins', "The McCrary's", 'ORS', 'The Chimes [British dance music act]', 'The Reflections [Harlem vocal group]', 'Metro', 'Robin S. and DJ Escape', 'James Waynes', 'The Quick', 'The Four Jacks', 'Love and Kisses', 'Red Miller Trio', 'Grace [1°]', 'The Heart and Soul Orchestra', 'Marvin and Johnny', 'Willie Dixon and the Allstars', 'Little Esther and Mel Walker', 'Roy Byrd and His Blues Jumpers', 'JR Castro', 'B.G. the Prince of Rap', 'B.B.', 'Joe Thomas [American saxophonist]', 'Rob Base and DJ E-Z Rock', 'the Soul Rockers', 'Paul Humphrey', 'D.O.N.S. and Terri B!', 'Jimmy James', 'Jack Dupree and Mr. Bear', 'Sonny Boy Williamson [2°]', 'Charo', 'Buddy Lucas and His Band of Tomorrow', 'Jorio', 'Sister Rosetta Tharpe and Marie Knight', 'David Morales and Janice Robinson', 'Chong', '2Virgins', 'the Soul Partners', 'Little Caesar [Harry Caesar]', 'Holland-Dozier', 'Joe Hinton', 'Zach Adam', 'Praxis', 'Fawn', 'Rochell and the Candles', 'Farley Project', 'Dianne Wesley', 'the Rhythm Bangers', 'Dan Grissom', 'Impact [1°]', 'Sagat', 'Electric', 'Mike Hitman Wilson', 'Rosko', 'Archibald and His Orchestra', 'Fancy Inc.', 'The Lisa Marie Experience', 'Crazibiza and Dave Audé', 'Roscoe Gordon', 'Angel City [electronic music group]', 'Cliq', 'Like Mike and Era Istrefi', 'Cause', 'the Sparkletones', 'Dana Ordway', 'The Blues Woman', 'Joe Fritz', 'Jimmy Bo Horne', 'The Sapphires', 'The Swallows', 'Queens', 'Wini Brown and Her Boyfriends', "Chubby Hip Shakin' Newsom", 'Switch [funk band]', 'Taffy', 'the Blue Notes', 'Sun [Sun Ho]', 'Little Willie Littlefield and Little Lola Wiggins', 'The Regents [American doo-wop group]', 'Martha', 'Floyd Hunt Quartette', 'Sharon Paige and Harold Melvin', 'David Joseph', 'Limmie', 'DJ Suede the Remix God', 'Ta Mara', 'The Boys [US R', 'Earl Bostic and His Orchestra', 'Déjà Vu [dance music act]',
       'Sphinx [electronic music duo]', 'RetroVision', 'Pee Wee Crayton and His Guitar', 'DJ Bill Bennett', 'Silvetti', 'Jennifer Saunders', 'Lil Louis', 'Earl King [Earl Johnson]', 'Hal Singer Sextette', 'Bouvier', 'C. J. Bolland', "Marc O'Tool Tall Paul vs INXS", 'K.I.D.', 'Effect', 'Doc Sausage and His Mad Lads', 'The Lovers [2°]', 'the Big Family', 'Memphis Slim and His House Rockers', 'Rudy Render', 'Charlotte [Charlotte Kelly]', 'Cheech', 'Ramsey Lewis and Earth, Wind & Fire', 'Les Cooper', 'Angello', 'Blaze', 'B quartet]', 'Jimmie Nelson and the Peter Rabbit Trio', 'Jackie Lee [American singer]', 'Joe Morris and His Orchestra', 'Chéri', 'Tony Moran', 'Mel', 'Luvndreams', 'the GB Experience', 'Da Mighty Dub Katz', 'Nellie Lutcher and Nat King Cole', 'Ko Ko Taylor', 'Taylor Girlz', 'Jess Glynne and DJ Cassidy', 'Karel Ullner', 'Q. Brooklyn', 'JX Riders and Skylar Stecker', 'The Larks [North Carolina band]', 'Jennifer Green', 'Space [french electronic music band]', 'Paul Williams [saxophonist]', 'E.V.E. (Ebony Vibe Everlasting)', 'Fireball', 'Slim [Marvin Scandrick]', 'Staxx', 'The Reflections [Detroit rock and roll band]', 'Jeff Morgan', 'Brenda', 'Hodges', 'Extasia', 'Goldlink', 'Absolutely Fabulous Pet Shop Boys', 'Raw Silk', 'the Emblems', 'George Lamond', 'Stick McGhee and His Buddies', 'The Rainbows', 'Laguna', 'Pet Shop Boys = The Collaboration', 'The Flaming Ember', 'Kevin Ross', 'Artie the 1 Man Party', 'Flip', 'Freddie Mitchell and His Orchestra', 'Troop 41', 'Joi', 'the Police', 'Twisted Dee', 'Pianoheadz', 'The Shades of Blue', 'Lipstique', 'the Soul Searchers', 'The Philarmonics', 'Cashis', 'Eastside Connection', 'Bell', 'The Coronets', 'Fast Life Yungstaz (F.L.Y.)', 'Bumble Bee Unlimited', 'Sheila', 'Fenix', 'Patty', 'Betty Boo and the Beatmasters', 'U.N.V.', "Suave'", 'The Fog [house project]', 'Original Five Blind Boys', 'Lennie Lewis and His Orchestra', 'Earl King [Earl Connelly]', 'His Cool Aid Chemists', 'Camille Howard and Her Trio',
       'Linx', 'DJ Duke', 'Dave Matthias', 'Herb Lance and the Classics', 'The Goodfellas', "Brown's Blues Blowers", 'Dyke and the Blazers', 'Skip', 'Eniac', 'S.O.U.L. S.Y.S.T.E.M. introducing Michelle Visage',
       'Jimmy Grissom', 'Trio [German band]', 'C.J. & Co.', 'The Positive Force', 'Jon of the Pleased Wimmin', 'Angel Clivillés', 'Arpeggio', 'Brainstorm [disco band]', 'Destiny', 'the Egg', 'Industry [2°]', 'Ed Wiley and His After Hour Rhythm', 'Black Machine', 'Herb Fisher and His Trio', 'Barona', 'Jessie Saunders', 'the Vagabonds', 'Dillon Francis and Kygo', 'The Geto Boys', '50 Cent and Olivia', 'Swinging Sax Kari and Orchestra', 'KMC [Italian dance act]', 'Déjà [soul duo]', 'DJ Vicious', 'Alton McClain', 'Meechie', 'Mel Walker', 'Lydia Murdoch', 'Fatback', 'The Ones [2°]', 'Solo [2°]', 'Who Da Funk?', '3rd Party', 'Jimmy Preston and His Prestonians', 'Chip Chocolate', 'Ecstasy', 'The Shadows [vocal group]', 'The Purple Ribbon All-Stars', "The Cats 'N Jammer Three", "Stretch 'N' Vern Present Maddogg", 'Marvin', 'Eunice', 'Richard Dimples Fields', 'Robbie Rivera and Axwell', 'Ron Butler', 'Karen Young [US singer]', 'Round2Crew', 'The Cardinals', 'Refugee Camp All-Stars', 'Mount Rushmore', 'Sonny Boy Williamson [1°]', 'Altar [dance music band]', 'Pacific Gas', 'Jimmy Wilson and His All-Stars', 'Willie Mabon and His Combo', 'The Corsairs', 'Mike Theodore Orchestra', 'Sunshine']
from collections import Counter
cntr = Counter()
for item in toget:
    if " [" in item:
        if len(item.split(" [")) > 1:
            #print(item)
            continue
        #val="["+item.split(" [")[1]
        #cntr[val] += 1
for item in cntr.most_common():
    print(item[0])
    
    
removes="""[country singer]
[singer]
[saxophonist]
[dance group]
[electronic music trio]
[German singer]
[Canadian singer]
[German dance group]
[British pop duo]
[country supergroup]
[English rock supergroup]
[American country supergroup]
[British rock band]
[English band]
[US band]
[metal band]
[drum and bass band]
[house music act]
[US soul band]
[British singer]
[British dance music act]
[Harlem vocal group]
[American saxophonist]
[electronic music group]
[funk band]
[American doo-wop group]
[dance music act]
[electronic music duo]
[American singer]
[North Carolina band]
[french electronic music band]
[Detroit rock and roll band]
[house project]
[German band]
[disco band]
[Italian dance act]
[soul duo]
[vocal group]
[US singer]
[dance music band]""".split("\n")

toget=['Johnny Sparrow and His Bows and Arrows', 'Whatnauts', 'Damage', 'Mercy Dee', 'Nile Rodgers and Tony Moran present Kimberly Davis', 'Little Son Jackson', 'Ideal [R', 'Mark Curry', 'Brothers in Rhythm', 'DJs from Mars', 'Eddie Williams and His Brown Buddies', 'John Greer and the Rhythm Rockers', 'DirtyFreqs', 'Nomad [house music act]', 'The Shades of Love', 'The Tabulations', 'the Ramblers', 'Rocko', 'Dirty Werk', 'Company', 'Joe Bennett', 'Annie Laurie and Paul Gayten and His Orchestra', 'Regina [Regina Richards]', 'Tim Myers', 'Richard Vission and Static Revenger', 'Ja-Kki', 'James and Smith', 'Mo Thugs and Bone Thugs-N-Harmony', 'Nokio', 'the Seen', 'Dynamix', "Mad'House", 'iAMSU!', 'The Street People [2°]', "the Family Cookin'", 'John Davis and the Monster Orchestra', 'The Boogie Man Orchestra', 'The Don [Nathaniel Pierre Jones]', 'Fantasy [US soul band]', 'Danny Overbea and His Combo', 'The X-Rays', 'Hadda Brooks Trio', 'the FBI', 'Pulse [2°]', 'Louie Louie', 'Do Ray and Me', 'Hannah [British singer]', 'Chris Cox and DJ Frankie', 'Brighter Side of Darkness', 'Howard Johnson [singer]', 'Pleasure [1°]', 'Roy C', 'Like Mike and David Guetta', 'Bill', 'B.M.U. (Black Men United)', 'Tim Letteer', 'The Moment of Truth', 'Nathaniel Mayer and the Fabulous Twilights', 'Loveland vs Darlene Lewis', 'JT', 'Whirlwind', 'Dirty Pop', 'Eddie Vinson and His Orchestra', 'Robert Knight', 'Shirley Gunter and the Queens', 'Emitt Slay Trio', 'Calvin Boze and His All-Stars', 'Grant Jones', 'the Bluebelles', 'Tami LaTrell', 'India [Linda Viera Caballero]', 'Hybrid Heights and Crystal Waters', 'TK n Cash', 'Loveland', 'Willie Mae Big Mama Thornton', 'Tim', 'Louis Brooks and His Hi-Toppers', 'The Four Blazes', 'Macho', 'Profyle', 'Andrea Brown', 'The Counts',
       'the Rebel MC', 'Young Thug and Travis Scott', 'Mike', 'Bazuka', 'Bobby Moore [saxophonist]', 'Gene Ammons and His Sextet', 'the Muffins', "The McCrary's", 'ORS', 'The Chimes [British dance music act]', 'The Reflections [Harlem vocal group]', 'Metro', 'Robin S. and DJ Escape', 'James Waynes', 'The Quick', 'The Four Jacks', 'Love and Kisses', 'Red Miller Trio', 'Grace [1°]', 'The Heart and Soul Orchestra', 'Marvin and Johnny', 'Willie Dixon and the Allstars', 'Little Esther and Mel Walker', 'Roy Byrd and His Blues Jumpers', 'JR Castro', 'B.G. the Prince of Rap', 'B.B.', 'Joe Thomas [American saxophonist]', 'Rob Base and DJ E-Z Rock', 'the Soul Rockers', 'Paul Humphrey', 'D.O.N.S. and Terri B!', 'Jimmy James', 'Jack Dupree and Mr. Bear', 'Sonny Boy Williamson [2°]', 'Charo', 'Buddy Lucas and His Band of Tomorrow', 'Jorio', 'Sister Rosetta Tharpe and Marie Knight', 'David Morales and Janice Robinson', 'Chong', '2Virgins', 'the Soul Partners', 'Little Caesar [Harry Caesar]', 'Holland-Dozier', 'Joe Hinton', 'Zach Adam', 'Praxis', 'Fawn', 'Rochell and the Candles', 'Farley Project', 'Dianne Wesley', 'the Rhythm Bangers', 'Dan Grissom', 'Impact [1°]', 'Sagat', 'Electric', 'Mike Hitman Wilson', 'Rosko', 'Archibald and His Orchestra', 'Fancy Inc.', 'The Lisa Marie Experience', 'Crazibiza and Dave Audé', 'Roscoe Gordon', 'Angel City [electronic music group]', 'Cliq', 'Like Mike and Era Istrefi', 'Cause', 'the Sparkletones', 'Dana Ordway', 'The Blues Woman', 'Joe Fritz', 'Jimmy Bo Horne', 'The Sapphires', 'The Swallows', 'Queens', 'Wini Brown and Her Boyfriends', "Chubby Hip Shakin' Newsom", 'Switch [funk band]', 'Taffy', 'the Blue Notes', 'Sun [Sun Ho]', 'Little Willie Littlefield and Little Lola Wiggins', 'The Regents [American doo-wop group]', 'Martha', 'Floyd Hunt Quartette', 'Sharon Paige and Harold Melvin', 'David Joseph', 'Limmie', 'DJ Suede the Remix God', 'Ta Mara', 'The Boys [US R', 'Earl Bostic and His Orchestra', 'Déjà Vu [dance music act]',
       'Sphinx [electronic music duo]', 'RetroVision', 'Pee Wee Crayton and His Guitar', 'DJ Bill Bennett', 'Silvetti', 'Jennifer Saunders', 'Lil Louis', 'Earl King [Earl Johnson]', 'Hal Singer Sextette', 'Bouvier', 'C. J. Bolland', "Marc O'Tool Tall Paul vs INXS", 'K.I.D.', 'Effect', 'Doc Sausage and His Mad Lads', 'The Lovers [2°]', 'the Big Family', 'Memphis Slim and His House Rockers', 'Rudy Render', 'Charlotte [Charlotte Kelly]', 'Cheech', 'Ramsey Lewis and Earth, Wind & Fire', 'Les Cooper', 'Angello', 'Blaze', 'B quartet]', 'Jimmie Nelson and the Peter Rabbit Trio', 'Jackie Lee [American singer]', 'Joe Morris and His Orchestra', 'Chéri', 'Tony Moran', 'Mel', 'Luvndreams', 'the GB Experience', 'Da Mighty Dub Katz', 'Nellie Lutcher and Nat King Cole', 'Ko Ko Taylor', 'Taylor Girlz', 'Jess Glynne and DJ Cassidy', 'Karel Ullner', 'Q. Brooklyn', 'JX Riders and Skylar Stecker', 'The Larks [North Carolina band]', 'Jennifer Green', 'Space [french electronic music band]', 'Paul Williams [saxophonist]', 'E.V.E. (Ebony Vibe Everlasting)', 'Fireball', 'Slim [Marvin Scandrick]', 'Staxx', 'The Reflections [Detroit rock and roll band]', 'Jeff Morgan', 'Brenda', 'Hodges', 'Extasia', 'Goldlink', 'Absolutely Fabulous Pet Shop Boys', 'Raw Silk', 'the Emblems', 'George Lamond', 'Stick McGhee and His Buddies', 'The Rainbows', 'Laguna', 'Pet Shop Boys = The Collaboration', 'The Flaming Ember', 'Kevin Ross', 'Artie the 1 Man Party', 'Flip', 'Freddie Mitchell and His Orchestra', 'Troop 41', 'Joi', 'the Police', 'Twisted Dee', 'Pianoheadz', 'The Shades of Blue', 'Lipstique', 'the Soul Searchers', 'The Philarmonics', 'Cashis', 'Eastside Connection', 'Bell', 'The Coronets', 'Fast Life Yungstaz (F.L.Y.)', 'Bumble Bee Unlimited', 'Sheila', 'Fenix', 'Patty', 'Betty Boo and the Beatmasters', 'U.N.V.', "Suave'", 'The Fog [house project]', 'Original Five Blind Boys', 'Lennie Lewis and His Orchestra', 'Earl King [Earl Connelly]', 'His Cool Aid Chemists', 'Camille Howard and Her Trio',
       'Linx', 'DJ Duke', 'Dave Matthias', 'Herb Lance and the Classics', 'The Goodfellas', "Brown's Blues Blowers", 'Dyke and the Blazers', 'Skip', 'Eniac', 'S.O.U.L. S.Y.S.T.E.M. introducing Michelle Visage',
       'Jimmy Grissom', 'Trio [German band]', 'C.J. & Co.', 'The Positive Force', 'Jon of the Pleased Wimmin', 'Angel Clivillés', 'Arpeggio', 'Brainstorm [disco band]', 'Destiny', 'the Egg', 'Industry [2°]', 'Ed Wiley and His After Hour Rhythm', 'Black Machine', 'Herb Fisher and His Trio', 'Barona', 'Jessie Saunders', 'the Vagabonds', 'Dillon Francis and Kygo', 'The Geto Boys', '50 Cent and Olivia', 'Swinging Sax Kari and Orchestra', 'KMC [Italian dance act]', 'Déjà [soul duo]', 'DJ Vicious', 'Alton McClain', 'Meechie', 'Mel Walker', 'Lydia Murdoch', 'Fatback', 'The Ones [2°]', 'Solo [2°]', 'Who Da Funk?', '3rd Party', 'Jimmy Preston and His Prestonians', 'Chip Chocolate', 'Ecstasy', 'The Shadows [vocal group]', 'The Purple Ribbon All-Stars', "The Cats 'N Jammer Three", "Stretch 'N' Vern Present Maddogg", 'Marvin', 'Eunice', 'Richard Dimples Fields', 'Robbie Rivera and Axwell', 'Ron Butler', 'Karen Young [US singer]', 'Round2Crew', 'The Cardinals', 'Refugee Camp All-Stars', 'Mount Rushmore', 'Sonny Boy Williamson [1°]', 'Altar [dance music band]', 'Pacific Gas', 'Jimmy Wilson and His All-Stars', 'Willie Mabon and His Combo', 'The Corsairs', 'Mike Theodore Orchestra', 'Sunshine']
togetX=[]
print(len(toget))
for item in toget:
    val = item
    for rm in removes:
        if rm in val:
            val = val.replace(rm, "").strip()
    togetX.append(val)
print(len(togetX))
print(togetX)


toget=['Johnny Sparrow and His Bows and Arrows', 'Whatnauts', 'Damage', 'Mercy Dee', 'Nile Rodgers and Tony Moran present Kimberly Davis', 'Little Son Jackson', 'Ideal [R', 'Mark Curry', 'Brothers in Rhythm', 'DJs from Mars', 'Eddie Williams and His Brown Buddies', 'John Greer and the Rhythm Rockers', 'DirtyFreqs', 'Nomad', 'The Shades of Love', 'The Tabulations', 'the Ramblers', 'Rocko', 'Dirty Werk', 'Company', 'Joe Bennett', 'Annie Laurie and Paul Gayten and His Orchestra', 'Regina [Regina Richards]', 'Tim Myers', 'Richard Vission and Static Revenger', 'Ja-Kki', 'James and Smith', 'Mo Thugs and Bone Thugs-N-Harmony', 'Nokio', 'the Seen', 'Dynamix', "Mad'House", 'iAMSU!', 'The Street People [2°]', "the Family Cookin'", 'John Davis and the Monster Orchestra', 'The Boogie Man Orchestra', 'The Don [Nathaniel Pierre Jones]', 'Fantasy', 'Danny Overbea and His Combo', 'The X-Rays', 'Hadda Brooks Trio', 'the FBI', 'Pulse [2°]', 'Louie Louie', 'Do Ray and Me', 'Hannah', 'Chris Cox and DJ Frankie', 'Brighter Side of Darkness', 'Howard Johnson', 'Pleasure [1°]', 'Roy C', 'Like Mike and David Guetta', 'Bill', 'B.M.U. (Black Men United)', 'Tim Letteer', 'The Moment of Truth', 'Nathaniel Mayer and the Fabulous Twilights', 'Loveland vs Darlene Lewis', 'JT', 'Whirlwind', 'Dirty Pop', 'Eddie Vinson and His Orchestra', 'Robert Knight', 'Shirley Gunter and the Queens', 'Emitt Slay Trio', 'Calvin Boze and His All-Stars', 'Grant Jones', 'the Bluebelles', 'Tami LaTrell', 'India [Linda Viera Caballero]', 'Hybrid Heights and Crystal Waters', 'TK n Cash', 'Loveland', 'Willie Mae Big Mama Thornton', 'Tim', 'Louis Brooks and His Hi-Toppers', 'The Four Blazes', 'Macho', 'Profyle', 'Andrea Brown', 'The Counts', 'the Rebel MC', 'Young Thug and Travis Scott', 'Mike', 'Bazuka', 'Bobby Moore', 'Gene Ammons and His Sextet', 'the Muffins', "The McCrary's", 'ORS', 'The Chimes', 'The Reflections', 'Metro', 'Robin S. and DJ Escape', 'James Waynes', 'The Quick', 'The Four Jacks', 'Love and Kisses', 'Red Miller Trio', 'Grace [1°]', 'The Heart and Soul Orchestra', 'Marvin and Johnny', 'Willie Dixon and the Allstars', 'Little Esther and Mel Walker', 'Roy Byrd and His Blues Jumpers', 'JR Castro', 'B.G. the Prince of Rap', 'B.B.', 'Joe Thomas', 'Rob Base and DJ E-Z Rock', 'the Soul Rockers', 'Paul Humphrey', 'D.O.N.S. and Terri B!', 'Jimmy James', 'Jack Dupree and Mr. Bear', 'Sonny Boy Williamson [2°]', 'Charo', 'Buddy Lucas and His Band of Tomorrow', 'Jorio', 'Sister Rosetta Tharpe and Marie Knight', 'David Morales and Janice Robinson', 'Chong', '2Virgins', 'the Soul Partners', 'Little Caesar [Harry Caesar]', 'Holland-Dozier', 'Joe Hinton', 'Zach Adam', 'Praxis', 'Fawn', 'Rochell and the Candles', 'Farley Project', 'Dianne Wesley', 'the Rhythm Bangers', 'Dan Grissom', 'Impact [1°]', 'Sagat', 'Electric', 'Mike Hitman Wilson', 'Rosko', 'Archibald and His Orchestra', 'Fancy Inc.', 'The Lisa Marie Experience', 'Crazibiza and Dave Audé', 'Roscoe Gordon', 'Angel City', 'Cliq', 'Like Mike and Era Istrefi', 'Cause', 'the Sparkletones', 'Dana Ordway', 'The Blues Woman', 'Joe Fritz', 'Jimmy Bo Horne', 'The Sapphires', 'The Swallows', 'Queens', 'Wini Brown and Her Boyfriends', "Chubby Hip Shakin' Newsom", 'Switch', 'Taffy', 'the Blue Notes', 'Sun [Sun Ho]', 'Little Willie Littlefield and Little Lola Wiggins', 'The Regents', 'Martha', 'Floyd Hunt Quartette', 'Sharon Paige and Harold Melvin', 'David Joseph', 'Limmie', 'DJ Suede the Remix God', 'Ta Mara', 'The Boys [US R', 'Earl Bostic and His Orchestra', 'Déjà Vu', 'Sphinx', 'RetroVision', 'Pee Wee Crayton and His Guitar', 'DJ Bill Bennett', 'Silvetti', 'Jennifer Saunders', 'Lil Louis', 'Earl King [Earl Johnson]', 'Hal Singer Sextette', 'Bouvier', 'C. J. Bolland', "Marc O'Tool Tall Paul vs INXS", 'K.I.D.', 'Effect', 'Doc Sausage and His Mad Lads', 'The Lovers [2°]', 'the Big Family', 'Memphis Slim and His House Rockers', 'Rudy Render', 'Charlotte [Charlotte Kelly]', 'Cheech', 'Ramsey Lewis and Earth, Wind & Fire', 'Les Cooper', 'Angello', 'Blaze', 'B quartet]', 'Jimmie Nelson and the Peter Rabbit Trio', 'Jackie Lee', 'Joe Morris and His Orchestra', 'Chéri', 'Tony Moran', 'Mel', 'Luvndreams', 'the GB Experience', 'Da Mighty Dub Katz', 'Nellie Lutcher and Nat King Cole', 'Ko Ko Taylor', 'Taylor Girlz', 'Jess Glynne and DJ Cassidy', 'Karel Ullner', 'Q. Brooklyn', 'JX Riders and Skylar Stecker', 'The Larks', 'Jennifer Green', 'Space', 'Paul Williams', 'E.V.E. (Ebony Vibe Everlasting)', 'Fireball', 'Slim [Marvin Scandrick]', 'Staxx', 'The Reflections', 'Jeff Morgan', 'Brenda', 'Hodges', 'Extasia', 'Goldlink', 'Absolutely Fabulous Pet Shop Boys', 'Raw Silk', 'the Emblems', 'George Lamond', 'Stick McGhee and His Buddies', 'The Rainbows', 'Laguna', 'Pet Shop Boys = The Collaboration', 'The Flaming Ember', 'Kevin Ross', 'Artie the 1 Man Party', 'Flip', 'Freddie Mitchell and His Orchestra', 'Troop 41', 'Joi', 'the Police', 'Twisted Dee', 'Pianoheadz', 'The Shades of Blue', 'Lipstique', 'the Soul Searchers', 'The Philarmonics', 'Cashis', 'Eastside Connection', 'Bell', 'The Coronets', 'Fast Life Yungstaz (F.L.Y.)', 'Bumble Bee Unlimited', 'Sheila', 'Fenix', 'Patty', 'Betty Boo and the Beatmasters', 'U.N.V.', "Suave'", 'The Fog', 'Original Five Blind Boys', 'Lennie Lewis and His Orchestra', 'Earl King [Earl Connelly]', 'His Cool Aid Chemists', 'Camille Howard and Her Trio', 'Linx', 'DJ Duke', 'Dave Matthias', 'Herb Lance and the Classics', 'The Goodfellas', "Brown's Blues Blowers", 'Dyke and the Blazers', 'Skip', 'Eniac', 'S.O.U.L. S.Y.S.T.E.M. introducing Michelle Visage', 'Jimmy Grissom', 'Trio', 'C.J. & Co.', 'The Positive Force', 'Jon of the Pleased Wimmin', 'Angel Clivillés', 'Arpeggio', 'Brainstorm', 'Destiny', 'the Egg', 'Industry [2°]', 'Ed Wiley and His After Hour Rhythm', 'Black Machine', 'Herb Fisher and His Trio', 'Barona', 'Jessie Saunders', 'the Vagabonds', 'Dillon Francis and Kygo', 'The Geto Boys', '50 Cent and Olivia', 'Swinging Sax Kari and Orchestra', 'KMC', 'Déjà', 'DJ Vicious', 'Alton McClain', 'Meechie', 'Mel Walker', 'Lydia Murdoch', 'Fatback', 'The Ones [2°]', 'Solo [2°]', 'Who Da Funk?', '3rd Party', 'Jimmy Preston and His Prestonians', 'Chip Chocolate', 'Ecstasy', 'The Shadows', 'The Purple Ribbon All-Stars', "The Cats 'N Jammer Three", "Stretch 'N' Vern Present Maddogg", 'Marvin', 'Eunice', 'Richard Dimples Fields', 'Robbie Rivera and Axwell', 'Ron Butler', 'Karen Young', 'Round2Crew', 'The Cardinals', 'Refugee Camp All-Stars', 'Mount Rushmore', 'Sonny Boy Williamson [1°]', 'Altar', 'Pacific Gas', 'Jimmy Wilson and His All-Stars', 'Willie Mabon and His Combo', 'The Corsairs', 'Mike Theodore Orchestra', 'Sunshine']

# Suffix

In [ ]:


mdbmaps["BillboardYE"].save()

In [ ]:
cad.getAlbumsData("George Frideric Handel: Messiah")

In [ ]:
updateManDB(2)

In [ ]:
updateManDB(2)

In [ ]:


mdbmaps["BillboardYE"].save()

In [ ]:
updateManDB(2)

In [ ]:
cad.getAlbumsData("Common Ground")

In [ ]:
updateManDB(2)

# Analyze Matches

In [ ]:
1/0
addAllMusic(mdbmaps, "Top40", "b67c4a39d846b0e14fcda8299193701a", "mn0000417717")   ### [Raybion Bros.]
addDiscogs(mdbmaps, "Top40", "b67c4a39d846b0e14fcda8299193701a", "2188194")    ### [Raybion Bros.]
### Butterfly Kisses

addAllMusic(mdbmaps, "Top40", "00d5d59798eeffef591f272260b0bbcc", "mn0000124572")   ### [Chuck-N-Blood]
### My Dogs

addAllMusic(mdbmaps, "Top40", "866a64f5a5e684f61aa573af7f63bc57", "mn0000233423")   ### [Magoo]
### Up Jumps Da Boogie

addAllMusic(mdbmaps, "Top40", "78f1feb9204bce3173d42798bc9bcb91", "mn0001808325")   ### [Kobe Bryant]
### Hold Me

addAllMusic(mdbmaps, "Top40", "d92e471232aa6a0bc4a03a49f16b0997", "mn0000186401")   ### [Karen O.]
addDiscogs(mdbmaps, "Top40", "d92e471232aa6a0bc4a03a49f16b0997", "245778")    ### [Karen O.]
### Hello Tomorrow

addAllMusic(mdbmaps, "Top40", "5edcd28a4505dc29e4edebd67052e614", "mn0000795894")   ### [Dolla]

addAllMusic(mdbmaps, "Top40", "7bcb752a835787affb80072d61387b22", "mn0001411030")   ### [Hollis]
### White Walls

addAllMusic(mdbmaps, "Top40", "c0051ae12f6048a807213c44b434aa1d", "mn0003348925")   ### [Jennifer Lawrence]
addDiscogs(mdbmaps, "Top40", "c0051ae12f6048a807213c44b434aa1d", "4210193")    ### [Jennifer Lawrence]
### The Hanging Tree

addAllMusic(mdbmaps, "Top40", "083b67cc423fe5482eaf1412aebb65b3", "mn0003323773")   ### [Lookas]
addDiscogs(mdbmaps, "Top40", "083b67cc423fe5482eaf1412aebb65b3", "3619379")    ### [Lookas]
### GDFR

addAllMusic(mdbmaps, "Top40", "f37bd04269e23d0c4c10c435a4f1dd7f", "mn0002943184")   ### [Mnek]
### Never Forget You  ,  So Good

addAllMusic(mdbmaps, "Top40", "1f9b34897f6440e2484f1ac5e00953f7", "mn0003462822")   ### [Gnash]
### I Hate U, I Love U  ,  Lights Down Low

addAllMusic(mdbmaps, "Top40", "f731f61bae9f9216f33f0cf76d403fee", "mn0000351749")   ### [Royce Da 5'9]
addDiscogs(mdbmaps, "Top40", "f731f61bae9f9216f33f0cf76d403fee", "40792")    ### [Royce Da 5'9]
### Not Alike

addAllMusic(mdbmaps, "Top40", "296504ec4590a1522dcf6afb838fd6f4", "mn0003332851")   ### [London On Da Track]
### Numbers

mdbmaps["Top40"].save()

In [ ]:
updateManDB(2)

In [ ]:
updateManDB(4)

In [ ]:
print([item[1]["ArtistName"] for item in toMatch["Top40"]])

In [ ]:
saveData = []
for fullName in sorted(list(cad.manyArtists.keys())):
    appendName = " & ".join(["[{0}]".format(manDB.renamed(indivName)) for indivName in sorted(list(cad.manyArtists[fullName]))])
    saveData.append([fullName, {"Guess": appendName, "Truth": ''}]) # = list(cad.manyArtists[fullName].keys())
saveFile(idata=saveData, ifile="tmp.yaml")

In [ ]:
fixedData = getFile("tmp.yaml")

In [ ]:
multis = []
multirenames = {}
for item in fixedData:
    if len(item[1]["Truth"]) > 0:
        name = item[1]["Truth"]
        if len(name.split("] & [")) == 1:
            multis.append(item[1]["Truth"][1:-1])
        else:
            rename = name[1:-1].split("] & [")
            multirenames[item[0]] = " ::: ".join(rename)
        
print("Found {0} renames".format(len(multirenames)))
print("Found {0} multi-artists".format(len(multis)))

#### Save MultiArtists (if found)

In [ ]:
knownFilename = "../multiartist/knownMultiArtists.yaml"
knownMultis = getFile(knownFilename)
print(len(multis))
print(len(knownMultis))
knownMultis += multis
print(len(knownMultis))
knownMultis = sorted(list(set(knownMultis)))
print(len(knownMultis))
saveFile(idata=knownMultis, ifile=knownFilename)

#### Save multi renames (if needed)

In [ ]:
multiManDB = masterArtistNameDB("multi", init=False)
#multiManDB.forceReload(getFile("main.yaml"))
multiManDB.addRenames(multirenames)

In [ ]:
multiManDB = masterArtistNameDB("multi", init=False)
multiManDB.addRenames(multirenames)
multiManDB.save()
saveFile(idata=multiManDB.getRenames(), ifile="multi.yaml")
multiManDB = masterArtistNameDB("multi", init=True)
multiManDB.forceReload(getFile("multi.yaml"))
multiManDB.checkForRecursives()
multiManDB = masterArtistNameDB("multi", init=False)

In [ ]:
#### Edit main.yaml (if needed)

In [ ]:
mdbmaps["Billboard"].getDF()

In [ ]:
mdf = mdbmaps["BillboardYE"].getDF()
mdf.shape
#mdf[mdf["DBMatches"] == 0]

# Multi Match

In [ ]:
cad.getTypeArtistAlbumData("Joe Rene")

In [ ]:
toMatch["Billboard"]

In [ ]:
manualAppends(cad, mType, toMatch, chartType, minAlbums=0, add=True)

In [ ]:
mdbmc.getMasterDF("Billboard")

In [ ]:
matchItAll(mdbmaps, getThresholdsWithoutAlbums(0.85), toMatch=toMatch, useAlbums=False)
reMatch()

In [ ]:
#singleArtistAlbumData["Billboard"]['Ernest']
saveFile(idata=[x[1]["ArtistName"] for x in toMatch["Top40"]], ifile="toget.p")

# Find Near Artist Name Matches

In [ ]:
toget, dbRenames = analyzeMatches(maindb, mdbmaps, chartType, maxVal=1.0, diffVal=1.0)

In [ ]:
if len(dbRenames) > 0:
    analyzeRenames(manDB, dbRenames)
    # Test for overap with manDB

    dbRenames = getFile(ifile="relDBRenames2.yaml")
    redos = {}
    dels  = []
    for oldername,bestname in dbRenames.items():
        if oldername in manDB.artistNameDB.keys(): # and bestname not in manDB.artistNameDB.keys():
            #print("Older=[{0}] \t Best=[{1}]".format(oldername,bestname))

            redos[bestname] = oldername
            dels.append(oldername)
            #print('PROBLEM ({0} in master DB): \t{1}  -->  {2}'.format(oldername, oldername, bestname))
            try:
                matchOlderData = mdbmaps[chartType].getArtistDataByName(oldername).getDict()
            except:
                matchOlderData = {}

            try:
                matchBestData  = mdbmaps[chartType].getArtistDataByName(bestname).getDict()
            except:
                matchBestData = {}

            #print("Older: {0}".format(matchOlderData))
            #print("Best:  {0}".format(matchBestData))
            #print("\n")
        else:
            print("{0}: {1}".format(oldername, bestname))

    print("# Renames: {0}".format(len(dbRenames)))
    print("# Dels: {0}".format(len(dels)))

In [ ]:
manDB = masterArtistNameDB("main", init=True)
manDB.forceReload(getFile("main.yaml"))
manDB.addRenames(getFile("relDBRenames2.yaml"))
manDB.checkForRecursives()

In [ ]:
manDB = masterArtistNameDB("main", init=True)
manDB.forceReload(getFile("main.yaml"))
manDB.addRenames(getFile("relDBRenames2.yaml"))
manDB.checkForRecursives()
manDB.save()
manDB = masterArtistNameDB("main", init=False)
saveFile(idata=manDB.dbRenames, ifile="main.yaml")
removeDBRenames(mdbmaps, chartType, dbRenames)

In [ ]:
removeDBRenames(mdbmaps, chartType, dbRenames)

In [ ]:
vals="""
[Sander Van Doom                0.9                Sander Van Doorn] 	 --> (Top40) 27813b44b4282de50171e13adb4ebcf9 / 183267 (Discogs)
"""

lines=vals.split("\n")
lines2=[x.split("({0}) ".format(chartType))[1] for x in lines if len(x) > 0]
names2=[x.split("({0}) ".format(chartType))[0] for x in lines if len(x) > 0]
lines3={x.split(" / ")[0]: x[x.find("(")+1:-1] for x in lines2}
names3=[x.split("    ")[0].strip()[1:] for x in names2]
if len(names3) != len(lines3):
    raise ValueError("Err")

for i,(k,v) in enumerate(lines3.items()):
    artistName = names3[i]
    #print("mdbmaps[chartType].addArtistDataByID(\'{0}\', \'{1}\', \'{2}\')".format(k,v,))
    print("### ----> {0}".format(artistName))
    print("mdbmaps[chartType].addArtistDataByID(\'{0}\', \'{1}\', None)".format(k,v))    
    albums = singleArtistAlbumData[chartType][artistName]
    print("### {0}".format("  ,  ".join(albums)))
    print("### ignores.append(\"{0}\")".format(artistName))
    print("\n")
    
    
#mdbmaps[chartType].addArtistDataByID("e309c2e3fc905eae304b71e10e82eb99", "AllMusic", '0000803469')
print("")
print("mdbmaps[chartType].save()")
print("saveFile(idata=list(set(ignores)), ifile=\"chartIgnores.yaml\")")

In [ ]:
### ----> Sander Van Doom
mdbmaps[chartType].addArtistDataByID('27813b44b4282de50171e13adb4ebcf9', 'Discogs', None)
### Grasshopper
### ignores.append("Sander Van Doom")



mdbmaps[chartType].save()
saveFile(idata=list(set(ignores)), ifile="chartIgnores.yaml")

In [ ]:
toget

****
****
****
****

# Multis

In [ ]:
ignores.append('Daryl Hall & John Oates')

In [ ]:
from multiArtist import multiartist
mularts  = multiartist(cutoff=0.9, discdata=None, exact=False)
knownMultiArtists = getFile("../multiartist/knownMultiArtists.yaml")
print(len(knownMultiArtists))
print(len(mdbmaps[chartType].getArtists()))
for idx,artistName in mdbmaps[chartType].getArtists().items():
    N = mularts.getArtistNames(artistName)
    if len(N) > 1:
        result = {subName: mdbmaps[chartType].isKnownByName(subName) for subName in N}
        if not any(result.values()):
            knownMultiArtists.append(artistName)
            #print(result)
        #print("ignores.append(\'{0}\')".format(artistName))

print(len(knownMultiArtists))
knownMultiArtists = list(set(knownMultiArtists))
print(len(knownMultiArtists))
        
#saveFile(idata = sorted(knownMultiArtists), ifile="../multiartist/knownMultiArtists.yaml")

In [ ]:
ignores = getFile("chartIgnores.yaml")
dels = []
for idx,artistName in mdbmaps[chartType].getArtists().items():
    if artistName in ignores:
        dels.append([idx,artistName])
        
print(len(dels))
for idx,artistName in dels:
    mdbmaps[chartType].removeArtistByID(idx)
    
mdbmaps[chartType].save()

In [ ]:
len(dels)

In [ ]:
adds="""addDiscogs(mdbmaps, "RYM", "c938cf68f94fed75cc26549337750387", "645741")   ### [Arditti String Quartet]
    addDiscogs(mdbmaps, "RYM", "20d20d433cce4f470529ec7541aa3bde", "276345")   ### [Deceased...]
    addDeezer(mdbmaps, "RYM", "c0bbfdcc5b92b78f5f11f55882d911c8", "8002136")   ### [Ling Tosite Sigure]
    addDiscogs(mdbmaps, "RYM", "e512fcff2eeff54f4b637fd084f8a06b", "1566620")   ### [SubRosa]
    addDiscogs(mdbmaps, "RYM", "7c02717b30de8c1464b6f686ff1845a4", "5125")   ### [Scraping Foetus Off the Wheel]
    addDiscogs(mdbmaps, "RYM", "a1b047738f828d4880fb4877b80d979c", "271870")   ### [Orchestre de la Suisse romande]
    addDiscogs(mdbmaps, "RYM", "c16063a9fea3d3d4684095fd5f68d71d", "330153")   ### [The Jimmy Giuffre 3]
    addDiscogs(mdbmaps, "RYM", "c4eb00d17e99f78059401a8e00e7dacc", "2219395")   ### [Sun Ra and His Solar Arkestra]
    addDiscogs(mdbmaps, "RYM", "d3c8efab1413492442397c2db3bd3a2e", "5698629")   ### [Standing on the Corner]
    addDiscogs(mdbmaps, "RYM", "76dea25af68db8985aa56146aa74bb3d", "5868607")   ### [Seiko Oomori]
    addDiscogs(mdbmaps, "RYM", "ae87ffccd622c9401a91cf041de05c8c", "3782979")   ### [Wędrowcy~Tułacze~Zbiegi]
    addDiscogs(mdbmaps, "RYM", "7a8afe2961d7b5ecb852678161d09c6f", "1930677")   ### [Mass of the Fermenting Dregs]
    addDiscogs(mdbmaps, "RYM", "1f9d0f4fc63ba3bfaa977bb53d5ec297", "1792452")   ### [Chór Polskiego Radia w Krakowie]
    addDiscogs(mdbmaps, "RYM", "938151a2dc408797cf03a119c9cf41ff", "1272551")   ### [qebrµs]
    addDiscogs(mdbmaps, "RYM", "4a417f8bf4b890c720c3678384ae1577", "999446")   ### [Gaza]
    addDiscogs(mdbmaps, "RYM", "789a7608dfcf89fb127dac972139aa3c", "272542")   ### [Asphyx]
    addDiscogs(mdbmaps, "RYM", "9f0948c41e2ee2e523387a725d623265", "263565")   ### [Eyehategod]
    addDiscogs(mdbmaps, "RYM", "e221271b326b0d9d0b473faf71048207", "4968274")   ### [father]
    addDiscogs(mdbmaps, "RYM", "6290942c7523a0243f9b8f3cd56f714b", "564753")   ### [Tallinna Kammerorkester]
    addDiscogs(mdbmaps, "RYM", "26292da02ddfcc6808f4808c05272b68", "256245")   ### [CCCP Fedeli alla Linea]
    addDiscogs(mdbmaps, "RYM", "89b5b278b1f3c1b105cb138724ccd2fd", "154944")   ### [Le Mystère des voix bulgares]
    addDiscogs(mdbmaps, "RYM", "457c1759fe0e158d9af951a8f99396a1", "1868577")   ### [George Russell and his Orchestra]
    addDiscogs(mdbmaps, "RYM", "81e01b185f9ca31d6b11677220199461", "264626")   ### [Cliff Jordan]    
    
    addDiscogs(mdbmaps, "RYM", "8d14b9131f9c697849bc5b70407b5273", "2483549")   ### [The Satellites]
    addDiscogs(mdbmaps, "RYM", "762821772c1072550d078a6d39fca7cf", "305667")   ### [Tin Pan Alley]
    addDiscogs(mdbmaps, "RYM", "0d93544c007400c690e0c0fe19251d18", "602138")   ### [Elias & His Zig Zag Jive Flutes]
    addDiscogs(mdbmaps, "RYM", "8c7f56c6720e2060f85e828f84a855bc", "392760")   ### [Pérez Prado y su Orquesta]
    addDiscogs(mdbmaps, "RYM", "e384be6fa7c15b5b6e9aab5326d8629c", "1489439")   ### [Johnny Duncan & The Blue Grass Boys]
    addDiscogs(mdbmaps, "RYM", "d99c322ec9e64a15f6fb8bcf95a663f0", "521963")   ### [Ray Charles With Chorus and Orchestra]
    addDiscogs(mdbmaps, "RYM", "9c4b83cb02899c42820b58a3339abea3", "521963")   ### [Ray Charles With Orchestra and Chorus]
    addDiscogs(mdbmaps, "RYM", "2d87f3a9b9a9e38f9dac2fd645883bff", "1600328")   ### [Bill Haley with The Saddlemen]
    addDiscogs(mdbmaps, "RYM", "5b023ef505fdc8be6e835d9c067e62c1", "1602284")   ### [Rita & the Tiaras]
    addDiscogs(mdbmaps, "RYM", "a00cdbfb8409cad1339cc8ce90033f01", "6708")   ### [Residents, Uninc.]
    addDiscogs(mdbmaps, "RYM", "20cae975550e8e8385cd88d3dc185c45", "3669108")   ### [Jimmy Dee and the Offbeats]
    addDiscogs(mdbmaps, "RYM", "79320017271ab5ed32ee94a56179dea7", "6025169")   ### [Shygirl]
    addDiscogs(mdbmaps, "RYM", "a9441c4ca72d40d97c9c84555ba7d74d", "305667")   ### [キャラメル]
    addDeezer(mdbmaps, "RYM", "04641d8d951d715ca5756000ff2db54d", "1418509")   ### [The Crooklyn Dodgers]
    addDeezer(mdbmaps, "RYM", "00433043c53d398788260b6b6d09d5cd", "357879")   ### [Domina]
    addDiscogs(mdbmaps, "RYM", "e24e3e28c82d52605bbdcf6731cd72fa", "98882")   ### [Double]
    addDiscogs(mdbmaps, "RYM", "8316104bbfbd61597ab33281fb7e7475", "2760")   ### [On the House]
    addDiscogs(mdbmaps, "RYM", "1e8536e43402f9abe2b62c00d737f34a", "1508404")   ### [Virtue]
    addDiscogs(mdbmaps, "RYM", "4fa818e91d057c8debc5022e4452e58e", "473561")   ### [Deux]
    addDeezer(mdbmaps, "RYM", "48bb720d403ae154a110f562492ef384", "2366")   ### [Eric B. and Rakim]
    addDiscogs(mdbmaps, "RYM", "8b7fa1b266a935d00d4f02cb99738d4f", "264096")   ### [Gen X]
    addDeezer(mdbmaps, "RYM", "74e3b69152a0166c815a1b81a1795ed7", "65383")   ### [Sträwberry Switchbläde]
    addDiscogs(mdbmaps, "RYM", "a1ea6d722062e7c45d23d37565dc139c", "240872")   ### [Double Dee]
    addDiscogs(mdbmaps, "RYM", "abc078064f6b0c58e3bfbf1634193203", "58096")   ### [Stretch]
    addDiscogs(mdbmaps, "RYM", "c42b0e8a4137bf545c46603870826940", "364861")   ### [The MASH]
    addDiscogs(mdbmaps, "RYM", "fe7c735cfc4d266af8722d2efab6c66d", "11650")   ### [Machine]
    addDiscogs(mdbmaps, "RYM", "9521e26abf89eeb11774270ef1fe0821", "256994")   ### [Avengers]
    addDiscogs(mdbmaps, "RYM", "eae4bc4aff6b4214eba2368eb8f04ca0", "112154")   ### [E L O]
    addDiscogs(mdbmaps, "RYM", "779c27b775d595de1b5aba08e32feca6", "65421")   ### [Ian Dury and The Blockheads]
    addDiscogs(mdbmaps, "RYM", "7fc92a88d903740a35469bac9e7e6c98", "1062022")   ### [シュガー・ベイブ]
    addDiscogs(mdbmaps, "RYM", "114280692506bd05d4cfd7539287d56a", "444482")   ### [The Victims]
    addDiscogs(mdbmaps, "RYM", "a75df1d87dc99e8c1b7e4347754c114f", "257157")   ### [The Germs]
    addDiscogs(mdbmaps, "RYM", "d0cb3fcd6f04042b02a565251463e26a", "245648")   ### [Ritchie Blackmore's Rainbow]
    addDiscogs(mdbmaps, "RYM", "aa288c1a76b7deab8d1a67c0001b930c", "115131")   ### [Dramatics]
    addDiscogs(mdbmaps, "RYM", "d9716555865aab1315630873178a9116", "396928")   ### [Macabre]
    addDiscogs(mdbmaps, "RYM", "4640400f563f1bd44e8ac06eb1478db2", "263609")   ### [Rods]
    addDiscogs(mdbmaps, "RYM", "11cb815a01ebfeabe610f23c3d9341a5", "353481")   ### [The 5 Stairsteps]
    addDeezer(mdbmaps, "RYM", "1006f8b5f4f6349136744e20772df67b", "498213")   ### [Freaky]
    addDiscogs(mdbmaps, "RYM", "8464f05f95a78438d901f20f3bb6102c", "1391314")   ### [DyE]
    addDiscogs(mdbmaps, "RYM", "28ca1aff4e0848b6816e06cb63ac6313", "391187")   ### [The Fascinations]
    addDiscogs(mdbmaps, "RYM", "6f0c8fc897cbcf79d91e3aa73a7b9ecc", "4089130")   ### [Yubin]
    addDiscogs(mdbmaps, "RYM", "5c572de79fc7c43498de75b0354359bf", "197436")   ### [Special A.K.A.]
    addDiscogs(mdbmaps, "RYM", "58de6b21791a14bf8d455512b58a5fe0", "2120637")   ### [Jack J]
    addDiscogs(mdbmaps, "RYM", "9656dda9976bea12537edd01394a69b5", "3141923")   ### [Liz]
    addDiscogs(mdbmaps, "RYM", "de8d0854ec9cdb690a21685c520bd313", "428943")   ### [Mama Cass Elliot]
    addDiscogs(mdbmaps, "RYM", "8499ff97669df7b8739828ea9cb60cd5", "142218")   ### [Georgie Fame and The Blue Flames]
    addDiscogs(mdbmaps, "RYM", "1d592382500a24f783afd6dcaedcedfa", "662431")   ### [Billy Riley and His Little Green Men]
    addDiscogs(mdbmaps, "RYM", "e480b8c771379c4f0d87a7a71612fd5a", "744898")   ### [Jerry Huffman]
    addDiscogs(mdbmaps, "RYM", "81c641a062289c6b6eb13768cf03f4ac", "607662")   ### [Dave "Diddlie" Day]
    addDiscogs(mdbmaps, "RYM", "2da780b145d312964895baf893b6419f", "302683")   ### [Les Baxter and His Orchestra]
    addDiscogs(mdbmaps, "RYM", "97ffef60fd834316646dc10cfe669e6c", "635174")   ### [Chas McDevitt Skiffle Group]
    addDiscogs(mdbmaps, "RYM", "24662fc735e06440b6960ffdfb8ec3f1", "252142")   ### [Little Joe and The Thrillers]
    addDiscogs(mdbmaps, "RYM", "6c5f3f2124352ff35fa1f72f9463900f", "1349319")   ### [Gary Crosby]
    addDiscogs(mdbmaps, "RYM", "d7f575ab9f0b30607ff94345d6b95872", "295179")   ### [Tennessee Ernie]
    addDiscogs(mdbmaps, "RYM", "6bcd487dbfedc87471efc505f1bf776f", "282897")   ### [Bill Haley with Haley's Comets]
    addDiscogs(mdbmaps, "RYM", "a83d5cae123c5c403f4e3439d5c239a7", "2020851")   ### [Robert "Bumps" Blackwell]
    addDiscogs(mdbmaps, "RYM", "c7d4d677f122b16218813e56a2d5682e", "721518")   ### [Sister Wynona Carr]
    addDiscogs(mdbmaps, "RYM", "a1ddb3086942d848bffcde444a502518", "607671")   ### [Kip Tyler and His Flips]
    addDiscogs(mdbmaps, "RYM", "2ab73b138ac5936c397f20fffe111626", "322498")   ### [Bill Justis and his Orchestra]
    addDiscogs(mdbmaps, "RYM", "cc2168c2b15e60fb8b7270e9caa10858", "1421291")   ### [The Avalons]
    addDiscogs(mdbmaps, "RYM", "00869fc7c59c5598535d4f59cb9ca515", "2063449")   ### [Chuck Miller]
    addDiscogs(mdbmaps, "RYM", "406d221aab9540f18a92ebac58e171b3", "2677871")   ### [Willis Jackson and His Orchestra]
    addDiscogs(mdbmaps, "RYM", "0686f8acab49dde36c9cb1d0f45984fb", "375010")   ### [Lennie Tristano and his Sextette]
    addDiscogs(mdbmaps, "RYM", "a72df6279de3f55526791a10580bbe93", "294687")   ### [Bobby Blue Bland]
    addDiscogs(mdbmaps, "RYM", "7b301a5f599813dd9117600b5ae9cf57", "3226218")   ### [Vann "Piano Man" Walls]
    addDiscogs(mdbmaps, "RYM", "c6274c2b0a6608c3930f1001ffb6cf79", "64677")   ### [The Capris]
    addDiscogs(mdbmaps, "RYM", "3c20ec217bfb071b6e6d24ab0ca1de16", "1127968")   ### [Janis and Her Boyfriends]
    addDiscogs(mdbmaps, "RYM", "1414229535e5e79906bdaa687fe1ee9c", "2725176")   ### [Sugar Boy and His Cane Cutters]
    addDiscogs(mdbmaps, "RYM", "b4d70417d7ed2892cdc24175f5dd716c", "655884")   ### [B. B. "Blues Boy" King and His Orchestra]
    addDiscogs(mdbmaps, "RYM", "08d803ac768046dc5bc5224356c6938d", "1291003")   ### [Cliff Richard and the Drifters]
    addDiscogs(mdbmaps, "RYM", "8b55bd482e160c12d8554f91c181eabd", "744896")   ### [Jody Chastain]
    addDiscogs(mdbmaps, "RYM", "95097c0957e750498f89e462fa7b61af", "2738368")   ### [Gene Summers and His Rebels]
    addDiscogs(mdbmaps, "RYM", "eede907c111ef66c324af58abb2d346d", "304460")   ### [Mitch Ryder and The Detroit Wheels]
    addDiscogs(mdbmaps, "RYM", "f30f9cbdac9e166e667578f2748c3337", "1963133")   ### [Eddie Bond and the Stompers]
    addDiscogs(mdbmaps, "RYM", "51fb70b05a30560d5f56031d1f3d5272", "727109")   ### [The Bubble Puppy]
    addDiscogs(mdbmaps, "RYM", "a96f9983b487da6e9c0d61371e7c262f", "278460")   ### [The Keggs]
    addDiscogs(mdbmaps, "RYM", "76176c7dcda521a36efea7f1b333343c", "926540")   ### [The Riders of the Mark]
    addDiscogs(mdbmaps, "RYM", "7721f47dc5b2b6e3ae05cf2aed37bee8", "278055")   ### [Ralph Nielsen and The Chancellors]
    addDiscogs(mdbmaps, "RYM", "fb1ee867421a22ef0821db5db8415250", "3475160")   ### [Elmo James and His Broomdusters]
    addDiscogs(mdbmaps, "RYM", "ad5c4ca8acfa8dfb772c917f63caf6e3", "240895")   ### [The Foundations]
    addDiscogs(mdbmaps, "RYM", "cc5af906775cf386c4d8f733c780f5bf", "379734")   ### [Rosie and the Originals]
    addDiscogs(mdbmaps, "RYM", "4a09b73e2ba7d2516dc4eb508a4b28c5", "252519")   ### [The Groupies]
    addDiscogs(mdbmaps, "RYM", "db77e2c91f05859818bbc65341b1efaf", "280863")   ### [The Cascades]
    addDiscogs(mdbmaps, "RYM", "fba4dba8da199da5439c70b1a3915d2e", "2697011")   ### [Glen and Dave]
    addDiscogs(mdbmaps, "RYM", "a885cb02488d122d857ba9dba24eeaea", "266331")   ### [The Walker Bros.]
    addDiscogs(mdbmaps, "RYM", "f75c1ba98fbc60b882057eeac1174c92", "307510")   ### [The Wimple Winch]
    addDiscogs(mdbmaps, "RYM", "c1371cfb2f2c4b5f3d443f23fbb10002", "2327974")   ### [The 23rd Turnoff]
    addDiscogs(mdbmaps, "RYM", "3edec8433d352e26a859f565d6f938d9", "1345226")   ### [Children of the Mushroom]
    addDiscogs(mdbmaps, "RYM", "9a4a5af300d4e154f9cab8b51b7fa664", "395324")   ### [Hugo Winterhalter and His Orchestra]
    addDiscogs(mdbmaps, "RYM", "1c160345a1094e366c788733f49b7a73", "392760")   ### [Perez "Prez" Prado and His Orchestra]
    addDiscogs(mdbmaps, "RYM", "750aa563b941ed5913eb1982f2b8d7d2", "846188")   ### [Clifton Chenier and His Band]
    addDiscogs(mdbmaps, "RYM", "ff65c9d025eff21ad5ae66386be16ac1", "755278")   ### [Lord Lebby]
    addDiscogs(mdbmaps, "RYM", "5b88cc94251ee1a39a02cd30d7c48f5f", "3110468")   ### [Duane Eddy and the Rebels]
    addDiscogs(mdbmaps, "RYM", "33606ff3a0c263997ab243da27914fc4", "375478")   ### [The Harp-Tones]
    addDiscogs(mdbmaps, "RYM", "e53e53ee0f95078840a33dcf55e807ef", "2424893")   ### [Ruth Brown and Her Rhythmakers]
    addDiscogs(mdbmaps, "RYM", "abd1e7cae237a8b1e8dc539175c1b32b", "383313")   ### [The Schoolboys]
    addDiscogs(mdbmaps, "RYM", "fb97bb4278e02a846d88b2804b4e9845", "1413694")   ### [Keith Courvale]
    addDiscogs(mdbmaps, "RYM", "60e3f8c244af6e88095632d8cf7c575f", "508390")   ### [Jimmy Rogers and His Trio]
    addDiscogs(mdbmaps, "RYM", "1141505be46249c6f4368e39d3544d29", "6760947")   ### [The Beale Street Gang]    
    
    addDiscogs(mdbmaps, "RYM", "cb0d16623707c4013acfc97237e86926", "135399")   ### [Ray Smith]
    addDiscogs(mdbmaps, "RYM", "ae1386196ea4f4ac5d14a8266ede974e", "3706334")   ### [The Night Caps]
    addDiscogs(mdbmaps, "RYM", "f317dbb458d93511e36931339435fdb3", "383368")   ### [Frankie Lymon and the Teenagers]
    addDeezer(mdbmaps, "RYM", "f317dbb458d93511e36931339435fdb3", "66326")   ### [Frankie Lymon and the Teenagers]
    addDiscogs(mdbmaps, "RYM", "52420c7fe125a7116be609aa1815b2ad", "3992270")   ### [Easyfun]
    addDeezer(mdbmaps, "RYM", "52420c7fe125a7116be609aa1815b2ad", "7850520")   ### [Easyfun]
    addDiscogs(mdbmaps, "RYM", "3367885125b724966722f5a1e8417945", "570739")   ### [Mala]
    addDeezer(mdbmaps, "RYM", "3367885125b724966722f5a1e8417945", "65201")   ### [Mala]
    addDiscogs(mdbmaps, "RYM", "f9795ab6880d2c9736b641be97305cf5", "662431")   ### [Billy Riley and The Little Green Men]
    addDiscogs(mdbmaps, "RYM", "641178c91e9e85ae9d6b7ec86fe21131", "277348")   ### [Gerry and The Pacemakers]
    addDiscogs(mdbmaps, "RYM", "903dbaeed29d13001b079a0696a83cac", "850981")   ### [Lorrie and Larry Collins]
    addDeezer(mdbmaps, "RYM", "903dbaeed29d13001b079a0696a83cac", "339693")   ### [Lorrie and Larry Collins]
    addDiscogs(mdbmaps, "RYM", "54bf926890e5387e818876765ed55300", "271132")   ### [Urban Jungle]
    addDiscogs(mdbmaps, "RYM", "7e9d9b799e134bb2f1b93c3f238cce76", "96923")   ### [Iron and Wine]
    addDeezer(mdbmaps, "RYM", "7e9d9b799e134bb2f1b93c3f238cce76", "1653")   ### [Iron and Wine]
    addDiscogs(mdbmaps, "RYM", "57c4a29a954fc1f2f03ce6f3d5bebd90", "49294")   ### [The Soul Sonic Force]
    addDeezer(mdbmaps, "RYM", "57c4a29a954fc1f2f03ce6f3d5bebd90", "4787821")   ### [The Soul Sonic Force]
    addDiscogs(mdbmaps, "RYM", "7ffd4b599c15df806aa10a82135c174b", "383338")   ### [The Channels]
    addDeezer(mdbmaps, "RYM", "27e2f90d42fce6c7125d05d277d3c77a", "1380133")   ### [Elmore James and His Broom Dusters]
    addDiscogs(mdbmaps, "RYM", "ae1386196ea4f4ac5d14a8266ede974e", "3706334")   ### [The Night Caps]

    addDiscogs(mdbmaps, "RYM", "97c1525f011b7db5bd5ff70315bdc197", "1613213")   ### [The Video All-stars]
    addDeezer(mdbmaps, "RYM", "97c1525f011b7db5bd5ff70315bdc197", "5291960")   ### [The Video All-stars]
    addDiscogs(mdbmaps, "RYM", "6f2a2844f9c7735595b3342ccf05a0f3", "3693069")   ### [Malton & Hamilton]
    addDeezer(mdbmaps, "RYM", "92d9711dc16996a8f55a2de5eeb7bb47", "299353")   ### [Super Mazembe Orchestra]
    addAllMusic(mdbmaps, "RYM", "92d9711dc16996a8f55a2de5eeb7bb47", "mw0000702755")   ### [Super Mazembe Orchestra]
    addDiscogs(mdbmaps, "RYM", "226536fd2c9b7cb072790bbee299d0df", "1125187")   ### [Cairo Jazz Band]
    addAllMusic(mdbmaps, "RYM", "226536fd2c9b7cb072790bbee299d0df", "mn0001286311")   ### [Cairo Jazz Band]

    addDiscogs(mdbmaps, "RYM", "b8995ef6f5289456e6dd30c715a5614b", "139654")   ### [Cortijo & su maquina del tiempo]
    addDeezer(mdbmaps, "RYM", "b8995ef6f5289456e6dd30c715a5614b", "93712")   ### [Cortijo & su maquina del tiempo]
    addDiscogs(mdbmaps, "RYM", "54a96810075a06ee674a7dbfa4f6eec0", "255386")   ### [The John Coltrane Quartette]
    addDeezer(mdbmaps, "RYM", "54a96810075a06ee674a7dbfa4f6eec0", "75111")   ### [The John Coltrane Quartette]
    addDiscogs(mdbmaps, "RYM", "b24a2b469e219c7cf760b4d1c0307987", "262227")   ### [The Village Fugs]
    addDeezer(mdbmaps, "RYM", "b24a2b469e219c7cf760b4d1c0307987", "66976")   ### [The Village Fugs]
    addDiscogs(mdbmaps, "RYM", "398460c1553d176d29800b281b1b7704", "677697")   ### [Aut'chose]
    addDeezer(mdbmaps, "RYM", "398460c1553d176d29800b281b1b7704", "121790582")   ### [Aut'chose]
    addDiscogs(mdbmaps, "RYM", "f14cbb05dd9055044b5b1f38c9699ed5", "66302")   ### [PiL]
    addDeezer(mdbmaps, "RYM", "f14cbb05dd9055044b5b1f38c9699ed5", "455089")   ### [PiL]
    addDiscogs(mdbmaps, "RYM", "d5d922291221dce1065cf851f8783fe9", "5951")   ### [Gainsbourg]
    addDeezer(mdbmaps, "RYM", "d5d922291221dce1065cf851f8783fe9", "1007")   ### [Gainsbourg]    
    addDiscogs(mdbmaps, "RYM", "be4f5b206219e2bc875cce2456a8faec", "633398")   ### [USA MI]
    addDiscogs(mdbmaps, "RYM", "ecbf0576685ebe4939a8ec667047c59b", "154551")   ### [Ghetto Boys]
    addDeezer(mdbmaps, "RYM", "ecbf0576685ebe4939a8ec667047c59b", "2828")   ### [Ghetto Boys]
    addDiscogs(mdbmaps, "RYM", "5c03d352e137adb452fec7cd64d05df7", "1394017")   ### [Roy ("Bald Head") Byrd]
    addDiscogs(mdbmaps, "RYM", "ff561b8942b87feba40a03a034f4d1b9", "4078404")   ### [Begiristain]
    addDiscogs(mdbmaps, "RYM", "76bd520b4aaab5e07858111a9886dc9e", "1939286")   ### [Jack Collier and Orchestra]
    addDiscogs(mdbmaps, "RYM", "b88c7ad00db61da1ea591634abf7d785", "4556436")   ### [NXzero]
    addDeezer(mdbmaps, "RYM", "b88c7ad00db61da1ea591634abf7d785", "1209354")   ### [NXzero]
    addDiscogs(mdbmaps, "RYM", "494ef0a399d60509533601be3d45c5c7", "1471950")   ### [DoomThrone]
    addDiscogs(mdbmaps, "RYM", "8dd495749630a864c1adf7ed9068ca44", "263201")   ### [The L.A. Guns]
    addDeezer(mdbmaps, "RYM", "8dd495749630a864c1adf7ed9068ca44", "94963")   ### [The L.A. Guns]
    addDiscogs(mdbmaps, "RYM", "b8257601f14d6dc866a95742650e605b", "108722")   ### [Limp Biscut]
    addDeezer(mdbmaps, "RYM", "16ad2629d260b8e9c199da2f026a5e76", "66994952")   ### [Korrozia Metalla]
    addDiscogs(mdbmaps, "RYM", "7340b8500d38286c8e9f5c073c466122", "80727")   ### [Sheila B. Devotion]
    addDeezer(mdbmaps, "RYM", "7340b8500d38286c8e9f5c073c466122", "3549")   ### [Sheila B. Devotion]
    addDiscogs(mdbmaps, "RYM", "fb385af8e7195d967dbb1c23d22158aa", "470098")   ### [I Cugini di Campagna]
    addDeezer(mdbmaps, "RYM", "fb385af8e7195d967dbb1c23d22158aa", "71646")   ### [I Cugini di Campagna]
    addDiscogs(mdbmaps, "RYM", "a533ad50e587afa35f6d69a9908bb114", "381534")   ### [Donny Osmond of The Osmonds]
    addDeezer(mdbmaps, "RYM", "a533ad50e587afa35f6d69a9908bb114", "2302")   ### [Donny Osmond of The Osmonds]
    addDiscogs(mdbmaps, "RYM", "1924e936c39620ef15cae4e7047be80f", "368604")   ### [King Brothers]
    addDiscogs(mdbmaps, "RYM", "4dcaa5d0e506bbdd336c6d1023181a4b", "252505")   ### [Sllednats]
    addDiscogs(mdbmaps, "RYM", "958a077ffc1f5abaeebf107e5a8adfd2", "1420163")   ### [Tom Glazer and the Do-Re-Mi Children's Chorus]
    addDiscogs(mdbmaps, "RYM", "710f42512024611e79c216f2bba42d89", "3118503")   ### [New Orleans Rhythm]    
    addDiscogs(mdbmaps, "RYM", "2e366b8d5a687ea05312368860f3bbbb", "296154")   ### [Elsa]
    addDiscogs(mdbmaps, "RYM", "635e6f146da4f888fea1e9900d749384", "2303563")   ### [Dave and the Derros]
    addDiscogs(mdbmaps, "RYM", "0b2178e7e2dd651f3e1aeea34035151a", "965883")   ### [Warwick Capper]
    addDiscogs(mdbmaps, "RYM", "9a6ac9e81c7bb168c0f0506310dbd02b", "12339")   ### [Ice MC]
    addDiscogs(mdbmaps, "RYM", "47b36024b7024303835b1d08be1cb718", "1313160")   ### [El Norte]
    addDiscogs(mdbmaps, "RYM", "b199adeeba809cf59835c98e0ab0e5ba", "777447")   ### [30 Odd Foot of Grunts]
    addDiscogs(mdbmaps, "RYM", "99ffa679a997d771f183d7a4f36843c1", "308825")   ### [Kevin Morane]
    addDiscogs(mdbmaps, "RYM", "dca0cf58c2f0e240fb8cb308c11cf625", "1021148")   ### [Childliners]
    addDiscogs(mdbmaps, "RYM", "afa2dfc18e915d93fbb050d40d0ed73e", "1006801")   ### [Norbi]
    addDiscogs(mdbmaps, "RYM", "5dcd3dc01a90a38dd77fffa7ccd01b93", "445344")   ### [Chris Waddle]
    addDiscogs(mdbmaps, "RYM", "fa9d8a0d150a464b833049daa064c344", "3903635")   ### [Jesulín]
    addDiscogs(mdbmaps, "RYM", "798dc49732042367cdce2996cae274f8", "802749")   ### [The Bandit Band]
    addDiscogs(mdbmaps, "RYM", "150f684f51be94261aec9d6c9736f906", "218966")   ### [Images]
    addDiscogs(mdbmaps, "RYM", "14fdbeaacc12f9484b80e45b831264ca", "457008")   ### [Hakkûhbar]
    addDiscogs(mdbmaps, "RYM", "513a1ddd23559420a7cd9d78f9547684", "1345353")   ### [The Beau Denturies]
    addDiscogs(mdbmaps, "RYM", "1edd37060bc6de68b0f5d016909d1864", "20956")   ### [Donald Byrd Brass]
    addDiscogs(mdbmaps, "RYM", "323cf70c8866a309e94bd0f80cfcc515", "2836613")   ### [the Pommodores]
    addDiscogs(mdbmaps, "RYM", "5f7d0cf4a5858fa89ab9623cee323166", "437613")   ### [Wilfrid Brambell]
    addDiscogs(mdbmaps, "RYM", "7a872557295cfc3f7a7086f7c0f0c4b5", "366273")   ### [The Kosher Five]
    addDiscogs(mdbmaps, "RYM", "4a7636adceec374fa6372be7a395eb90", "1442118")   ### [Nela Pocisková]    
    addDeezer(mdbmaps, "RYM", "5820b50f2b57ea416986b51321b4f15a", "214954")   ### [dARI]
    addDiscogs(mdbmaps, "RYM", "f843b987aae855280e6b4f06505755ba", "322068")   ### [Manchester United Football Club]
    addDiscogs(mdbmaps, "RYM", "567de8acafc6725ad7a222cb7aebdc10", "324556")   ### [Pow woW]
    addDiscogs(mdbmaps, "RYM", "4d5540abdf793742c0ab2d486952f62b", "5943286")   ### [Threatin]    
    addDiscogs(mdbmaps, "RYM", "afc28559dce7af8dfc24bef840756c5f", "2606")   ### [Your Favorite Martian]
    addDeezer(mdbmaps, "RYM", "cc40a99cfb60ac11efe53dfaf7192aa7", "2307081")   ### [Bart Baker]
    addDiscogs(mdbmaps, "RYM", "3ab8390b315e35b8d0feb8baae1c6b60", "613854")   ### [Kyo
    addDeezer(mdbmaps, "RYM", "9633df06d51223375c083678d3aa1f47", "4841295")   ### [Jumex]
    addDeezer(mdbmaps, "RYM", "bd335a99bf9bd07f0ecea434abc62e50", "50452132")   ### [Slayyyter]
    addDiscogs(mdbmaps, "RYM", "29665f3e314ac303defbd4424611cde3", "331401")   ### [The Robins]
    addDiscogs(mdbmaps, "RYM", "b9652f9fa2c850d9fe6da6fa9a4e0039", "4966057")   ### [Moon Mullins and his Night Raiders]
    addDiscogs(mdbmaps, "RYM", "079eb9a2f0db5ab134148f8c602b0bde", "1461247")   ### [The John Barry Seven and Orchestra]
    addDiscogs(mdbmaps, "RYM", "f7e73d71ee79a46dc2e5428e3390b9ed", "370771")   ### [The Beverly's All Stars]
    addDiscogs(mdbmaps, "RYM", "741041ef487bf0aef5f934ec4e69ff01", "157480")   ### [Luis Enríquez Bacalov]
    addDeezer(mdbmaps, "RYM", "4b16c7d324511b725977a4e19c50ca52", "3524")   ### [The Sweet]
    addDiscogs(mdbmaps, "RYM", "4b16c7d324511b725977a4e19c50ca52", "253929")   ### [The Sweet]
    addDeezer(mdbmaps, "RYM", "de6f7d93d538f1fff448f98d30bb6f9f", "2519")   ### [BIG]
    addDiscogs(mdbmaps, "RYM", "de6f7d93d538f1fff448f98d30bb6f9f", "65049")   ### [BIG]
    addDeezer(mdbmaps, "RYM", "430fa4204c8da9ef1b33cd1c8d94a30b", "4183")   ### [Fatlip]
    addDiscogs(mdbmaps, "RYM", "430fa4204c8da9ef1b33cd1c8d94a30b", "53627")   ### [Fatlip]
    addDeezer(mdbmaps, "RYM", "a55c05d5618ad60e894f85d49077b897", "6840")   ### [The Clipse]
    addAllMusic(mdbmaps, "RYM", "d4e2cc671a40fd407ac5ffd13afe2619", "mn0003903838")   ### [Kim Lip]
    addDeezer(mdbmaps, "RYM", "4af2b4acd0ca8eede64f51f9e6e30c04", "11814761")   ### [Cartier'GOD]
    addDeezer(mdbmaps, "RYM", "c874e309f4461fc19542c20e5e2a6f43", "503190")   ### [Liturgy]
    addDiscogs(mdbmaps, "RYM", "0c6ef78b580aa5b30f0cff4594c3d1ed", "7606376")   ### [Sewerslvt]
    addDiscogs(mdbmaps, "RYM", "d41050f9da22b67593ff5b469cf157da", "50182")   ### [ナンシー・シナトラとリー・ヘイズルウッド]
    addDiscogs(mdbmaps, "RYM", "02bfaf56ddaf8a6cccf382834b76a456", "83725")   ### [Meirelles e os Copa 5]
    addDiscogs(mdbmaps, "RYM", "b605584f86d2db6de44565682761907f", "2583851")   ### [Memphis Slim and the House Rockers]
    addDiscogs(mdbmaps, "RYM", "2c43c2cd93785d8cfc0e56536f9a5541", "2582177")   ### [Professor Longhair and His Blues Scholars]
    addDiscogs(mdbmaps, "RYM", "bc8b9f7b2561460c702d22ad3a0c0a15", "596029")   ### [Lonnie Donegan and His Skiffle Group]
    addDiscogs(mdbmaps, "RYM", "0879feb9c0b1d5f3eead7873c56b248d", "450656")   ### [Jackie Brenston and His Delta Cats]
    addDiscogs(mdbmaps, "RYM", "96ffc21ae710d82d87118a7c81be8a8d", "388889")   ### [The Factory]
    addDiscogs(mdbmaps, "RYM", "82ec25cf11ec51d3b89737f0e76f34d4", "7145937")   ### [C'est la Key]
    addDiscogs(mdbmaps, "RYM", "ae25052ad0f5b8192425223041bcda63", "43032")   ### [Koji Kondo]
    addDiscogs(mdbmaps, "RYM", "3eb2a9f33044038f5ac0dfce95473d3f", "517226")   ### [Smarki Smark]
    addDiscogs(mdbmaps, "RYM", "401047f446fd94208fdbd8b326ff3ae9", "352678")   ### [Ko Otani]
    addDiscogs(mdbmaps, "RYM", "ca2577a5936067144a45e00da069170b", "109947")   ### [Mclusky]
    addDiscogs(mdbmaps, "RYM", "35916743cb13af35180fc4092a8af174", "3718032")   ### [Yu Miyake]
    addDiscogs(mdbmaps, "RYM", "6afbd86876892e2989d592f42830665e", "1625262")   ### [Mario Galaxy Orchestra]
    addDiscogs(mdbmaps, "RYM", "283a24c1889712bc4404cefc2585eaac", "1387807")   ### [Shibusashirazu]
    addDiscogs(mdbmaps, "RYM", "b8fdbd005a1ea5443b399b4902389be3", "833163")   ### [José Van Dam]
    addDiscogs(mdbmaps, "RYM", "48435035a72262a5c872c40127e4bcae", "5220036")   ### [Shōgo Sakai]
    addDiscogs(mdbmaps, "RYM", "b58586ce55750a36ce1e2846924567dd", "835650")   ### [Collegium Vocale Gent]
    addAllMusic(mdbmaps, "RYM", "166162c241859e2ac7df9e72c3a968c2", "mn0000142767")   ### [Dave Brubeck Quartet]
    addDiscogs(mdbmaps, "RYM", "166162c241859e2ac7df9e72c3a968c2", "251689")   ### [Dave Brubeck Quartet]
    addDiscogs(mdbmaps, "RYM", "a12c3ff6f6f1b321115187fbd2836167", "3943018")   ### [Professor Longhair and his New Orleans Boys]
    addAllMusic(mdbmaps, "RYM", "d3ed51828201f55461b8e50eb70a3ca0", "mn0001488326")   ### [Sun Ra and His Arkestra]
    addDiscogs(mdbmaps, "RYM", "d3ed51828201f55461b8e50eb70a3ca0", "2219395")   ### [Sun Ra and His Arkestra]
    addDiscogs(mdbmaps, "RYM", "47a2e9ff0265dfca51166c188d1dc6a0", "262128")   ### [Art Blakey and The Jazz Messengers]
    addDiscogs(mdbmaps, "RYM", "40a416471254ef5f86c722e7b61ce757", "521963")   ### [Ray Charles His Orchestra and Chorus]
    addDiscogs(mdbmaps, "RYM", "75c916c0bacddbcc17919985eb793deb", "287108")   ### [Mickey and Sylvia]
    addDeezer(mdbmaps, "RYM", "68d0b3546273c06aeeb390eda1211506", "2379021")   ### [Gosia Andrzejewicz]
    addDiscogs(mdbmaps, "RYM", "68d0b3546273c06aeeb390eda1211506", "1163486")   ### [Gosia Andrzejewicz]
    addDeezer(mdbmaps, "RYM", "fa6af02bf507a0fcdb38b679f0c8122d", "76577")   ### [Doda]
    addDeezer(mdbmaps, "RYM", "fc1c73048b67fda50174f53c23ffb9bb", "71203")   ### [Tony Martin]
    addDiscogs(mdbmaps, "RYM", "fc1c73048b67fda50174f53c23ffb9bb", "520764")   ### [Tony Martin]
    addDeezer(mdbmaps, "RYM", "f378d592733c455c132ef0125310216b", "248191")   ### [The Avons]
    addDiscogs(mdbmaps, "RYM", "f378d592733c455c132ef0125310216b", "324390")   ### [The Avons]
    addDeezer(mdbmaps, "RYM", "bce67689ce5606d76bf67f9e8d939ef1", "160609")   ### [Tommy Steele and The Steelmen]
    addDiscogs(mdbmaps, "RYM", "bce67689ce5606d76bf67f9e8d939ef1", "662149")   ### [Tommy Steele and The Steelmen]
    addDeezer(mdbmaps, "RYM", "2a0c745c899051ddfbb77783cf6ee779", "6073616")   ### [Ecco2K]
    addDiscogs(mdbmaps, "RYM", "2a0c745c899051ddfbb77783cf6ee779", "4905641")   ### [Ecco2K]
    addDeezer(mdbmaps, "RYM", "38dd191cc6a9586233f97f5fc3d5aa2a", "78598292")   ### [LOONA ODD EYE CIRCLE]
    addDiscogs(mdbmaps, "RYM", "38dd191cc6a9586233f97f5fc3d5aa2a", "6080850")   ### [LOONA ODD EYE CIRCLE]    
    addDeezer(mdbmaps, "RYM", "345a3a754d70941142c97ddc23875531", "74930292")   ### [Josephine Foster and the Supposed]
    addAllMusic(mdbmaps, "RYM", "345a3a754d70941142c97ddc23875531", "mn0001972461")   ### [Josephine Foster and the Supposed]
    addDeezer(mdbmaps, "RYM", "1d20ba9c582ce56a3ee6b58afac51a11", "253719")   ### [Beaver and Krause]
    addDiscogs(mdbmaps, "RYM", "1d20ba9c582ce56a3ee6b58afac51a11", "84683")   ### [Beaver and Krause]
    addAllMusic(mdbmaps, "RYM", "895cb1f647a19af849f485064fe3318f", "mn0001633598")   ### [Palace Music]
    addAllMusic(mdbmaps, "RYM", "41eed152d1d60026a09852a9be6707d8", "mn0000531986")   ### [Bowie]
    addDiscogs(mdbmaps, "RYM", "594ba96d4fb061a007a06f112c258b95", "135307")   ### [Palace Brothers]    
    addAllMusic(mdbmaps, "RYM", "f8622b59427a58f6d10228b274130eb0", "mn0000503695")   ### [The Africa '70]
    addDeezer(mdbmaps, "RYM", "1e46268cc0f1f32a140be18ea04f9efe", "1153")   ### [Paco Ibáñez]
    addAllMusic(mdbmaps, "RYM", "1e46268cc0f1f32a140be18ea04f9efe", "mn0000548044")   ### [Paco Ibáñez]
    addDeezer(mdbmaps, "RYM", "99f6bc5d57d6feafe8eb0144d88d9b5e", "180374")   ### [Sixteen Horsepower]
    addDiscogs(mdbmaps, "RYM", "99f6bc5d57d6feafe8eb0144d88d9b5e", "207906")   ### [Sixteen Horsepower]
    addDeezer(mdbmaps, "RYM", "b0cbcf7d16e2aa86f275335cfc7a3fb9", "14985")   ### [Nits]
    addAllMusic(mdbmaps, "RYM", "b0cbcf7d16e2aa86f275335cfc7a3fb9", "mn0000355554")   ### [Nits]
    addDeezer(mdbmaps, "RYM", "be121b29e9bdc34f0c81c2f21f171936", "157515")   ### [Nektar]
    addAllMusic(mdbmaps, "RYM", "be121b29e9bdc34f0c81c2f21f171936", "mn0000380106")   ### [Nektar]
    addDiscogs(mdbmaps, "RYM", "5507ffc4752cd2ae73b6686fac4fe2d3", "1232916")   ### [Spriguns]
    addDeezer(mdbmaps, "RYM", "61f8fa1151a6ff2bc1b0caf9b049101c", "4068826")   ### [Eulenspygel]
    addDiscogs(mdbmaps, "RYM", "61f8fa1151a6ff2bc1b0caf9b049101c", "363905")   ### [Eulenspygel]""".split("\n")


rms="""Could not add artist data from PrimaryKey: a1b047738f828d4880fb4877b80d979c
Could not add artist data from PrimaryKey: c16063a9fea3d3d4684095fd5f68d71d
Could not add artist data from PrimaryKey: c4eb00d17e99f78059401a8e00e7dacc
Could not add artist data from PrimaryKey: d3c8efab1413492442397c2db3bd3a2e
Could not add artist data from PrimaryKey: 9f0948c41e2ee2e523387a725d623265
Could not add artist data from PrimaryKey: 6290942c7523a0243f9b8f3cd56f714b
Could not add artist data from PrimaryKey: 26292da02ddfcc6808f4808c05272b68
Could not add artist data from PrimaryKey: 89b5b278b1f3c1b105cb138724ccd2fd
Could not add artist data from PrimaryKey: 457c1759fe0e158d9af951a8f99396a1
Could not add artist data from PrimaryKey: 8d14b9131f9c697849bc5b70407b5273
Could not add artist data from PrimaryKey: 8c7f56c6720e2060f85e828f84a855bc
Could not add artist data from PrimaryKey: 2d87f3a9b9a9e38f9dac2fd645883bff
Could not add artist data from PrimaryKey: 5b023ef505fdc8be6e835d9c067e62c1
Could not add artist data from PrimaryKey: a00cdbfb8409cad1339cc8ce90033f01
Could not add artist data from PrimaryKey: 20cae975550e8e8385cd88d3dc185c45
Could not add artist data from PrimaryKey: a9441c4ca72d40d97c9c84555ba7d74d
Could not add artist data from PrimaryKey: 8316104bbfbd61597ab33281fb7e7475
Could not add artist data from PrimaryKey: 8b7fa1b266a935d00d4f02cb99738d4f
Could not add artist data from PrimaryKey: eae4bc4aff6b4214eba2368eb8f04ca0
Could not add artist data from PrimaryKey: 779c27b775d595de1b5aba08e32feca6
Could not add artist data from PrimaryKey: 7fc92a88d903740a35469bac9e7e6c98
Could not add artist data from PrimaryKey: d0cb3fcd6f04042b02a565251463e26a
Could not add artist data from PrimaryKey: aa288c1a76b7deab8d1a67c0001b930c
Could not add artist data from PrimaryKey: 11cb815a01ebfeabe610f23c3d9341a5
Could not add artist data from PrimaryKey: 5c572de79fc7c43498de75b0354359bf
Could not add artist data from PrimaryKey: 8499ff97669df7b8739828ea9cb60cd5
Could not add artist data from PrimaryKey: 1d592382500a24f783afd6dcaedcedfa
Could not add artist data from PrimaryKey: 2da780b145d312964895baf893b6419f
Could not add artist data from PrimaryKey: 97ffef60fd834316646dc10cfe669e6c
Could not add artist data from PrimaryKey: 24662fc735e06440b6960ffdfb8ec3f1
Could not add artist data from PrimaryKey: d7f575ab9f0b30607ff94345d6b95872
Could not add artist data from PrimaryKey: 6bcd487dbfedc87471efc505f1bf776f
Could not add artist data from PrimaryKey: a83d5cae123c5c403f4e3439d5c239a7
Could not add artist data from PrimaryKey: c7d4d677f122b16218813e56a2d5682e
Could not add artist data from PrimaryKey: a1ddb3086942d848bffcde444a502518
Could not add artist data from PrimaryKey: 2ab73b138ac5936c397f20fffe111626
Could not add artist data from PrimaryKey: 406d221aab9540f18a92ebac58e171b3
Could not add artist data from PrimaryKey: 0686f8acab49dde36c9cb1d0f45984fb
Could not add artist data from PrimaryKey: 3c20ec217bfb071b6e6d24ab0ca1de16
Could not add artist data from PrimaryKey: b4d70417d7ed2892cdc24175f5dd716c
Could not add artist data from PrimaryKey: 08d803ac768046dc5bc5224356c6938d
Could not add artist data from PrimaryKey: 95097c0957e750498f89e462fa7b61af
Could not add artist data from PrimaryKey: eede907c111ef66c324af58abb2d346d
Could not add artist data from PrimaryKey: f30f9cbdac9e166e667578f2748c3337
Could not add artist data from PrimaryKey: 7721f47dc5b2b6e3ae05cf2aed37bee8
Could not add artist data from PrimaryKey: fb1ee867421a22ef0821db5db8415250
Could not add artist data from PrimaryKey: cc5af906775cf386c4d8f733c780f5bf
Could not add artist data from PrimaryKey: a885cb02488d122d857ba9dba24eeaea
Could not add artist data from PrimaryKey: 3edec8433d352e26a859f565d6f938d9
Could not add artist data from PrimaryKey: 9a4a5af300d4e154f9cab8b51b7fa664
Could not add artist data from PrimaryKey: 1c160345a1094e366c788733f49b7a73
Could not add artist data from PrimaryKey: 750aa563b941ed5913eb1982f2b8d7d2
Could not add artist data from PrimaryKey: 5b88cc94251ee1a39a02cd30d7c48f5f
Could not add artist data from PrimaryKey: e53e53ee0f95078840a33dcf55e807ef
Could not add artist data from PrimaryKey: 641178c91e9e85ae9d6b7ec86fe21131
Could not add artist data from PrimaryKey: 903dbaeed29d13001b079a0696a83cac
Could not add artist data from PrimaryKey: 903dbaeed29d13001b079a0696a83cac
Could not add artist data from PrimaryKey: 7e9d9b799e134bb2f1b93c3f238cce76
Could not add artist data from PrimaryKey: 7e9d9b799e134bb2f1b93c3f238cce76
Could not add artist data from PrimaryKey: 97c1525f011b7db5bd5ff70315bdc197
Could not add artist data from PrimaryKey: 97c1525f011b7db5bd5ff70315bdc197
Could not add artist data from PrimaryKey: 92d9711dc16996a8f55a2de5eeb7bb47
Could not add artist data from PrimaryKey: 92d9711dc16996a8f55a2de5eeb7bb47
Could not add artist data from PrimaryKey: 226536fd2c9b7cb072790bbee299d0df
Could not add artist data from PrimaryKey: 226536fd2c9b7cb072790bbee299d0df
Could not add artist data from PrimaryKey: b8995ef6f5289456e6dd30c715a5614b
Could not add artist data from PrimaryKey: b8995ef6f5289456e6dd30c715a5614b
Could not add artist data from PrimaryKey: 54a96810075a06ee674a7dbfa4f6eec0
Could not add artist data from PrimaryKey: 54a96810075a06ee674a7dbfa4f6eec0
Could not add artist data from PrimaryKey: b24a2b469e219c7cf760b4d1c0307987
Could not add artist data from PrimaryKey: b24a2b469e219c7cf760b4d1c0307987
Could not add artist data from PrimaryKey: 398460c1553d176d29800b281b1b7704
Could not add artist data from PrimaryKey: 398460c1553d176d29800b281b1b7704
Could not add artist data from PrimaryKey: d5d922291221dce1065cf851f8783fe9
Could not add artist data from PrimaryKey: d5d922291221dce1065cf851f8783fe9
Could not add artist data from PrimaryKey: ecbf0576685ebe4939a8ec667047c59b
Could not add artist data from PrimaryKey: ecbf0576685ebe4939a8ec667047c59b
Could not add artist data from PrimaryKey: 76bd520b4aaab5e07858111a9886dc9e
Could not add artist data from PrimaryKey: b88c7ad00db61da1ea591634abf7d785
Could not add artist data from PrimaryKey: b88c7ad00db61da1ea591634abf7d785
Could not add artist data from PrimaryKey: b8257601f14d6dc866a95742650e605b
Could not add artist data from PrimaryKey: a533ad50e587afa35f6d69a9908bb114
Could not add artist data from PrimaryKey: a533ad50e587afa35f6d69a9908bb114
Could not add artist data from PrimaryKey: 4dcaa5d0e506bbdd336c6d1023181a4b
Could not add artist data from PrimaryKey: 9a6ac9e81c7bb168c0f0506310dbd02b
Could not add artist data from PrimaryKey: 1edd37060bc6de68b0f5d016909d1864
Could not add artist data from PrimaryKey: 567de8acafc6725ad7a222cb7aebdc10
Could not add artist data from PrimaryKey: afc28559dce7af8dfc24bef840756c5f
Could not add artist data from PrimaryKey: de6f7d93d538f1fff448f98d30bb6f9f
Could not add artist data from PrimaryKey: de6f7d93d538f1fff448f98d30bb6f9f
Could not add artist data from PrimaryKey: d41050f9da22b67593ff5b469cf157da
Could not add artist data from PrimaryKey: 594ba96d4fb061a007a06f112c258b95""".split("\n")

In [ ]:
rmIDs = [x.split()[-1] for x in rms]

for line in adds:
    if len(line.strip()) == 0:
        continue
    bad = False
    for rmID in rmIDs:
        if rmID in line:
            bad = True
            break
            
    if not bad:
        print("    {0}".format(line))